In [1]:
import pandas as pd
import numpy as np
import pickle

# ------------------------------
# 1. Game Logic
# ------------------------------

def check_winner(board):
    """
    Returns 1 if Player 1 wins, 2 if Player 2 wins, or None if no winner.
    """
    wins = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),  # rows
        (0, 3, 6), (1, 4, 7), (2, 5, 8),  # columns
        (0, 4, 8), (2, 4, 6)              # diagonals
    ]
    for (i, j, k) in wins:
        if board[i] != 0 and board[i] == board[j] == board[k]:
            return board[i]
    return None

# ------------------------------
# 2. Terminal Outcome Averaging
# ------------------------------

def get_terminal_outcomes(board, turn):
    """
    Recursively explores all game continuations from the current board.
    Returns a list of outcomes: 1 if Player 1 wins, -1 if Player 2 wins, 0 for a draw.
    """
    winner = check_winner(board)
    if winner is not None:
        # Convert winner to evaluation: Player 1 win (1) -> 1, Player 2 win (2) -> -1.
        return [1] if winner == 1 else [-1]
    if all(cell != 0 for cell in board):
        return [0]  # Draw

    outcomes = []
    for i in range(9):
        if board[i] == 0:
            board[i] = turn
            # Toggle turn: if current turn is 1, next is 2; if current is 2, next is 1.
            outcomes.extend(get_terminal_outcomes(board, 2 if turn == 1 else 1))
            board[i] = 0  # Backtrack
    return outcomes

# ------------------------------
# 3. Dataset Generation
# ------------------------------

def generate_states_average(board, turn, states):
    """
    Recursively generates all valid game states and stores the average terminal evaluation from each state.
    """
    board_tuple = tuple(board)
    if board_tuple in states:
        return

    outcomes = get_terminal_outcomes(board, turn)
    average_score = sum(outcomes) / len(outcomes)
    states[board_tuple] = average_score

    if check_winner(board) is not None or all(cell != 0 for cell in board):
        return

    for i in range(9):
        if board[i] == 0:
            board[i] = turn
            generate_states_average(board, 2 if turn == 1 else 1, states)
            board[i] = 0

# ------------------------------
# 4. Main Execution
# ------------------------------

if __name__ == "__main__":
    states = {}
    empty_board = [0] * 9  # 0 represents an empty cell.
    generate_states_average(empty_board, 1, states)

    data = []
    for board_tuple, avg_eval in states.items():
        board_3x3 = np.array(board_tuple).reshape((3, 3))
        data.append((board_3x3, avg_eval))

    # Save the data as a pickle file ("eval.pkl")
    with open("eval.pkl", "wb") as f:
        pickle.dump(data, f)

    print(f"Saved {len(data)} board states to 'eval.pkl'")


Saved 5478 board states to 'eval.pkl'


In [7]:
with open("eval.pkl", "rb") as f:
    data = pickle.load(f)

# Example: inspect first item
board, score = data[10]
print(board)
print("Score:", score)


[[1 2 1]
 [2 1 2]
 [2 1 1]]
Score: 1.0


In [9]:
import pickle
import numpy as np
from collections import OrderedDict

with open("eval.pkl", "rb") as f:
    data = pickle.load(f)

# Flatten and store in ordered dict
states = OrderedDict()
for board, score in data:
    flat_board = tuple(board.flatten())  # tuple of 9 ints
    states[flat_board] = score

# Write to a Python file
with open("states.py", "w") as f:
    f.write("from collections import OrderedDict\n\n")
    f.write("states = OrderedDict([\n")
    for board, score in states.items():
        f.write(f"    ({board}, {score:.6f}),\n")
    f.write("])\n")

print(f"Wrote {len(states)} states to 'states.py'")

Wrote 5478 states to 'states.py'


In [1]:
from utils import State, Action, load_data  # Adjust imports as needed
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

def load_aug_data() -> list[tuple[State, float]]:
    with open("augmented_data.pkl", "rb") as f:
        data = pickle.load(f)
    new_data = []
    for row in data:
        row_data, utility = row
        board, fill_num, prev_local_action = row_data
        state = State(board=board, fill_num=fill_num, prev_local_action=prev_local_action)
        new_data.append((state, utility))
    return new_data

data = load_aug_data()
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [3]:
from collections import OrderedDict

states = OrderedDict([
    ((0, 0, 0, 0, 0, 0, 0, 0, 0), 0.208804),
    ((1, 0, 0, 0, 0, 0, 0, 0, 0), 0.243617),
    ((1, 2, 0, 0, 0, 0, 0, 0, 0), 0.352236),
    ((1, 2, 1, 0, 0, 0, 0, 0, 0), 0.209459),
    ((1, 2, 1, 2, 0, 0, 0, 0, 0), 0.372549),
    ((1, 2, 1, 2, 1, 0, 0, 0, 0), 0.777778),
    ((1, 2, 1, 2, 1, 2, 0, 0, 0), 1.000000),
    ((1, 2, 1, 2, 1, 2, 1, 0, 0), 1.000000),
    ((1, 2, 1, 2, 1, 2, 0, 1, 0), 1.000000),
    ((1, 2, 1, 2, 1, 2, 2, 1, 0), 1.000000),
    ((1, 2, 1, 2, 1, 2, 2, 1, 1), 1.000000),
    ((1, 2, 1, 2, 1, 2, 0, 1, 2), 1.000000),
    ((1, 2, 1, 2, 1, 2, 1, 1, 2), 1.000000),
    ((1, 2, 1, 2, 1, 2, 0, 0, 1), 1.000000),
    ((1, 2, 1, 2, 1, 0, 2, 0, 0), 0.600000),
    ((1, 2, 1, 2, 1, 1, 2, 0, 0), 0.500000),
    ((1, 2, 1, 2, 1, 1, 2, 2, 0), 1.000000),
    ((1, 2, 1, 2, 1, 1, 2, 2, 1), 1.000000),
    ((1, 2, 1, 2, 1, 1, 2, 0, 2), 0.000000),
    ((1, 2, 1, 2, 1, 1, 2, 1, 2), 0.000000),
    ((1, 2, 1, 2, 1, 0, 2, 1, 0), 0.500000),
    ((1, 2, 1, 2, 1, 0, 2, 1, 2), 0.000000),
    ((1, 2, 1, 2, 1, 0, 2, 0, 1), 1.000000),
    ((1, 2, 1, 2, 1, 0, 0, 2, 0), 1.000000),
    ((1, 2, 1, 2, 1, 1, 0, 2, 0), 1.000000),
    ((1, 2, 1, 2, 1, 1, 0, 2, 2), 1.000000),
    ((1, 2, 1, 2, 1, 1, 1, 2, 2), 1.000000),
    ((1, 2, 1, 2, 1, 0, 1, 2, 0), 1.000000),
    ((1, 2, 1, 2, 1, 0, 0, 2, 1), 1.000000),
    ((1, 2, 1, 2, 1, 0, 0, 0, 2), 0.600000),
    ((1, 2, 1, 2, 1, 1, 0, 0, 2), 0.500000),
    ((1, 2, 1, 2, 1, 0, 1, 0, 2), 1.000000),
    ((1, 2, 1, 2, 1, 0, 0, 1, 2), 0.500000),
    ((1, 2, 1, 2, 0, 1, 0, 0, 0), 0.428571),
    ((1, 2, 1, 2, 2, 1, 0, 0, 0), 0.200000),
    ((1, 2, 1, 2, 2, 1, 1, 0, 0), -0.500000),
    ((1, 2, 1, 2, 2, 1, 1, 2, 0), -1.000000),
    ((1, 2, 1, 2, 2, 1, 1, 0, 2), 0.000000),
    ((1, 2, 1, 2, 2, 1, 1, 1, 2), 0.000000),
    ((1, 2, 1, 2, 2, 1, 0, 1, 0), 0.500000),
    ((1, 2, 1, 2, 2, 1, 2, 1, 0), 1.000000),
    ((1, 2, 1, 2, 2, 1, 2, 1, 1), 1.000000),
    ((1, 2, 1, 2, 2, 1, 0, 1, 2), 0.000000),
    ((1, 2, 1, 2, 2, 1, 0, 0, 1), 1.000000),
    ((1, 2, 1, 2, 0, 1, 2, 0, 0), 0.600000),
    ((1, 2, 1, 2, 0, 1, 2, 1, 0), 0.500000),
    ((1, 2, 1, 2, 0, 1, 2, 1, 2), 0.000000),
    ((1, 2, 1, 2, 0, 1, 2, 0, 1), 1.000000),
    ((1, 2, 1, 2, 0, 1, 0, 2, 0), 0.600000),
    ((1, 2, 1, 2, 0, 1, 1, 2, 0), 0.000000),
    ((1, 2, 1, 2, 0, 1, 1, 2, 2), 1.000000),
    ((1, 2, 1, 2, 0, 1, 0, 2, 1), 1.000000),
    ((1, 2, 1, 2, 0, 1, 0, 0, 2), 0.333333),
    ((1, 2, 1, 2, 0, 1, 1, 0, 2), 0.500000),
    ((1, 2, 1, 2, 0, 1, 0, 1, 2), 0.000000),
    ((1, 2, 1, 2, 0, 0, 1, 0, 0), 0.047619),
    ((1, 2, 1, 2, 2, 0, 1, 0, 0), -0.666667),
    ((1, 2, 1, 2, 2, 0, 1, 1, 0), -0.500000),
    ((1, 2, 1, 2, 2, 2, 1, 1, 0), -1.000000),
    ((1, 2, 1, 2, 2, 0, 1, 1, 2), 0.000000),
    ((1, 2, 1, 2, 2, 0, 1, 0, 1), -1.000000),
    ((1, 2, 1, 2, 2, 2, 1, 0, 1), -1.000000),
    ((1, 2, 1, 2, 2, 0, 1, 2, 1), -1.000000),
    ((1, 2, 1, 2, 0, 2, 1, 0, 0), 0.200000),
    ((1, 2, 1, 2, 0, 2, 1, 1, 0), 0.000000),
    ((1, 2, 1, 2, 0, 2, 1, 1, 2), 1.000000),
    ((1, 2, 1, 2, 0, 2, 1, 0, 1), 0.000000),
    ((1, 2, 1, 2, 0, 2, 1, 2, 1), 1.000000),
    ((1, 2, 1, 2, 1, 2, 1, 2, 1), 1.000000),
    ((1, 2, 1, 2, 0, 0, 1, 2, 0), 0.200000),
    ((1, 2, 1, 2, 0, 0, 1, 2, 1), 0.000000),
    ((1, 2, 1, 2, 0, 0, 1, 0, 2), 0.600000),
    ((1, 2, 1, 2, 0, 0, 1, 1, 2), 0.500000),
    ((1, 2, 1, 2, 0, 0, 0, 1, 0), 0.333333),
    ((1, 2, 1, 2, 2, 0, 0, 1, 0), 0.000000),
    ((1, 2, 1, 2, 2, 0, 0, 1, 1), 0.000000),
    ((1, 2, 1, 2, 2, 2, 0, 1, 1), -1.000000),
    ((1, 2, 1, 2, 2, 0, 2, 1, 1), 1.000000),
    ((1, 2, 1, 2, 0, 2, 0, 1, 0), 0.333333),
    ((1, 2, 1, 2, 0, 2, 0, 1, 1), 0.000000),
    ((1, 2, 1, 2, 0, 2, 2, 1, 1), 1.000000),
    ((1, 2, 1, 2, 0, 0, 2, 1, 0), 0.666667),
    ((1, 2, 1, 2, 0, 0, 2, 1, 1), 1.000000),
    ((1, 2, 1, 2, 0, 0, 0, 1, 2), 0.333333),
    ((1, 2, 1, 2, 0, 0, 0, 0, 1), 0.333333),
    ((1, 2, 1, 2, 2, 0, 0, 0, 1), -0.200000),
    ((1, 2, 1, 2, 0, 2, 0, 0, 1), 0.200000),
    ((1, 2, 1, 2, 0, 0, 2, 0, 1), 1.000000),
    ((1, 2, 1, 2, 0, 0, 0, 2, 1), 0.500000),
    ((1, 2, 1, 0, 2, 0, 0, 0, 0), -0.086957),
    ((1, 2, 1, 1, 2, 0, 0, 0, 0), -0.058824),
    ((1, 2, 1, 1, 2, 2, 0, 0, 0), 0.200000),
    ((1, 2, 1, 1, 2, 2, 1, 0, 0), 1.000000),
    ((1, 2, 1, 1, 2, 2, 0, 1, 0), 0.500000),
    ((1, 2, 1, 1, 2, 2, 2, 1, 0), 0.000000),
    ((1, 2, 1, 1, 2, 2, 2, 1, 1), 0.000000),
    ((1, 2, 1, 1, 2, 2, 0, 1, 2), 1.000000),
    ((1, 2, 1, 1, 2, 2, 1, 1, 2), 1.000000),
    ((1, 2, 1, 1, 2, 2, 0, 0, 1), -0.500000),
    ((1, 2, 1, 1, 2, 2, 2, 0, 1), 0.000000),
    ((1, 2, 1, 1, 2, 2, 0, 2, 1), -1.000000),
    ((1, 2, 1, 1, 2, 0, 2, 0, 0), -0.333333),
    ((1, 2, 1, 1, 2, 1, 2, 0, 0), -0.500000),
    ((1, 2, 1, 1, 2, 1, 2, 2, 0), -1.000000),
    ((1, 2, 1, 1, 2, 1, 2, 0, 2), 0.000000),
    ((1, 2, 1, 1, 2, 1, 2, 1, 2), 0.000000),
    ((1, 2, 1, 1, 2, 0, 2, 1, 0), 0.000000),
    ((1, 2, 1, 1, 2, 0, 2, 1, 2), 0.000000),
    ((1, 2, 1, 1, 2, 0, 2, 0, 1), -0.500000),
    ((1, 2, 1, 1, 2, 0, 2, 2, 1), -1.000000),
    ((1, 2, 1, 1, 2, 0, 0, 2, 0), -1.000000),
    ((1, 2, 1, 1, 2, 0, 0, 0, 2), 0.200000),
    ((1, 2, 1, 1, 2, 1, 0, 0, 2), -0.500000),
    ((1, 2, 1, 1, 2, 1, 0, 2, 2), -1.000000),
    ((1, 2, 1, 1, 2, 0, 1, 0, 2), 1.000000),
    ((1, 2, 1, 1, 2, 0, 0, 1, 2), 0.500000),
    ((1, 2, 1, 0, 2, 1, 0, 0, 0), -0.058824),
    ((1, 2, 1, 0, 2, 1, 2, 0, 0), 0.200000),
    ((1, 2, 1, 0, 2, 1, 2, 1, 0), 0.500000),
    ((1, 2, 1, 0, 2, 1, 2, 1, 2), 0.000000),
    ((1, 2, 1, 0, 2, 1, 2, 0, 1), 1.000000),
    ((1, 2, 1, 0, 2, 1, 0, 2, 0), -1.000000),
    ((1, 2, 1, 0, 2, 1, 0, 0, 2), -0.333333),
    ((1, 2, 1, 0, 2, 1, 1, 0, 2), -0.500000),
    ((1, 2, 1, 0, 2, 1, 1, 2, 2), -1.000000),
    ((1, 2, 1, 0, 2, 1, 0, 1, 2), 0.000000),
    ((1, 2, 1, 0, 2, 0, 1, 0, 0), -0.294118),
    ((1, 2, 1, 0, 2, 2, 1, 0, 0), -0.200000),
    ((1, 2, 1, 0, 2, 2, 1, 1, 0), 0.000000),
    ((1, 2, 1, 0, 2, 2, 1, 1, 2), 1.000000),
    ((1, 2, 1, 0, 2, 2, 1, 0, 1), -1.000000),
    ((1, 2, 1, 0, 2, 2, 1, 2, 1), -1.000000),
    ((1, 2, 1, 0, 2, 0, 1, 2, 0), -1.000000),
    ((1, 2, 1, 0, 2, 0, 1, 0, 2), 0.200000),
    ((1, 2, 1, 0, 2, 0, 1, 1, 2), 0.500000),
    ((1, 2, 1, 0, 2, 0, 0, 1, 0), 0.166667),
    ((1, 2, 1, 0, 2, 2, 0, 1, 0), 0.000000),
    ((1, 2, 1, 0, 2, 2, 0, 1, 1), -0.500000),
    ((1, 2, 1, 0, 2, 2, 2, 1, 1), 0.000000),
    ((1, 2, 1, 0, 2, 0, 2, 1, 0), 0.333333),
    ((1, 2, 1, 0, 2, 0, 2, 1, 1), 0.500000),
    ((1, 2, 1, 0, 2, 0, 0, 1, 2), 0.333333),
    ((1, 2, 1, 0, 2, 0, 0, 0, 1), -0.294118),
    ((1, 2, 1, 0, 2, 2, 0, 0, 1), -0.666667),
    ((1, 2, 1, 0, 2, 0, 2, 0, 1), 0.200000),
    ((1, 2, 1, 0, 2, 0, 0, 2, 1), -1.000000),
    ((1, 2, 1, 0, 0, 2, 0, 0, 0), 0.372549),
    ((1, 2, 1, 1, 0, 2, 0, 0, 0), 0.428571),
    ((1, 2, 1, 1, 0, 2, 2, 0, 0), 0.333333),
    ((1, 2, 1, 1, 1, 2, 2, 0, 0), 0.500000),
    ((1, 2, 1, 1, 1, 2, 2, 2, 0), 1.000000),
    ((1, 2, 1, 1, 1, 2, 2, 2, 1), 1.000000),
    ((1, 2, 1, 1, 1, 2, 2, 0, 2), 0.000000),
    ((1, 2, 1, 1, 1, 2, 2, 1, 2), 0.000000),
    ((1, 2, 1, 1, 0, 2, 2, 1, 0), 0.000000),
    ((1, 2, 1, 1, 0, 2, 2, 1, 2), 0.000000),
    ((1, 2, 1, 1, 0, 2, 2, 0, 1), 0.500000),
    ((1, 2, 1, 1, 0, 2, 2, 2, 1), 1.000000),
    ((1, 2, 1, 1, 0, 2, 0, 2, 0), 0.600000),
    ((1, 2, 1, 1, 1, 2, 0, 2, 0), 1.000000),
    ((1, 2, 1, 1, 1, 2, 0, 2, 2), 1.000000),
    ((1, 2, 1, 1, 1, 2, 1, 2, 2), 1.000000),
    ((1, 2, 1, 1, 0, 2, 1, 2, 0), 1.000000),
    ((1, 2, 1, 1, 0, 2, 0, 2, 1), 0.000000),
    ((1, 2, 1, 1, 0, 2, 0, 0, 2), 0.600000),
    ((1, 2, 1, 1, 1, 2, 0, 0, 2), 0.500000),
    ((1, 2, 1, 1, 0, 2, 1, 0, 2), 1.000000),
    ((1, 2, 1, 1, 0, 2, 0, 1, 2), 0.500000),
    ((1, 2, 1, 0, 1, 2, 0, 0, 0), 0.777778),
    ((1, 2, 1, 0, 1, 2, 2, 0, 0), 0.600000),
    ((1, 2, 1, 0, 1, 2, 2, 1, 0), 0.500000),
    ((1, 2, 1, 0, 1, 2, 2, 1, 2), 0.000000),
    ((1, 2, 1, 0, 1, 2, 2, 0, 1), 1.000000),
    ((1, 2, 1, 0, 1, 2, 0, 2, 0), 1.000000),
    ((1, 2, 1, 0, 1, 2, 1, 2, 0), 1.000000),
    ((1, 2, 1, 0, 1, 2, 0, 2, 1), 1.000000),
    ((1, 2, 1, 0, 1, 2, 0, 0, 2), 0.600000),
    ((1, 2, 1, 0, 1, 2, 1, 0, 2), 1.000000),
    ((1, 2, 1, 0, 1, 2, 0, 1, 2), 0.500000),
    ((1, 2, 1, 0, 0, 2, 1, 0, 0), 0.333333),
    ((1, 2, 1, 0, 0, 2, 1, 2, 0), 0.500000),
    ((1, 2, 1, 0, 0, 2, 1, 2, 1), 0.000000),
    ((1, 2, 1, 0, 0, 2, 1, 0, 2), 1.000000),
    ((1, 2, 1, 0, 0, 2, 1, 1, 2), 1.000000),
    ((1, 2, 1, 0, 0, 2, 0, 1, 0), 0.333333),
    ((1, 2, 1, 0, 0, 2, 2, 1, 0), 0.333333),
    ((1, 2, 1, 0, 0, 2, 2, 1, 1), 0.500000),
    ((1, 2, 1, 0, 0, 2, 0, 1, 2), 0.666667),
    ((1, 2, 1, 0, 0, 2, 0, 0, 1), 0.047619),
    ((1, 2, 1, 0, 0, 2, 2, 0, 1), 0.600000),
    ((1, 2, 1, 0, 0, 2, 0, 2, 1), 0.200000),
    ((1, 2, 1, 0, 0, 0, 2, 0, 0), 0.185185),
    ((1, 2, 1, 1, 0, 0, 2, 0, 0), -0.166667),
    ((1, 2, 1, 1, 0, 0, 2, 2, 0), -0.333333),
    ((1, 2, 1, 1, 1, 0, 2, 2, 0), 0.000000),
    ((1, 2, 1, 1, 1, 0, 2, 2, 2), -1.000000),
    ((1, 2, 1, 1, 0, 1, 2, 2, 0), -1.000000),
    ((1, 2, 1, 1, 0, 1, 2, 2, 2), -1.000000),
    ((1, 2, 1, 1, 0, 0, 2, 2, 1), 0.000000),
    ((1, 2, 1, 1, 0, 0, 2, 0, 2), -0.333333),
    ((1, 2, 1, 1, 1, 0, 2, 0, 2), -0.500000),
    ((1, 2, 1, 1, 0, 1, 2, 0, 2), -0.500000),
    ((1, 2, 1, 1, 0, 0, 2, 1, 2), 0.000000),
    ((1, 2, 1, 0, 1, 0, 2, 0, 0), 0.238095),
    ((1, 2, 1, 0, 1, 0, 2, 2, 0), 0.200000),
    ((1, 2, 1, 0, 1, 1, 2, 2, 0), 0.000000),
    ((1, 2, 1, 0, 1, 1, 2, 2, 2), -1.000000),
    ((1, 2, 1, 0, 1, 0, 2, 2, 1), 1.000000),
    ((1, 2, 1, 0, 1, 0, 2, 0, 2), -0.333333),
    ((1, 2, 1, 0, 1, 1, 2, 0, 2), -0.500000),
    ((1, 2, 1, 0, 1, 0, 2, 1, 2), 0.000000),
    ((1, 2, 1, 0, 0, 1, 2, 0, 0), 0.047619),
    ((1, 2, 1, 0, 0, 1, 2, 2, 0), -0.200000),
    ((1, 2, 1, 0, 0, 1, 2, 2, 1), 1.000000),
    ((1, 2, 1, 0, 0, 1, 2, 0, 2), -0.333333),
    ((1, 2, 1, 0, 0, 1, 2, 1, 2), 0.000000),
    ((1, 2, 1, 0, 0, 0, 2, 1, 0), 0.333333),
    ((1, 2, 1, 0, 0, 0, 2, 1, 2), 0.000000),
    ((1, 2, 1, 0, 0, 0, 2, 0, 1), 0.555556),
    ((1, 2, 1, 0, 0, 0, 2, 2, 1), 0.500000),
    ((1, 2, 1, 0, 0, 0, 0, 2, 0), 0.200000),
    ((1, 2, 1, 1, 0, 0, 0, 2, 0), -0.058824),
    ((1, 2, 1, 1, 0, 0, 0, 2, 2), -0.200000),
    ((1, 2, 1, 1, 1, 0, 0, 2, 2), 0.000000),
    ((1, 2, 1, 1, 0, 1, 0, 2, 2), -1.000000),
    ((1, 2, 1, 1, 0, 0, 1, 2, 2), 1.000000),
    ((1, 2, 1, 0, 1, 0, 0, 2, 0), 0.555556),
    ((1, 2, 1, 0, 1, 0, 0, 2, 2), 0.200000),
    ((1, 2, 1, 0, 1, 1, 0, 2, 2), 0.000000),
    ((1, 2, 1, 0, 1, 0, 1, 2, 2), 1.000000),
    ((1, 2, 1, 0, 0, 1, 0, 2, 0), -0.058824),
    ((1, 2, 1, 0, 0, 1, 0, 2, 2), -0.333333),
    ((1, 2, 1, 0, 0, 1, 1, 2, 2), 0.000000),
    ((1, 2, 1, 0, 0, 0, 1, 2, 0), 0.285714),
    ((1, 2, 1, 0, 0, 0, 1, 2, 2), 0.500000),
    ((1, 2, 1, 0, 0, 0, 0, 2, 1), 0.285714),
    ((1, 2, 1, 0, 0, 0, 0, 0, 2), 0.185185),
    ((1, 2, 1, 1, 0, 0, 0, 0, 2), 0.047619),
    ((1, 2, 1, 0, 1, 0, 0, 0, 2), 0.238095),
    ((1, 2, 1, 0, 0, 1, 0, 0, 2), -0.166667),
    ((1, 2, 1, 0, 0, 0, 1, 0, 2), 0.555556),
    ((1, 2, 1, 0, 0, 0, 0, 1, 2), 0.333333),
    ((1, 2, 0, 1, 0, 0, 0, 0, 0), 0.238901),
    ((1, 2, 2, 1, 0, 0, 0, 0, 0), 0.452055),
    ((1, 2, 2, 1, 1, 0, 0, 0, 0), 0.733333),
    ((1, 2, 2, 1, 1, 2, 0, 0, 0), 0.500000),
    ((1, 2, 2, 1, 1, 2, 1, 0, 0), 1.000000),
    ((1, 2, 2, 1, 1, 2, 0, 1, 0), 0.000000),
    ((1, 2, 2, 1, 1, 2, 2, 1, 0), 1.000000),
    ((1, 2, 2, 1, 1, 2, 2, 1, 1), 1.000000),
    ((1, 2, 2, 1, 1, 2, 0, 1, 2), -1.000000),
    ((1, 2, 2, 1, 1, 2, 0, 0, 1), 1.000000),
    ((1, 2, 2, 1, 1, 0, 2, 0, 0), 1.000000),
    ((1, 2, 2, 1, 1, 1, 2, 0, 0), 1.000000),
    ((1, 2, 2, 1, 1, 0, 2, 1, 0), 1.000000),
    ((1, 2, 2, 1, 1, 0, 2, 1, 2), 1.000000),
    ((1, 2, 2, 1, 1, 1, 2, 1, 2), 1.000000),
    ((1, 2, 2, 1, 1, 0, 2, 0, 1), 1.000000),
    ((1, 2, 2, 1, 1, 0, 0, 2, 0), 1.000000),
    ((1, 2, 2, 1, 1, 1, 0, 2, 0), 1.000000),
    ((1, 2, 2, 1, 1, 0, 1, 2, 0), 1.000000),
    ((1, 2, 2, 1, 1, 0, 0, 2, 1), 1.000000),
    ((1, 2, 2, 1, 1, 0, 0, 0, 2), 0.500000),
    ((1, 2, 2, 1, 1, 1, 0, 0, 2), 1.000000),
    ((1, 2, 2, 1, 1, 0, 1, 0, 2), 1.000000),
    ((1, 2, 2, 1, 1, 0, 0, 1, 2), 0.000000),
    ((1, 2, 2, 1, 0, 1, 0, 0, 0), 0.333333),
    ((1, 2, 2, 1, 2, 1, 0, 0, 0), -0.200000),
    ((1, 2, 2, 1, 2, 1, 1, 0, 0), 1.000000),
    ((1, 2, 2, 1, 2, 1, 0, 1, 0), 0.000000),
    ((1, 2, 2, 1, 2, 1, 2, 1, 0), -1.000000),
    ((1, 2, 2, 1, 2, 1, 0, 1, 2), 1.000000),
    ((1, 2, 2, 1, 2, 1, 1, 1, 2), 1.000000),
    ((1, 2, 2, 1, 2, 1, 0, 0, 1), -1.000000),
    ((1, 2, 2, 1, 2, 1, 2, 0, 1), -1.000000),
    ((1, 2, 2, 1, 2, 1, 0, 2, 1), -1.000000),
    ((1, 2, 2, 1, 0, 1, 2, 0, 0), 0.200000),
    ((1, 2, 2, 1, 0, 1, 2, 1, 0), 0.000000),
    ((1, 2, 2, 1, 0, 1, 2, 1, 2), 1.000000),
    ((1, 2, 2, 1, 0, 1, 2, 0, 1), 0.000000),
    ((1, 2, 2, 1, 0, 1, 2, 2, 1), 1.000000),
    ((1, 2, 2, 1, 1, 1, 2, 2, 1), 1.000000),
    ((1, 2, 2, 1, 0, 1, 0, 2, 0), 0.500000),
    ((1, 2, 2, 1, 0, 1, 1, 2, 0), 1.000000),
    ((1, 2, 2, 1, 0, 1, 0, 2, 1), 0.000000),
    ((1, 2, 2, 1, 0, 1, 0, 0, 2), 1.000000),
    ((1, 2, 2, 1, 0, 1, 1, 0, 2), 1.000000),
    ((1, 2, 2, 1, 0, 1, 0, 1, 2), 1.000000),
    ((1, 2, 2, 1, 0, 0, 1, 0, 0), 1.000000),
    ((1, 2, 2, 1, 0, 0, 0, 1, 0), 0.428571),
    ((1, 2, 2, 1, 2, 0, 0, 1, 0), 0.200000),
    ((1, 2, 2, 1, 2, 0, 1, 1, 0), 1.000000),
    ((1, 2, 2, 1, 2, 0, 0, 1, 1), 0.000000),
    ((1, 2, 2, 1, 2, 2, 0, 1, 1), 1.000000),
    ((1, 2, 2, 1, 2, 2, 1, 1, 1), 1.000000),
    ((1, 2, 2, 1, 2, 0, 2, 1, 1), -1.000000),
    ((1, 2, 2, 1, 0, 2, 0, 1, 0), 0.600000),
    ((1, 2, 2, 1, 0, 2, 1, 1, 0), 1.000000),
    ((1, 2, 2, 1, 0, 2, 0, 1, 1), 1.000000),
    ((1, 2, 2, 1, 0, 2, 2, 1, 1), 1.000000),
    ((1, 2, 2, 1, 0, 0, 2, 1, 0), 0.333333),
    ((1, 2, 2, 1, 0, 0, 2, 1, 1), 0.000000),
    ((1, 2, 2, 1, 0, 0, 0, 1, 2), 0.600000),
    ((1, 2, 2, 1, 0, 0, 1, 1, 2), 1.000000),
    ((1, 2, 2, 1, 0, 0, 0, 0, 1), 0.333333),
    ((1, 2, 2, 1, 2, 0, 0, 0, 1), -0.200000),
    ((1, 2, 2, 1, 2, 0, 1, 0, 1), 1.000000),
    ((1, 2, 2, 1, 0, 2, 0, 0, 1), 1.000000),
    ((1, 2, 2, 1, 0, 2, 1, 0, 1), 1.000000),
    ((1, 2, 2, 1, 0, 0, 2, 0, 1), 0.200000),
    ((1, 2, 2, 1, 0, 0, 0, 2, 1), 0.500000),
    ((1, 2, 2, 1, 0, 0, 1, 2, 1), 1.000000),
    ((1, 2, 0, 1, 2, 0, 0, 0, 0), -0.068493),
    ((1, 2, 0, 1, 2, 1, 0, 0, 0), -0.294118),
    ((1, 2, 0, 1, 2, 1, 2, 0, 0), -0.666667),
    ((1, 2, 0, 1, 2, 1, 2, 1, 0), -0.500000),
    ((1, 2, 0, 1, 2, 1, 2, 1, 2), 0.000000),
    ((1, 2, 0, 1, 2, 1, 2, 0, 1), -1.000000),
    ((1, 2, 0, 1, 2, 1, 2, 2, 1), -1.000000),
    ((1, 2, 0, 1, 2, 1, 0, 2, 0), -1.000000),
    ((1, 2, 0, 1, 2, 1, 0, 0, 2), 0.200000),
    ((1, 2, 0, 1, 2, 1, 1, 0, 2), 1.000000),
    ((1, 2, 0, 1, 2, 1, 0, 1, 2), 0.500000),
    ((1, 2, 0, 1, 2, 0, 1, 0, 0), 1.000000),
    ((1, 2, 0, 1, 2, 0, 0, 1, 0), 0.238095),
    ((1, 2, 0, 1, 2, 2, 0, 1, 0), 0.600000),
    ((1, 2, 0, 1, 2, 2, 1, 1, 0), 1.000000),
    ((1, 2, 0, 1, 2, 2, 0, 1, 1), 0.500000),
    ((1, 2, 0, 1, 2, 2, 2, 1, 1), 0.000000),
    ((1, 2, 0, 1, 2, 0, 2, 1, 0), -0.333333),
    ((1, 2, 0, 1, 2, 0, 2, 1, 1), -0.500000),
    ((1, 2, 0, 1, 2, 0, 0, 1, 2), 0.600000),
    ((1, 2, 0, 1, 2, 0, 1, 1, 2), 1.000000),
    ((1, 2, 0, 1, 2, 0, 0, 0, 1), -0.294118),
    ((1, 2, 0, 1, 2, 2, 0, 0, 1), 0.200000),
    ((1, 2, 0, 1, 2, 2, 1, 0, 1), 1.000000),
    ((1, 2, 0, 1, 2, 0, 2, 0, 1), -0.666667),
    ((1, 2, 0, 1, 2, 0, 0, 2, 1), -1.000000),
    ((1, 2, 0, 1, 0, 2, 0, 0, 0), 0.493671),
    ((1, 2, 0, 1, 1, 2, 0, 0, 0), 0.555556),
    ((1, 2, 0, 1, 1, 2, 2, 0, 0), 0.600000),
    ((1, 2, 0, 1, 1, 2, 2, 1, 0), 0.500000),
    ((1, 2, 0, 1, 1, 2, 2, 1, 2), 0.000000),
    ((1, 2, 0, 1, 1, 2, 2, 0, 1), 1.000000),
    ((1, 2, 0, 1, 1, 2, 0, 2, 0), 1.000000),
    ((1, 2, 0, 1, 1, 2, 1, 2, 0), 1.000000),
    ((1, 2, 0, 1, 1, 2, 0, 2, 1), 1.000000),
    ((1, 2, 0, 1, 1, 2, 0, 0, 2), 0.200000),
    ((1, 2, 0, 1, 1, 2, 1, 0, 2), 1.000000),
    ((1, 2, 0, 1, 1, 2, 0, 1, 2), -0.500000),
    ((1, 2, 0, 1, 0, 2, 1, 0, 0), 1.000000),
    ((1, 2, 0, 1, 0, 2, 0, 1, 0), 0.428571),
    ((1, 2, 0, 1, 0, 2, 2, 1, 0), 0.333333),
    ((1, 2, 0, 1, 0, 2, 2, 1, 1), 0.500000),
    ((1, 2, 0, 1, 0, 2, 0, 1, 2), 0.200000),
    ((1, 2, 0, 1, 0, 2, 1, 1, 2), 1.000000),
    ((1, 2, 0, 1, 0, 2, 0, 0, 1), 0.555556),
    ((1, 2, 0, 1, 0, 2, 2, 0, 1), 0.600000),
    ((1, 2, 0, 1, 0, 2, 0, 2, 1), 0.500000),
    ((1, 2, 0, 1, 0, 2, 1, 2, 1), 1.000000),
    ((1, 2, 0, 1, 0, 0, 2, 0, 0), 0.074074),
    ((1, 2, 0, 1, 1, 0, 2, 0, 0), 0.555556),
    ((1, 2, 0, 1, 1, 0, 2, 2, 0), 0.500000),
    ((1, 2, 0, 1, 1, 1, 2, 2, 0), 1.000000),
    ((1, 2, 0, 1, 1, 0, 2, 2, 1), 1.000000),
    ((1, 2, 0, 1, 1, 0, 2, 0, 2), 0.200000),
    ((1, 2, 0, 1, 1, 1, 2, 0, 2), 1.000000),
    ((1, 2, 0, 1, 1, 0, 2, 1, 2), 0.500000),
    ((1, 2, 0, 1, 0, 1, 2, 0, 0), -0.142857),
    ((1, 2, 0, 1, 0, 1, 2, 2, 0), -0.200000),
    ((1, 2, 0, 1, 0, 1, 2, 2, 1), 0.000000),
    ((1, 2, 0, 1, 0, 1, 2, 0, 2), 0.200000),
    ((1, 2, 0, 1, 0, 1, 2, 1, 2), 0.500000),
    ((1, 2, 0, 1, 0, 0, 2, 1, 0), 0.166667),
    ((1, 2, 0, 1, 0, 0, 2, 1, 2), 0.333333),
    ((1, 2, 0, 1, 0, 0, 2, 0, 1), 0.047619),
    ((1, 2, 0, 1, 0, 0, 2, 2, 1), 0.200000),
    ((1, 2, 0, 1, 0, 0, 0, 2, 0), 0.245902),
    ((1, 2, 0, 1, 1, 0, 0, 2, 0), 0.733333),
    ((1, 2, 0, 1, 1, 0, 0, 2, 2), 0.500000),
    ((1, 2, 0, 1, 1, 1, 0, 2, 2), 1.000000),
    ((1, 2, 0, 1, 1, 0, 1, 2, 2), 1.000000),
    ((1, 2, 0, 1, 0, 1, 0, 2, 0), 0.000000),
    ((1, 2, 0, 1, 0, 1, 0, 2, 2), 0.000000),
    ((1, 2, 0, 1, 0, 1, 1, 2, 2), 1.000000),
    ((1, 2, 0, 1, 0, 0, 1, 2, 0), 1.000000),
    ((1, 2, 0, 1, 0, 0, 0, 2, 1), 0.285714),
    ((1, 2, 0, 1, 0, 0, 0, 0, 2), 0.291139),
    ((1, 2, 0, 1, 1, 0, 0, 0, 2), 0.333333),
    ((1, 2, 0, 1, 0, 1, 0, 0, 2), 0.333333),
    ((1, 2, 0, 1, 0, 0, 1, 0, 2), 1.000000),
    ((1, 2, 0, 1, 0, 0, 0, 1, 2), 0.428571),
    ((1, 2, 0, 0, 1, 0, 0, 0, 0), 0.568710),
    ((1, 2, 2, 0, 1, 0, 0, 0, 0), 0.616438),
    ((1, 2, 2, 0, 1, 1, 0, 0, 0), 0.777778),
    ((1, 2, 2, 2, 1, 1, 0, 0, 0), 0.600000),
    ((1, 2, 2, 2, 1, 1, 1, 0, 0), 0.500000),
    ((1, 2, 2, 2, 1, 1, 1, 2, 0), 1.000000),
    ((1, 2, 2, 2, 1, 1, 1, 2, 1), 1.000000),
    ((1, 2, 2, 2, 1, 1, 1, 0, 2), 0.000000),
    ((1, 2, 2, 2, 1, 1, 1, 1, 2), 0.000000),
    ((1, 2, 2, 2, 1, 1, 0, 1, 0), 0.500000),
    ((1, 2, 2, 2, 1, 1, 2, 1, 0), 1.000000),
    ((1, 2, 2, 2, 1, 1, 2, 1, 1), 1.000000),
    ((1, 2, 2, 2, 1, 1, 0, 1, 2), 0.000000),
    ((1, 2, 2, 2, 1, 1, 0, 0, 1), 1.000000),
    ((1, 2, 2, 0, 1, 1, 2, 0, 0), 1.000000),
    ((1, 2, 2, 0, 1, 1, 2, 1, 0), 1.000000),
    ((1, 2, 2, 0, 1, 1, 2, 1, 2), 1.000000),
    ((1, 2, 2, 0, 1, 1, 2, 0, 1), 1.000000),
    ((1, 2, 2, 0, 1, 1, 0, 2, 0), 1.000000),
    ((1, 2, 2, 0, 1, 1, 1, 2, 0), 1.000000),
    ((1, 2, 2, 0, 1, 1, 1, 2, 2), 1.000000),
    ((1, 2, 2, 1, 1, 1, 1, 2, 2), 1.000000),
    ((1, 2, 2, 0, 1, 1, 0, 2, 1), 1.000000),
    ((1, 2, 2, 0, 1, 1, 0, 0, 2), 0.600000),
    ((1, 2, 2, 0, 1, 1, 1, 0, 2), 0.500000),
    ((1, 2, 2, 0, 1, 1, 0, 1, 2), 0.500000),
    ((1, 2, 2, 0, 1, 0, 1, 0, 0), 0.555556),
    ((1, 2, 2, 2, 1, 0, 1, 0, 0), 0.600000),
    ((1, 2, 2, 2, 1, 0, 1, 1, 0), 0.500000),
    ((1, 2, 2, 2, 1, 2, 1, 1, 0), 1.000000),
    ((1, 2, 2, 2, 1, 2, 1, 1, 1), 1.000000),
    ((1, 2, 2, 2, 1, 0, 1, 1, 2), 0.000000),
    ((1, 2, 2, 2, 1, 0, 1, 0, 1), 1.000000),
    ((1, 2, 2, 0, 1, 2, 1, 0, 0), 0.500000),
    ((1, 2, 2, 0, 1, 2, 1, 1, 0), 0.000000),
    ((1, 2, 2, 0, 1, 2, 1, 1, 2), -1.000000),
    ((1, 2, 2, 0, 1, 2, 1, 0, 1), 1.000000),
    ((1, 2, 2, 0, 1, 0, 1, 2, 0), 1.000000),
    ((1, 2, 2, 0, 1, 0, 1, 2, 1), 1.000000),
    ((1, 2, 2, 0, 1, 0, 1, 0, 2), 0.200000),
    ((1, 2, 2, 0, 1, 0, 1, 1, 2), -0.500000),
    ((1, 2, 2, 0, 1, 0, 0, 1, 0), 0.428571),
    ((1, 2, 2, 2, 1, 0, 0, 1, 0), 0.600000),
    ((1, 2, 2, 2, 1, 0, 0, 1, 1), 1.000000),
    ((1, 2, 2, 0, 1, 2, 0, 1, 0), 0.200000),
    ((1, 2, 2, 0, 1, 2, 0, 1, 1), 1.000000),
    ((1, 2, 2, 0, 1, 0, 2, 1, 0), 1.000000),
    ((1, 2, 2, 0, 1, 0, 2, 1, 1), 1.000000),
    ((1, 2, 2, 0, 1, 0, 0, 1, 2), 0.000000),
    ((1, 2, 2, 0, 1, 0, 0, 0, 1), 1.000000),
    ((1, 2, 0, 2, 1, 0, 0, 0, 0), 0.696203),
    ((1, 2, 0, 2, 1, 1, 0, 0, 0), 0.619048),
    ((1, 2, 0, 2, 1, 1, 2, 0, 0), 0.600000),
    ((1, 2, 0, 2, 1, 1, 2, 1, 0), 0.500000),
    ((1, 2, 0, 2, 1, 1, 2, 1, 2), 0.000000),
    ((1, 2, 0, 2, 1, 1, 2, 0, 1), 1.000000),
    ((1, 2, 0, 2, 1, 1, 0, 2, 0), 1.000000),
    ((1, 2, 0, 2, 1, 1, 1, 2, 0), 1.000000),
    ((1, 2, 0, 2, 1, 1, 1, 2, 2), 1.000000),
    ((1, 2, 0, 2, 1, 1, 0, 2, 1), 1.000000),
    ((1, 2, 0, 2, 1, 1, 0, 0, 2), 0.333333),
    ((1, 2, 0, 2, 1, 1, 1, 0, 2), 0.500000),
    ((1, 2, 0, 2, 1, 1, 0, 1, 2), 0.000000),
    ((1, 2, 0, 2, 1, 0, 1, 0, 0), 0.777778),
    ((1, 2, 0, 2, 1, 2, 1, 0, 0), 1.000000),
    ((1, 2, 0, 2, 1, 2, 1, 1, 0), 1.000000),
    ((1, 2, 0, 2, 1, 2, 1, 1, 2), 1.000000),
    ((1, 2, 0, 2, 1, 2, 1, 0, 1), 1.000000),
    ((1, 2, 0, 2, 1, 0, 1, 2, 0), 1.000000),
    ((1, 2, 0, 2, 1, 0, 1, 2, 1), 1.000000),
    ((1, 2, 0, 2, 1, 0, 1, 0, 2), 0.600000),
    ((1, 2, 0, 2, 1, 0, 1, 1, 2), 0.500000),
    ((1, 2, 0, 2, 1, 0, 0, 1, 0), 0.619048),
    ((1, 2, 0, 2, 1, 2, 0, 1, 0), 1.000000),
    ((1, 2, 0, 2, 1, 2, 0, 1, 1), 1.000000),
    ((1, 2, 0, 2, 1, 0, 2, 1, 0), 0.600000),
    ((1, 2, 0, 2, 1, 0, 2, 1, 1), 1.000000),
    ((1, 2, 0, 2, 1, 0, 0, 1, 2), 0.333333),
    ((1, 2, 0, 2, 1, 0, 0, 0, 1), 1.000000),
    ((1, 2, 0, 0, 1, 2, 0, 0, 0), 0.616438),
    ((1, 2, 0, 0, 1, 2, 1, 0, 0), 0.733333),
    ((1, 2, 0, 0, 1, 2, 1, 2, 0), 1.000000),
    ((1, 2, 0, 0, 1, 2, 1, 2, 1), 1.000000),
    ((1, 2, 0, 0, 1, 2, 1, 0, 2), 0.500000),
    ((1, 2, 0, 0, 1, 2, 1, 1, 2), 0.000000),
    ((1, 2, 0, 0, 1, 2, 0, 1, 0), 0.428571),
    ((1, 2, 0, 0, 1, 2, 2, 1, 0), 0.600000),
    ((1, 2, 0, 0, 1, 2, 2, 1, 1), 1.000000),
    ((1, 2, 0, 0, 1, 2, 0, 1, 2), 0.000000),
    ((1, 2, 0, 0, 1, 2, 0, 0, 1), 1.000000),
    ((1, 2, 0, 0, 1, 0, 2, 0, 0), 0.493671),
    ((1, 2, 0, 0, 1, 1, 2, 0, 0), 0.555556),
    ((1, 2, 0, 0, 1, 1, 2, 2, 0), 0.500000),
    ((1, 2, 0, 0, 1, 1, 2, 2, 1), 1.000000),
    ((1, 2, 0, 0, 1, 1, 2, 0, 2), 0.200000),
    ((1, 2, 0, 0, 1, 1, 2, 1, 2), 0.500000),
    ((1, 2, 0, 0, 1, 0, 2, 1, 0), 0.619048),
    ((1, 2, 0, 0, 1, 0, 2, 1, 2), 0.333333),
    ((1, 2, 0, 0, 1, 0, 2, 0, 1), 1.000000),
    ((1, 2, 0, 0, 1, 0, 0, 2, 0), 0.761194),
    ((1, 2, 0, 0, 1, 1, 0, 2, 0), 0.777778),
    ((1, 2, 0, 0, 1, 1, 0, 2, 2), 0.600000),
    ((1, 2, 0, 0, 1, 1, 1, 2, 2), 1.000000),
    ((1, 2, 0, 0, 1, 0, 1, 2, 0), 1.000000),
    ((1, 2, 0, 0, 1, 0, 1, 2, 2), 1.000000),
    ((1, 2, 0, 0, 1, 0, 0, 2, 1), 1.000000),
    ((1, 2, 0, 0, 1, 0, 0, 0, 2), 0.333333),
    ((1, 2, 0, 0, 1, 1, 0, 0, 2), 0.428571),
    ((1, 2, 0, 0, 1, 0, 1, 0, 2), 0.555556),
    ((1, 2, 0, 0, 1, 0, 0, 1, 2), 0.166667),
    ((1, 2, 0, 0, 0, 1, 0, 0, 0), 0.310811),
    ((1, 2, 2, 0, 0, 1, 0, 0, 0), 0.490196),
    ((1, 2, 2, 0, 0, 1, 1, 0, 0), 0.619048),
    ((1, 2, 2, 2, 0, 1, 1, 0, 0), 0.666667),
    ((1, 2, 2, 2, 0, 1, 1, 1, 0), 0.500000),
    ((1, 2, 2, 2, 2, 1, 1, 1, 0), 1.000000),
    ((1, 2, 2, 2, 2, 1, 1, 1, 1), 1.000000),
    ((1, 2, 2, 2, 0, 1, 1, 1, 2), 0.000000),
    ((1, 2, 2, 2, 0, 1, 1, 0, 1), 1.000000),
    ((1, 2, 2, 2, 2, 1, 1, 0, 1), 1.000000),
    ((1, 2, 2, 2, 0, 1, 1, 2, 1), 1.000000),
    ((1, 2, 2, 0, 2, 1, 1, 0, 0), 0.600000),
    ((1, 2, 2, 0, 2, 1, 1, 1, 0), 1.000000),
    ((1, 2, 2, 0, 2, 1, 1, 1, 2), 1.000000),
    ((1, 2, 2, 0, 2, 1, 1, 0, 1), 0.000000),
    ((1, 2, 2, 0, 2, 1, 1, 2, 1), -1.000000),
    ((1, 2, 2, 0, 0, 1, 1, 2, 0), 0.600000),
    ((1, 2, 2, 0, 0, 1, 1, 2, 1), 0.000000),
    ((1, 2, 2, 0, 0, 1, 1, 0, 2), 0.600000),
    ((1, 2, 2, 0, 0, 1, 1, 1, 2), 0.500000),
    ((1, 2, 2, 0, 0, 1, 0, 1, 0), 0.500000),
    ((1, 2, 2, 2, 0, 1, 0, 1, 0), 0.666667),
    ((1, 2, 2, 2, 0, 1, 0, 1, 1), 1.000000),
    ((1, 2, 2, 2, 2, 1, 0, 1, 1), 1.000000),
    ((1, 2, 2, 2, 0, 1, 2, 1, 1), 1.000000),
    ((1, 2, 2, 0, 2, 1, 0, 1, 0), 0.333333),
    ((1, 2, 2, 0, 2, 1, 0, 1, 1), 0.000000),
    ((1, 2, 2, 0, 2, 1, 2, 1, 1), -1.000000),
    ((1, 2, 2, 0, 0, 1, 2, 1, 0), 0.333333),
    ((1, 2, 2, 0, 0, 1, 2, 1, 1), 0.000000),
    ((1, 2, 2, 0, 0, 1, 0, 1, 2), 0.666667),
    ((1, 2, 2, 0, 0, 1, 0, 0, 1), 0.238095),
    ((1, 2, 2, 2, 0, 1, 0, 0, 1), 1.000000),
    ((1, 2, 2, 0, 2, 1, 0, 0, 1), -0.333333),
    ((1, 2, 2, 0, 0, 1, 2, 0, 1), 0.200000),
    ((1, 2, 2, 0, 0, 1, 0, 2, 1), 0.200000),
    ((1, 2, 0, 2, 0, 1, 0, 0, 0), 0.518519),
    ((1, 2, 0, 2, 0, 1, 1, 0, 0), 0.333333),
    ((1, 2, 0, 2, 2, 1, 1, 0, 0), 0.000000),
    ((1, 2, 0, 2, 2, 1, 1, 1, 0), 0.500000),
    ((1, 2, 0, 2, 2, 1, 1, 1, 2), 0.000000),
    ((1, 2, 0, 2, 2, 1, 1, 0, 1), 0.000000),
    ((1, 2, 0, 2, 2, 1, 1, 2, 1), -1.000000),
    ((1, 2, 0, 2, 0, 1, 1, 2, 0), 0.333333),
    ((1, 2, 0, 2, 0, 1, 1, 2, 1), 0.000000),
    ((1, 2, 0, 2, 0, 1, 1, 0, 2), 0.333333),
    ((1, 2, 0, 2, 0, 1, 1, 1, 2), 0.000000),
    ((1, 2, 0, 2, 0, 1, 0, 1, 0), 0.500000),
    ((1, 2, 0, 2, 2, 1, 0, 1, 0), 0.666667),
    ((1, 2, 0, 2, 2, 1, 0, 1, 1), 1.000000),
    ((1, 2, 0, 2, 2, 1, 2, 1, 1), 1.000000),
    ((1, 2, 0, 2, 0, 1, 2, 1, 0), 0.666667),
    ((1, 2, 0, 2, 0, 1, 2, 1, 1), 1.000000),
    ((1, 2, 0, 2, 0, 1, 0, 1, 2), 0.000000),
    ((1, 2, 0, 2, 0, 1, 0, 0, 1), 0.777778),
    ((1, 2, 0, 2, 2, 1, 0, 0, 1), 0.600000),
    ((1, 2, 0, 2, 0, 1, 2, 0, 1), 1.000000),
    ((1, 2, 0, 2, 0, 1, 0, 2, 1), 0.500000),
    ((1, 2, 0, 0, 2, 1, 0, 0, 0), 0.043478),
    ((1, 2, 0, 0, 2, 1, 1, 0, 0), 0.176471),
    ((1, 2, 0, 0, 2, 1, 1, 2, 0), -1.000000),
    ((1, 2, 0, 0, 2, 1, 1, 0, 2), 0.200000),
    ((1, 2, 0, 0, 2, 1, 1, 1, 2), 0.500000),
    ((1, 2, 0, 0, 2, 1, 0, 1, 0), 0.333333),
    ((1, 2, 0, 0, 2, 1, 2, 1, 0), 0.000000),
    ((1, 2, 0, 0, 2, 1, 2, 1, 1), 0.000000),
    ((1, 2, 0, 0, 2, 1, 0, 1, 2), 0.333333),
    ((1, 2, 0, 0, 2, 1, 0, 0, 1), -0.058824),
    ((1, 2, 0, 0, 2, 1, 2, 0, 1), -0.200000),
    ((1, 2, 0, 0, 2, 1, 0, 2, 1), -1.000000),
    ((1, 2, 0, 0, 0, 1, 2, 0, 0), 0.215686),
    ((1, 2, 0, 0, 0, 1, 2, 1, 0), 0.333333),
    ((1, 2, 0, 0, 0, 1, 2, 1, 2), 0.333333),
    ((1, 2, 0, 0, 0, 1, 2, 0, 1), 0.333333),
    ((1, 2, 0, 0, 0, 1, 2, 2, 1), 0.500000),
    ((1, 2, 0, 0, 0, 1, 0, 2, 0), 0.300000),
    ((1, 2, 0, 0, 0, 1, 1, 2, 0), 0.411765),
    ((1, 2, 0, 0, 0, 1, 1, 2, 2), 0.600000),
    ((1, 2, 0, 0, 0, 1, 0, 2, 1), 0.285714),
    ((1, 2, 0, 0, 0, 1, 0, 0, 2), 0.259259),
    ((1, 2, 0, 0, 0, 1, 1, 0, 2), 0.428571),
    ((1, 2, 0, 0, 0, 1, 0, 1, 2), 0.333333),
    ((1, 2, 0, 0, 0, 0, 1, 0, 0), 0.438753),
    ((1, 2, 2, 0, 0, 0, 1, 0, 0), 0.616438),
    ((1, 2, 2, 0, 0, 0, 1, 1, 0), 0.555556),
    ((1, 2, 2, 2, 0, 0, 1, 1, 0), 0.600000),
    ((1, 2, 2, 2, 0, 0, 1, 1, 1), 1.000000),
    ((1, 2, 2, 0, 2, 0, 1, 1, 0), 1.000000),
    ((1, 2, 2, 0, 2, 0, 1, 1, 1), 1.000000),
    ((1, 2, 2, 0, 0, 2, 1, 1, 0), 0.500000),
    ((1, 2, 2, 0, 0, 2, 1, 1, 1), 1.000000),
    ((1, 2, 2, 0, 0, 0, 1, 1, 2), 0.200000),
    ((1, 2, 2, 0, 0, 0, 1, 0, 1), 0.733333),
    ((1, 2, 2, 2, 0, 0, 1, 0, 1), 1.000000),
    ((1, 2, 2, 0, 2, 0, 1, 0, 1), 0.500000),
    ((1, 2, 2, 0, 0, 2, 1, 0, 1), 1.000000),
    ((1, 2, 2, 0, 0, 0, 1, 2, 1), 0.500000),
    ((1, 2, 0, 2, 0, 0, 1, 0, 0), 0.372549),
    ((1, 2, 0, 2, 0, 0, 1, 1, 0), 0.428571),
    ((1, 2, 0, 2, 2, 0, 1, 1, 0), 0.200000),
    ((1, 2, 0, 2, 2, 0, 1, 1, 1), 1.000000),
    ((1, 2, 0, 2, 0, 2, 1, 1, 0), 0.600000),
    ((1, 2, 0, 2, 0, 2, 1, 1, 1), 1.000000),
    ((1, 2, 0, 2, 0, 0, 1, 1, 2), 0.333333),
    ((1, 2, 0, 2, 0, 0, 1, 0, 1), 0.333333),
    ((1, 2, 0, 2, 2, 0, 1, 0, 1), -0.200000),
    ((1, 2, 0, 2, 0, 2, 1, 0, 1), 0.500000),
    ((1, 2, 0, 2, 0, 0, 1, 2, 1), 0.200000),
    ((1, 2, 0, 0, 2, 0, 1, 0, 0), 0.134328),
    ((1, 2, 0, 0, 2, 0, 1, 1, 0), 0.555556),
    ((1, 2, 0, 0, 2, 2, 1, 1, 0), 0.500000),
    ((1, 2, 0, 0, 2, 2, 1, 1, 1), 1.000000),
    ((1, 2, 0, 0, 2, 0, 1, 1, 2), 0.600000),
    ((1, 2, 0, 0, 2, 0, 1, 0, 1), 0.000000),
    ((1, 2, 0, 0, 2, 2, 1, 0, 1), 0.000000),
    ((1, 2, 0, 0, 2, 0, 1, 2, 1), -1.000000),
    ((1, 2, 0, 0, 0, 2, 1, 0, 0), 0.522388),
    ((1, 2, 0, 0, 0, 2, 1, 1, 0), 0.555556),
    ((1, 2, 0, 0, 0, 2, 1, 1, 2), 0.600000),
    ((1, 2, 0, 0, 0, 2, 1, 0, 1), 0.466667),
    ((1, 2, 0, 0, 0, 2, 1, 2, 1), 0.500000),
    ((1, 2, 0, 0, 0, 0, 1, 2, 0), 0.508197),
    ((1, 2, 0, 0, 0, 0, 1, 2, 1), 0.285714),
    ((1, 2, 0, 0, 0, 0, 1, 0, 2), 0.493671),
    ((1, 2, 0, 0, 0, 0, 1, 1, 2), 0.428571),
    ((1, 2, 0, 0, 0, 0, 0, 1, 0), 0.407407),
    ((1, 2, 2, 0, 0, 0, 0, 1, 0), 0.490196),
    ((1, 2, 2, 0, 0, 0, 0, 1, 1), 0.555556),
    ((1, 2, 2, 2, 0, 0, 0, 1, 1), 1.000000),
    ((1, 2, 2, 0, 2, 0, 0, 1, 1), 0.200000),
    ((1, 2, 2, 0, 0, 2, 0, 1, 1), 1.000000),
    ((1, 2, 2, 0, 0, 0, 2, 1, 1), 0.200000),
    ((1, 2, 0, 2, 0, 0, 0, 1, 0), 0.518519),
    ((1, 2, 0, 2, 0, 0, 0, 1, 1), 0.777778),
    ((1, 2, 0, 2, 2, 0, 0, 1, 1), 0.600000),
    ((1, 2, 0, 2, 0, 2, 0, 1, 1), 0.500000),
    ((1, 2, 0, 2, 0, 0, 2, 1, 1), 1.000000),
    ((1, 2, 0, 0, 2, 0, 0, 1, 0), 0.296296),
    ((1, 2, 0, 0, 2, 0, 0, 1, 1), 0.238095),
    ((1, 2, 0, 0, 2, 2, 0, 1, 1), 0.200000),
    ((1, 2, 0, 0, 2, 0, 2, 1, 1), 0.000000),
    ((1, 2, 0, 0, 0, 2, 0, 1, 0), 0.450980),
    ((1, 2, 0, 0, 0, 2, 0, 1, 1), 0.555556),
    ((1, 2, 0, 0, 0, 2, 2, 1, 1), 0.600000),
    ((1, 2, 0, 0, 0, 0, 2, 1, 0), 0.368421),
    ((1, 2, 0, 0, 0, 0, 2, 1, 1), 0.428571),
    ((1, 2, 0, 0, 0, 0, 0, 1, 2), 0.333333),
    ((1, 2, 0, 0, 0, 0, 0, 0, 1), 0.319728),
    ((1, 2, 2, 0, 0, 0, 0, 0, 1), 0.452055),
    ((1, 2, 0, 2, 0, 0, 0, 0, 1), 0.561644),
    ((1, 2, 0, 0, 2, 0, 0, 0, 1), -0.069767),
    ((1, 2, 0, 0, 0, 2, 0, 0, 1), 0.397260),
    ((1, 2, 0, 0, 0, 0, 2, 0, 1), 0.341772),
    ((1, 2, 0, 0, 0, 0, 0, 2, 1), 0.298246),
    ((1, 0, 2, 0, 0, 0, 0, 0, 0), 0.197740),
    ((1, 1, 2, 0, 0, 0, 0, 0, 0), -0.048611),
    ((1, 1, 2, 2, 0, 0, 0, 0, 0), 0.074074),
    ((1, 1, 2, 2, 1, 0, 0, 0, 0), 0.555556),
    ((1, 1, 2, 2, 1, 2, 0, 0, 0), 0.500000),
    ((1, 1, 2, 2, 1, 2, 1, 0, 0), 0.000000),
    ((1, 1, 2, 2, 1, 2, 1, 2, 0), 1.000000),
    ((1, 1, 2, 2, 1, 2, 1, 2, 1), 1.000000),
    ((1, 1, 2, 2, 1, 2, 1, 0, 2), -1.000000),
    ((1, 1, 2, 2, 1, 2, 0, 1, 0), 1.000000),
    ((1, 1, 2, 2, 1, 2, 0, 0, 1), 1.000000),
    ((1, 1, 2, 2, 1, 0, 2, 0, 0), 1.000000),
    ((1, 1, 2, 2, 1, 1, 2, 0, 0), 1.000000),
    ((1, 1, 2, 2, 1, 1, 2, 2, 0), 1.000000),
    ((1, 1, 2, 2, 1, 1, 2, 2, 1), 1.000000),
    ((1, 1, 2, 2, 1, 1, 2, 0, 2), 1.000000),
    ((1, 1, 2, 2, 1, 1, 2, 1, 2), 1.000000),
    ((1, 1, 2, 2, 1, 0, 2, 1, 0), 1.000000),
    ((1, 1, 2, 2, 1, 0, 2, 0, 1), 1.000000),
    ((1, 1, 2, 2, 1, 0, 0, 2, 0), 0.600000),
    ((1, 1, 2, 2, 1, 1, 0, 2, 0), 0.500000),
    ((1, 1, 2, 2, 1, 1, 0, 2, 2), 0.000000),
    ((1, 1, 2, 2, 1, 1, 1, 2, 2), 0.000000),
    ((1, 1, 2, 2, 1, 0, 1, 2, 0), 0.500000),
    ((1, 1, 2, 2, 1, 0, 1, 2, 2), 0.000000),
    ((1, 1, 2, 2, 1, 0, 0, 2, 1), 1.000000),
    ((1, 1, 2, 2, 1, 0, 0, 0, 2), 0.200000),
    ((1, 1, 2, 2, 1, 1, 0, 0, 2), 0.500000),
    ((1, 1, 2, 2, 1, 0, 1, 0, 2), -0.500000),
    ((1, 1, 2, 2, 1, 0, 0, 1, 2), 1.000000),
    ((1, 1, 2, 2, 0, 1, 0, 0, 0), 0.166667),
    ((1, 1, 2, 2, 2, 1, 0, 0, 0), -0.333333),
    ((1, 1, 2, 2, 2, 1, 1, 0, 0), 0.000000),
    ((1, 1, 2, 2, 2, 1, 1, 2, 0), 0.000000),
    ((1, 1, 2, 2, 2, 1, 1, 2, 1), 0.000000),
    ((1, 1, 2, 2, 2, 1, 1, 0, 2), 0.000000),
    ((1, 1, 2, 2, 2, 1, 1, 1, 2), 0.000000),
    ((1, 1, 2, 2, 2, 1, 0, 1, 0), -0.500000),
    ((1, 1, 2, 2, 2, 1, 2, 1, 0), -1.000000),
    ((1, 1, 2, 2, 2, 1, 0, 1, 2), 0.000000),
    ((1, 1, 2, 2, 2, 1, 0, 0, 1), -0.500000),
    ((1, 1, 2, 2, 2, 1, 2, 0, 1), -1.000000),
    ((1, 1, 2, 2, 2, 1, 0, 2, 1), 0.000000),
    ((1, 1, 2, 2, 0, 1, 2, 0, 0), 0.333333),
    ((1, 1, 2, 2, 0, 1, 2, 1, 0), 0.000000),
    ((1, 1, 2, 2, 0, 1, 2, 1, 2), 1.000000),
    ((1, 1, 2, 2, 0, 1, 2, 0, 1), 0.000000),
    ((1, 1, 2, 2, 0, 1, 2, 2, 1), 1.000000),
    ((1, 1, 2, 2, 0, 1, 0, 2, 0), 0.333333),
    ((1, 1, 2, 2, 0, 1, 1, 2, 0), 0.000000),
    ((1, 1, 2, 2, 0, 1, 1, 2, 2), 0.000000),
    ((1, 1, 2, 2, 0, 1, 0, 2, 1), 0.500000),
    ((1, 1, 2, 2, 0, 1, 0, 0, 2), 0.333333),
    ((1, 1, 2, 2, 0, 1, 1, 0, 2), 0.000000),
    ((1, 1, 2, 2, 0, 1, 0, 1, 2), 0.500000),
    ((1, 1, 2, 2, 0, 0, 1, 0, 0), -0.166667),
    ((1, 1, 2, 2, 2, 0, 1, 0, 0), -0.333333),
    ((1, 1, 2, 2, 2, 0, 1, 1, 0), -0.500000),
    ((1, 1, 2, 2, 2, 2, 1, 1, 0), -1.000000),
    ((1, 1, 2, 2, 2, 0, 1, 1, 2), 0.000000),
    ((1, 1, 2, 2, 2, 0, 1, 0, 1), -0.500000),
    ((1, 1, 2, 2, 2, 2, 1, 0, 1), -1.000000),
    ((1, 1, 2, 2, 2, 0, 1, 2, 1), 0.000000),
    ((1, 1, 2, 2, 0, 2, 1, 0, 0), -0.333333),
    ((1, 1, 2, 2, 0, 2, 1, 1, 0), -1.000000),
    ((1, 1, 2, 2, 0, 2, 1, 1, 2), -1.000000),
    ((1, 1, 2, 2, 0, 2, 1, 0, 1), 0.000000),
    ((1, 1, 2, 2, 0, 2, 1, 2, 1), 1.000000),
    ((1, 1, 2, 2, 0, 0, 1, 2, 0), 0.333333),
    ((1, 1, 2, 2, 0, 0, 1, 2, 1), 0.500000),
    ((1, 1, 2, 2, 0, 0, 1, 0, 2), -0.333333),
    ((1, 1, 2, 2, 0, 0, 1, 1, 2), -0.500000),
    ((1, 1, 2, 2, 0, 0, 0, 1, 0), -0.142857),
    ((1, 1, 2, 2, 2, 0, 0, 1, 0), -0.666667),
    ((1, 1, 2, 2, 2, 0, 0, 1, 1), -1.000000),
    ((1, 1, 2, 2, 2, 2, 0, 1, 1), -1.000000),
    ((1, 1, 2, 2, 2, 0, 2, 1, 1), -1.000000),
    ((1, 1, 2, 2, 0, 2, 0, 1, 0), -0.200000),
    ((1, 1, 2, 2, 0, 2, 0, 1, 1), 0.000000),
    ((1, 1, 2, 2, 0, 2, 2, 1, 1), 1.000000),
    ((1, 1, 2, 2, 1, 2, 2, 1, 1), 1.000000),
    ((1, 1, 2, 2, 0, 0, 2, 1, 0), 0.200000),
    ((1, 1, 2, 2, 0, 0, 2, 1, 1), 0.000000),
    ((1, 1, 2, 2, 0, 0, 0, 1, 2), 0.200000),
    ((1, 1, 2, 2, 0, 0, 0, 0, 1), 0.047619),
    ((1, 1, 2, 2, 2, 0, 0, 0, 1), -0.666667),
    ((1, 1, 2, 2, 0, 2, 0, 0, 1), 0.200000),
    ((1, 1, 2, 2, 0, 0, 2, 0, 1), 0.200000),
    ((1, 1, 2, 2, 0, 0, 0, 2, 1), 0.600000),
    ((1, 1, 2, 0, 2, 0, 0, 0, 0), -0.276596),
    ((1, 1, 2, 1, 2, 0, 0, 0, 0), -0.125000),
    ((1, 1, 2, 1, 2, 2, 0, 0, 0), -0.200000),
    ((1, 1, 2, 1, 2, 2, 1, 0, 0), 1.000000),
    ((1, 1, 2, 1, 2, 2, 0, 1, 0), -1.000000),
    ((1, 1, 2, 1, 2, 2, 2, 1, 0), -1.000000),
    ((1, 1, 2, 1, 2, 2, 0, 1, 2), -1.000000),
    ((1, 1, 2, 1, 2, 2, 0, 0, 1), 0.000000),
    ((1, 1, 2, 1, 2, 2, 2, 0, 1), -1.000000),
    ((1, 1, 2, 1, 2, 2, 0, 2, 1), 1.000000),
    ((1, 1, 2, 1, 2, 2, 1, 2, 1), 1.000000),
    ((1, 1, 2, 1, 2, 0, 2, 0, 0), -1.000000),
    ((1, 1, 2, 1, 2, 0, 0, 2, 0), 0.200000),
    ((1, 1, 2, 1, 2, 1, 0, 2, 0), 0.000000),
    ((1, 1, 2, 1, 2, 1, 2, 2, 0), -1.000000),
    ((1, 1, 2, 1, 2, 1, 0, 2, 2), 1.000000),
    ((1, 1, 2, 1, 2, 1, 1, 2, 2), 1.000000),
    ((1, 1, 2, 1, 2, 0, 1, 2, 0), 1.000000),
    ((1, 1, 2, 1, 2, 0, 0, 2, 1), 0.000000),
    ((1, 1, 2, 1, 2, 0, 2, 2, 1), -1.000000),
    ((1, 1, 2, 1, 2, 0, 0, 0, 2), -0.200000),
    ((1, 1, 2, 1, 2, 1, 0, 0, 2), 0.000000),
    ((1, 1, 2, 1, 2, 1, 2, 0, 2), -1.000000),
    ((1, 1, 2, 1, 2, 0, 1, 0, 2), 1.000000),
    ((1, 1, 2, 1, 2, 0, 0, 1, 2), -1.000000),
    ((1, 1, 2, 1, 2, 0, 2, 1, 2), -1.000000),
    ((1, 1, 2, 0, 2, 1, 0, 0, 0), -0.157895),
    ((1, 1, 2, 0, 2, 1, 2, 0, 0), -1.000000),
    ((1, 1, 2, 0, 2, 1, 0, 2, 0), 0.000000),
    ((1, 1, 2, 0, 2, 1, 1, 2, 0), 0.500000),
    ((1, 1, 2, 0, 2, 1, 1, 2, 2), 1.000000),
    ((1, 1, 2, 0, 2, 1, 0, 2, 1), -0.500000),
    ((1, 1, 2, 0, 2, 1, 2, 2, 1), -1.000000),
    ((1, 1, 2, 0, 2, 1, 0, 0, 2), 0.000000),
    ((1, 1, 2, 0, 2, 1, 1, 0, 2), 0.500000),
    ((1, 1, 2, 0, 2, 1, 0, 1, 2), -0.500000),
    ((1, 1, 2, 0, 2, 1, 2, 1, 2), -1.000000),
    ((1, 1, 2, 0, 2, 0, 1, 0, 0), 0.047619),
    ((1, 1, 2, 0, 2, 2, 1, 0, 0), -0.200000),
    ((1, 1, 2, 0, 2, 2, 1, 1, 0), -1.000000),
    ((1, 1, 2, 0, 2, 2, 1, 1, 2), -1.000000),
    ((1, 1, 2, 0, 2, 2, 1, 0, 1), 0.000000),
    ((1, 1, 2, 0, 2, 2, 1, 2, 1), 1.000000),
    ((1, 1, 2, 0, 2, 0, 1, 2, 0), 0.600000),
    ((1, 1, 2, 0, 2, 0, 1, 2, 1), 0.500000),
    ((1, 1, 2, 0, 2, 0, 1, 0, 2), 0.200000),
    ((1, 1, 2, 0, 2, 0, 1, 1, 2), -0.500000),
    ((1, 1, 2, 0, 2, 0, 0, 1, 0), -0.789474),
    ((1, 1, 2, 0, 2, 2, 0, 1, 0), -1.000000),
    ((1, 1, 2, 0, 2, 2, 0, 1, 1), -1.000000),
    ((1, 1, 2, 0, 2, 2, 2, 1, 1), -1.000000),
    ((1, 1, 2, 0, 2, 0, 2, 1, 0), -1.000000),
    ((1, 1, 2, 0, 2, 0, 0, 1, 2), -0.666667),
    ((1, 1, 2, 0, 2, 0, 0, 0, 1), -0.368421),
    ((1, 1, 2, 0, 2, 2, 0, 0, 1), -0.333333),
    ((1, 1, 2, 0, 2, 0, 2, 0, 1), -1.000000),
    ((1, 1, 2, 0, 2, 0, 0, 2, 1), 0.000000),
    ((1, 1, 2, 0, 0, 2, 0, 0, 0), -0.023256),
    ((1, 1, 2, 1, 0, 2, 0, 0, 0), -0.058824),
    ((1, 1, 2, 1, 0, 2, 2, 0, 0), -0.333333),
    ((1, 1, 2, 1, 1, 2, 2, 0, 0), 0.000000),
    ((1, 1, 2, 1, 1, 2, 2, 2, 0), 1.000000),
    ((1, 1, 2, 1, 1, 2, 2, 2, 1), 1.000000),
    ((1, 1, 2, 1, 1, 2, 2, 0, 2), -1.000000),
    ((1, 1, 2, 1, 0, 2, 2, 1, 0), -1.000000),
    ((1, 1, 2, 1, 0, 2, 2, 1, 2), -1.000000),
    ((1, 1, 2, 1, 0, 2, 2, 0, 1), 0.000000),
    ((1, 1, 2, 1, 0, 2, 2, 2, 1), 1.000000),
    ((1, 1, 2, 1, 0, 2, 0, 2, 0), 0.600000),
    ((1, 1, 2, 1, 1, 2, 0, 2, 0), 0.000000),
    ((1, 1, 2, 1, 1, 2, 0, 2, 2), -1.000000),
    ((1, 1, 2, 1, 0, 2, 1, 2, 0), 1.000000),
    ((1, 1, 2, 1, 0, 2, 0, 2, 1), 1.000000),
    ((1, 1, 2, 1, 0, 2, 0, 0, 2), -1.000000),
    ((1, 1, 2, 0, 1, 2, 0, 0, 0), 0.285714),
    ((1, 1, 2, 0, 1, 2, 2, 0, 0), 0.500000),
    ((1, 1, 2, 0, 1, 2, 2, 1, 0), 1.000000),
    ((1, 1, 2, 0, 1, 2, 2, 0, 1), 1.000000),
    ((1, 1, 2, 0, 1, 2, 0, 2, 0), 0.200000),
    ((1, 1, 2, 0, 1, 2, 1, 2, 0), 0.000000),
    ((1, 1, 2, 0, 1, 2, 1, 2, 2), -1.000000),
    ((1, 1, 2, 0, 1, 2, 0, 2, 1), 1.000000),
    ((1, 1, 2, 0, 1, 2, 0, 0, 2), -1.000000),
    ((1, 1, 2, 0, 0, 2, 1, 0, 0), -0.058824),
    ((1, 1, 2, 0, 0, 2, 1, 2, 0), 0.600000),
    ((1, 1, 2, 0, 0, 2, 1, 2, 1), 1.000000),
    ((1, 1, 2, 0, 0, 2, 1, 0, 2), -1.000000),
    ((1, 1, 2, 0, 0, 2, 0, 1, 0), -0.529412),
    ((1, 1, 2, 0, 0, 2, 2, 1, 0), -0.200000),
    ((1, 1, 2, 0, 0, 2, 2, 1, 1), 0.000000),
    ((1, 1, 2, 0, 0, 2, 0, 1, 2), -1.000000),
    ((1, 1, 2, 0, 0, 2, 0, 0, 1), 0.238095),
    ((1, 1, 2, 0, 0, 2, 2, 0, 1), 0.200000),
    ((1, 1, 2, 0, 0, 2, 0, 2, 1), 1.000000),
    ((1, 1, 2, 0, 0, 0, 2, 0, 0), -0.090909),
    ((1, 1, 2, 1, 0, 0, 2, 0, 0), -0.578947),
    ((1, 1, 2, 1, 0, 0, 2, 2, 0), -0.333333),
    ((1, 1, 2, 1, 1, 0, 2, 2, 0), 0.000000),
    ((1, 1, 2, 1, 1, 0, 2, 2, 2), -1.000000),
    ((1, 1, 2, 1, 0, 1, 2, 2, 0), -1.000000),
    ((1, 1, 2, 1, 0, 1, 2, 2, 2), -1.000000),
    ((1, 1, 2, 1, 0, 0, 2, 2, 1), 0.000000),
    ((1, 1, 2, 1, 0, 0, 2, 0, 2), -1.000000),
    ((1, 1, 2, 1, 1, 0, 2, 0, 2), -1.000000),
    ((1, 1, 2, 1, 0, 1, 2, 0, 2), -1.000000),
    ((1, 1, 2, 1, 0, 0, 2, 1, 2), -1.000000),
    ((1, 1, 2, 0, 1, 0, 2, 0, 0), 0.333333),
    ((1, 1, 2, 0, 1, 0, 2, 2, 0), 0.200000),
    ((1, 1, 2, 0, 1, 1, 2, 2, 0), 0.000000),
    ((1, 1, 2, 0, 1, 1, 2, 2, 2), -1.000000),
    ((1, 1, 2, 0, 1, 0, 2, 2, 1), 1.000000),
    ((1, 1, 2, 0, 1, 0, 2, 0, 2), -0.200000),
    ((1, 1, 2, 0, 1, 1, 2, 0, 2), 0.000000),
    ((1, 1, 2, 0, 1, 0, 2, 1, 2), 1.000000),
    ((1, 1, 2, 0, 0, 1, 2, 0, 0), -0.157895),
    ((1, 1, 2, 0, 0, 1, 2, 2, 0), -0.333333),
    ((1, 1, 2, 0, 0, 1, 2, 2, 1), 0.000000),
    ((1, 1, 2, 0, 0, 1, 2, 0, 2), -0.333333),
    ((1, 1, 2, 0, 0, 1, 2, 1, 2), 0.000000),
    ((1, 1, 2, 0, 0, 0, 2, 1, 0), -0.125000),
    ((1, 1, 2, 0, 0, 0, 2, 1, 2), -0.200000),
    ((1, 1, 2, 0, 0, 0, 2, 0, 1), 0.125000),
    ((1, 1, 2, 0, 0, 0, 2, 2, 1), 0.200000),
    ((1, 1, 2, 0, 0, 0, 0, 2, 0), 0.185185),
    ((1, 1, 2, 1, 0, 0, 0, 2, 0), 0.047619),
    ((1, 1, 2, 1, 0, 0, 0, 2, 2), -0.200000),
    ((1, 1, 2, 1, 1, 0, 0, 2, 2), -1.000000),
    ((1, 1, 2, 1, 0, 1, 0, 2, 2), 0.000000),
    ((1, 1, 2, 1, 0, 0, 1, 2, 2), 1.000000),
    ((1, 1, 2, 0, 1, 0, 0, 2, 0), 0.047619),
    ((1, 1, 2, 0, 1, 0, 0, 2, 2), -0.666667),
    ((1, 1, 2, 0, 1, 1, 0, 2, 2), -0.500000),
    ((1, 1, 2, 0, 1, 0, 1, 2, 2), -0.500000),
    ((1, 1, 2, 0, 0, 1, 0, 2, 0), 0.000000),
    ((1, 1, 2, 0, 0, 1, 0, 2, 2), 0.000000),
    ((1, 1, 2, 0, 0, 1, 1, 2, 2), 0.500000),
    ((1, 1, 2, 0, 0, 0, 1, 2, 0), 0.428571),
    ((1, 1, 2, 0, 0, 0, 1, 2, 2), 0.200000),
    ((1, 1, 2, 0, 0, 0, 0, 2, 1), 0.428571),
    ((1, 1, 2, 0, 0, 0, 0, 0, 2), -0.217391),
    ((1, 1, 2, 1, 0, 0, 0, 0, 2), -0.529412),
    ((1, 1, 2, 0, 1, 0, 0, 0, 2), -0.294118),
    ((1, 1, 2, 0, 0, 1, 0, 0, 2), 0.000000),
    ((1, 1, 2, 0, 0, 0, 1, 0, 2), -0.058824),
    ((1, 1, 2, 0, 0, 0, 0, 1, 2), -0.294118),
    ((1, 0, 2, 1, 0, 0, 0, 0, 0), 0.103926),
    ((1, 0, 2, 1, 2, 0, 0, 0, 0), -0.107692),
    ((1, 0, 2, 1, 2, 1, 0, 0, 0), -0.125000),
    ((1, 0, 2, 1, 2, 1, 2, 0, 0), -1.000000),
    ((1, 0, 2, 1, 2, 1, 0, 2, 0), -0.200000),
    ((1, 0, 2, 1, 2, 1, 1, 2, 0), 1.000000),
    ((1, 0, 2, 1, 2, 1, 0, 2, 1), -1.000000),
    ((1, 0, 2, 1, 2, 1, 2, 2, 1), -1.000000),
    ((1, 0, 2, 1, 2, 1, 0, 0, 2), 0.200000),
    ((1, 0, 2, 1, 2, 1, 1, 0, 2), 1.000000),
    ((1, 0, 2, 1, 2, 1, 0, 1, 2), 0.000000),
    ((1, 0, 2, 1, 2, 1, 2, 1, 2), -1.000000),
    ((1, 0, 2, 1, 2, 0, 1, 0, 0), 1.000000),
    ((1, 0, 2, 1, 2, 0, 0, 1, 0), -0.125000),
    ((1, 0, 2, 1, 2, 2, 0, 1, 0), -0.200000),
    ((1, 0, 2, 1, 2, 2, 1, 1, 0), 1.000000),
    ((1, 0, 2, 1, 2, 2, 0, 1, 1), 0.000000),
    ((1, 0, 2, 1, 2, 2, 2, 1, 1), -1.000000),
    ((1, 0, 2, 1, 2, 0, 2, 1, 0), -1.000000),
    ((1, 0, 2, 1, 2, 0, 0, 1, 2), -0.200000),
    ((1, 0, 2, 1, 2, 0, 1, 1, 2), 1.000000),
    ((1, 0, 2, 1, 2, 0, 0, 0, 1), -0.125000),
    ((1, 0, 2, 1, 2, 2, 0, 0, 1), 0.200000),
    ((1, 0, 2, 1, 2, 2, 1, 0, 1), 1.000000),
    ((1, 0, 2, 1, 2, 0, 2, 0, 1), -1.000000),
    ((1, 0, 2, 1, 2, 0, 0, 2, 1), -0.200000),
    ((1, 0, 2, 1, 2, 0, 1, 2, 1), 1.000000),
    ((1, 0, 2, 1, 0, 2, 0, 0, 0), 0.194030),
    ((1, 0, 2, 1, 1, 2, 0, 0, 0), 0.285714),
    ((1, 0, 2, 1, 1, 2, 2, 0, 0), 0.200000),
    ((1, 0, 2, 1, 1, 2, 2, 1, 0), 0.000000),
    ((1, 0, 2, 1, 1, 2, 2, 1, 2), -1.000000),
    ((1, 0, 2, 1, 1, 2, 2, 0, 1), 1.000000),
    ((1, 0, 2, 1, 1, 2, 0, 2, 0), 0.500000),
    ((1, 0, 2, 1, 1, 2, 1, 2, 0), 1.000000),
    ((1, 0, 2, 1, 1, 2, 0, 2, 1), 1.000000),
    ((1, 0, 2, 1, 1, 2, 0, 0, 2), -1.000000),
    ((1, 0, 2, 1, 0, 2, 1, 0, 0), 1.000000),
    ((1, 0, 2, 1, 0, 2, 0, 1, 0), -0.058824),
    ((1, 0, 2, 1, 0, 2, 2, 1, 0), -0.333333),
    ((1, 0, 2, 1, 0, 2, 2, 1, 1), 0.000000),
    ((1, 0, 2, 1, 0, 2, 0, 1, 2), -1.000000),
    ((1, 0, 2, 1, 0, 2, 0, 0, 1), 0.555556),
    ((1, 0, 2, 1, 0, 2, 2, 0, 1), 0.200000),
    ((1, 0, 2, 1, 0, 2, 0, 2, 1), 1.000000),
    ((1, 0, 2, 1, 0, 2, 1, 2, 1), 1.000000),
    ((1, 0, 2, 1, 0, 0, 2, 0, 0), -0.090909),
    ((1, 0, 2, 1, 1, 0, 2, 0, 0), 0.333333),
    ((1, 0, 2, 1, 1, 0, 2, 2, 0), 0.500000),
    ((1, 0, 2, 1, 1, 1, 2, 2, 0), 1.000000),
    ((1, 0, 2, 1, 1, 0, 2, 2, 1), 1.000000),
    ((1, 0, 2, 1, 1, 0, 2, 0, 2), -0.200000),
    ((1, 0, 2, 1, 1, 1, 2, 0, 2), 1.000000),
    ((1, 0, 2, 1, 1, 0, 2, 1, 2), 0.000000),
    ((1, 0, 2, 1, 0, 1, 2, 0, 0), -0.125000),
    ((1, 0, 2, 1, 0, 1, 2, 2, 0), -0.200000),
    ((1, 0, 2, 1, 0, 1, 2, 2, 1), 0.000000),
    ((1, 0, 2, 1, 0, 1, 2, 0, 2), -0.200000),
    ((1, 0, 2, 1, 0, 1, 2, 1, 2), 0.000000),
    ((1, 0, 2, 1, 0, 0, 2, 1, 0), -0.157895),
    ((1, 0, 2, 1, 0, 0, 2, 1, 2), -0.333333),
    ((1, 0, 2, 1, 0, 0, 2, 0, 1), 0.125000),
    ((1, 0, 2, 1, 0, 0, 2, 2, 1), 0.200000),
    ((1, 0, 2, 1, 0, 0, 0, 2, 0), 0.232877),
    ((1, 0, 2, 1, 1, 0, 0, 2, 0), 0.466667),
    ((1, 0, 2, 1, 1, 0, 0, 2, 2), 0.000000),
    ((1, 0, 2, 1, 1, 1, 0, 2, 2), 1.000000),
    ((1, 0, 2, 1, 1, 0, 1, 2, 2), 1.000000),
    ((1, 0, 2, 1, 0, 1, 0, 2, 0), 0.111111),
    ((1, 0, 2, 1, 0, 1, 0, 2, 2), 0.500000),
    ((1, 0, 2, 1, 0, 1, 1, 2, 2), 1.000000),
    ((1, 0, 2, 1, 0, 0, 1, 2, 0), 1.000000),
    ((1, 0, 2, 1, 0, 0, 0, 2, 1), 0.333333),
    ((1, 0, 2, 1, 0, 0, 0, 0, 2), -0.044776),
    ((1, 0, 2, 1, 1, 0, 0, 0, 2), 0.000000),
    ((1, 0, 2, 1, 0, 1, 0, 0, 2), 0.333333),
    ((1, 0, 2, 1, 0, 0, 1, 0, 2), 1.000000),
    ((1, 0, 2, 1, 0, 0, 0, 1, 2), -0.058824),
    ((1, 0, 2, 0, 1, 0, 0, 0, 0), 0.401361),
    ((1, 0, 2, 2, 1, 0, 0, 0, 0), 0.493671),
    ((1, 0, 2, 2, 1, 1, 0, 0, 0), 0.619048),
    ((1, 0, 2, 2, 1, 1, 2, 0, 0), 1.000000),
    ((1, 0, 2, 2, 1, 1, 2, 1, 0), 1.000000),
    ((1, 0, 2, 2, 1, 1, 2, 1, 2), 1.000000),
    ((1, 0, 2, 2, 1, 1, 2, 0, 1), 1.000000),
    ((1, 0, 2, 2, 1, 1, 0, 2, 0), 0.600000),
    ((1, 0, 2, 2, 1, 1, 1, 2, 0), 0.500000),
    ((1, 0, 2, 2, 1, 1, 1, 2, 2), 0.000000),
    ((1, 0, 2, 2, 1, 1, 0, 2, 1), 1.000000),
    ((1, 0, 2, 2, 1, 1, 0, 0, 2), 0.333333),
    ((1, 0, 2, 2, 1, 1, 1, 0, 2), 0.000000),
    ((1, 0, 2, 2, 1, 1, 0, 1, 2), 0.500000),
    ((1, 0, 2, 2, 1, 0, 1, 0, 0), 0.238095),
    ((1, 0, 2, 2, 1, 2, 1, 0, 0), 0.200000),
    ((1, 0, 2, 2, 1, 2, 1, 1, 0), 0.000000),
    ((1, 0, 2, 2, 1, 2, 1, 1, 2), -1.000000),
    ((1, 0, 2, 2, 1, 2, 1, 0, 1), 1.000000),
    ((1, 0, 2, 2, 1, 0, 1, 2, 0), 0.600000),
    ((1, 0, 2, 2, 1, 0, 1, 2, 1), 1.000000),
    ((1, 0, 2, 2, 1, 0, 1, 0, 2), -0.333333),
    ((1, 0, 2, 2, 1, 0, 1, 1, 2), -0.500000),
    ((1, 0, 2, 2, 1, 0, 0, 1, 0), 0.555556),
    ((1, 0, 2, 2, 1, 2, 0, 1, 0), 0.500000),
    ((1, 0, 2, 2, 1, 2, 0, 1, 1), 1.000000),
    ((1, 0, 2, 2, 1, 0, 2, 1, 0), 1.000000),
    ((1, 0, 2, 2, 1, 0, 2, 1, 1), 1.000000),
    ((1, 0, 2, 2, 1, 0, 0, 1, 2), 0.200000),
    ((1, 0, 2, 2, 1, 0, 0, 0, 1), 1.000000),
    ((1, 0, 2, 0, 1, 2, 0, 0, 0), 0.298246),
    ((1, 0, 2, 0, 1, 2, 1, 0, 0), 0.285714),
    ((1, 0, 2, 0, 1, 2, 1, 2, 0), 0.500000),
    ((1, 0, 2, 0, 1, 2, 1, 2, 1), 1.000000),
    ((1, 0, 2, 0, 1, 2, 1, 0, 2), -1.000000),
    ((1, 0, 2, 0, 1, 2, 0, 1, 0), 0.285714),
    ((1, 0, 2, 0, 1, 2, 2, 1, 0), 0.500000),
    ((1, 0, 2, 0, 1, 2, 2, 1, 1), 1.000000),
    ((1, 0, 2, 0, 1, 2, 0, 1, 2), -1.000000),
    ((1, 0, 2, 0, 1, 2, 0, 0, 1), 1.000000),
    ((1, 0, 2, 0, 1, 0, 2, 0, 0), 0.561644),
    ((1, 0, 2, 0, 1, 1, 2, 0, 0), 0.777778),
    ((1, 0, 2, 0, 1, 1, 2, 2, 0), 0.500000),
    ((1, 0, 2, 0, 1, 1, 2, 2, 1), 1.000000),
    ((1, 0, 2, 0, 1, 1, 2, 0, 2), 0.600000),
    ((1, 0, 2, 0, 1, 1, 2, 1, 2), 1.000000),
    ((1, 0, 2, 0, 1, 0, 2, 1, 0), 0.777778),
    ((1, 0, 2, 0, 1, 0, 2, 1, 2), 0.600000),
    ((1, 0, 2, 0, 1, 0, 2, 0, 1), 1.000000),
    ((1, 0, 2, 0, 1, 0, 0, 2, 0), 0.397260),
    ((1, 0, 2, 0, 1, 1, 0, 2, 0), 0.555556),
    ((1, 0, 2, 0, 1, 1, 0, 2, 2), 0.200000),
    ((1, 0, 2, 0, 1, 1, 1, 2, 2), 0.500000),
    ((1, 0, 2, 0, 1, 0, 1, 2, 0), 0.555556),
    ((1, 0, 2, 0, 1, 0, 1, 2, 2), 0.200000),
    ((1, 0, 2, 0, 1, 0, 0, 2, 1), 1.000000),
    ((1, 0, 2, 0, 1, 0, 0, 0, 2), 0.069767),
    ((1, 0, 2, 0, 1, 1, 0, 0, 2), 0.428571),
    ((1, 0, 2, 0, 1, 0, 1, 0, 2), -0.058824),
    ((1, 0, 2, 0, 1, 0, 0, 1, 2), 0.176471),
    ((1, 0, 2, 0, 0, 1, 0, 0, 0), 0.263158),
    ((1, 0, 2, 2, 0, 1, 0, 0, 0), 0.368421),
    ((1, 0, 2, 2, 0, 1, 1, 0, 0), 0.333333),
    ((1, 0, 2, 2, 2, 1, 1, 0, 0), 0.333333),
    ((1, 0, 2, 2, 2, 1, 1, 1, 0), 0.500000),
    ((1, 0, 2, 2, 2, 1, 1, 1, 2), 0.000000),
    ((1, 0, 2, 2, 2, 1, 1, 0, 1), 0.500000),
    ((1, 0, 2, 2, 2, 1, 1, 2, 1), 0.000000),
    ((1, 0, 2, 2, 0, 1, 1, 2, 0), 0.333333),
    ((1, 0, 2, 2, 0, 1, 1, 2, 1), 0.500000),
    ((1, 0, 2, 2, 0, 1, 1, 0, 2), 0.000000),
    ((1, 0, 2, 2, 0, 1, 1, 1, 2), 0.000000),
    ((1, 0, 2, 2, 0, 1, 0, 1, 0), 0.333333),
    ((1, 0, 2, 2, 2, 1, 0, 1, 0), 0.000000),
    ((1, 0, 2, 2, 2, 1, 0, 1, 1), 0.000000),
    ((1, 0, 2, 2, 2, 1, 2, 1, 1), -1.000000),
    ((1, 0, 2, 2, 0, 1, 2, 1, 0), 0.333333),
    ((1, 0, 2, 2, 0, 1, 2, 1, 1), 0.000000),
    ((1, 0, 2, 2, 0, 1, 0, 1, 2), 0.333333),
    ((1, 0, 2, 2, 0, 1, 0, 0, 1), 0.428571),
    ((1, 0, 2, 2, 2, 1, 0, 0, 1), 0.000000),
    ((1, 0, 2, 2, 0, 1, 2, 0, 1), 0.200000),
    ((1, 0, 2, 2, 0, 1, 0, 2, 1), 0.600000),
    ((1, 0, 2, 0, 2, 1, 0, 0, 0), -0.021277),
    ((1, 0, 2, 0, 2, 1, 1, 0, 0), 0.428571),
    ((1, 0, 2, 0, 2, 1, 1, 2, 0), 0.200000),
    ((1, 0, 2, 0, 2, 1, 1, 2, 1), -0.500000),
    ((1, 0, 2, 0, 2, 1, 1, 0, 2), 0.600000),
    ((1, 0, 2, 0, 2, 1, 1, 1, 2), 0.500000),
    ((1, 0, 2, 0, 2, 1, 0, 1, 0), 0.052632),
    ((1, 0, 2, 0, 2, 1, 2, 1, 0), -1.000000),
    ((1, 0, 2, 0, 2, 1, 0, 1, 2), 0.000000),
    ((1, 0, 2, 0, 2, 1, 0, 0, 1), -0.368421),
    ((1, 0, 2, 0, 2, 1, 2, 0, 1), -1.000000),
    ((1, 0, 2, 0, 2, 1, 0, 2, 1), -0.666667),
    ((1, 0, 2, 0, 0, 1, 2, 0, 0), 0.181818),
    ((1, 0, 2, 0, 0, 1, 2, 1, 0), 0.263158),
    ((1, 0, 2, 0, 0, 1, 2, 1, 2), 0.333333),
    ((1, 0, 2, 0, 0, 1, 2, 0, 1), 0.125000),
    ((1, 0, 2, 0, 0, 1, 2, 2, 1), 0.200000),
    ((1, 0, 2, 0, 0, 1, 0, 2, 0), 0.215686),
    ((1, 0, 2, 0, 0, 1, 1, 2, 0), 0.428571),
    ((1, 0, 2, 0, 0, 1, 1, 2, 2), 0.600000),
    ((1, 0, 2, 0, 0, 1, 0, 2, 1), 0.047619),
    ((1, 0, 2, 0, 0, 1, 0, 0, 2), 0.296296),
    ((1, 0, 2, 0, 0, 1, 1, 0, 2), 0.428571),
    ((1, 0, 2, 0, 0, 1, 0, 1, 2), 0.333333),
    ((1, 0, 2, 0, 0, 0, 1, 0, 0), 0.315011),
    ((1, 0, 2, 2, 0, 0, 1, 0, 0), 0.185185),
    ((1, 0, 2, 2, 0, 0, 1, 1, 0), 0.047619),
    ((1, 0, 2, 2, 2, 0, 1, 1, 0), 0.200000),
    ((1, 0, 2, 2, 2, 0, 1, 1, 1), 1.000000),
    ((1, 0, 2, 2, 0, 2, 1, 1, 0), -0.200000),
    ((1, 0, 2, 2, 0, 2, 1, 1, 1), 1.000000),
    ((1, 0, 2, 2, 0, 0, 1, 1, 2), -0.333333),
    ((1, 0, 2, 2, 0, 0, 1, 0, 1), 0.555556),
    ((1, 0, 2, 2, 2, 0, 1, 0, 1), 0.200000),
    ((1, 0, 2, 2, 0, 2, 1, 0, 1), 0.500000),
    ((1, 0, 2, 2, 0, 0, 1, 2, 1), 0.600000),
    ((1, 0, 2, 0, 2, 0, 1, 0, 0), 0.291139),
    ((1, 0, 2, 0, 2, 0, 1, 1, 0), 0.333333),
    ((1, 0, 2, 0, 2, 2, 1, 1, 0), 0.000000),
    ((1, 0, 2, 0, 2, 2, 1, 1, 1), 1.000000),
    ((1, 0, 2, 0, 2, 0, 1, 1, 2), 0.200000),
    ((1, 0, 2, 0, 2, 0, 1, 0, 1), 0.333333),
    ((1, 0, 2, 0, 2, 2, 1, 0, 1), 0.500000),
    ((1, 0, 2, 0, 2, 0, 1, 2, 1), 0.200000),
    ((1, 0, 2, 0, 0, 2, 1, 0, 0), 0.245902),
    ((1, 0, 2, 0, 0, 2, 1, 1, 0), 0.000000),
    ((1, 0, 2, 0, 0, 2, 1, 1, 2), -1.000000),
    ((1, 0, 2, 0, 0, 2, 1, 0, 1), 0.733333),
    ((1, 0, 2, 0, 0, 2, 1, 2, 1), 1.000000),
    ((1, 0, 2, 0, 0, 0, 1, 2, 0), 0.493671),
    ((1, 0, 2, 0, 0, 0, 1, 2, 1), 0.555556),
    ((1, 0, 2, 0, 0, 0, 1, 0, 2), 0.095890),
    ((1, 0, 2, 0, 0, 0, 1, 1, 2), -0.058824),
    ((1, 0, 2, 0, 0, 0, 0, 1, 0), 0.144928),
    ((1, 0, 2, 2, 0, 0, 0, 1, 0), 0.215686),
    ((1, 0, 2, 2, 0, 0, 0, 1, 1), 0.333333),
    ((1, 0, 2, 2, 2, 0, 0, 1, 1), -0.200000),
    ((1, 0, 2, 2, 0, 2, 0, 1, 1), 0.500000),
    ((1, 0, 2, 2, 0, 0, 2, 1, 1), 0.200000),
    ((1, 0, 2, 0, 2, 0, 0, 1, 0), -0.136364),
    ((1, 0, 2, 0, 2, 0, 0, 1, 1), -0.125000),
    ((1, 0, 2, 0, 2, 2, 0, 1, 1), -0.200000),
    ((1, 0, 2, 0, 2, 0, 2, 1, 1), -1.000000),
    ((1, 0, 2, 0, 0, 2, 0, 1, 0), 0.000000),
    ((1, 0, 2, 0, 0, 2, 0, 1, 1), 0.333333),
    ((1, 0, 2, 0, 0, 2, 2, 1, 1), 0.200000),
    ((1, 0, 2, 0, 0, 0, 2, 1, 0), 0.181818),
    ((1, 0, 2, 0, 0, 0, 2, 1, 1), 0.125000),
    ((1, 0, 2, 0, 0, 0, 0, 1, 2), 0.043478),
    ((1, 0, 2, 0, 0, 0, 0, 0, 1), 0.256018),
    ((1, 0, 2, 2, 0, 0, 0, 0, 1), 0.341772),
    ((1, 0, 2, 0, 2, 0, 0, 0, 1), -0.136364),
    ((1, 0, 2, 0, 0, 2, 0, 0, 1), 0.452055),
    ((1, 0, 2, 0, 0, 0, 2, 0, 1), 0.138462),
    ((1, 0, 2, 0, 0, 0, 0, 2, 1), 0.341772),
    ((1, 0, 0, 2, 0, 0, 0, 0, 0), 0.352236),
    ((1, 1, 0, 2, 0, 0, 0, 0, 0), 0.238901),
    ((1, 1, 0, 2, 2, 0, 0, 0, 0), -0.068493),
    ((1, 1, 1, 2, 2, 0, 0, 0, 0), 1.000000),
    ((1, 1, 0, 2, 2, 1, 0, 0, 0), 0.238095),
    ((1, 1, 0, 2, 2, 1, 2, 0, 0), 0.200000),
    ((1, 1, 1, 2, 2, 1, 2, 0, 0), 1.000000),
    ((1, 1, 0, 2, 2, 1, 2, 1, 0), 0.000000),
    ((1, 1, 0, 2, 2, 1, 2, 1, 2), 1.000000),
    ((1, 1, 1, 2, 2, 1, 2, 1, 2), 1.000000),
    ((1, 1, 0, 2, 2, 1, 2, 0, 1), 0.000000),
    ((1, 1, 0, 2, 2, 1, 2, 2, 1), 1.000000),
    ((1, 1, 1, 2, 2, 1, 2, 2, 1), 1.000000),
    ((1, 1, 0, 2, 2, 1, 0, 2, 0), 0.600000),
    ((1, 1, 1, 2, 2, 1, 0, 2, 0), 1.000000),
    ((1, 1, 0, 2, 2, 1, 1, 2, 0), 0.500000),
    ((1, 1, 0, 2, 2, 1, 1, 2, 2), 1.000000),
    ((1, 1, 1, 2, 2, 1, 1, 2, 2), 1.000000),
    ((1, 1, 0, 2, 2, 1, 0, 2, 1), 0.500000),
    ((1, 1, 0, 2, 2, 1, 0, 0, 2), 0.600000),
    ((1, 1, 1, 2, 2, 1, 0, 0, 2), 1.000000),
    ((1, 1, 0, 2, 2, 1, 1, 0, 2), 0.500000),
    ((1, 1, 0, 2, 2, 1, 0, 1, 2), 0.500000),
    ((1, 1, 0, 2, 2, 0, 1, 0, 0), -0.058824),
    ((1, 1, 0, 2, 2, 2, 1, 0, 0), -1.000000),
    ((1, 1, 0, 2, 2, 0, 1, 2, 0), 0.200000),
    ((1, 1, 1, 2, 2, 0, 1, 2, 0), 1.000000),
    ((1, 1, 0, 2, 2, 0, 1, 2, 1), -0.500000),
    ((1, 1, 0, 2, 2, 2, 1, 2, 1), -1.000000),
    ((1, 1, 0, 2, 2, 0, 1, 0, 2), 0.200000),
    ((1, 1, 1, 2, 2, 0, 1, 0, 2), 1.000000),
    ((1, 1, 0, 2, 2, 0, 1, 1, 2), -0.500000),
    ((1, 1, 0, 2, 2, 2, 1, 1, 2), -1.000000),
    ((1, 1, 0, 2, 2, 0, 0, 1, 0), -0.294118),
    ((1, 1, 0, 2, 2, 2, 0, 1, 0), -1.000000),
    ((1, 1, 0, 2, 2, 0, 2, 1, 0), -0.200000),
    ((1, 1, 1, 2, 2, 0, 2, 1, 0), 1.000000),
    ((1, 1, 0, 2, 2, 0, 2, 1, 1), -1.000000),
    ((1, 1, 0, 2, 2, 2, 2, 1, 1), -1.000000),
    ((1, 1, 0, 2, 2, 0, 0, 1, 2), 0.200000),
    ((1, 1, 1, 2, 2, 0, 0, 1, 2), 1.000000),
    ((1, 1, 0, 2, 2, 0, 0, 0, 1), -0.294118),
    ((1, 1, 0, 2, 2, 2, 0, 0, 1), -1.000000),
    ((1, 1, 0, 2, 2, 0, 2, 0, 1), -0.200000),
    ((1, 1, 1, 2, 2, 0, 2, 0, 1), 1.000000),
    ((1, 1, 0, 2, 2, 0, 0, 2, 1), 0.200000),
    ((1, 1, 1, 2, 2, 0, 0, 2, 1), 1.000000),
    ((1, 1, 0, 2, 0, 2, 0, 0, 0), 0.245902),
    ((1, 1, 1, 2, 0, 2, 0, 0, 0), 1.000000),
    ((1, 1, 0, 2, 1, 2, 0, 0, 0), 0.733333),
    ((1, 1, 0, 2, 1, 2, 2, 0, 0), 1.000000),
    ((1, 1, 1, 2, 1, 2, 2, 0, 0), 1.000000),
    ((1, 1, 0, 2, 1, 2, 2, 1, 0), 1.000000),
    ((1, 1, 0, 2, 1, 2, 2, 0, 1), 1.000000),
    ((1, 1, 0, 2, 1, 2, 0, 2, 0), 1.000000),
    ((1, 1, 1, 2, 1, 2, 0, 2, 0), 1.000000),
    ((1, 1, 0, 2, 1, 2, 1, 2, 0), 1.000000),
    ((1, 1, 0, 2, 1, 2, 1, 2, 2), 1.000000),
    ((1, 1, 1, 2, 1, 2, 1, 2, 2), 1.000000),
    ((1, 1, 0, 2, 1, 2, 0, 2, 1), 1.000000),
    ((1, 1, 0, 2, 1, 2, 0, 0, 2), 0.500000),
    ((1, 1, 1, 2, 1, 2, 0, 0, 2), 1.000000),
    ((1, 1, 0, 2, 1, 2, 1, 0, 2), 0.000000),
    ((1, 1, 0, 2, 1, 2, 0, 1, 2), 1.000000),
    ((1, 1, 0, 2, 0, 2, 1, 0, 0), -0.058824),
    ((1, 1, 0, 2, 0, 2, 1, 2, 0), 0.600000),
    ((1, 1, 1, 2, 0, 2, 1, 2, 0), 1.000000),
    ((1, 1, 0, 2, 0, 2, 1, 2, 1), 0.000000),
    ((1, 1, 0, 2, 0, 2, 1, 0, 2), -0.200000),
    ((1, 1, 1, 2, 0, 2, 1, 0, 2), 1.000000),
    ((1, 1, 0, 2, 0, 2, 1, 1, 2), -1.000000),
    ((1, 1, 0, 2, 0, 2, 0, 1, 0), 0.000000),
    ((1, 1, 0, 2, 0, 2, 2, 1, 0), 0.500000),
    ((1, 1, 1, 2, 0, 2, 2, 1, 0), 1.000000),
    ((1, 1, 0, 2, 0, 2, 2, 1, 1), 0.000000),
    ((1, 1, 0, 2, 0, 2, 0, 1, 2), 0.000000),
    ((1, 1, 1, 2, 0, 2, 0, 1, 2), 1.000000),
    ((1, 1, 0, 2, 0, 2, 0, 0, 1), 0.285714),
    ((1, 1, 0, 2, 0, 2, 2, 0, 1), 0.500000),
    ((1, 1, 1, 2, 0, 2, 2, 0, 1), 1.000000),
    ((1, 1, 0, 2, 0, 2, 0, 2, 1), 0.500000),
    ((1, 1, 1, 2, 0, 2, 0, 2, 1), 1.000000),
    ((1, 1, 0, 2, 0, 0, 2, 0, 0), 0.452055),
    ((1, 1, 1, 2, 0, 0, 2, 0, 0), 1.000000),
    ((1, 1, 0, 2, 1, 0, 2, 0, 0), 0.733333),
    ((1, 1, 0, 2, 1, 0, 2, 2, 0), 0.500000),
    ((1, 1, 1, 2, 1, 0, 2, 2, 0), 1.000000),
    ((1, 1, 0, 2, 1, 1, 2, 2, 0), 0.000000),
    ((1, 1, 0, 2, 1, 1, 2, 2, 2), -1.000000),
    ((1, 1, 0, 2, 1, 0, 2, 2, 1), 1.000000),
    ((1, 1, 0, 2, 1, 0, 2, 0, 2), 0.500000),
    ((1, 1, 1, 2, 1, 0, 2, 0, 2), 1.000000),
    ((1, 1, 0, 2, 1, 1, 2, 0, 2), 0.000000),
    ((1, 1, 0, 2, 1, 0, 2, 1, 2), 1.000000),
    ((1, 1, 0, 2, 0, 1, 2, 0, 0), 0.428571),
    ((1, 1, 0, 2, 0, 1, 2, 2, 0), 0.600000),
    ((1, 1, 1, 2, 0, 1, 2, 2, 0), 1.000000),
    ((1, 1, 0, 2, 0, 1, 2, 2, 1), 1.000000),
    ((1, 1, 0, 2, 0, 1, 2, 0, 2), 0.600000),
    ((1, 1, 1, 2, 0, 1, 2, 0, 2), 1.000000),
    ((1, 1, 0, 2, 0, 1, 2, 1, 2), 1.000000),
    ((1, 1, 0, 2, 0, 0, 2, 1, 0), 0.333333),
    ((1, 1, 0, 2, 0, 0, 2, 1, 2), 1.000000),
    ((1, 1, 1, 2, 0, 0, 2, 1, 2), 1.000000),
    ((1, 1, 0, 2, 0, 0, 2, 0, 1), 0.333333),
    ((1, 1, 0, 2, 0, 0, 2, 2, 1), 1.000000),
    ((1, 1, 1, 2, 0, 0, 2, 2, 1), 1.000000),
    ((1, 1, 0, 2, 0, 0, 0, 2, 0), 0.493671),
    ((1, 1, 1, 2, 0, 0, 0, 2, 0), 1.000000),
    ((1, 1, 0, 2, 1, 0, 0, 2, 0), 0.555556),
    ((1, 1, 0, 2, 1, 0, 0, 2, 2), 0.200000),
    ((1, 1, 1, 2, 1, 0, 0, 2, 2), 1.000000),
    ((1, 1, 0, 2, 1, 1, 0, 2, 2), -0.500000),
    ((1, 1, 0, 2, 1, 0, 1, 2, 2), 0.500000),
    ((1, 1, 0, 2, 0, 1, 0, 2, 0), 0.428571),
    ((1, 1, 0, 2, 0, 1, 0, 2, 2), 0.200000),
    ((1, 1, 1, 2, 0, 1, 0, 2, 2), 1.000000),
    ((1, 1, 0, 2, 0, 1, 1, 2, 2), 0.500000),
    ((1, 1, 0, 2, 0, 0, 1, 2, 0), 0.428571),
    ((1, 1, 0, 2, 0, 0, 1, 2, 2), 0.600000),
    ((1, 1, 1, 2, 0, 0, 1, 2, 2), 1.000000),
    ((1, 1, 0, 2, 0, 0, 0, 2, 1), 0.555556),
    ((1, 1, 0, 2, 0, 0, 0, 0, 2), 0.291139),
    ((1, 1, 1, 2, 0, 0, 0, 0, 2), 1.000000),
    ((1, 1, 0, 2, 1, 0, 0, 0, 2), 0.333333),
    ((1, 1, 0, 2, 0, 1, 0, 0, 2), 0.428571),
    ((1, 1, 0, 2, 0, 0, 1, 0, 2), 0.047619),
    ((1, 1, 0, 2, 0, 0, 0, 1, 2), 0.333333),
    ((1, 0, 1, 2, 0, 0, 0, 0, 0), 0.438753),
    ((1, 0, 1, 2, 2, 0, 0, 0, 0), 0.134328),
    ((1, 0, 1, 2, 2, 1, 0, 0, 0), 0.555556),
    ((1, 0, 1, 2, 2, 1, 2, 0, 0), 1.000000),
    ((1, 0, 1, 2, 2, 1, 2, 1, 0), 1.000000),
    ((1, 0, 1, 2, 2, 1, 2, 1, 2), 1.000000),
    ((1, 0, 1, 2, 2, 1, 2, 0, 1), 1.000000),
    ((1, 0, 1, 2, 2, 1, 0, 2, 0), 0.500000),
    ((1, 0, 1, 2, 2, 1, 1, 2, 0), 0.000000),
    ((1, 0, 1, 2, 2, 1, 1, 2, 2), 1.000000),
    ((1, 0, 1, 2, 2, 1, 0, 2, 1), 1.000000),
    ((1, 0, 1, 2, 2, 1, 0, 0, 2), 0.600000),
    ((1, 0, 1, 2, 2, 1, 1, 0, 2), 0.500000),
    ((1, 0, 1, 2, 2, 1, 0, 1, 2), 0.500000),
    ((1, 0, 1, 2, 2, 0, 1, 0, 0), -0.294118),
    ((1, 0, 1, 2, 2, 2, 1, 0, 0), -1.000000),
    ((1, 0, 1, 2, 2, 0, 1, 2, 0), -0.200000),
    ((1, 0, 1, 2, 2, 0, 1, 2, 1), -1.000000),
    ((1, 0, 1, 2, 2, 2, 1, 2, 1), -1.000000),
    ((1, 0, 1, 2, 2, 0, 1, 0, 2), 0.200000),
    ((1, 0, 1, 2, 2, 0, 1, 1, 2), -0.500000),
    ((1, 0, 1, 2, 2, 2, 1, 1, 2), -1.000000),
    ((1, 0, 1, 2, 2, 0, 0, 1, 0), 0.176471),
    ((1, 0, 1, 2, 2, 2, 0, 1, 0), -1.000000),
    ((1, 0, 1, 2, 2, 0, 2, 1, 0), 0.600000),
    ((1, 0, 1, 2, 2, 0, 2, 1, 1), 0.000000),
    ((1, 0, 1, 2, 2, 2, 2, 1, 1), -1.000000),
    ((1, 0, 1, 2, 2, 0, 0, 1, 2), 0.200000),
    ((1, 0, 1, 2, 2, 0, 0, 0, 1), 0.000000),
    ((1, 0, 1, 2, 2, 2, 0, 0, 1), -1.000000),
    ((1, 0, 1, 2, 2, 0, 2, 0, 1), 0.500000),
    ((1, 0, 1, 2, 2, 0, 0, 2, 1), 0.000000),
    ((1, 0, 1, 2, 0, 2, 0, 0, 0), 0.508197),
    ((1, 0, 1, 2, 1, 2, 0, 0, 0), 1.000000),
    ((1, 0, 1, 2, 1, 2, 2, 0, 0), 1.000000),
    ((1, 0, 1, 2, 1, 2, 2, 1, 0), 1.000000),
    ((1, 0, 1, 2, 1, 2, 2, 1, 2), 1.000000),
    ((1, 1, 1, 2, 1, 2, 2, 1, 2), 1.000000),
    ((1, 0, 1, 2, 1, 2, 2, 0, 1), 1.000000),
    ((1, 0, 1, 2, 1, 2, 0, 2, 0), 1.000000),
    ((1, 0, 1, 2, 1, 2, 1, 2, 0), 1.000000),
    ((1, 0, 1, 2, 1, 2, 0, 2, 1), 1.000000),
    ((1, 0, 1, 2, 1, 2, 0, 0, 2), 1.000000),
    ((1, 0, 1, 2, 1, 2, 1, 0, 2), 1.000000),
    ((1, 0, 1, 2, 1, 2, 0, 1, 2), 1.000000),
    ((1, 0, 1, 2, 0, 2, 1, 0, 0), 0.285714),
    ((1, 0, 1, 2, 0, 2, 1, 2, 0), 0.500000),
    ((1, 0, 1, 2, 0, 2, 1, 2, 1), 0.000000),
    ((1, 0, 1, 2, 0, 2, 1, 0, 2), 0.500000),
    ((1, 0, 1, 2, 0, 2, 1, 1, 2), 0.000000),
    ((1, 0, 1, 2, 0, 2, 0, 1, 0), 0.411765),
    ((1, 0, 1, 2, 0, 2, 2, 1, 0), 0.600000),
    ((1, 0, 1, 2, 0, 2, 2, 1, 1), 0.000000),
    ((1, 0, 1, 2, 0, 2, 0, 1, 2), 0.600000),
    ((1, 0, 1, 2, 0, 2, 0, 0, 1), 0.285714),
    ((1, 0, 1, 2, 0, 2, 2, 0, 1), 0.500000),
    ((1, 0, 1, 2, 0, 2, 0, 2, 1), 0.500000),
    ((1, 0, 1, 2, 0, 0, 2, 0, 0), 0.616438),
    ((1, 0, 1, 2, 1, 0, 2, 0, 0), 0.555556),
    ((1, 0, 1, 2, 1, 0, 2, 2, 0), 0.500000),
    ((1, 0, 1, 2, 1, 1, 2, 2, 0), 0.000000),
    ((1, 0, 1, 2, 1, 1, 2, 2, 2), -1.000000),
    ((1, 0, 1, 2, 1, 0, 2, 2, 1), 1.000000),
    ((1, 0, 1, 2, 1, 0, 2, 0, 2), 0.200000),
    ((1, 0, 1, 2, 1, 1, 2, 0, 2), -0.500000),
    ((1, 0, 1, 2, 1, 0, 2, 1, 2), 0.500000),
    ((1, 0, 1, 2, 0, 1, 2, 0, 0), 0.555556),
    ((1, 0, 1, 2, 0, 1, 2, 2, 0), 0.500000),
    ((1, 0, 1, 2, 0, 1, 2, 2, 1), 1.000000),
    ((1, 0, 1, 2, 0, 1, 2, 0, 2), 0.200000),
    ((1, 0, 1, 2, 0, 1, 2, 1, 2), 0.500000),
    ((1, 0, 1, 2, 0, 0, 2, 1, 0), 0.619048),
    ((1, 0, 1, 2, 0, 0, 2, 1, 2), 0.600000),
    ((1, 0, 1, 2, 0, 0, 2, 0, 1), 0.733333),
    ((1, 0, 1, 2, 0, 0, 2, 2, 1), 1.000000),
    ((1, 0, 1, 2, 0, 0, 0, 2, 0), 0.522388),
    ((1, 0, 1, 2, 1, 0, 0, 2, 0), 0.733333),
    ((1, 0, 1, 2, 1, 0, 0, 2, 2), 0.500000),
    ((1, 0, 1, 2, 1, 1, 0, 2, 2), 0.000000),
    ((1, 0, 1, 2, 1, 0, 1, 2, 2), 1.000000),
    ((1, 0, 1, 2, 0, 1, 0, 2, 0), 0.555556),
    ((1, 0, 1, 2, 0, 1, 0, 2, 2), 0.600000),
    ((1, 0, 1, 2, 0, 1, 1, 2, 2), 1.000000),
    ((1, 0, 1, 2, 0, 0, 1, 2, 0), 0.333333),
    ((1, 0, 1, 2, 0, 0, 1, 2, 2), 1.000000),
    ((1, 0, 1, 2, 0, 0, 0, 2, 1), 0.466667),
    ((1, 0, 1, 2, 0, 0, 0, 0, 2), 0.493671),
    ((1, 0, 1, 2, 1, 0, 0, 0, 2), 0.555556),
    ((1, 0, 1, 2, 0, 1, 0, 0, 2), 0.428571),
    ((1, 0, 1, 2, 0, 0, 1, 0, 2), 0.555556),
    ((1, 0, 1, 2, 0, 0, 0, 1, 2), 0.428571),
    ((1, 0, 0, 2, 1, 0, 0, 0, 0), 0.568710),
    ((1, 0, 0, 2, 1, 2, 0, 0, 0), 0.761194),
    ((1, 0, 0, 2, 1, 2, 1, 0, 0), 0.555556),
    ((1, 0, 0, 2, 1, 2, 1, 2, 0), 1.000000),
    ((1, 0, 0, 2, 1, 2, 1, 2, 1), 1.000000),
    ((1, 0, 0, 2, 1, 2, 1, 0, 2), 0.200000),
    ((1, 0, 0, 2, 1, 2, 1, 1, 2), 0.000000),
    ((1, 0, 0, 2, 1, 2, 0, 1, 0), 0.777778),
    ((1, 0, 0, 2, 1, 2, 2, 1, 0), 1.000000),
    ((1, 0, 0, 2, 1, 2, 2, 1, 1), 1.000000),
    ((1, 0, 0, 2, 1, 2, 0, 1, 2), 0.600000),
    ((1, 0, 0, 2, 1, 2, 0, 0, 1), 1.000000),
    ((1, 0, 0, 2, 1, 0, 2, 0, 0), 0.616438),
    ((1, 0, 0, 2, 1, 1, 2, 0, 0), 0.428571),
    ((1, 0, 0, 2, 1, 1, 2, 2, 0), 0.200000),
    ((1, 0, 0, 2, 1, 1, 2, 2, 1), 1.000000),
    ((1, 0, 0, 2, 1, 1, 2, 0, 2), 0.000000),
    ((1, 0, 0, 2, 1, 1, 2, 1, 2), 0.500000),
    ((1, 0, 0, 2, 1, 0, 2, 1, 0), 0.777778),
    ((1, 0, 0, 2, 1, 0, 2, 1, 2), 0.600000),
    ((1, 0, 0, 2, 1, 0, 2, 0, 1), 1.000000),
    ((1, 0, 0, 2, 1, 0, 0, 2, 0), 0.616438),
    ((1, 0, 0, 2, 1, 1, 0, 2, 0), 0.428571),
    ((1, 0, 0, 2, 1, 1, 0, 2, 2), 0.000000),
    ((1, 0, 0, 2, 1, 1, 1, 2, 2), 0.500000),
    ((1, 0, 0, 2, 1, 0, 1, 2, 0), 0.777778),
    ((1, 0, 0, 2, 1, 0, 1, 2, 2), 0.600000),
    ((1, 0, 0, 2, 1, 0, 0, 2, 1), 1.000000),
    ((1, 0, 0, 2, 1, 0, 0, 0, 2), 0.333333),
    ((1, 0, 0, 2, 1, 1, 0, 0, 2), 0.166667),
    ((1, 0, 0, 2, 1, 0, 1, 0, 2), 0.238095),
    ((1, 0, 0, 2, 1, 0, 0, 1, 2), 0.428571),
    ((1, 0, 0, 2, 0, 1, 0, 0, 0), 0.407407),
    ((1, 0, 0, 2, 2, 1, 0, 0, 0), 0.296296),
    ((1, 0, 0, 2, 2, 1, 1, 0, 0), 0.166667),
    ((1, 0, 0, 2, 2, 1, 1, 2, 0), 0.000000),
    ((1, 0, 0, 2, 2, 1, 1, 2, 1), -0.500000),
    ((1, 0, 0, 2, 2, 1, 1, 0, 2), 0.333333),
    ((1, 0, 0, 2, 2, 1, 1, 1, 2), 0.000000),
    ((1, 0, 0, 2, 2, 1, 0, 1, 0), 0.333333),
    ((1, 0, 0, 2, 2, 1, 2, 1, 0), 0.333333),
    ((1, 0, 0, 2, 2, 1, 2, 1, 1), 0.000000),
    ((1, 0, 0, 2, 2, 1, 0, 1, 2), 0.333333),
    ((1, 0, 0, 2, 2, 1, 0, 0, 1), 0.238095),
    ((1, 0, 0, 2, 2, 1, 2, 0, 1), 0.200000),
    ((1, 0, 0, 2, 2, 1, 0, 2, 1), 0.200000),
    ((1, 0, 0, 2, 0, 1, 2, 0, 0), 0.490196),
    ((1, 0, 0, 2, 0, 1, 2, 1, 0), 0.500000),
    ((1, 0, 0, 2, 0, 1, 2, 1, 2), 0.666667),
    ((1, 0, 0, 2, 0, 1, 2, 0, 1), 0.555556),
    ((1, 0, 0, 2, 0, 1, 2, 2, 1), 1.000000),
    ((1, 0, 0, 2, 0, 1, 0, 2, 0), 0.450980),
    ((1, 0, 0, 2, 0, 1, 1, 2, 0), 0.333333),
    ((1, 0, 0, 2, 0, 1, 1, 2, 2), 0.666667),
    ((1, 0, 0, 2, 0, 1, 0, 2, 1), 0.555556),
    ((1, 0, 0, 2, 0, 1, 0, 0, 2), 0.333333),
    ((1, 0, 0, 2, 0, 1, 1, 0, 2), 0.333333),
    ((1, 0, 0, 2, 0, 1, 0, 1, 2), 0.333333),
    ((1, 0, 0, 2, 0, 0, 1, 0, 0), 0.209459),
    ((1, 0, 0, 2, 2, 0, 1, 0, 0), -0.086957),
    ((1, 0, 0, 2, 2, 0, 1, 1, 0), -0.058824),
    ((1, 0, 0, 2, 2, 2, 1, 1, 0), -1.000000),
    ((1, 0, 0, 2, 2, 0, 1, 1, 2), -0.333333),
    ((1, 0, 0, 2, 2, 0, 1, 0, 1), -0.294118),
    ((1, 0, 0, 2, 2, 2, 1, 0, 1), -1.000000),
    ((1, 0, 0, 2, 2, 0, 1, 2, 1), -0.666667),
    ((1, 0, 0, 2, 0, 2, 1, 0, 0), 0.200000),
    ((1, 0, 0, 2, 0, 2, 1, 1, 0), -0.058824),
    ((1, 0, 0, 2, 0, 2, 1, 1, 2), -0.333333),
    ((1, 0, 0, 2, 0, 2, 1, 0, 1), 0.285714),
    ((1, 0, 0, 2, 0, 2, 1, 2, 1), 0.200000),
    ((1, 0, 0, 2, 0, 0, 1, 2, 0), 0.372549),
    ((1, 0, 0, 2, 0, 0, 1, 2, 1), 0.047619),
    ((1, 0, 0, 2, 0, 0, 1, 0, 2), 0.185185),
    ((1, 0, 0, 2, 0, 0, 1, 1, 2), -0.166667),
    ((1, 0, 0, 2, 0, 0, 0, 1, 0), 0.310811),
    ((1, 0, 0, 2, 2, 0, 0, 1, 0), 0.043478),
    ((1, 0, 0, 2, 2, 0, 0, 1, 1), -0.058824),
    ((1, 0, 0, 2, 2, 2, 0, 1, 1), -1.000000),
    ((1, 0, 0, 2, 2, 0, 2, 1, 1), -0.333333),
    ((1, 0, 0, 2, 0, 2, 0, 1, 0), 0.300000),
    ((1, 0, 0, 2, 0, 2, 0, 1, 1), 0.285714),
    ((1, 0, 0, 2, 0, 2, 2, 1, 1), 0.200000),
    ((1, 0, 0, 2, 0, 0, 2, 1, 0), 0.490196),
    ((1, 0, 0, 2, 0, 0, 2, 1, 1), 0.238095),
    ((1, 0, 0, 2, 0, 0, 0, 1, 2), 0.259259),
    ((1, 0, 0, 2, 0, 0, 0, 0, 1), 0.319728),
    ((1, 0, 0, 2, 2, 0, 0, 0, 1), -0.069767),
    ((1, 0, 0, 2, 0, 2, 0, 0, 1), 0.298246),
    ((1, 0, 0, 2, 0, 0, 2, 0, 1), 0.452055),
    ((1, 0, 0, 2, 0, 0, 0, 2, 1), 0.397260),
    ((1, 0, 0, 0, 2, 0, 0, 0, 0), 0.035755),
    ((1, 1, 0, 0, 2, 0, 0, 0, 0), -0.006565),
    ((1, 1, 0, 0, 2, 2, 0, 0, 0), -0.044776),
    ((1, 1, 1, 0, 2, 2, 0, 0, 0), 1.000000),
    ((1, 1, 0, 1, 2, 2, 0, 0, 0), 0.333333),
    ((1, 1, 0, 1, 2, 2, 2, 0, 0), 0.200000),
    ((1, 1, 1, 1, 2, 2, 2, 0, 0), 1.000000),
    ((1, 1, 0, 1, 2, 2, 2, 1, 0), 0.000000),
    ((1, 1, 0, 1, 2, 2, 2, 1, 2), 1.000000),
    ((1, 1, 1, 1, 2, 2, 2, 1, 2), 1.000000),
    ((1, 1, 0, 1, 2, 2, 2, 0, 1), 0.000000),
    ((1, 1, 0, 1, 2, 2, 2, 2, 1), 1.000000),
    ((1, 1, 1, 1, 2, 2, 2, 2, 1), 1.000000),
    ((1, 1, 0, 1, 2, 2, 0, 2, 0), 1.000000),
    ((1, 1, 1, 1, 2, 2, 0, 2, 0), 1.000000),
    ((1, 1, 0, 1, 2, 2, 1, 2, 0), 1.000000),
    ((1, 1, 0, 1, 2, 2, 0, 2, 1), 1.000000),
    ((1, 1, 0, 1, 2, 2, 0, 0, 2), 0.500000),
    ((1, 1, 1, 1, 2, 2, 0, 0, 2), 1.000000),
    ((1, 1, 0, 1, 2, 2, 1, 0, 2), 1.000000),
    ((1, 1, 0, 1, 2, 2, 0, 1, 2), 0.000000),
    ((1, 1, 0, 0, 2, 2, 1, 0, 0), 0.000000),
    ((1, 1, 0, 0, 2, 2, 1, 2, 0), 0.500000),
    ((1, 1, 1, 0, 2, 2, 1, 2, 0), 1.000000),
    ((1, 1, 0, 0, 2, 2, 1, 2, 1), 0.000000),
    ((1, 1, 0, 0, 2, 2, 1, 0, 2), 0.000000),
    ((1, 1, 1, 0, 2, 2, 1, 0, 2), 1.000000),
    ((1, 1, 0, 0, 2, 2, 1, 1, 2), -1.000000),
    ((1, 1, 0, 0, 2, 2, 0, 1, 0), -0.529412),
    ((1, 1, 0, 0, 2, 2, 2, 1, 0), -0.200000),
    ((1, 1, 1, 0, 2, 2, 2, 1, 0), 1.000000),
    ((1, 1, 0, 0, 2, 2, 2, 1, 1), -1.000000),
    ((1, 1, 0, 0, 2, 2, 0, 1, 2), -0.200000),
    ((1, 1, 1, 0, 2, 2, 0, 1, 2), 1.000000),
    ((1, 1, 0, 0, 2, 2, 0, 0, 1), -0.058824),
    ((1, 1, 0, 0, 2, 2, 2, 0, 1), -0.200000),
    ((1, 1, 1, 0, 2, 2, 2, 0, 1), 1.000000),
    ((1, 1, 0, 0, 2, 2, 0, 2, 1), 0.600000),
    ((1, 1, 1, 0, 2, 2, 0, 2, 1), 1.000000),
    ((1, 1, 0, 0, 2, 0, 2, 0, 0), -0.107692),
    ((1, 1, 1, 0, 2, 0, 2, 0, 0), 1.000000),
    ((1, 1, 0, 1, 2, 0, 2, 0, 0), -0.125000),
    ((1, 1, 0, 1, 2, 0, 2, 2, 0), -0.200000),
    ((1, 1, 1, 1, 2, 0, 2, 2, 0), 1.000000),
    ((1, 1, 0, 1, 2, 1, 2, 2, 0), -1.000000),
    ((1, 1, 0, 1, 2, 1, 2, 2, 2), -1.000000),
    ((1, 1, 0, 1, 2, 0, 2, 2, 1), 0.000000),
    ((1, 1, 0, 1, 2, 0, 2, 0, 2), -0.200000),
    ((1, 1, 1, 1, 2, 0, 2, 0, 2), 1.000000),
    ((1, 1, 0, 1, 2, 1, 2, 0, 2), -1.000000),
    ((1, 1, 0, 1, 2, 0, 2, 1, 2), 0.000000),
    ((1, 1, 0, 0, 2, 1, 2, 0, 0), -0.125000),
    ((1, 1, 0, 0, 2, 1, 2, 2, 0), -0.200000),
    ((1, 1, 1, 0, 2, 1, 2, 2, 0), 1.000000),
    ((1, 1, 0, 0, 2, 1, 2, 2, 1), 0.000000),
    ((1, 1, 0, 0, 2, 1, 2, 0, 2), -0.200000),
    ((1, 1, 1, 0, 2, 1, 2, 0, 2), 1.000000),
    ((1, 1, 0, 0, 2, 1, 2, 1, 2), 0.000000),
    ((1, 1, 0, 0, 2, 0, 2, 1, 0), -0.125000),
    ((1, 1, 0, 0, 2, 0, 2, 1, 2), 0.200000),
    ((1, 1, 1, 0, 2, 0, 2, 1, 2), 1.000000),
    ((1, 1, 0, 0, 2, 0, 2, 0, 1), -0.125000),
    ((1, 1, 0, 0, 2, 0, 2, 2, 1), 0.200000),
    ((1, 1, 1, 0, 2, 0, 2, 2, 1), 1.000000),
    ((1, 1, 0, 0, 2, 0, 0, 2, 0), 0.341772),
    ((1, 1, 1, 0, 2, 0, 0, 2, 0), 1.000000),
    ((1, 1, 0, 1, 2, 0, 0, 2, 0), 0.333333),
    ((1, 1, 0, 1, 2, 0, 0, 2, 2), 0.500000),
    ((1, 1, 1, 1, 2, 0, 0, 2, 2), 1.000000),
    ((1, 1, 0, 1, 2, 1, 0, 2, 2), 0.000000),
    ((1, 1, 0, 1, 2, 0, 1, 2, 2), 1.000000),
    ((1, 1, 0, 0, 2, 1, 0, 2, 0), 0.238095),
    ((1, 1, 0, 0, 2, 1, 0, 2, 2), 0.600000),
    ((1, 1, 1, 0, 2, 1, 0, 2, 2), 1.000000),
    ((1, 1, 0, 0, 2, 1, 1, 2, 2), 1.000000),
    ((1, 1, 0, 0, 2, 0, 1, 2, 0), 0.555556),
    ((1, 1, 0, 0, 2, 0, 1, 2, 2), 1.000000),
    ((1, 1, 1, 0, 2, 0, 1, 2, 2), 1.000000),
    ((1, 1, 0, 0, 2, 0, 0, 2, 1), 0.238095),
    ((1, 1, 0, 0, 2, 0, 0, 0, 2), 0.139241),
    ((1, 1, 1, 0, 2, 0, 0, 0, 2), 1.000000),
    ((1, 1, 0, 1, 2, 0, 0, 0, 2), 0.111111),
    ((1, 1, 0, 0, 2, 1, 0, 0, 2), 0.238095),
    ((1, 1, 0, 0, 2, 0, 1, 0, 2), 0.333333),
    ((1, 1, 0, 0, 2, 0, 0, 1, 2), -0.142857),
    ((1, 0, 1, 0, 2, 0, 0, 0, 0), 0.129252),
    ((1, 0, 1, 0, 2, 2, 0, 0, 0), 0.134328),
    ((1, 0, 1, 1, 2, 2, 0, 0, 0), 0.555556),
    ((1, 0, 1, 1, 2, 2, 2, 0, 0), 0.600000),
    ((1, 0, 1, 1, 2, 2, 2, 1, 0), 0.500000),
    ((1, 0, 1, 1, 2, 2, 2, 1, 2), 1.000000),
    ((1, 0, 1, 1, 2, 2, 2, 0, 1), 0.500000),
    ((1, 0, 1, 1, 2, 2, 2, 2, 1), 1.000000),
    ((1, 0, 1, 1, 2, 2, 0, 2, 0), 0.500000),
    ((1, 0, 1, 1, 2, 2, 1, 2, 0), 1.000000),
    ((1, 0, 1, 1, 2, 2, 0, 2, 1), 0.000000),
    ((1, 0, 1, 1, 2, 2, 0, 0, 2), 1.000000),
    ((1, 0, 1, 1, 2, 2, 1, 0, 2), 1.000000),
    ((1, 0, 1, 1, 2, 2, 0, 1, 2), 1.000000),
    ((1, 0, 1, 0, 2, 2, 1, 0, 0), 0.000000),
    ((1, 0, 1, 0, 2, 2, 1, 2, 0), 0.000000),
    ((1, 0, 1, 0, 2, 2, 1, 2, 1), -1.000000),
    ((1, 0, 1, 0, 2, 2, 1, 0, 2), 0.500000),
    ((1, 0, 1, 0, 2, 2, 1, 1, 2), 0.000000),
    ((1, 0, 1, 0, 2, 2, 0, 1, 0), 0.176471),
    ((1, 0, 1, 0, 2, 2, 2, 1, 0), 0.200000),
    ((1, 0, 1, 0, 2, 2, 2, 1, 1), -0.500000),
    ((1, 0, 1, 0, 2, 2, 0, 1, 2), 0.600000),
    ((1, 0, 1, 0, 2, 2, 0, 0, 1), -0.294118),
    ((1, 0, 1, 0, 2, 2, 2, 0, 1), 0.200000),
    ((1, 0, 1, 0, 2, 2, 0, 2, 1), -0.200000),
    ((1, 0, 1, 0, 2, 0, 2, 0, 0), 0.291139),
    ((1, 0, 1, 1, 2, 0, 2, 0, 0), 0.047619),
    ((1, 0, 1, 1, 2, 0, 2, 2, 0), -0.200000),
    ((1, 0, 1, 1, 2, 1, 2, 2, 0), -1.000000),
    ((1, 0, 1, 1, 2, 1, 2, 2, 2), -1.000000),
    ((1, 0, 1, 1, 2, 0, 2, 2, 1), 0.000000),
    ((1, 0, 1, 1, 2, 0, 2, 0, 2), 0.200000),
    ((1, 0, 1, 1, 2, 1, 2, 0, 2), -0.500000),
    ((1, 0, 1, 1, 2, 0, 2, 1, 2), 0.500000),
    ((1, 0, 1, 0, 2, 1, 2, 0, 0), 0.333333),
    ((1, 0, 1, 0, 2, 1, 2, 2, 0), 0.000000),
    ((1, 0, 1, 0, 2, 1, 2, 2, 1), 1.000000),
    ((1, 0, 1, 0, 2, 1, 2, 0, 2), 0.200000),
    ((1, 0, 1, 0, 2, 1, 2, 1, 2), 0.500000),
    ((1, 0, 1, 0, 2, 0, 2, 1, 0), 0.428571),
    ((1, 0, 1, 0, 2, 0, 2, 1, 2), 0.600000),
    ((1, 0, 1, 0, 2, 0, 2, 0, 1), 0.333333),
    ((1, 0, 1, 0, 2, 0, 2, 2, 1), 0.500000),
    ((1, 0, 1, 0, 2, 0, 0, 2, 0), 0.017544),
    ((1, 0, 1, 1, 2, 0, 0, 2, 0), 0.000000),
    ((1, 0, 1, 1, 2, 0, 0, 2, 2), 0.000000),
    ((1, 0, 1, 1, 2, 1, 0, 2, 2), -1.000000),
    ((1, 0, 1, 1, 2, 0, 1, 2, 2), 1.000000),
    ((1, 0, 1, 0, 2, 1, 0, 2, 0), 0.000000),
    ((1, 0, 1, 0, 2, 1, 0, 2, 2), -0.200000),
    ((1, 0, 1, 0, 2, 1, 1, 2, 2), 0.000000),
    ((1, 0, 1, 0, 2, 0, 1, 2, 0), 0.000000),
    ((1, 0, 1, 0, 2, 0, 1, 2, 2), 0.500000),
    ((1, 0, 1, 0, 2, 0, 0, 2, 1), 0.000000),
    ((1, 0, 1, 0, 2, 0, 0, 0, 2), 0.291139),
    ((1, 0, 1, 1, 2, 0, 0, 0, 2), 0.333333),
    ((1, 0, 1, 0, 2, 1, 0, 0, 2), 0.047619),
    ((1, 0, 1, 0, 2, 0, 1, 0, 2), 0.333333),
    ((1, 0, 1, 0, 2, 0, 0, 1, 2), 0.428571),
    ((1, 0, 0, 1, 2, 0, 0, 0, 0), -0.006565),
    ((1, 0, 0, 1, 2, 2, 0, 0, 0), 0.341772),
    ((1, 0, 0, 1, 2, 2, 1, 0, 0), 1.000000),
    ((1, 0, 0, 1, 2, 2, 0, 1, 0), 0.238095),
    ((1, 0, 0, 1, 2, 2, 2, 1, 0), 0.000000),
    ((1, 0, 0, 1, 2, 2, 2, 1, 1), -0.500000),
    ((1, 0, 0, 1, 2, 2, 0, 1, 2), 0.600000),
    ((1, 0, 0, 1, 2, 2, 1, 1, 2), 1.000000),
    ((1, 0, 0, 1, 2, 2, 0, 0, 1), 0.238095),
    ((1, 0, 0, 1, 2, 2, 2, 0, 1), 0.000000),
    ((1, 0, 0, 1, 2, 2, 0, 2, 1), 0.600000),
    ((1, 0, 0, 1, 2, 2, 1, 2, 1), 1.000000),
    ((1, 0, 0, 1, 2, 0, 2, 0, 0), -0.276596),
    ((1, 0, 0, 1, 2, 1, 2, 0, 0), -0.789474),
    ((1, 0, 0, 1, 2, 1, 2, 2, 0), -1.000000),
    ((1, 0, 0, 1, 2, 1, 2, 2, 1), -1.000000),
    ((1, 0, 0, 1, 2, 1, 2, 0, 2), -0.666667),
    ((1, 0, 0, 1, 2, 1, 2, 1, 2), -0.500000),
    ((1, 0, 0, 1, 2, 0, 2, 1, 0), -0.157895),
    ((1, 0, 0, 1, 2, 0, 2, 1, 2), 0.000000),
    ((1, 0, 0, 1, 2, 0, 2, 0, 1), -0.368421),
    ((1, 0, 0, 1, 2, 0, 2, 2, 1), -0.333333),
    ((1, 0, 0, 1, 2, 0, 0, 2, 0), -0.044776),
    ((1, 0, 0, 1, 2, 1, 0, 2, 0), -0.529412),
    ((1, 0, 0, 1, 2, 1, 0, 2, 2), -0.200000),
    ((1, 0, 0, 1, 2, 1, 1, 2, 2), 1.000000),
    ((1, 0, 0, 1, 2, 0, 1, 2, 0), 1.000000),
    ((1, 0, 0, 1, 2, 0, 0, 2, 1), -0.058824),
    ((1, 0, 0, 1, 2, 0, 0, 0, 2), 0.139241),
    ((1, 0, 0, 1, 2, 1, 0, 0, 2), -0.142857),
    ((1, 0, 0, 1, 2, 0, 1, 0, 2), 1.000000),
    ((1, 0, 0, 1, 2, 0, 0, 1, 2), 0.238095),
    ((1, 0, 0, 0, 2, 1, 0, 0, 0), 0.055556),
    ((1, 0, 0, 0, 2, 1, 2, 0, 0), -0.136364),
    ((1, 0, 0, 0, 2, 1, 2, 1, 0), 0.052632),
    ((1, 0, 0, 0, 2, 1, 2, 1, 2), 0.000000),
    ((1, 0, 0, 0, 2, 1, 2, 0, 1), -0.125000),
    ((1, 0, 0, 0, 2, 1, 2, 2, 1), -0.200000),
    ((1, 0, 0, 0, 2, 1, 0, 2, 0), -0.069767),
    ((1, 0, 0, 0, 2, 1, 1, 2, 0), 0.176471),
    ((1, 0, 0, 0, 2, 1, 1, 2, 2), 0.600000),
    ((1, 0, 0, 0, 2, 1, 0, 2, 1), -0.294118),
    ((1, 0, 0, 0, 2, 1, 0, 0, 2), 0.148148),
    ((1, 0, 0, 0, 2, 1, 1, 0, 2), 0.428571),
    ((1, 0, 0, 0, 2, 1, 0, 1, 2), 0.166667),
    ((1, 0, 0, 0, 2, 0, 1, 0, 0), 0.129252),
    ((1, 0, 0, 0, 2, 2, 1, 0, 0), 0.017544),
    ((1, 0, 0, 0, 2, 2, 1, 1, 0), 0.000000),
    ((1, 0, 0, 0, 2, 2, 1, 1, 2), -0.200000),
    ((1, 0, 0, 0, 2, 2, 1, 0, 1), 0.000000),
    ((1, 0, 0, 0, 2, 2, 1, 2, 1), -0.200000),
    ((1, 0, 0, 0, 2, 0, 1, 2, 0), 0.134328),
    ((1, 0, 0, 0, 2, 0, 1, 2, 1), -0.294118),
    ((1, 0, 0, 0, 2, 0, 1, 0, 2), 0.291139),
    ((1, 0, 0, 0, 2, 0, 1, 1, 2), 0.047619),
    ((1, 0, 0, 0, 2, 0, 0, 1, 0), 0.055556),
    ((1, 0, 0, 0, 2, 2, 0, 1, 0), -0.069767),
    ((1, 0, 0, 0, 2, 2, 0, 1, 1), -0.294118),
    ((1, 0, 0, 0, 2, 2, 2, 1, 1), -0.666667),
    ((1, 0, 0, 0, 2, 0, 2, 1, 0), -0.021277),
    ((1, 0, 0, 0, 2, 0, 2, 1, 1), -0.368421),
    ((1, 0, 0, 0, 2, 0, 0, 1, 2), 0.148148),
    ((1, 0, 0, 0, 2, 0, 0, 0, 1), -0.092308),
    ((1, 0, 0, 0, 2, 2, 0, 0, 1), -0.069767),
    ((1, 0, 0, 0, 2, 0, 2, 0, 1), -0.136364),
    ((1, 0, 0, 0, 2, 0, 0, 2, 1), -0.069767),
    ((1, 0, 0, 0, 0, 2, 0, 0, 0), 0.309021),
    ((1, 1, 0, 0, 0, 2, 0, 0, 0), 0.165468),
    ((1, 1, 0, 0, 0, 2, 2, 0, 0), 0.232877),
    ((1, 1, 1, 0, 0, 2, 2, 0, 0), 1.000000),
    ((1, 1, 0, 1, 0, 2, 2, 0, 0), 0.047619),
    ((1, 1, 0, 1, 0, 2, 2, 2, 0), 0.600000),
    ((1, 1, 1, 1, 0, 2, 2, 2, 0), 1.000000),
    ((1, 1, 0, 1, 1, 2, 2, 2, 0), 0.000000),
    ((1, 1, 0, 1, 1, 2, 2, 2, 2), -1.000000),
    ((1, 1, 0, 1, 0, 2, 2, 2, 1), 1.000000),
    ((1, 1, 0, 1, 0, 2, 2, 0, 2), -0.200000),
    ((1, 1, 1, 1, 0, 2, 2, 0, 2), 1.000000),
    ((1, 1, 0, 1, 1, 2, 2, 0, 2), -1.000000),
    ((1, 1, 0, 1, 0, 2, 2, 1, 2), 0.000000),
    ((1, 1, 0, 0, 1, 2, 2, 0, 0), 0.466667),
    ((1, 1, 0, 0, 1, 2, 2, 2, 0), 0.500000),
    ((1, 1, 1, 0, 1, 2, 2, 2, 0), 1.000000),
    ((1, 1, 0, 0, 1, 2, 2, 2, 1), 1.000000),
    ((1, 1, 0, 0, 1, 2, 2, 0, 2), 0.000000),
    ((1, 1, 1, 0, 1, 2, 2, 0, 2), 1.000000),
    ((1, 1, 0, 0, 1, 2, 2, 1, 2), 1.000000),
    ((1, 1, 0, 0, 0, 2, 2, 1, 0), 0.111111),
    ((1, 1, 0, 0, 0, 2, 2, 1, 2), 0.500000),
    ((1, 1, 1, 0, 0, 2, 2, 1, 2), 1.000000),
    ((1, 1, 0, 0, 0, 2, 2, 0, 1), 0.333333),
    ((1, 1, 0, 0, 0, 2, 2, 2, 1), 1.000000),
    ((1, 1, 1, 0, 0, 2, 2, 2, 1), 1.000000),
    ((1, 1, 0, 0, 0, 2, 0, 2, 0), 0.561644),
    ((1, 1, 1, 0, 0, 2, 0, 2, 0), 1.000000),
    ((1, 1, 0, 1, 0, 2, 0, 2, 0), 0.555556),
    ((1, 1, 0, 1, 0, 2, 0, 2, 2), 0.000000),
    ((1, 1, 1, 1, 0, 2, 0, 2, 2), 1.000000),
    ((1, 1, 0, 1, 1, 2, 0, 2, 2), -1.000000),
    ((1, 1, 0, 1, 0, 2, 1, 2, 2), 1.000000),
    ((1, 1, 0, 0, 1, 2, 0, 2, 0), 0.333333),
    ((1, 1, 0, 0, 1, 2, 0, 2, 2), -0.200000),
    ((1, 1, 1, 0, 1, 2, 0, 2, 2), 1.000000),
    ((1, 1, 0, 0, 1, 2, 1, 2, 2), 0.000000),
    ((1, 1, 0, 0, 0, 2, 1, 2, 0), 0.555556),
    ((1, 1, 0, 0, 0, 2, 1, 2, 2), 0.500000),
    ((1, 1, 1, 0, 0, 2, 1, 2, 2), 1.000000),
    ((1, 1, 0, 0, 0, 2, 0, 2, 1), 0.777778),
    ((1, 1, 0, 0, 0, 2, 0, 0, 2), 0.017544),
    ((1, 1, 1, 0, 0, 2, 0, 0, 2), 1.000000),
    ((1, 1, 0, 1, 0, 2, 0, 0, 2), 0.000000),
    ((1, 1, 0, 0, 1, 2, 0, 0, 2), 0.000000),
    ((1, 1, 0, 0, 0, 2, 1, 0, 2), 0.000000),
    ((1, 1, 0, 0, 0, 2, 0, 1, 2), 0.000000),
    ((1, 0, 1, 0, 0, 2, 0, 0, 0), 0.438753),
    ((1, 0, 1, 0, 0, 2, 2, 0, 0), 0.493671),
    ((1, 0, 1, 1, 0, 2, 2, 0, 0), 0.428571),
    ((1, 0, 1, 1, 0, 2, 2, 2, 0), 0.600000),
    ((1, 0, 1, 1, 1, 2, 2, 2, 0), 0.000000),
    ((1, 0, 1, 1, 1, 2, 2, 2, 2), -1.000000),
    ((1, 0, 1, 1, 0, 2, 2, 2, 1), 1.000000),
    ((1, 0, 1, 1, 0, 2, 2, 0, 2), 0.200000),
    ((1, 0, 1, 1, 1, 2, 2, 0, 2), -0.500000),
    ((1, 0, 1, 1, 0, 2, 2, 1, 2), 0.500000),
    ((1, 0, 1, 0, 1, 2, 2, 0, 0), 0.555556),
    ((1, 0, 1, 0, 1, 2, 2, 2, 0), 0.500000),
    ((1, 0, 1, 0, 1, 2, 2, 2, 1), 1.000000),
    ((1, 0, 1, 0, 1, 2, 2, 0, 2), 0.200000),
    ((1, 0, 1, 0, 1, 2, 2, 1, 2), 0.500000),
    ((1, 0, 1, 0, 0, 2, 2, 1, 0), 0.428571),
    ((1, 0, 1, 0, 0, 2, 2, 1, 2), 0.600000),
    ((1, 0, 1, 0, 0, 2, 2, 0, 1), 0.555556),
    ((1, 0, 1, 0, 0, 2, 2, 2, 1), 1.000000),
    ((1, 0, 1, 0, 0, 2, 0, 2, 0), 0.522388),
    ((1, 0, 1, 1, 0, 2, 0, 2, 0), 0.555556),
    ((1, 0, 1, 1, 0, 2, 0, 2, 2), 0.500000),
    ((1, 0, 1, 1, 1, 2, 0, 2, 2), 0.000000),
    ((1, 0, 1, 1, 0, 2, 1, 2, 2), 1.000000),
    ((1, 0, 1, 0, 1, 2, 0, 2, 0), 0.733333),
    ((1, 0, 1, 0, 1, 2, 0, 2, 2), 0.500000),
    ((1, 0, 1, 0, 1, 2, 1, 2, 2), 1.000000),
    ((1, 0, 1, 0, 0, 2, 1, 2, 0), 0.466667),
    ((1, 0, 1, 0, 0, 2, 1, 2, 2), 1.000000),
    ((1, 0, 1, 0, 0, 2, 0, 2, 1), 0.333333),
    ((1, 0, 1, 0, 0, 2, 0, 0, 2), 0.616438),
    ((1, 0, 1, 1, 0, 2, 0, 0, 2), 0.555556),
    ((1, 0, 1, 0, 1, 2, 0, 0, 2), 0.555556),
    ((1, 0, 1, 0, 0, 2, 1, 0, 2), 0.733333),
    ((1, 0, 1, 0, 0, 2, 0, 1, 2), 0.619048),
    ((1, 0, 0, 1, 0, 2, 0, 0, 0), 0.315011),
    ((1, 0, 0, 1, 0, 2, 2, 0, 0), 0.185185),
    ((1, 0, 0, 1, 1, 2, 2, 0, 0), 0.047619),
    ((1, 0, 0, 1, 1, 2, 2, 2, 0), 0.200000),
    ((1, 0, 0, 1, 1, 2, 2, 2, 1), 1.000000),
    ((1, 0, 0, 1, 1, 2, 2, 0, 2), -0.666667),
    ((1, 0, 0, 1, 1, 2, 2, 1, 2), -0.500000),
    ((1, 0, 0, 1, 0, 2, 2, 1, 0), 0.000000),
    ((1, 0, 0, 1, 0, 2, 2, 1, 2), 0.000000),
    ((1, 0, 0, 1, 0, 2, 2, 0, 1), 0.428571),
    ((1, 0, 0, 1, 0, 2, 2, 2, 1), 1.000000),
    ((1, 0, 0, 1, 0, 2, 0, 2, 0), 0.561644),
    ((1, 0, 0, 1, 1, 2, 0, 2, 0), 0.333333),
    ((1, 0, 0, 1, 1, 2, 0, 2, 2), -0.200000),
    ((1, 0, 0, 1, 1, 2, 1, 2, 2), 1.000000),
    ((1, 0, 0, 1, 0, 2, 1, 2, 0), 1.000000),
    ((1, 0, 0, 1, 0, 2, 0, 2, 1), 0.777778),
    ((1, 0, 0, 1, 0, 2, 0, 0, 2), 0.134328),
    ((1, 0, 0, 1, 1, 2, 0, 0, 2), -0.294118),
    ((1, 0, 0, 1, 0, 2, 1, 0, 2), 1.000000),
    ((1, 0, 0, 1, 0, 2, 0, 1, 2), 0.176471),
    ((1, 0, 0, 0, 1, 2, 0, 0, 0), 0.453237),
    ((1, 0, 0, 0, 1, 2, 2, 0, 0), 0.397260),
    ((1, 0, 0, 0, 1, 2, 2, 1, 0), 0.555556),
    ((1, 0, 0, 0, 1, 2, 2, 1, 2), 0.200000),
    ((1, 0, 0, 0, 1, 2, 2, 0, 1), 1.000000),
    ((1, 0, 0, 0, 1, 2, 0, 2, 0), 0.522388),
    ((1, 0, 0, 0, 1, 2, 1, 2, 0), 0.733333),
    ((1, 0, 0, 0, 1, 2, 1, 2, 2), 0.500000),
    ((1, 0, 0, 0, 1, 2, 0, 2, 1), 1.000000),
    ((1, 0, 0, 0, 1, 2, 0, 0, 2), 0.150000),
    ((1, 0, 0, 0, 1, 2, 1, 0, 2), 0.285714),
    ((1, 0, 0, 0, 1, 2, 0, 1, 2), 0.176471),
    ((1, 0, 0, 0, 0, 2, 1, 0, 0), 0.297710),
    ((1, 0, 0, 0, 0, 2, 1, 2, 0), 0.522388),
    ((1, 0, 0, 0, 0, 2, 1, 2, 1), 0.333333),
    ((1, 0, 0, 0, 0, 2, 1, 0, 2), 0.245902),
    ((1, 0, 0, 0, 0, 2, 1, 1, 2), -0.058824),
    ((1, 0, 0, 0, 0, 2, 0, 1, 0), 0.194030),
    ((1, 0, 0, 0, 0, 2, 2, 1, 0), 0.215686),
    ((1, 0, 0, 0, 0, 2, 2, 1, 1), 0.047619),
    ((1, 0, 0, 0, 0, 2, 0, 1, 2), 0.209302),
    ((1, 0, 0, 0, 0, 2, 0, 0, 1), 0.319728),
    ((1, 0, 0, 0, 0, 2, 2, 0, 1), 0.341772),
    ((1, 0, 0, 0, 0, 2, 0, 2, 1), 0.561644),
    ((1, 0, 0, 0, 0, 0, 2, 0, 0), 0.197740),
    ((1, 1, 0, 0, 0, 0, 2, 0, 0), 0.103926),
    ((1, 1, 0, 0, 0, 0, 2, 2, 0), 0.194030),
    ((1, 1, 1, 0, 0, 0, 2, 2, 0), 1.000000),
    ((1, 1, 0, 1, 0, 0, 2, 2, 0), -0.058824),
    ((1, 1, 0, 1, 0, 0, 2, 2, 2), -1.000000),
    ((1, 1, 0, 0, 1, 0, 2, 2, 0), 0.285714),
    ((1, 1, 0, 0, 1, 0, 2, 2, 2), -1.000000),
    ((1, 1, 0, 0, 0, 1, 2, 2, 0), -0.058824),
    ((1, 1, 0, 0, 0, 1, 2, 2, 2), -1.000000),
    ((1, 1, 0, 0, 0, 0, 2, 2, 1), 0.555556),
    ((1, 1, 0, 0, 0, 0, 2, 0, 2), -0.044776),
    ((1, 1, 1, 0, 0, 0, 2, 0, 2), 1.000000),
    ((1, 1, 0, 1, 0, 0, 2, 0, 2), -0.529412),
    ((1, 1, 0, 0, 1, 0, 2, 0, 2), 0.000000),
    ((1, 1, 0, 0, 0, 1, 2, 0, 2), -0.058824),
    ((1, 1, 0, 0, 0, 0, 2, 1, 2), 0.333333),
    ((1, 0, 1, 0, 0, 0, 2, 0, 0), 0.315011),
    ((1, 0, 1, 0, 0, 0, 2, 2, 0), 0.245902),
    ((1, 0, 1, 1, 0, 0, 2, 2, 0), -0.058824),
    ((1, 0, 1, 1, 0, 0, 2, 2, 2), -1.000000),
    ((1, 0, 1, 0, 1, 0, 2, 2, 0), 0.285714),
    ((1, 0, 1, 0, 1, 0, 2, 2, 2), -1.000000),
    ((1, 0, 1, 0, 0, 1, 2, 2, 0), 0.000000),
    ((1, 0, 1, 0, 0, 1, 2, 2, 2), -1.000000),
    ((1, 0, 1, 0, 0, 0, 2, 2, 1), 0.733333),
    ((1, 0, 1, 0, 0, 0, 2, 0, 2), 0.095890),
    ((1, 0, 1, 1, 0, 0, 2, 0, 2), -0.058824),
    ((1, 0, 1, 0, 1, 0, 2, 0, 2), -0.058824),
    ((1, 0, 1, 0, 0, 1, 2, 0, 2), -0.058824),
    ((1, 0, 1, 0, 0, 0, 2, 1, 2), 0.428571),
    ((1, 0, 0, 1, 0, 0, 2, 0, 0), -0.048611),
    ((1, 0, 0, 1, 0, 0, 2, 2, 0), -0.023256),
    ((1, 0, 0, 1, 1, 0, 2, 2, 0), 0.285714),
    ((1, 0, 0, 1, 1, 0, 2, 2, 2), -1.000000),
    ((1, 0, 0, 1, 0, 1, 2, 2, 0), -0.529412),
    ((1, 0, 0, 1, 0, 1, 2, 2, 2), -1.000000),
    ((1, 0, 0, 1, 0, 0, 2, 2, 1), 0.238095),
    ((1, 0, 0, 1, 0, 0, 2, 0, 2), -0.217391),
    ((1, 0, 0, 1, 1, 0, 2, 0, 2), -0.294118),
    ((1, 0, 0, 1, 0, 1, 2, 0, 2), -0.294118),
    ((1, 0, 0, 1, 0, 0, 2, 1, 2), 0.000000),
    ((1, 0, 0, 0, 1, 0, 2, 0, 0), 0.401361),
    ((1, 0, 0, 0, 1, 0, 2, 2, 0), 0.298246),
    ((1, 0, 0, 0, 1, 1, 2, 2, 0), 0.285714),
    ((1, 0, 0, 0, 1, 1, 2, 2, 2), -1.000000),
    ((1, 0, 0, 0, 1, 0, 2, 2, 1), 1.000000),
    ((1, 0, 0, 0, 1, 0, 2, 0, 2), 0.069767),
    ((1, 0, 0, 0, 1, 1, 2, 0, 2), 0.176471),
    ((1, 0, 0, 0, 1, 0, 2, 1, 2), 0.428571),
    ((1, 0, 0, 0, 0, 1, 2, 0, 0), 0.144928),
    ((1, 0, 0, 0, 0, 1, 2, 2, 0), 0.000000),
    ((1, 0, 0, 0, 0, 1, 2, 2, 1), 0.333333),
    ((1, 0, 0, 0, 0, 1, 2, 0, 2), 0.043478),
    ((1, 0, 0, 0, 0, 1, 2, 1, 2), 0.333333),
    ((1, 0, 0, 0, 0, 0, 2, 1, 0), 0.263158),
    ((1, 0, 0, 0, 0, 0, 2, 1, 2), 0.296296),
    ((1, 0, 0, 0, 0, 0, 2, 0, 1), 0.256018),
    ((1, 0, 0, 0, 0, 0, 2, 2, 1), 0.452055),
    ((1, 0, 0, 0, 0, 0, 0, 2, 0), 0.309021),
    ((1, 1, 0, 0, 0, 0, 0, 2, 0), 0.315011),
    ((1, 1, 0, 0, 0, 0, 0, 2, 2), 0.134328),
    ((1, 1, 1, 0, 0, 0, 0, 2, 2), 1.000000),
    ((1, 1, 0, 1, 0, 0, 0, 2, 2), 0.000000),
    ((1, 1, 0, 0, 1, 0, 0, 2, 2), -0.294118),
    ((1, 1, 0, 0, 0, 1, 0, 2, 2), 0.176471),
    ((1, 1, 0, 0, 0, 0, 1, 2, 2), 0.555556),
    ((1, 0, 1, 0, 0, 0, 0, 2, 0), 0.297710),
    ((1, 0, 1, 0, 0, 0, 0, 2, 2), 0.245902),
    ((1, 0, 1, 1, 0, 0, 0, 2, 2), 0.000000),
    ((1, 0, 1, 0, 1, 0, 0, 2, 2), 0.285714),
    ((1, 0, 1, 0, 0, 1, 0, 2, 2), -0.058824),
    ((1, 0, 1, 0, 0, 0, 1, 2, 2), 0.733333),
    ((1, 0, 0, 1, 0, 0, 0, 2, 0), 0.165468),
    ((1, 0, 0, 1, 0, 0, 0, 2, 2), 0.017544),
    ((1, 0, 0, 1, 1, 0, 0, 2, 2), 0.000000),
    ((1, 0, 0, 1, 0, 1, 0, 2, 2), 0.000000),
    ((1, 0, 0, 1, 0, 0, 1, 2, 2), 1.000000),
    ((1, 0, 0, 0, 1, 0, 0, 2, 0), 0.453237),
    ((1, 0, 0, 0, 1, 0, 0, 2, 2), 0.150000),
    ((1, 0, 0, 0, 1, 1, 0, 2, 2), 0.176471),
    ((1, 0, 0, 0, 1, 0, 1, 2, 2), 0.555556),
    ((1, 0, 0, 0, 0, 1, 0, 2, 0), 0.194030),
    ((1, 0, 0, 0, 0, 1, 0, 2, 2), 0.209302),
    ((1, 0, 0, 0, 0, 1, 1, 2, 2), 0.619048),
    ((1, 0, 0, 0, 0, 0, 1, 2, 0), 0.438753),
    ((1, 0, 0, 0, 0, 0, 1, 2, 2), 0.616438),
    ((1, 0, 0, 0, 0, 0, 0, 2, 1), 0.319728),
    ((1, 0, 0, 0, 0, 0, 0, 0, 2), 0.199110),
    ((1, 1, 0, 0, 0, 0, 0, 0, 2), 0.047619),
    ((1, 0, 1, 0, 0, 0, 0, 0, 2), 0.315011),
    ((1, 0, 0, 1, 0, 0, 0, 0, 2), 0.047619),
    ((1, 0, 0, 0, 1, 0, 0, 0, 2), 0.194030),
    ((1, 0, 0, 0, 0, 1, 0, 0, 2), 0.220779),
    ((1, 0, 0, 0, 0, 0, 1, 0, 2), 0.315011),
    ((1, 0, 0, 0, 0, 0, 0, 1, 2), 0.220779),
    ((0, 1, 0, 0, 0, 0, 0, 0, 0), 0.137064),
    ((2, 1, 0, 0, 0, 0, 0, 0, 0), 0.132555),
    ((2, 1, 1, 0, 0, 0, 0, 0, 0), -0.048611),
    ((2, 1, 1, 2, 0, 0, 0, 0, 0), -0.023256),
    ((2, 1, 1, 2, 1, 0, 0, 0, 0), 0.285714),
    ((2, 1, 1, 2, 1, 2, 0, 0, 0), 0.500000),
    ((2, 1, 1, 2, 1, 2, 1, 0, 0), 1.000000),
    ((2, 1, 1, 2, 1, 2, 0, 1, 0), 1.000000),
    ((2, 1, 1, 2, 1, 2, 0, 0, 1), 0.000000),
    ((2, 1, 1, 2, 1, 2, 2, 0, 1), -1.000000),
    ((2, 1, 1, 2, 1, 2, 0, 2, 1), 1.000000),
    ((2, 1, 1, 2, 1, 2, 1, 2, 1), 1.000000),
    ((2, 1, 1, 2, 1, 0, 2, 0, 0), -1.000000),
    ((2, 1, 1, 2, 1, 0, 0, 2, 0), 0.200000),
    ((2, 1, 1, 2, 1, 1, 0, 2, 0), 0.000000),
    ((2, 1, 1, 2, 1, 1, 2, 2, 0), -1.000000),
    ((2, 1, 1, 2, 1, 1, 0, 2, 2), 1.000000),
    ((2, 1, 1, 2, 1, 1, 1, 2, 2), 1.000000),
    ((2, 1, 1, 2, 1, 0, 1, 2, 0), 1.000000),
    ((2, 1, 1, 2, 1, 0, 0, 2, 1), 0.000000),
    ((2, 1, 1, 2, 1, 0, 2, 2, 1), -1.000000),
    ((2, 1, 1, 2, 1, 0, 0, 0, 2), 0.500000),
    ((2, 1, 1, 2, 1, 1, 0, 0, 2), 0.000000),
    ((2, 1, 1, 2, 1, 1, 2, 0, 2), -1.000000),
    ((2, 1, 1, 2, 1, 0, 1, 0, 2), 1.000000),
    ((2, 1, 1, 2, 1, 0, 0, 1, 2), 1.000000),
    ((2, 1, 1, 2, 0, 1, 0, 0, 0), -0.058824),
    ((2, 1, 1, 2, 2, 1, 0, 0, 0), -0.200000),
    ((2, 1, 1, 2, 2, 1, 1, 0, 0), 0.000000),
    ((2, 1, 1, 2, 2, 1, 1, 2, 0), 1.000000),
    ((2, 1, 1, 2, 2, 1, 1, 2, 1), 1.000000),
    ((2, 1, 1, 2, 2, 1, 1, 0, 2), -1.000000),
    ((2, 1, 1, 2, 2, 1, 0, 1, 0), -1.000000),
    ((2, 1, 1, 2, 2, 1, 2, 1, 0), -1.000000),
    ((2, 1, 1, 2, 2, 1, 0, 1, 2), -1.000000),
    ((2, 1, 1, 2, 2, 1, 0, 0, 1), 1.000000),
    ((2, 1, 1, 2, 0, 1, 2, 0, 0), -1.000000),
    ((2, 1, 1, 2, 0, 1, 0, 2, 0), 0.600000),
    ((2, 1, 1, 2, 0, 1, 1, 2, 0), 1.000000),
    ((2, 1, 1, 2, 0, 1, 1, 2, 2), 1.000000),
    ((2, 1, 1, 2, 0, 1, 0, 2, 1), 1.000000),
    ((2, 1, 1, 2, 0, 1, 0, 0, 2), -0.333333),
    ((2, 1, 1, 2, 0, 1, 1, 0, 2), 0.000000),
    ((2, 1, 1, 2, 0, 1, 0, 1, 2), -1.000000),
    ((2, 1, 1, 2, 0, 1, 2, 1, 2), -1.000000),
    ((2, 1, 1, 2, 0, 0, 1, 0, 0), 0.238095),
    ((2, 1, 1, 2, 2, 0, 1, 0, 0), -0.333333),
    ((2, 1, 1, 2, 2, 0, 1, 1, 0), -1.000000),
    ((2, 1, 1, 2, 2, 2, 1, 1, 0), -1.000000),
    ((2, 1, 1, 2, 2, 0, 1, 1, 2), -1.000000),
    ((2, 1, 1, 2, 2, 0, 1, 0, 1), 0.000000),
    ((2, 1, 1, 2, 2, 2, 1, 0, 1), -1.000000),
    ((2, 1, 1, 2, 2, 0, 1, 2, 1), 1.000000),
    ((2, 1, 1, 2, 0, 2, 1, 0, 0), 0.200000),
    ((2, 1, 1, 2, 0, 2, 1, 1, 0), 0.000000),
    ((2, 1, 1, 2, 0, 2, 1, 1, 2), 1.000000),
    ((2, 1, 1, 2, 1, 2, 1, 1, 2), 1.000000),
    ((2, 1, 1, 2, 0, 2, 1, 0, 1), 0.000000),
    ((2, 1, 1, 2, 0, 2, 1, 2, 1), 1.000000),
    ((2, 1, 1, 2, 0, 0, 1, 2, 0), 1.000000),
    ((2, 1, 1, 2, 0, 0, 1, 2, 1), 1.000000),
    ((2, 1, 1, 2, 0, 0, 1, 0, 2), 0.200000),
    ((2, 1, 1, 2, 0, 0, 1, 1, 2), 0.000000),
    ((2, 1, 1, 2, 0, 0, 0, 1, 0), -0.529412),
    ((2, 1, 1, 2, 2, 0, 0, 1, 0), -1.000000),
    ((2, 1, 1, 2, 2, 0, 0, 1, 1), -1.000000),
    ((2, 1, 1, 2, 2, 2, 0, 1, 1), -1.000000),
    ((2, 1, 1, 2, 2, 0, 2, 1, 1), -1.000000),
    ((2, 1, 1, 2, 0, 2, 0, 1, 0), -0.200000),
    ((2, 1, 1, 2, 0, 2, 0, 1, 1), -1.000000),
    ((2, 1, 1, 2, 0, 2, 2, 1, 1), -1.000000),
    ((2, 1, 1, 2, 0, 0, 2, 1, 0), -1.000000),
    ((2, 1, 1, 2, 0, 0, 0, 1, 2), -0.200000),
    ((2, 1, 1, 2, 0, 0, 0, 0, 1), -0.058824),
    ((2, 1, 1, 2, 2, 0, 0, 0, 1), -0.200000),
    ((2, 1, 1, 2, 0, 2, 0, 0, 1), -0.333333),
    ((2, 1, 1, 2, 0, 0, 2, 0, 1), -1.000000),
    ((2, 1, 1, 2, 0, 0, 0, 2, 1), 0.600000),
    ((2, 1, 1, 0, 2, 0, 0, 0, 0), -0.276596),
    ((2, 1, 1, 1, 2, 0, 0, 0, 0), -0.157895),
    ((2, 1, 1, 1, 2, 2, 0, 0, 0), -0.333333),
    ((2, 1, 1, 1, 2, 2, 1, 0, 0), -0.500000),
    ((2, 1, 1, 1, 2, 2, 1, 2, 0), 0.000000),
    ((2, 1, 1, 1, 2, 2, 1, 2, 1), 0.000000),
    ((2, 1, 1, 1, 2, 2, 1, 0, 2), -1.000000),
    ((2, 1, 1, 1, 2, 2, 0, 1, 0), -0.500000),
    ((2, 1, 1, 1, 2, 2, 2, 1, 0), 0.000000),
    ((2, 1, 1, 1, 2, 2, 2, 1, 1), 0.000000),
    ((2, 1, 1, 1, 2, 2, 0, 1, 2), -1.000000),
    ((2, 1, 1, 1, 2, 2, 0, 0, 1), 0.000000),
    ((2, 1, 1, 1, 2, 2, 2, 0, 1), 0.000000),
    ((2, 1, 1, 1, 2, 2, 0, 2, 1), 0.000000),
    ((2, 1, 1, 1, 2, 0, 2, 0, 0), 0.000000),
    ((2, 1, 1, 1, 2, 1, 2, 0, 0), 0.000000),
    ((2, 1, 1, 1, 2, 1, 2, 2, 0), 1.000000),
    ((2, 1, 1, 1, 2, 1, 2, 2, 1), 1.000000),
    ((2, 1, 1, 1, 2, 1, 2, 0, 2), -1.000000),
    ((2, 1, 1, 1, 2, 0, 2, 1, 0), -0.500000),
    ((2, 1, 1, 1, 2, 0, 2, 1, 2), -1.000000),
    ((2, 1, 1, 1, 2, 0, 2, 0, 1), 0.500000),
    ((2, 1, 1, 1, 2, 0, 2, 2, 1), 1.000000),
    ((2, 1, 1, 1, 2, 0, 0, 2, 0), 0.000000),
    ((2, 1, 1, 1, 2, 1, 0, 2, 0), 0.000000),
    ((2, 1, 1, 1, 2, 1, 0, 2, 2), -1.000000),
    ((2, 1, 1, 1, 2, 0, 1, 2, 0), -0.500000),
    ((2, 1, 1, 1, 2, 0, 1, 2, 2), -1.000000),
    ((2, 1, 1, 1, 2, 0, 0, 2, 1), 0.500000),
    ((2, 1, 1, 1, 2, 0, 0, 0, 2), -1.000000),
    ((2, 1, 1, 0, 2, 1, 0, 0, 0), -0.125000),
    ((2, 1, 1, 0, 2, 1, 2, 0, 0), -0.200000),
    ((2, 1, 1, 0, 2, 1, 2, 1, 0), -1.000000),
    ((2, 1, 1, 0, 2, 1, 2, 1, 2), -1.000000),
    ((2, 1, 1, 0, 2, 1, 2, 0, 1), 1.000000),
    ((2, 1, 1, 0, 2, 1, 0, 2, 0), 0.200000),
    ((2, 1, 1, 0, 2, 1, 1, 2, 0), 0.000000),
    ((2, 1, 1, 0, 2, 1, 1, 2, 2), -1.000000),
    ((2, 1, 1, 0, 2, 1, 0, 2, 1), 1.000000),
    ((2, 1, 1, 0, 2, 1, 0, 0, 2), -1.000000),
    ((2, 1, 1, 0, 2, 0, 1, 0, 0), -0.368421),
    ((2, 1, 1, 0, 2, 2, 1, 0, 0), -0.666667),
    ((2, 1, 1, 0, 2, 2, 1, 1, 0), -1.000000),
    ((2, 1, 1, 0, 2, 2, 1, 1, 2), -1.000000),
    ((2, 1, 1, 0, 2, 2, 1, 0, 1), -0.500000),
    ((2, 1, 1, 0, 2, 2, 1, 2, 1), 0.000000),
    ((2, 1, 1, 0, 2, 0, 1, 2, 0), 0.000000),
    ((2, 1, 1, 0, 2, 0, 1, 2, 1), 0.500000),
    ((2, 1, 1, 0, 2, 0, 1, 0, 2), -1.000000),
    ((2, 1, 1, 0, 2, 0, 0, 1, 0), -0.789474),
    ((2, 1, 1, 0, 2, 2, 0, 1, 0), -0.666667),
    ((2, 1, 1, 0, 2, 2, 0, 1, 1), -0.500000),
    ((2, 1, 1, 0, 2, 2, 2, 1, 1), 0.000000),
    ((2, 1, 1, 0, 2, 0, 2, 1, 0), -0.666667),
    ((2, 1, 1, 0, 2, 0, 2, 1, 1), -0.500000),
    ((2, 1, 1, 0, 2, 0, 0, 1, 2), -1.000000),
    ((2, 1, 1, 0, 2, 0, 0, 0, 1), 0.047619),
    ((2, 1, 1, 0, 2, 2, 0, 0, 1), -0.333333),
    ((2, 1, 1, 0, 2, 0, 2, 0, 1), 0.200000),
    ((2, 1, 1, 0, 2, 0, 0, 2, 1), 0.600000),
    ((2, 1, 1, 0, 0, 2, 0, 0, 0), 0.074074),
    ((2, 1, 1, 1, 0, 2, 0, 0, 0), 0.166667),
    ((2, 1, 1, 1, 0, 2, 2, 0, 0), 0.333333),
    ((2, 1, 1, 1, 1, 2, 2, 0, 0), 0.500000),
    ((2, 1, 1, 1, 1, 2, 2, 2, 0), 0.000000),
    ((2, 1, 1, 1, 1, 2, 2, 2, 1), 0.000000),
    ((2, 1, 1, 1, 1, 2, 2, 0, 2), 1.000000),
    ((2, 1, 1, 1, 1, 2, 2, 1, 2), 1.000000),
    ((2, 1, 1, 1, 0, 2, 2, 1, 0), 0.500000),
    ((2, 1, 1, 1, 0, 2, 2, 1, 2), 1.000000),
    ((2, 1, 1, 1, 0, 2, 2, 0, 1), 0.000000),
    ((2, 1, 1, 1, 0, 2, 2, 2, 1), 0.000000),
    ((2, 1, 1, 1, 0, 2, 0, 2, 0), 0.333333),
    ((2, 1, 1, 1, 1, 2, 0, 2, 0), 0.500000),
    ((2, 1, 1, 1, 1, 2, 0, 2, 2), 1.000000),
    ((2, 1, 1, 1, 1, 2, 1, 2, 2), 1.000000),
    ((2, 1, 1, 1, 0, 2, 1, 2, 0), 0.500000),
    ((2, 1, 1, 1, 0, 2, 1, 2, 2), 1.000000),
    ((2, 1, 1, 1, 0, 2, 0, 2, 1), 0.000000),
    ((2, 1, 1, 1, 0, 2, 0, 0, 2), 0.333333),
    ((2, 1, 1, 1, 1, 2, 0, 0, 2), 1.000000),
    ((2, 1, 1, 1, 0, 2, 1, 0, 2), 0.000000),
    ((2, 1, 1, 1, 0, 2, 0, 1, 2), 0.000000),
    ((2, 1, 1, 0, 1, 2, 0, 0, 0), 0.555556),
    ((2, 1, 1, 0, 1, 2, 2, 0, 0), 0.200000),
    ((2, 1, 1, 0, 1, 2, 2, 1, 0), 1.000000),
    ((2, 1, 1, 0, 1, 2, 2, 0, 1), -0.500000),
    ((2, 1, 1, 0, 1, 2, 2, 2, 1), 0.000000),
    ((2, 1, 1, 0, 1, 2, 0, 2, 0), 0.600000),
    ((2, 1, 1, 0, 1, 2, 1, 2, 0), 1.000000),
    ((2, 1, 1, 0, 1, 2, 0, 2, 1), 0.500000),
    ((2, 1, 1, 0, 1, 2, 0, 0, 2), 1.000000),
    ((2, 1, 1, 0, 1, 2, 1, 0, 2), 1.000000),
    ((2, 1, 1, 0, 1, 2, 0, 1, 2), 1.000000),
    ((2, 1, 1, 0, 0, 2, 1, 0, 0), 0.047619),
    ((2, 1, 1, 0, 0, 2, 1, 2, 0), 0.600000),
    ((2, 1, 1, 0, 0, 2, 1, 2, 1), 0.500000),
    ((2, 1, 1, 0, 0, 2, 1, 0, 2), 0.200000),
    ((2, 1, 1, 0, 0, 2, 1, 1, 2), 0.000000),
    ((2, 1, 1, 0, 0, 2, 0, 1, 0), -0.142857),
    ((2, 1, 1, 0, 0, 2, 2, 1, 0), 0.200000),
    ((2, 1, 1, 0, 0, 2, 2, 1, 1), -0.500000),
    ((2, 1, 1, 0, 0, 2, 0, 1, 2), 0.200000),
    ((2, 1, 1, 0, 0, 2, 0, 0, 1), -0.166667),
    ((2, 1, 1, 0, 0, 2, 2, 0, 1), -0.333333),
    ((2, 1, 1, 0, 0, 2, 0, 2, 1), 0.333333),
    ((2, 1, 1, 0, 0, 0, 2, 0, 0), -0.217391),
    ((2, 1, 1, 1, 0, 0, 2, 0, 0), 0.000000),
    ((2, 1, 1, 1, 0, 0, 2, 2, 0), 0.000000),
    ((2, 1, 1, 1, 1, 0, 2, 2, 0), -0.500000),
    ((2, 1, 1, 1, 1, 0, 2, 2, 2), -1.000000),
    ((2, 1, 1, 1, 0, 1, 2, 2, 0), 0.000000),
    ((2, 1, 1, 1, 0, 1, 2, 2, 2), -1.000000),
    ((2, 1, 1, 1, 0, 0, 2, 2, 1), 0.500000),
    ((2, 1, 1, 1, 0, 0, 2, 0, 2), -0.333333),
    ((2, 1, 1, 1, 1, 0, 2, 0, 2), 0.000000),
    ((2, 1, 1, 1, 0, 1, 2, 0, 2), -1.000000),
    ((2, 1, 1, 1, 0, 0, 2, 1, 2), 0.000000),
    ((2, 1, 1, 0, 1, 0, 2, 0, 0), -0.294118),
    ((2, 1, 1, 0, 1, 0, 2, 2, 0), -0.666667),
    ((2, 1, 1, 0, 1, 1, 2, 2, 0), -1.000000),
    ((2, 1, 1, 0, 1, 1, 2, 2, 2), -1.000000),
    ((2, 1, 1, 0, 1, 0, 2, 2, 1), -0.500000),
    ((2, 1, 1, 0, 1, 0, 2, 0, 2), -0.200000),
    ((2, 1, 1, 0, 1, 1, 2, 0, 2), -1.000000),
    ((2, 1, 1, 0, 1, 0, 2, 1, 2), 1.000000),
    ((2, 1, 1, 0, 0, 1, 2, 0, 0), -0.529412),
    ((2, 1, 1, 0, 0, 1, 2, 2, 0), -0.200000),
    ((2, 1, 1, 0, 0, 1, 2, 2, 1), 1.000000),
    ((2, 1, 1, 0, 0, 1, 2, 0, 2), -1.000000),
    ((2, 1, 1, 0, 0, 1, 2, 1, 2), -1.000000),
    ((2, 1, 1, 0, 0, 0, 2, 1, 0), -0.294118),
    ((2, 1, 1, 0, 0, 0, 2, 1, 2), -0.200000),
    ((2, 1, 1, 0, 0, 0, 2, 0, 1), -0.058824),
    ((2, 1, 1, 0, 0, 0, 2, 2, 1), 0.200000),
    ((2, 1, 1, 0, 0, 0, 0, 2, 0), 0.185185),
    ((2, 1, 1, 1, 0, 0, 0, 2, 0), 0.000000),
    ((2, 1, 1, 1, 0, 0, 0, 2, 2), -0.333333),
    ((2, 1, 1, 1, 1, 0, 0, 2, 2), 0.000000),
    ((2, 1, 1, 1, 0, 1, 0, 2, 2), -1.000000),
    ((2, 1, 1, 1, 0, 0, 1, 2, 2), 0.000000),
    ((2, 1, 1, 0, 1, 0, 0, 2, 0), 0.047619),
    ((2, 1, 1, 0, 1, 0, 0, 2, 2), 0.200000),
    ((2, 1, 1, 0, 1, 1, 0, 2, 2), 0.000000),
    ((2, 1, 1, 0, 1, 0, 1, 2, 2), 1.000000),
    ((2, 1, 1, 0, 0, 1, 0, 2, 0), 0.047619),
    ((2, 1, 1, 0, 0, 1, 0, 2, 2), -0.333333),
    ((2, 1, 1, 0, 0, 1, 1, 2, 2), 0.000000),
    ((2, 1, 1, 0, 0, 0, 1, 2, 0), 0.428571),
    ((2, 1, 1, 0, 0, 0, 1, 2, 2), 0.200000),
    ((2, 1, 1, 0, 0, 0, 0, 2, 1), 0.428571),
    ((2, 1, 1, 0, 0, 0, 0, 0, 2), -0.090909),
    ((2, 1, 1, 1, 0, 0, 0, 0, 2), -0.157895),
    ((2, 1, 1, 0, 1, 0, 0, 0, 2), 0.333333),
    ((2, 1, 1, 0, 0, 1, 0, 0, 2), -0.578947),
    ((2, 1, 1, 0, 0, 0, 1, 0, 2), 0.125000),
    ((2, 1, 1, 0, 0, 0, 0, 1, 2), -0.125000),
    ((2, 1, 0, 1, 0, 0, 0, 0, 0), 0.120253),
    ((2, 1, 2, 1, 0, 0, 0, 0, 0), 0.185185),
    ((2, 1, 2, 1, 1, 0, 0, 0, 0), 0.555556),
    ((2, 1, 2, 1, 1, 2, 0, 0, 0), 0.200000),
    ((2, 1, 2, 1, 1, 2, 1, 0, 0), -0.500000),
    ((2, 1, 2, 1, 1, 2, 1, 2, 0), 0.000000),
    ((2, 1, 2, 1, 1, 2, 1, 2, 1), 0.000000),
    ((2, 1, 2, 1, 1, 2, 1, 0, 2), -1.000000),
    ((2, 1, 2, 1, 1, 2, 0, 1, 0), 1.000000),
    ((2, 1, 2, 1, 1, 2, 0, 0, 1), 0.500000),
    ((2, 1, 2, 1, 1, 2, 2, 0, 1), 1.000000),
    ((2, 1, 2, 1, 1, 2, 2, 1, 1), 1.000000),
    ((2, 1, 2, 1, 1, 2, 0, 2, 1), 0.000000),
    ((2, 1, 2, 1, 1, 0, 2, 0, 0), 1.000000),
    ((2, 1, 2, 1, 1, 1, 2, 0, 0), 1.000000),
    ((2, 1, 2, 1, 1, 0, 2, 1, 0), 1.000000),
    ((2, 1, 2, 1, 1, 0, 2, 0, 1), 1.000000),
    ((2, 1, 2, 1, 1, 0, 2, 2, 1), 1.000000),
    ((2, 1, 2, 1, 1, 1, 2, 2, 1), 1.000000),
    ((2, 1, 2, 1, 1, 0, 0, 2, 0), 0.600000),
    ((2, 1, 2, 1, 1, 1, 0, 2, 0), 1.000000),
    ((2, 1, 2, 1, 1, 0, 1, 2, 0), 0.500000),
    ((2, 1, 2, 1, 1, 0, 1, 2, 2), 1.000000),
    ((2, 1, 2, 1, 1, 1, 1, 2, 2), 1.000000),
    ((2, 1, 2, 1, 1, 0, 0, 2, 1), 0.500000),
    ((2, 1, 2, 1, 1, 0, 0, 0, 2), 0.500000),
    ((2, 1, 2, 1, 1, 1, 0, 0, 2), 1.000000),
    ((2, 1, 2, 1, 1, 0, 1, 0, 2), 0.000000),
    ((2, 1, 2, 1, 1, 0, 0, 1, 2), 1.000000),
    ((2, 1, 2, 1, 0, 1, 0, 0, 0), 0.047619),
    ((2, 1, 2, 1, 2, 1, 0, 0, 0), -0.666667),
    ((2, 1, 2, 1, 2, 1, 1, 0, 0), -0.500000),
    ((2, 1, 2, 1, 2, 1, 1, 2, 0), 0.000000),
    ((2, 1, 2, 1, 2, 1, 1, 2, 1), 0.000000),
    ((2, 1, 2, 1, 2, 1, 1, 0, 2), -1.000000),
    ((2, 1, 2, 1, 2, 1, 0, 1, 0), -1.000000),
    ((2, 1, 2, 1, 2, 1, 2, 1, 0), -1.000000),
    ((2, 1, 2, 1, 2, 1, 0, 1, 2), -1.000000),
    ((2, 1, 2, 1, 2, 1, 0, 0, 1), -0.500000),
    ((2, 1, 2, 1, 2, 1, 2, 0, 1), -1.000000),
    ((2, 1, 2, 1, 2, 1, 0, 2, 1), 0.000000),
    ((2, 1, 2, 1, 0, 1, 2, 0, 0), 0.200000),
    ((2, 1, 2, 1, 0, 1, 2, 1, 0), 0.000000),
    ((2, 1, 2, 1, 0, 1, 2, 1, 2), 1.000000),
    ((2, 1, 2, 1, 1, 1, 2, 1, 2), 1.000000),
    ((2, 1, 2, 1, 0, 1, 2, 0, 1), 0.000000),
    ((2, 1, 2, 1, 0, 1, 2, 2, 1), 1.000000),
    ((2, 1, 2, 1, 0, 1, 0, 2, 0), 0.600000),
    ((2, 1, 2, 1, 0, 1, 1, 2, 0), 0.500000),
    ((2, 1, 2, 1, 0, 1, 1, 2, 2), 1.000000),
    ((2, 1, 2, 1, 0, 1, 0, 2, 1), 0.500000),
    ((2, 1, 2, 1, 0, 1, 0, 0, 2), 0.200000),
    ((2, 1, 2, 1, 0, 1, 1, 0, 2), 0.000000),
    ((2, 1, 2, 1, 0, 1, 0, 1, 2), 0.000000),
    ((2, 1, 2, 1, 0, 0, 1, 0, 0), 0.000000),
    ((2, 1, 2, 1, 2, 0, 1, 0, 0), 0.000000),
    ((2, 1, 2, 1, 2, 0, 1, 1, 0), 0.000000),
    ((2, 1, 2, 1, 2, 2, 1, 1, 0), 1.000000),
    ((2, 1, 2, 1, 2, 2, 1, 1, 1), 1.000000),
    ((2, 1, 2, 1, 2, 0, 1, 1, 2), -1.000000),
    ((2, 1, 2, 1, 2, 0, 1, 0, 1), 0.500000),
    ((2, 1, 2, 1, 2, 2, 1, 0, 1), 1.000000),
    ((2, 1, 2, 1, 2, 0, 1, 2, 1), 0.000000),
    ((2, 1, 2, 1, 0, 2, 1, 0, 0), 0.000000),
    ((2, 1, 2, 1, 0, 2, 1, 1, 0), 0.000000),
    ((2, 1, 2, 1, 0, 2, 1, 1, 2), -1.000000),
    ((2, 1, 2, 1, 0, 2, 1, 0, 1), 0.500000),
    ((2, 1, 2, 1, 0, 2, 1, 2, 1), 0.000000),
    ((2, 1, 2, 1, 0, 0, 1, 2, 0), 0.333333),
    ((2, 1, 2, 1, 0, 0, 1, 2, 1), 0.000000),
    ((2, 1, 2, 1, 0, 0, 1, 0, 2), -0.333333),
    ((2, 1, 2, 1, 0, 0, 1, 1, 2), -1.000000),
    ((2, 1, 2, 1, 0, 0, 0, 1, 0), 0.047619),
    ((2, 1, 2, 1, 2, 0, 0, 1, 0), -0.333333),
    ((2, 1, 2, 1, 2, 0, 0, 1, 1), 0.000000),
    ((2, 1, 2, 1, 2, 2, 0, 1, 1), 1.000000),
    ((2, 1, 2, 1, 2, 0, 2, 1, 1), -1.000000),
    ((2, 1, 2, 1, 0, 2, 0, 1, 0), 0.600000),
    ((2, 1, 2, 1, 0, 2, 0, 1, 1), 1.000000),
    ((2, 1, 2, 1, 0, 2, 2, 1, 1), 1.000000),
    ((2, 1, 2, 1, 0, 0, 2, 1, 0), 0.200000),
    ((2, 1, 2, 1, 0, 0, 2, 1, 1), 0.000000),
    ((2, 1, 2, 1, 0, 0, 0, 1, 2), -0.200000),
    ((2, 1, 2, 1, 0, 0, 0, 0, 1), 0.333333),
    ((2, 1, 2, 1, 2, 0, 0, 0, 1), 0.000000),
    ((2, 1, 2, 1, 0, 2, 0, 0, 1), 0.666667),
    ((2, 1, 2, 1, 0, 0, 2, 0, 1), 0.333333),
    ((2, 1, 2, 1, 0, 0, 0, 2, 1), 0.333333),
    ((2, 1, 0, 1, 2, 0, 0, 0, 0), -0.160000),
    ((2, 1, 0, 1, 2, 1, 0, 0, 0), -0.368421),
    ((2, 1, 0, 1, 2, 1, 2, 0, 0), -0.333333),
    ((2, 1, 0, 1, 2, 1, 2, 1, 0), -1.000000),
    ((2, 1, 0, 1, 2, 1, 2, 1, 2), -1.000000),
    ((2, 1, 0, 1, 2, 1, 2, 0, 1), 0.000000),
    ((2, 1, 0, 1, 2, 1, 2, 2, 1), 1.000000),
    ((2, 1, 0, 1, 2, 1, 0, 2, 0), 0.000000),
    ((2, 1, 0, 1, 2, 1, 1, 2, 0), -0.500000),
    ((2, 1, 0, 1, 2, 1, 1, 2, 2), -1.000000),
    ((2, 1, 0, 1, 2, 1, 0, 2, 1), 0.500000),
    ((2, 1, 0, 1, 2, 1, 0, 0, 2), -1.000000),
    ((2, 1, 0, 1, 2, 0, 1, 0, 0), -0.157895),
    ((2, 1, 0, 1, 2, 2, 1, 0, 0), 0.000000),
    ((2, 1, 0, 1, 2, 2, 1, 1, 0), 0.000000),
    ((2, 1, 0, 1, 2, 2, 1, 1, 2), -1.000000),
    ((2, 1, 0, 1, 2, 2, 1, 0, 1), 0.500000),
    ((2, 1, 0, 1, 2, 2, 1, 2, 1), 0.000000),
    ((2, 1, 0, 1, 2, 0, 1, 2, 0), -0.333333),
    ((2, 1, 0, 1, 2, 0, 1, 2, 1), 0.000000),
    ((2, 1, 0, 1, 2, 0, 1, 0, 2), -1.000000),
    ((2, 1, 0, 1, 2, 0, 0, 1, 0), -0.368421),
    ((2, 1, 0, 1, 2, 2, 0, 1, 0), 0.000000),
    ((2, 1, 0, 1, 2, 2, 0, 1, 1), 0.500000),
    ((2, 1, 0, 1, 2, 2, 2, 1, 1), 0.000000),
    ((2, 1, 0, 1, 2, 0, 2, 1, 0), -0.666667),
    ((2, 1, 0, 1, 2, 0, 2, 1, 1), -0.500000),
    ((2, 1, 0, 1, 2, 0, 0, 1, 2), -1.000000),
    ((2, 1, 0, 1, 2, 0, 0, 0, 1), 0.166667),
    ((2, 1, 0, 1, 2, 2, 0, 0, 1), 0.333333),
    ((2, 1, 0, 1, 2, 0, 2, 0, 1), 0.000000),
    ((2, 1, 0, 1, 2, 0, 0, 2, 1), 0.333333),
    ((2, 1, 0, 1, 0, 2, 0, 0, 0), 0.228070),
    ((2, 1, 0, 1, 1, 2, 0, 0, 0), 0.428571),
    ((2, 1, 0, 1, 1, 2, 2, 0, 0), 0.600000),
    ((2, 1, 0, 1, 1, 2, 2, 1, 0), 1.000000),
    ((2, 1, 0, 1, 1, 2, 2, 0, 1), 0.500000),
    ((2, 1, 0, 1, 1, 2, 2, 2, 1), 0.000000),
    ((2, 1, 0, 1, 1, 2, 0, 2, 0), 0.333333),
    ((2, 1, 0, 1, 1, 2, 1, 2, 0), 0.500000),
    ((2, 1, 0, 1, 1, 2, 1, 2, 2), 1.000000),
    ((2, 1, 0, 1, 1, 2, 0, 2, 1), 0.000000),
    ((2, 1, 0, 1, 1, 2, 0, 0, 2), 0.600000),
    ((2, 1, 0, 1, 1, 2, 1, 0, 2), 0.000000),
    ((2, 1, 0, 1, 1, 2, 0, 1, 2), 1.000000),
    ((2, 1, 0, 1, 0, 2, 1, 0, 0), 0.000000),
    ((2, 1, 0, 1, 0, 2, 1, 2, 0), 0.333333),
    ((2, 1, 0, 1, 0, 2, 1, 2, 1), 0.000000),
    ((2, 1, 0, 1, 0, 2, 1, 0, 2), -0.333333),
    ((2, 1, 0, 1, 0, 2, 1, 1, 2), -1.000000),
    ((2, 1, 0, 1, 0, 2, 0, 1, 0), 0.238095),
    ((2, 1, 0, 1, 0, 2, 2, 1, 0), 0.600000),
    ((2, 1, 0, 1, 0, 2, 2, 1, 1), 0.500000),
    ((2, 1, 0, 1, 0, 2, 0, 1, 2), -0.200000),
    ((2, 1, 0, 1, 0, 2, 0, 0, 1), 0.333333),
    ((2, 1, 0, 1, 0, 2, 2, 0, 1), 0.333333),
    ((2, 1, 0, 1, 0, 2, 0, 2, 1), 0.000000),
    ((2, 1, 0, 1, 0, 0, 2, 0, 0), 0.185185),
    ((2, 1, 0, 1, 1, 0, 2, 0, 0), 0.555556),
    ((2, 1, 0, 1, 1, 0, 2, 2, 0), 0.200000),
    ((2, 1, 0, 1, 1, 1, 2, 2, 0), 1.000000),
    ((2, 1, 0, 1, 1, 0, 2, 2, 1), 0.500000),
    ((2, 1, 0, 1, 1, 0, 2, 0, 2), 0.500000),
    ((2, 1, 0, 1, 1, 1, 2, 0, 2), 1.000000),
    ((2, 1, 0, 1, 1, 0, 2, 1, 2), 1.000000),
    ((2, 1, 0, 1, 0, 1, 2, 0, 0), 0.047619),
    ((2, 1, 0, 1, 0, 1, 2, 2, 0), 0.600000),
    ((2, 1, 0, 1, 0, 1, 2, 2, 1), 1.000000),
    ((2, 1, 0, 1, 0, 1, 2, 0, 2), -0.200000),
    ((2, 1, 0, 1, 0, 1, 2, 1, 2), 0.000000),
    ((2, 1, 0, 1, 0, 0, 2, 1, 0), 0.047619),
    ((2, 1, 0, 1, 0, 0, 2, 1, 2), 0.200000),
    ((2, 1, 0, 1, 0, 0, 2, 0, 1), 0.333333),
    ((2, 1, 0, 1, 0, 0, 2, 2, 1), 0.666667),
    ((2, 1, 0, 1, 0, 0, 0, 2, 0), 0.228070),
    ((2, 1, 0, 1, 1, 0, 0, 2, 0), 0.428571),
    ((2, 1, 0, 1, 1, 0, 0, 2, 2), 0.600000),
    ((2, 1, 0, 1, 1, 1, 0, 2, 2), 1.000000),
    ((2, 1, 0, 1, 1, 0, 1, 2, 2), 1.000000),
    ((2, 1, 0, 1, 0, 1, 0, 2, 0), 0.238095),
    ((2, 1, 0, 1, 0, 1, 0, 2, 2), -0.200000),
    ((2, 1, 0, 1, 0, 1, 1, 2, 2), 0.000000),
    ((2, 1, 0, 1, 0, 0, 1, 2, 0), 0.166667),
    ((2, 1, 0, 1, 0, 0, 1, 2, 2), 0.333333),
    ((2, 1, 0, 1, 0, 0, 0, 2, 1), 0.333333),
    ((2, 1, 0, 1, 0, 0, 0, 0, 2), 0.000000),
    ((2, 1, 0, 1, 1, 0, 0, 0, 2), 0.555556),
    ((2, 1, 0, 1, 0, 1, 0, 0, 2), -0.125000),
    ((2, 1, 0, 1, 0, 0, 1, 0, 2), -0.157895),
    ((2, 1, 0, 1, 0, 0, 0, 1, 2), -0.125000),
    ((2, 1, 0, 0, 1, 0, 0, 0, 0), 0.391121),
    ((2, 1, 2, 0, 1, 0, 0, 0, 0), 0.493671),
    ((2, 1, 2, 0, 1, 1, 0, 0, 0), 0.555556),
    ((2, 1, 2, 2, 1, 1, 0, 0, 0), 0.200000),
    ((2, 1, 2, 2, 1, 1, 1, 0, 0), 0.500000),
    ((2, 1, 2, 2, 1, 1, 1, 2, 0), 0.000000),
    ((2, 1, 2, 2, 1, 1, 1, 2, 1), 0.000000),
    ((2, 1, 2, 2, 1, 1, 1, 0, 2), 1.000000),
    ((2, 1, 2, 2, 1, 1, 1, 1, 2), 1.000000),
    ((2, 1, 2, 2, 1, 1, 0, 1, 0), 1.000000),
    ((2, 1, 2, 2, 1, 1, 0, 0, 1), -0.500000),
    ((2, 1, 2, 2, 1, 1, 2, 0, 1), -1.000000),
    ((2, 1, 2, 2, 1, 1, 0, 2, 1), 0.000000),
    ((2, 1, 2, 0, 1, 1, 2, 0, 0), 0.500000),
    ((2, 1, 2, 0, 1, 1, 2, 1, 0), 1.000000),
    ((2, 1, 2, 0, 1, 1, 2, 0, 1), 0.000000),
    ((2, 1, 2, 0, 1, 1, 2, 2, 1), 1.000000),
    ((2, 1, 2, 0, 1, 1, 0, 2, 0), 0.600000),
    ((2, 1, 2, 0, 1, 1, 1, 2, 0), 0.500000),
    ((2, 1, 2, 0, 1, 1, 1, 2, 2), 1.000000),
    ((2, 1, 2, 0, 1, 1, 0, 2, 1), 0.500000),
    ((2, 1, 2, 0, 1, 1, 0, 0, 2), 1.000000),
    ((2, 1, 2, 0, 1, 1, 1, 0, 2), 1.000000),
    ((2, 1, 2, 0, 1, 1, 0, 1, 2), 1.000000),
    ((2, 1, 2, 0, 1, 0, 1, 0, 0), 0.428571),
    ((2, 1, 2, 2, 1, 0, 1, 0, 0), 0.600000),
    ((2, 1, 2, 2, 1, 0, 1, 1, 0), 1.000000),
    ((2, 1, 2, 2, 1, 0, 1, 0, 1), 0.500000),
    ((2, 1, 2, 2, 1, 2, 1, 0, 1), 1.000000),
    ((2, 1, 2, 2, 1, 2, 1, 1, 1), 1.000000),
    ((2, 1, 2, 2, 1, 0, 1, 2, 1), 0.000000),
    ((2, 1, 2, 0, 1, 2, 1, 0, 0), 0.200000),
    ((2, 1, 2, 0, 1, 2, 1, 1, 0), 1.000000),
    ((2, 1, 2, 0, 1, 2, 1, 0, 1), 0.500000),
    ((2, 1, 2, 0, 1, 2, 1, 2, 1), 0.000000),
    ((2, 1, 2, 0, 1, 0, 1, 2, 0), 0.333333),
    ((2, 1, 2, 0, 1, 0, 1, 2, 1), 0.000000),
    ((2, 1, 2, 0, 1, 0, 1, 0, 2), 0.600000),
    ((2, 1, 2, 0, 1, 0, 1, 1, 2), 1.000000),
    ((2, 1, 2, 0, 1, 0, 0, 1, 0), 1.000000),
    ((2, 1, 2, 0, 1, 0, 0, 0, 1), 0.428571),
    ((2, 1, 2, 2, 1, 0, 0, 0, 1), 0.200000),
    ((2, 1, 2, 2, 1, 0, 0, 1, 1), 1.000000),
    ((2, 1, 2, 0, 1, 2, 0, 0, 1), 0.600000),
    ((2, 1, 2, 0, 1, 2, 0, 1, 1), 1.000000),
    ((2, 1, 2, 0, 1, 0, 2, 0, 1), 0.600000),
    ((2, 1, 2, 0, 1, 0, 2, 1, 1), 1.000000),
    ((2, 1, 2, 0, 1, 0, 0, 2, 1), 0.333333),
    ((2, 1, 0, 2, 1, 0, 0, 0, 0), 0.373134),
    ((2, 1, 0, 2, 1, 1, 0, 0, 0), 0.176471),
    ((2, 1, 0, 2, 1, 1, 2, 0, 0), -1.000000),
    ((2, 1, 0, 2, 1, 1, 0, 2, 0), 0.000000),
    ((2, 1, 0, 2, 1, 1, 1, 2, 0), 0.500000),
    ((2, 1, 0, 2, 1, 1, 1, 2, 2), 1.000000),
    ((2, 1, 0, 2, 1, 1, 0, 2, 1), -0.500000),
    ((2, 1, 0, 2, 1, 1, 2, 2, 1), -1.000000),
    ((2, 1, 0, 2, 1, 1, 0, 0, 2), 0.600000),
    ((2, 1, 0, 2, 1, 1, 1, 0, 2), 1.000000),
    ((2, 1, 0, 2, 1, 1, 0, 1, 2), 1.000000),
    ((2, 1, 0, 2, 1, 0, 1, 0, 0), 0.777778),
    ((2, 1, 0, 2, 1, 2, 1, 0, 0), 1.000000),
    ((2, 1, 0, 2, 1, 2, 1, 1, 0), 1.000000),
    ((2, 1, 0, 2, 1, 2, 1, 0, 1), 1.000000),
    ((2, 1, 0, 2, 1, 2, 1, 2, 1), 1.000000),
    ((2, 1, 0, 2, 1, 0, 1, 2, 0), 0.600000),
    ((2, 1, 0, 2, 1, 0, 1, 2, 1), 0.500000),
    ((2, 1, 0, 2, 1, 0, 1, 0, 2), 1.000000),
    ((2, 1, 0, 2, 1, 0, 1, 1, 2), 1.000000),
    ((2, 1, 0, 2, 1, 0, 0, 1, 0), 1.000000),
    ((2, 1, 0, 2, 1, 0, 0, 0, 1), 0.176471),
    ((2, 1, 0, 2, 1, 2, 0, 0, 1), 0.600000),
    ((2, 1, 0, 2, 1, 2, 0, 1, 1), 1.000000),
    ((2, 1, 0, 2, 1, 0, 2, 0, 1), -1.000000),
    ((2, 1, 0, 2, 1, 0, 0, 2, 1), 0.000000),
    ((2, 1, 0, 0, 1, 2, 0, 0, 0), 0.493671),
    ((2, 1, 0, 0, 1, 2, 1, 0, 0), 0.555556),
    ((2, 1, 0, 0, 1, 2, 1, 2, 0), 0.600000),
    ((2, 1, 0, 0, 1, 2, 1, 2, 1), 0.500000),
    ((2, 1, 0, 0, 1, 2, 1, 0, 2), 0.500000),
    ((2, 1, 0, 0, 1, 2, 1, 1, 2), 1.000000),
    ((2, 1, 0, 0, 1, 2, 0, 1, 0), 1.000000),
    ((2, 1, 0, 0, 1, 2, 0, 0, 1), 0.428571),
    ((2, 1, 0, 0, 1, 2, 2, 0, 1), 0.200000),
    ((2, 1, 0, 0, 1, 2, 2, 1, 1), 1.000000),
    ((2, 1, 0, 0, 1, 2, 0, 2, 1), 0.333333),
    ((2, 1, 0, 0, 1, 0, 2, 0, 0), 0.134328),
    ((2, 1, 0, 0, 1, 1, 2, 0, 0), 0.000000),
    ((2, 1, 0, 0, 1, 1, 2, 2, 0), -0.200000),
    ((2, 1, 0, 0, 1, 1, 2, 2, 1), 0.000000),
    ((2, 1, 0, 0, 1, 1, 2, 0, 2), 0.000000),
    ((2, 1, 0, 0, 1, 1, 2, 1, 2), 1.000000),
    ((2, 1, 0, 0, 1, 0, 2, 1, 0), 1.000000),
    ((2, 1, 0, 0, 1, 0, 2, 0, 1), 0.176471),
    ((2, 1, 0, 0, 1, 0, 2, 2, 1), 0.000000),
    ((2, 1, 0, 0, 1, 0, 0, 2, 0), 0.296296),
    ((2, 1, 0, 0, 1, 1, 0, 2, 0), 0.238095),
    ((2, 1, 0, 0, 1, 1, 0, 2, 2), 0.600000),
    ((2, 1, 0, 0, 1, 1, 1, 2, 2), 1.000000),
    ((2, 1, 0, 0, 1, 0, 1, 2, 0), 0.619048),
    ((2, 1, 0, 0, 1, 0, 1, 2, 2), 1.000000),
    ((2, 1, 0, 0, 1, 0, 0, 2, 1), 0.166667),
    ((2, 1, 0, 0, 1, 0, 0, 0, 2), 0.561644),
    ((2, 1, 0, 0, 1, 1, 0, 0, 2), 0.555556),
    ((2, 1, 0, 0, 1, 0, 1, 0, 2), 0.777778),
    ((2, 1, 0, 0, 1, 0, 0, 1, 2), 1.000000),
    ((2, 1, 0, 0, 0, 1, 0, 0, 0), 0.055556),
    ((2, 1, 2, 0, 0, 1, 0, 0, 0), 0.185185),
    ((2, 1, 2, 0, 0, 1, 1, 0, 0), 0.333333),
    ((2, 1, 2, 2, 0, 1, 1, 0, 0), 0.666667),
    ((2, 1, 2, 2, 0, 1, 1, 1, 0), 1.000000),
    ((2, 1, 2, 2, 2, 1, 1, 1, 0), 1.000000),
    ((2, 1, 2, 2, 2, 1, 1, 1, 1), 1.000000),
    ((2, 1, 2, 2, 0, 1, 1, 1, 2), 1.000000),
    ((2, 1, 2, 2, 0, 1, 1, 0, 1), 0.500000),
    ((2, 1, 2, 2, 2, 1, 1, 0, 1), 1.000000),
    ((2, 1, 2, 2, 0, 1, 1, 2, 1), 0.000000),
    ((2, 1, 2, 0, 2, 1, 1, 0, 0), 0.000000),
    ((2, 1, 2, 0, 2, 1, 1, 1, 0), 0.000000),
    ((2, 1, 2, 0, 2, 1, 1, 1, 2), -1.000000),
    ((2, 1, 2, 0, 2, 1, 1, 0, 1), 0.500000),
    ((2, 1, 2, 0, 2, 1, 1, 2, 1), 0.000000),
    ((2, 1, 2, 0, 0, 1, 1, 2, 0), 0.333333),
    ((2, 1, 2, 0, 0, 1, 1, 2, 1), 0.000000),
    ((2, 1, 2, 0, 0, 1, 1, 0, 2), 0.333333),
    ((2, 1, 2, 0, 0, 1, 1, 1, 2), 0.000000),
    ((2, 1, 2, 0, 0, 1, 0, 1, 0), 0.047619),
    ((2, 1, 2, 2, 0, 1, 0, 1, 0), 0.600000),
    ((2, 1, 2, 2, 0, 1, 0, 1, 1), 0.000000),
    ((2, 1, 2, 2, 2, 1, 0, 1, 1), 1.000000),
    ((2, 1, 2, 2, 0, 1, 2, 1, 1), -1.000000),
    ((2, 1, 2, 0, 2, 1, 0, 1, 0), -0.333333),
    ((2, 1, 2, 0, 2, 1, 0, 1, 1), 0.000000),
    ((2, 1, 2, 0, 2, 1, 2, 1, 1), -1.000000),
    ((2, 1, 2, 0, 0, 1, 2, 1, 0), -0.200000),
    ((2, 1, 2, 0, 0, 1, 2, 1, 1), -1.000000),
    ((2, 1, 2, 0, 0, 1, 0, 1, 2), 0.200000),
    ((2, 1, 2, 0, 0, 1, 0, 0, 1), 0.000000),
    ((2, 1, 2, 2, 0, 1, 0, 0, 1), 0.000000),
    ((2, 1, 2, 0, 2, 1, 0, 0, 1), 0.000000),
    ((2, 1, 2, 0, 0, 1, 2, 0, 1), -0.333333),
    ((2, 1, 2, 0, 0, 1, 0, 2, 1), 0.333333),
    ((2, 1, 0, 2, 0, 1, 0, 0, 0), 0.173913),
    ((2, 1, 0, 2, 0, 1, 1, 0, 0), 0.500000),
    ((2, 1, 0, 2, 2, 1, 1, 0, 0), 0.333333),
    ((2, 1, 0, 2, 2, 1, 1, 1, 0), 0.000000),
    ((2, 1, 0, 2, 2, 1, 1, 1, 2), -1.000000),
    ((2, 1, 0, 2, 2, 1, 1, 0, 1), 1.000000),
    ((2, 1, 0, 2, 2, 1, 1, 2, 1), 1.000000),
    ((2, 1, 0, 2, 0, 1, 1, 2, 0), 0.666667),
    ((2, 1, 0, 2, 0, 1, 1, 2, 1), 0.500000),
    ((2, 1, 0, 2, 0, 1, 1, 0, 2), 0.333333),
    ((2, 1, 0, 2, 0, 1, 1, 1, 2), 0.000000),
    ((2, 1, 0, 2, 0, 1, 0, 1, 0), -0.058824),
    ((2, 1, 0, 2, 2, 1, 0, 1, 0), -0.333333),
    ((2, 1, 0, 2, 2, 1, 0, 1, 1), 0.000000),
    ((2, 1, 0, 2, 2, 1, 2, 1, 1), -1.000000),
    ((2, 1, 0, 2, 0, 1, 2, 1, 0), -1.000000),
    ((2, 1, 0, 2, 0, 1, 0, 1, 2), -0.200000),
    ((2, 1, 0, 2, 0, 1, 0, 0, 1), 0.176471),
    ((2, 1, 0, 2, 2, 1, 0, 0, 1), 0.600000),
    ((2, 1, 0, 2, 0, 1, 2, 0, 1), -1.000000),
    ((2, 1, 0, 2, 0, 1, 0, 2, 1), 0.200000),
    ((2, 1, 0, 0, 2, 1, 0, 0, 0), -0.148936),
    ((2, 1, 0, 0, 2, 1, 1, 0, 0), 0.052632),
    ((2, 1, 0, 0, 2, 1, 1, 2, 0), 0.000000),
    ((2, 1, 0, 0, 2, 1, 1, 2, 1), 0.500000),
    ((2, 1, 0, 0, 2, 1, 1, 0, 2), -1.000000),
    ((2, 1, 0, 0, 2, 1, 0, 1, 0), -0.578947),
    ((2, 1, 0, 0, 2, 1, 2, 1, 0), -1.000000),
    ((2, 1, 0, 0, 2, 1, 2, 1, 1), -1.000000),
    ((2, 1, 0, 0, 2, 1, 0, 1, 2), -1.000000),
    ((2, 1, 0, 0, 2, 1, 0, 0, 1), 0.238095),
    ((2, 1, 0, 0, 2, 1, 2, 0, 1), -0.200000),
    ((2, 1, 0, 0, 2, 1, 0, 2, 1), 0.600000),
    ((2, 1, 0, 0, 0, 1, 2, 0, 0), -0.209302),
    ((2, 1, 0, 0, 0, 1, 2, 1, 0), -0.529412),
    ((2, 1, 0, 0, 0, 1, 2, 1, 2), -0.200000),
    ((2, 1, 0, 0, 0, 1, 2, 0, 1), -0.058824),
    ((2, 1, 0, 0, 0, 1, 2, 2, 1), 0.600000),
    ((2, 1, 0, 0, 0, 1, 0, 2, 0), 0.259259),
    ((2, 1, 0, 0, 0, 1, 1, 2, 0), 0.333333),
    ((2, 1, 0, 0, 0, 1, 1, 2, 2), 0.333333),
    ((2, 1, 0, 0, 0, 1, 0, 2, 1), 0.428571),
    ((2, 1, 0, 0, 0, 1, 0, 0, 2), 0.000000),
    ((2, 1, 0, 0, 0, 1, 1, 0, 2), 0.263158),
    ((2, 1, 0, 0, 0, 1, 0, 1, 2), -0.125000),
    ((2, 1, 0, 0, 0, 0, 1, 0, 0), 0.263158),
    ((2, 1, 2, 0, 0, 0, 1, 0, 0), 0.296296),
    ((2, 1, 2, 0, 0, 0, 1, 1, 0), 0.333333),
    ((2, 1, 2, 2, 0, 0, 1, 1, 0), 1.000000),
    ((2, 1, 2, 2, 0, 0, 1, 1, 1), 1.000000),
    ((2, 1, 2, 0, 2, 0, 1, 1, 0), 0.200000),
    ((2, 1, 2, 0, 2, 0, 1, 1, 1), 1.000000),
    ((2, 1, 2, 0, 0, 2, 1, 1, 0), 0.500000),
    ((2, 1, 2, 0, 0, 2, 1, 1, 1), 1.000000),
    ((2, 1, 2, 0, 0, 0, 1, 1, 2), -0.200000),
    ((2, 1, 2, 0, 0, 0, 1, 0, 1), 0.428571),
    ((2, 1, 2, 2, 0, 0, 1, 0, 1), 0.600000),
    ((2, 1, 2, 0, 2, 0, 1, 0, 1), 0.600000),
    ((2, 1, 2, 0, 0, 2, 1, 0, 1), 0.600000),
    ((2, 1, 2, 0, 0, 0, 1, 2, 1), 0.000000),
    ((2, 1, 0, 2, 0, 0, 1, 0, 0), 0.490196),
    ((2, 1, 0, 2, 0, 0, 1, 1, 0), 0.333333),
    ((2, 1, 0, 2, 2, 0, 1, 1, 0), -0.200000),
    ((2, 1, 0, 2, 2, 0, 1, 1, 1), 1.000000),
    ((2, 1, 0, 2, 0, 2, 1, 1, 0), 0.500000),
    ((2, 1, 0, 2, 0, 2, 1, 1, 1), 1.000000),
    ((2, 1, 0, 2, 0, 0, 1, 1, 2), 0.200000),
    ((2, 1, 0, 2, 0, 0, 1, 0, 1), 0.619048),
    ((2, 1, 0, 2, 2, 0, 1, 0, 1), 0.600000),
    ((2, 1, 0, 2, 0, 2, 1, 0, 1), 0.600000),
    ((2, 1, 0, 2, 0, 0, 1, 2, 1), 0.666667),
    ((2, 1, 0, 0, 2, 0, 1, 0, 0), -0.021277),
    ((2, 1, 0, 0, 2, 0, 1, 1, 0), -0.125000),
    ((2, 1, 0, 0, 2, 2, 1, 1, 0), -0.200000),
    ((2, 1, 0, 0, 2, 2, 1, 1, 1), 1.000000),
    ((2, 1, 0, 0, 2, 0, 1, 1, 2), -1.000000),
    ((2, 1, 0, 0, 2, 0, 1, 0, 1), 0.428571),
    ((2, 1, 0, 0, 2, 2, 1, 0, 1), 0.200000),
    ((2, 1, 0, 0, 2, 0, 1, 2, 1), 0.333333),
    ((2, 1, 0, 0, 0, 2, 1, 0, 0), 0.215686),
    ((2, 1, 0, 0, 0, 2, 1, 1, 0), 0.111111),
    ((2, 1, 0, 0, 0, 2, 1, 1, 2), -0.200000),
    ((2, 1, 0, 0, 0, 2, 1, 0, 1), 0.428571),
    ((2, 1, 0, 0, 0, 2, 1, 2, 1), 0.333333),
    ((2, 1, 0, 0, 0, 0, 1, 2, 0), 0.368421),
    ((2, 1, 0, 0, 0, 0, 1, 2, 1), 0.333333),
    ((2, 1, 0, 0, 0, 0, 1, 0, 2), 0.181818),
    ((2, 1, 0, 0, 0, 0, 1, 1, 2), -0.125000),
    ((2, 1, 0, 0, 0, 0, 0, 1, 0), -0.085339),
    ((2, 1, 2, 0, 0, 0, 0, 1, 0), 0.189873),
    ((2, 1, 2, 0, 0, 0, 0, 1, 1), 0.333333),
    ((2, 1, 2, 2, 0, 0, 0, 1, 1), 0.500000),
    ((2, 1, 2, 0, 2, 0, 0, 1, 1), 0.200000),
    ((2, 1, 2, 0, 0, 2, 0, 1, 1), 1.000000),
    ((2, 1, 2, 0, 0, 0, 2, 1, 1), -0.200000),
    ((2, 1, 0, 2, 0, 0, 0, 1, 0), -0.044776),
    ((2, 1, 0, 2, 0, 0, 0, 1, 1), 0.000000),
    ((2, 1, 0, 2, 2, 0, 0, 1, 1), -0.200000),
    ((2, 1, 0, 2, 0, 2, 0, 1, 1), 0.000000),
    ((2, 1, 0, 2, 0, 0, 2, 1, 1), -1.000000),
    ((2, 1, 0, 0, 2, 0, 0, 1, 0), -0.404255),
    ((2, 1, 0, 0, 2, 0, 0, 1, 1), -0.142857),
    ((2, 1, 0, 0, 2, 2, 0, 1, 1), 0.200000),
    ((2, 1, 0, 0, 2, 0, 2, 1, 1), -0.666667),
    ((2, 1, 0, 0, 0, 2, 0, 1, 0), 0.139241),
    ((2, 1, 0, 0, 0, 2, 0, 1, 1), 0.333333),
    ((2, 1, 0, 0, 0, 2, 2, 1, 1), 0.200000),
    ((2, 1, 0, 0, 0, 0, 2, 1, 0), -0.232877),
    ((2, 1, 0, 0, 0, 0, 2, 1, 1), -0.294118),
    ((2, 1, 0, 0, 0, 0, 0, 1, 2), -0.107692),
    ((2, 1, 0, 0, 0, 0, 0, 0, 1), 0.220779),
    ((2, 1, 2, 0, 0, 0, 0, 0, 1), 0.296296),
    ((2, 1, 0, 2, 0, 0, 0, 0, 1), 0.209302),
    ((2, 1, 0, 0, 2, 0, 0, 0, 1), 0.148148),
    ((2, 1, 0, 0, 0, 2, 0, 0, 1), 0.259259),
    ((2, 1, 0, 0, 0, 0, 2, 0, 1), 0.043478),
    ((2, 1, 0, 0, 0, 0, 0, 2, 1), 0.333333),
    ((0, 1, 2, 0, 0, 0, 0, 0, 0), 0.132555),
    ((0, 1, 2, 1, 0, 0, 0, 0, 0), 0.055556),
    ((0, 1, 2, 1, 2, 0, 0, 0, 0), -0.148936),
    ((0, 1, 2, 1, 2, 1, 0, 0, 0), -0.368421),
    ((0, 1, 2, 1, 2, 1, 2, 0, 0), -1.000000),
    ((0, 1, 2, 1, 2, 1, 0, 2, 0), 0.000000),
    ((0, 1, 2, 1, 2, 1, 1, 2, 0), 0.500000),
    ((0, 1, 2, 1, 2, 1, 1, 2, 2), 1.000000),
    ((0, 1, 2, 1, 2, 1, 0, 2, 1), -0.500000),
    ((0, 1, 2, 1, 2, 1, 2, 2, 1), -1.000000),
    ((0, 1, 2, 1, 2, 1, 0, 0, 2), -0.333333),
    ((0, 1, 2, 1, 2, 1, 1, 0, 2), 0.000000),
    ((0, 1, 2, 1, 2, 1, 0, 1, 2), -1.000000),
    ((0, 1, 2, 1, 2, 1, 2, 1, 2), -1.000000),
    ((0, 1, 2, 1, 2, 0, 1, 0, 0), 0.238095),
    ((0, 1, 2, 1, 2, 2, 1, 0, 0), 0.600000),
    ((0, 1, 2, 1, 2, 2, 1, 1, 0), 0.000000),
    ((0, 1, 2, 1, 2, 2, 1, 1, 2), -1.000000),
    ((0, 1, 2, 1, 2, 2, 1, 0, 1), 1.000000),
    ((0, 1, 2, 1, 2, 2, 1, 2, 1), 1.000000),
    ((0, 1, 2, 1, 2, 0, 1, 2, 0), 0.600000),
    ((0, 1, 2, 1, 2, 0, 1, 2, 1), 0.500000),
    ((0, 1, 2, 1, 2, 0, 1, 0, 2), -0.200000),
    ((0, 1, 2, 1, 2, 0, 1, 1, 2), -1.000000),
    ((0, 1, 2, 1, 2, 0, 0, 1, 0), -0.578947),
    ((0, 1, 2, 1, 2, 2, 0, 1, 0), -0.333333),
    ((0, 1, 2, 1, 2, 2, 0, 1, 1), 0.000000),
    ((0, 1, 2, 1, 2, 2, 2, 1, 1), -1.000000),
    ((0, 1, 2, 1, 2, 0, 2, 1, 0), -1.000000),
    ((0, 1, 2, 1, 2, 0, 0, 1, 2), -1.000000),
    ((0, 1, 2, 1, 2, 0, 0, 0, 1), 0.052632),
    ((0, 1, 2, 1, 2, 2, 0, 0, 1), 0.333333),
    ((0, 1, 2, 1, 2, 0, 2, 0, 1), -1.000000),
    ((0, 1, 2, 1, 2, 0, 0, 2, 1), 0.000000),
    ((0, 1, 2, 1, 0, 2, 0, 0, 0), 0.173913),
    ((0, 1, 2, 1, 1, 2, 0, 0, 0), 0.176471),
    ((0, 1, 2, 1, 1, 2, 2, 0, 0), 0.600000),
    ((0, 1, 2, 1, 1, 2, 2, 1, 0), 1.000000),
    ((0, 1, 2, 1, 1, 2, 2, 0, 1), 1.000000),
    ((0, 1, 2, 1, 1, 2, 2, 2, 1), 1.000000),
    ((0, 1, 2, 1, 1, 2, 0, 2, 0), 0.000000),
    ((0, 1, 2, 1, 1, 2, 1, 2, 0), -0.500000),
    ((0, 1, 2, 1, 1, 2, 1, 2, 2), -1.000000),
    ((0, 1, 2, 1, 1, 2, 0, 2, 1), 0.500000),
    ((0, 1, 2, 1, 1, 2, 0, 0, 2), -1.000000),
    ((0, 1, 2, 1, 0, 2, 1, 0, 0), 0.176471),
    ((0, 1, 2, 1, 0, 2, 1, 2, 0), 0.200000),
    ((0, 1, 2, 1, 0, 2, 1, 2, 1), 0.500000),
    ((0, 1, 2, 1, 0, 2, 1, 0, 2), -1.000000),
    ((0, 1, 2, 1, 0, 2, 0, 1, 0), -0.058824),
    ((0, 1, 2, 1, 0, 2, 2, 1, 0), -0.200000),
    ((0, 1, 2, 1, 0, 2, 2, 1, 1), 0.000000),
    ((0, 1, 2, 1, 0, 2, 0, 1, 2), -1.000000),
    ((0, 1, 2, 1, 0, 2, 0, 0, 1), 0.500000),
    ((0, 1, 2, 1, 0, 2, 2, 0, 1), 0.333333),
    ((0, 1, 2, 1, 0, 2, 0, 2, 1), 0.666667),
    ((0, 1, 2, 1, 0, 0, 2, 0, 0), 0.000000),
    ((0, 1, 2, 1, 1, 0, 2, 0, 0), 0.555556),
    ((0, 1, 2, 1, 1, 0, 2, 2, 0), 0.600000),
    ((0, 1, 2, 1, 1, 1, 2, 2, 0), 1.000000),
    ((0, 1, 2, 1, 1, 0, 2, 2, 1), 1.000000),
    ((0, 1, 2, 1, 1, 0, 2, 0, 2), 0.000000),
    ((0, 1, 2, 1, 1, 1, 2, 0, 2), 1.000000),
    ((0, 1, 2, 1, 1, 0, 2, 1, 2), 1.000000),
    ((0, 1, 2, 1, 0, 1, 2, 0, 0), -0.125000),
    ((0, 1, 2, 1, 0, 1, 2, 2, 0), -0.200000),
    ((0, 1, 2, 1, 0, 1, 2, 2, 1), 0.000000),
    ((0, 1, 2, 1, 0, 1, 2, 0, 2), -0.200000),
    ((0, 1, 2, 1, 0, 1, 2, 1, 2), 0.000000),
    ((0, 1, 2, 1, 0, 0, 2, 1, 0), -0.125000),
    ((0, 1, 2, 1, 0, 0, 2, 1, 2), -0.200000),
    ((0, 1, 2, 1, 0, 0, 2, 0, 1), 0.263158),
    ((0, 1, 2, 1, 0, 0, 2, 2, 1), 0.333333),
    ((0, 1, 2, 1, 0, 0, 0, 2, 0), 0.259259),
    ((0, 1, 2, 1, 1, 0, 0, 2, 0), 0.238095),
    ((0, 1, 2, 1, 1, 0, 0, 2, 2), -0.200000),
    ((0, 1, 2, 1, 1, 1, 0, 2, 2), 1.000000),
    ((0, 1, 2, 1, 1, 0, 1, 2, 2), 0.000000),
    ((0, 1, 2, 1, 0, 1, 0, 2, 0), 0.238095),
    ((0, 1, 2, 1, 0, 1, 0, 2, 2), 0.600000),
    ((0, 1, 2, 1, 0, 1, 1, 2, 2), 1.000000),
    ((0, 1, 2, 1, 0, 0, 1, 2, 0), 0.428571),
    ((0, 1, 2, 1, 0, 0, 1, 2, 2), 0.600000),
    ((0, 1, 2, 1, 0, 0, 0, 2, 1), 0.333333),
    ((0, 1, 2, 1, 0, 0, 0, 0, 2), -0.209302),
    ((0, 1, 2, 1, 1, 0, 0, 0, 2), 0.000000),
    ((0, 1, 2, 1, 0, 1, 0, 0, 2), 0.047619),
    ((0, 1, 2, 1, 0, 0, 1, 0, 2), -0.058824),
    ((0, 1, 2, 1, 0, 0, 0, 1, 2), -0.529412),
    ((0, 1, 2, 0, 1, 0, 0, 0, 0), 0.391121),
    ((0, 1, 2, 2, 1, 0, 0, 0, 0), 0.493671),
    ((0, 1, 2, 2, 1, 1, 0, 0, 0), 0.428571),
    ((0, 1, 2, 2, 1, 1, 2, 0, 0), 0.600000),
    ((0, 1, 2, 2, 1, 1, 2, 1, 0), 1.000000),
    ((0, 1, 2, 2, 1, 1, 2, 0, 1), 0.000000),
    ((0, 1, 2, 2, 1, 1, 2, 2, 1), 1.000000),
    ((0, 1, 2, 2, 1, 1, 0, 2, 0), 0.333333),
    ((0, 1, 2, 2, 1, 1, 1, 2, 0), 0.000000),
    ((0, 1, 2, 2, 1, 1, 1, 2, 2), 0.000000),
    ((0, 1, 2, 2, 1, 1, 0, 2, 1), 0.500000),
    ((0, 1, 2, 2, 1, 1, 0, 0, 2), 0.600000),
    ((0, 1, 2, 2, 1, 1, 1, 0, 2), 0.500000),
    ((0, 1, 2, 2, 1, 1, 0, 1, 2), 1.000000),
    ((0, 1, 2, 2, 1, 0, 1, 0, 0), 0.428571),
    ((0, 1, 2, 2, 1, 2, 1, 0, 0), 0.600000),
    ((0, 1, 2, 2, 1, 2, 1, 1, 0), 1.000000),
    ((0, 1, 2, 2, 1, 2, 1, 0, 1), 1.000000),
    ((0, 1, 2, 2, 1, 2, 1, 2, 1), 1.000000),
    ((0, 1, 2, 2, 1, 0, 1, 2, 0), 0.333333),
    ((0, 1, 2, 2, 1, 0, 1, 2, 1), 0.500000),
    ((0, 1, 2, 2, 1, 0, 1, 0, 2), 0.200000),
    ((0, 1, 2, 2, 1, 0, 1, 1, 2), 1.000000),
    ((0, 1, 2, 2, 1, 0, 0, 1, 0), 1.000000),
    ((0, 1, 2, 2, 1, 0, 0, 0, 1), 0.555556),
    ((0, 1, 2, 2, 1, 2, 0, 0, 1), 1.000000),
    ((0, 1, 2, 2, 1, 2, 0, 1, 1), 1.000000),
    ((0, 1, 2, 2, 1, 0, 2, 0, 1), 0.500000),
    ((0, 1, 2, 2, 1, 0, 2, 1, 1), 1.000000),
    ((0, 1, 2, 2, 1, 0, 0, 2, 1), 0.600000),
    ((0, 1, 2, 0, 1, 2, 0, 0, 0), 0.373134),
    ((0, 1, 2, 0, 1, 2, 1, 0, 0), 0.176471),
    ((0, 1, 2, 0, 1, 2, 1, 2, 0), 0.000000),
    ((0, 1, 2, 0, 1, 2, 1, 2, 1), 0.500000),
    ((0, 1, 2, 0, 1, 2, 1, 0, 2), -1.000000),
    ((0, 1, 2, 0, 1, 2, 0, 1, 0), 1.000000),
    ((0, 1, 2, 0, 1, 2, 0, 0, 1), 0.777778),
    ((0, 1, 2, 0, 1, 2, 2, 0, 1), 1.000000),
    ((0, 1, 2, 0, 1, 2, 2, 1, 1), 1.000000),
    ((0, 1, 2, 0, 1, 2, 0, 2, 1), 0.600000),
    ((0, 1, 2, 0, 1, 0, 2, 0, 0), 0.561644),
    ((0, 1, 2, 0, 1, 1, 2, 0, 0), 0.555556),
    ((0, 1, 2, 0, 1, 1, 2, 2, 0), 0.600000),
    ((0, 1, 2, 0, 1, 1, 2, 2, 1), 1.000000),
    ((0, 1, 2, 0, 1, 1, 2, 0, 2), 0.500000),
    ((0, 1, 2, 0, 1, 1, 2, 1, 2), 1.000000),
    ((0, 1, 2, 0, 1, 0, 2, 1, 0), 1.000000),
    ((0, 1, 2, 0, 1, 0, 2, 0, 1), 0.777778),
    ((0, 1, 2, 0, 1, 0, 2, 2, 1), 1.000000),
    ((0, 1, 2, 0, 1, 0, 0, 2, 0), 0.296296),
    ((0, 1, 2, 0, 1, 1, 0, 2, 0), 0.428571),
    ((0, 1, 2, 0, 1, 1, 0, 2, 2), 0.200000),
    ((0, 1, 2, 0, 1, 1, 1, 2, 2), 0.500000),
    ((0, 1, 2, 0, 1, 0, 1, 2, 0), 0.166667),
    ((0, 1, 2, 0, 1, 0, 1, 2, 2), 0.000000),
    ((0, 1, 2, 0, 1, 0, 0, 2, 1), 0.619048),
    ((0, 1, 2, 0, 1, 0, 0, 0, 2), 0.134328),
    ((0, 1, 2, 0, 1, 1, 0, 0, 2), 0.555556),
    ((0, 1, 2, 0, 1, 0, 1, 0, 2), 0.176471),
    ((0, 1, 2, 0, 1, 0, 0, 1, 2), 1.000000),
    ((0, 1, 2, 0, 0, 1, 0, 0, 0), 0.120253),
    ((0, 1, 2, 2, 0, 1, 0, 0, 0), 0.228070),
    ((0, 1, 2, 2, 0, 1, 1, 0, 0), 0.333333),
    ((0, 1, 2, 2, 2, 1, 1, 0, 0), 0.333333),
    ((0, 1, 2, 2, 2, 1, 1, 1, 0), 0.500000),
    ((0, 1, 2, 2, 2, 1, 1, 1, 2), 0.000000),
    ((0, 1, 2, 2, 2, 1, 1, 0, 1), 0.500000),
    ((0, 1, 2, 2, 2, 1, 1, 2, 1), 0.000000),
    ((0, 1, 2, 2, 0, 1, 1, 2, 0), 0.000000),
    ((0, 1, 2, 2, 0, 1, 1, 2, 1), 0.000000),
    ((0, 1, 2, 2, 0, 1, 1, 0, 2), 0.333333),
    ((0, 1, 2, 2, 0, 1, 1, 1, 2), 0.500000),
    ((0, 1, 2, 2, 0, 1, 0, 1, 0), 0.238095),
    ((0, 1, 2, 2, 2, 1, 0, 1, 0), 0.000000),
    ((0, 1, 2, 2, 2, 1, 0, 1, 1), 0.000000),
    ((0, 1, 2, 2, 2, 1, 2, 1, 1), -1.000000),
    ((0, 1, 2, 2, 0, 1, 2, 1, 0), -0.200000),
    ((0, 1, 2, 2, 0, 1, 2, 1, 1), -1.000000),
    ((0, 1, 2, 2, 0, 1, 0, 1, 2), 0.600000),
    ((0, 1, 2, 2, 0, 1, 0, 0, 1), 0.000000),
    ((0, 1, 2, 2, 2, 1, 0, 0, 1), 0.000000),
    ((0, 1, 2, 2, 0, 1, 2, 0, 1), -0.333333),
    ((0, 1, 2, 2, 0, 1, 0, 2, 1), 0.333333),
    ((0, 1, 2, 0, 2, 1, 0, 0, 0), -0.160000),
    ((0, 1, 2, 0, 2, 1, 1, 0, 0), 0.166667),
    ((0, 1, 2, 0, 2, 1, 1, 2, 0), 0.333333),
    ((0, 1, 2, 0, 2, 1, 1, 2, 1), 0.000000),
    ((0, 1, 2, 0, 2, 1, 1, 0, 2), 0.000000),
    ((0, 1, 2, 0, 2, 1, 1, 1, 2), -0.500000),
    ((0, 1, 2, 0, 2, 1, 0, 1, 0), -0.368421),
    ((0, 1, 2, 0, 2, 1, 2, 1, 0), -1.000000),
    ((0, 1, 2, 0, 2, 1, 0, 1, 2), -0.666667),
    ((0, 1, 2, 0, 2, 1, 0, 0, 1), -0.157895),
    ((0, 1, 2, 0, 2, 1, 2, 0, 1), -1.000000),
    ((0, 1, 2, 0, 2, 1, 0, 2, 1), -0.333333),
    ((0, 1, 2, 0, 0, 1, 2, 0, 0), 0.000000),
    ((0, 1, 2, 0, 0, 1, 2, 1, 0), -0.125000),
    ((0, 1, 2, 0, 0, 1, 2, 1, 2), 0.200000),
    ((0, 1, 2, 0, 0, 1, 2, 0, 1), -0.157895),
    ((0, 1, 2, 0, 0, 1, 2, 2, 1), 0.333333),
    ((0, 1, 2, 0, 0, 1, 0, 2, 0), 0.228070),
    ((0, 1, 2, 0, 0, 1, 1, 2, 0), 0.333333),
    ((0, 1, 2, 0, 0, 1, 1, 2, 2), 0.666667),
    ((0, 1, 2, 0, 0, 1, 0, 2, 1), 0.166667),
    ((0, 1, 2, 0, 0, 1, 0, 0, 2), 0.185185),
    ((0, 1, 2, 0, 0, 1, 1, 0, 2), 0.333333),
    ((0, 1, 2, 0, 0, 1, 0, 1, 2), 0.047619),
    ((0, 1, 2, 0, 0, 0, 1, 0, 0), 0.220779),
    ((0, 1, 2, 2, 0, 0, 1, 0, 0), 0.259259),
    ((0, 1, 2, 2, 0, 0, 1, 1, 0), 0.333333),
    ((0, 1, 2, 2, 2, 0, 1, 1, 0), 0.200000),
    ((0, 1, 2, 2, 2, 0, 1, 1, 1), 1.000000),
    ((0, 1, 2, 2, 0, 2, 1, 1, 0), 0.000000),
    ((0, 1, 2, 2, 0, 2, 1, 1, 1), 1.000000),
    ((0, 1, 2, 2, 0, 0, 1, 1, 2), 0.200000),
    ((0, 1, 2, 2, 0, 0, 1, 0, 1), 0.428571),
    ((0, 1, 2, 2, 2, 0, 1, 0, 1), 0.200000),
    ((0, 1, 2, 2, 0, 2, 1, 0, 1), 0.600000),
    ((0, 1, 2, 2, 0, 0, 1, 2, 1), 0.333333),
    ((0, 1, 2, 0, 2, 0, 1, 0, 0), 0.148148),
    ((0, 1, 2, 0, 2, 0, 1, 1, 0), -0.142857),
    ((0, 1, 2, 0, 2, 2, 1, 1, 0), -0.200000),
    ((0, 1, 2, 0, 2, 2, 1, 1, 1), 1.000000),
    ((0, 1, 2, 0, 2, 0, 1, 1, 2), -0.666667),
    ((0, 1, 2, 0, 2, 0, 1, 0, 1), 0.428571),
    ((0, 1, 2, 0, 2, 2, 1, 0, 1), 0.600000),
    ((0, 1, 2, 0, 2, 0, 1, 2, 1), 0.333333),
    ((0, 1, 2, 0, 0, 2, 1, 0, 0), 0.209302),
    ((0, 1, 2, 0, 0, 2, 1, 1, 0), 0.000000),
    ((0, 1, 2, 0, 0, 2, 1, 1, 2), -1.000000),
    ((0, 1, 2, 0, 0, 2, 1, 0, 1), 0.619048),
    ((0, 1, 2, 0, 0, 2, 1, 2, 1), 0.666667),
    ((0, 1, 2, 0, 0, 0, 1, 2, 0), 0.333333),
    ((0, 1, 2, 0, 0, 0, 1, 2, 1), 0.333333),
    ((0, 1, 2, 0, 0, 0, 1, 0, 2), 0.043478),
    ((0, 1, 2, 0, 0, 0, 1, 1, 2), -0.294118),
    ((0, 1, 2, 0, 0, 0, 0, 1, 0), -0.085339),
    ((0, 1, 2, 2, 0, 0, 0, 1, 0), 0.139241),
    ((0, 1, 2, 2, 0, 0, 0, 1, 1), 0.111111),
    ((0, 1, 2, 2, 2, 0, 0, 1, 1), -0.200000),
    ((0, 1, 2, 2, 0, 2, 0, 1, 1), 0.500000),
    ((0, 1, 2, 2, 0, 0, 2, 1, 1), -0.200000),
    ((0, 1, 2, 0, 2, 0, 0, 1, 0), -0.404255),
    ((0, 1, 2, 0, 2, 0, 0, 1, 1), -0.125000),
    ((0, 1, 2, 0, 2, 2, 0, 1, 1), -0.200000),
    ((0, 1, 2, 0, 2, 0, 2, 1, 1), -1.000000),
    ((0, 1, 2, 0, 0, 2, 0, 1, 0), -0.044776),
    ((0, 1, 2, 0, 0, 2, 0, 1, 1), 0.333333),
    ((0, 1, 2, 0, 0, 2, 2, 1, 1), 0.200000),
    ((0, 1, 2, 0, 0, 0, 2, 1, 0), -0.107692),
    ((0, 1, 2, 0, 0, 0, 2, 1, 1), -0.125000),
    ((0, 1, 2, 0, 0, 0, 0, 1, 2), -0.232877),
    ((0, 1, 2, 0, 0, 0, 0, 0, 1), 0.263158),
    ((0, 1, 2, 2, 0, 0, 0, 0, 1), 0.215686),
    ((0, 1, 2, 0, 2, 0, 0, 0, 1), -0.021277),
    ((0, 1, 2, 0, 0, 2, 0, 0, 1), 0.490196),
    ((0, 1, 2, 0, 0, 0, 2, 0, 1), 0.181818),
    ((0, 1, 2, 0, 0, 0, 0, 2, 1), 0.368421),
    ((0, 1, 0, 2, 0, 0, 0, 0, 0), 0.223610),
    ((0, 1, 1, 2, 0, 0, 0, 0, 0), 0.165468),
    ((0, 1, 1, 2, 2, 0, 0, 0, 0), -0.044776),
    ((0, 1, 1, 2, 2, 1, 0, 0, 0), 0.333333),
    ((0, 1, 1, 2, 2, 1, 2, 0, 0), 0.500000),
    ((0, 1, 1, 2, 2, 1, 2, 1, 0), 0.000000),
    ((0, 1, 1, 2, 2, 1, 2, 1, 2), 1.000000),
    ((0, 1, 1, 2, 2, 1, 2, 0, 1), 1.000000),
    ((0, 1, 1, 2, 2, 1, 0, 2, 0), 1.000000),
    ((0, 1, 1, 2, 2, 1, 1, 2, 0), 1.000000),
    ((0, 1, 1, 2, 2, 1, 1, 2, 2), 1.000000),
    ((0, 1, 1, 2, 2, 1, 0, 2, 1), 1.000000),
    ((0, 1, 1, 2, 2, 1, 0, 0, 2), 0.200000),
    ((0, 1, 1, 2, 2, 1, 1, 0, 2), 0.000000),
    ((0, 1, 1, 2, 2, 1, 0, 1, 2), 0.000000),
    ((0, 1, 1, 2, 2, 0, 1, 0, 0), -0.058824),
    ((0, 1, 1, 2, 2, 2, 1, 0, 0), -1.000000),
    ((0, 1, 1, 2, 2, 0, 1, 2, 0), 0.600000),
    ((0, 1, 1, 2, 2, 0, 1, 2, 1), 0.000000),
    ((0, 1, 1, 2, 2, 2, 1, 2, 1), -1.000000),
    ((0, 1, 1, 2, 2, 0, 1, 0, 2), -0.200000),
    ((0, 1, 1, 2, 2, 0, 1, 1, 2), -1.000000),
    ((0, 1, 1, 2, 2, 2, 1, 1, 2), -1.000000),
    ((0, 1, 1, 2, 2, 0, 0, 1, 0), -0.529412),
    ((0, 1, 1, 2, 2, 2, 0, 1, 0), -1.000000),
    ((0, 1, 1, 2, 2, 0, 2, 1, 0), -0.200000),
    ((0, 1, 1, 2, 2, 0, 2, 1, 1), -1.000000),
    ((0, 1, 1, 2, 2, 2, 2, 1, 1), -1.000000),
    ((0, 1, 1, 2, 2, 0, 0, 1, 2), -0.200000),
    ((0, 1, 1, 2, 2, 0, 0, 0, 1), 0.000000),
    ((0, 1, 1, 2, 2, 2, 0, 0, 1), -1.000000),
    ((0, 1, 1, 2, 2, 0, 2, 0, 1), 0.000000),
    ((0, 1, 1, 2, 2, 0, 0, 2, 1), 0.500000),
    ((0, 1, 1, 2, 0, 2, 0, 0, 0), 0.245902),
    ((0, 1, 1, 2, 1, 2, 0, 0, 0), 0.733333),
    ((0, 1, 1, 2, 1, 2, 2, 0, 0), 0.500000),
    ((0, 1, 1, 2, 1, 2, 2, 1, 0), 1.000000),
    ((0, 1, 1, 2, 1, 2, 2, 0, 1), 0.000000),
    ((0, 1, 1, 2, 1, 2, 2, 2, 1), 1.000000),
    ((1, 1, 1, 2, 1, 2, 2, 2, 1), 1.000000),
    ((0, 1, 1, 2, 1, 2, 0, 2, 0), 1.000000),
    ((0, 1, 1, 2, 1, 2, 1, 2, 0), 1.000000),
    ((0, 1, 1, 2, 1, 2, 0, 2, 1), 1.000000),
    ((0, 1, 1, 2, 1, 2, 0, 0, 2), 1.000000),
    ((0, 1, 1, 2, 1, 2, 1, 0, 2), 1.000000),
    ((0, 1, 1, 2, 1, 2, 0, 1, 2), 1.000000),
    ((0, 1, 1, 2, 0, 2, 1, 0, 0), 0.285714),
    ((0, 1, 1, 2, 0, 2, 1, 2, 0), 0.500000),
    ((0, 1, 1, 2, 0, 2, 1, 2, 1), 0.000000),
    ((0, 1, 1, 2, 0, 2, 1, 0, 2), 0.500000),
    ((0, 1, 1, 2, 0, 2, 1, 1, 2), 0.000000),
    ((0, 1, 1, 2, 0, 2, 0, 1, 0), 0.000000),
    ((0, 1, 1, 2, 0, 2, 2, 1, 0), 0.000000),
    ((0, 1, 1, 2, 0, 2, 2, 1, 1), -1.000000),
    ((0, 1, 1, 2, 0, 2, 0, 1, 2), 0.500000),
    ((0, 1, 1, 2, 0, 2, 0, 0, 1), -0.058824),
    ((0, 1, 1, 2, 0, 2, 2, 0, 1), -0.200000),
    ((0, 1, 1, 2, 0, 2, 0, 2, 1), 0.600000),
    ((0, 1, 1, 2, 0, 0, 2, 0, 0), 0.017544),
    ((0, 1, 1, 2, 1, 0, 2, 0, 0), 0.000000),
    ((0, 1, 1, 2, 1, 0, 2, 2, 0), -0.200000),
    ((0, 1, 1, 2, 1, 1, 2, 2, 0), -1.000000),
    ((0, 1, 1, 2, 1, 1, 2, 2, 2), -1.000000),
    ((0, 1, 1, 2, 1, 0, 2, 2, 1), 0.000000),
    ((0, 1, 1, 2, 1, 0, 2, 0, 2), 0.000000),
    ((0, 1, 1, 2, 1, 1, 2, 0, 2), -1.000000),
    ((0, 1, 1, 2, 1, 0, 2, 1, 2), 1.000000),
    ((0, 1, 1, 2, 0, 1, 2, 0, 0), 0.000000),
    ((0, 1, 1, 2, 0, 1, 2, 2, 0), 0.000000),
    ((0, 1, 1, 2, 0, 1, 2, 2, 1), 1.000000),
    ((0, 1, 1, 2, 0, 1, 2, 0, 2), -0.200000),
    ((0, 1, 1, 2, 0, 1, 2, 1, 2), 0.000000),
    ((0, 1, 1, 2, 0, 0, 2, 1, 0), 0.000000),
    ((0, 1, 1, 2, 0, 0, 2, 1, 2), 0.500000),
    ((0, 1, 1, 2, 0, 0, 2, 0, 1), 0.000000),
    ((0, 1, 1, 2, 0, 0, 2, 2, 1), 0.500000),
    ((0, 1, 1, 2, 0, 0, 0, 2, 0), 0.561644),
    ((0, 1, 1, 2, 1, 0, 0, 2, 0), 0.333333),
    ((0, 1, 1, 2, 1, 0, 0, 2, 2), 0.500000),
    ((0, 1, 1, 2, 1, 1, 0, 2, 2), 0.000000),
    ((0, 1, 1, 2, 1, 0, 1, 2, 2), 1.000000),
    ((0, 1, 1, 2, 0, 1, 0, 2, 0), 0.555556),
    ((0, 1, 1, 2, 0, 1, 0, 2, 2), 0.600000),
    ((0, 1, 1, 2, 0, 1, 1, 2, 2), 1.000000),
    ((0, 1, 1, 2, 0, 0, 1, 2, 0), 0.777778),
    ((0, 1, 1, 2, 0, 0, 1, 2, 2), 1.000000),
    ((0, 1, 1, 2, 0, 0, 0, 2, 1), 0.555556),
    ((0, 1, 1, 2, 0, 0, 0, 0, 2), 0.232877),
    ((0, 1, 1, 2, 1, 0, 0, 0, 2), 0.466667),
    ((0, 1, 1, 2, 0, 1, 0, 0, 2), 0.047619),
    ((0, 1, 1, 2, 0, 0, 1, 0, 2), 0.333333),
    ((0, 1, 1, 2, 0, 0, 0, 1, 2), 0.111111),
    ((0, 1, 0, 2, 1, 0, 0, 0, 0), 0.438753),
    ((0, 1, 0, 2, 1, 2, 0, 0, 0), 0.761194),
    ((0, 1, 0, 2, 1, 2, 1, 0, 0), 0.777778),
    ((0, 1, 0, 2, 1, 2, 1, 2, 0), 1.000000),
    ((0, 1, 0, 2, 1, 2, 1, 2, 1), 1.000000),
    ((0, 1, 0, 2, 1, 2, 1, 0, 2), 0.500000),
    ((0, 1, 0, 2, 1, 2, 1, 1, 2), 1.000000),
    ((0, 1, 0, 2, 1, 2, 0, 1, 0), 1.000000),
    ((0, 1, 0, 2, 1, 2, 0, 0, 1), 0.777778),
    ((0, 1, 0, 2, 1, 2, 2, 0, 1), 0.500000),
    ((0, 1, 0, 2, 1, 2, 2, 1, 1), 1.000000),
    ((0, 1, 0, 2, 1, 2, 0, 2, 1), 1.000000),
    ((0, 1, 0, 2, 1, 0, 2, 0, 0), 0.245902),
    ((0, 1, 0, 2, 1, 1, 2, 0, 0), -0.058824),
    ((0, 1, 0, 2, 1, 1, 2, 2, 0), -0.333333),
    ((0, 1, 0, 2, 1, 1, 2, 2, 1), 0.000000),
    ((0, 1, 0, 2, 1, 1, 2, 0, 2), -0.200000),
    ((0, 1, 0, 2, 1, 1, 2, 1, 2), 1.000000),
    ((0, 1, 0, 2, 1, 0, 2, 1, 0), 1.000000),
    ((0, 1, 0, 2, 1, 0, 2, 0, 1), 0.285714),
    ((0, 1, 0, 2, 1, 0, 2, 2, 1), 0.200000),
    ((0, 1, 0, 2, 1, 0, 0, 2, 0), 0.372549),
    ((0, 1, 0, 2, 1, 1, 0, 2, 0), 0.000000),
    ((0, 1, 0, 2, 1, 1, 0, 2, 2), 0.000000),
    ((0, 1, 0, 2, 1, 1, 1, 2, 2), 0.500000),
    ((0, 1, 0, 2, 1, 0, 1, 2, 0), 0.619048),
    ((0, 1, 0, 2, 1, 0, 1, 2, 2), 0.600000),
    ((0, 1, 0, 2, 1, 0, 0, 2, 1), 0.428571),
    ((0, 1, 0, 2, 1, 0, 0, 0, 2), 0.397260),
    ((0, 1, 0, 2, 1, 1, 0, 0, 2), 0.238095),
    ((0, 1, 0, 2, 1, 0, 1, 0, 2), 0.555556),
    ((0, 1, 0, 2, 1, 0, 0, 1, 2), 1.000000),
    ((0, 1, 0, 2, 0, 1, 0, 0, 0), 0.220779),
    ((0, 1, 0, 2, 2, 1, 0, 0, 0), 0.185185),
    ((0, 1, 0, 2, 2, 1, 1, 0, 0), 0.333333),
    ((0, 1, 0, 2, 2, 1, 1, 2, 0), 0.666667),
    ((0, 1, 0, 2, 2, 1, 1, 2, 1), 0.500000),
    ((0, 1, 0, 2, 2, 1, 1, 0, 2), 0.000000),
    ((0, 1, 0, 2, 2, 1, 1, 1, 2), -0.500000),
    ((0, 1, 0, 2, 2, 1, 0, 1, 0), -0.166667),
    ((0, 1, 0, 2, 2, 1, 2, 1, 0), -0.333333),
    ((0, 1, 0, 2, 2, 1, 2, 1, 1), -1.000000),
    ((0, 1, 0, 2, 2, 1, 0, 1, 2), 0.000000),
    ((0, 1, 0, 2, 2, 1, 0, 0, 1), 0.238095),
    ((0, 1, 0, 2, 2, 1, 2, 0, 1), -0.200000),
    ((0, 1, 0, 2, 2, 1, 0, 2, 1), 0.600000),
    ((0, 1, 0, 2, 0, 1, 2, 0, 0), 0.069767),
    ((0, 1, 0, 2, 0, 1, 2, 1, 0), -0.058824),
    ((0, 1, 0, 2, 0, 1, 2, 1, 2), 0.600000),
    ((0, 1, 0, 2, 0, 1, 2, 0, 1), -0.058824),
    ((0, 1, 0, 2, 0, 1, 2, 2, 1), 0.600000),
    ((0, 1, 0, 2, 0, 1, 0, 2, 0), 0.370370),
    ((0, 1, 0, 2, 0, 1, 1, 2, 0), 0.500000),
    ((0, 1, 0, 2, 0, 1, 1, 2, 2), 0.666667),
    ((0, 1, 0, 2, 0, 1, 0, 2, 1), 0.428571),
    ((0, 1, 0, 2, 0, 1, 0, 0, 2), 0.259259),
    ((0, 1, 0, 2, 0, 1, 1, 0, 2), 0.333333),
    ((0, 1, 0, 2, 0, 1, 0, 1, 2), 0.238095),
    ((0, 1, 0, 2, 0, 0, 1, 0, 0), 0.310811),
    ((0, 1, 0, 2, 2, 0, 1, 0, 0), 0.043478),
    ((0, 1, 0, 2, 2, 0, 1, 1, 0), -0.294118),
    ((0, 1, 0, 2, 2, 2, 1, 1, 0), -1.000000),
    ((0, 1, 0, 2, 2, 0, 1, 1, 2), -0.666667),
    ((0, 1, 0, 2, 2, 0, 1, 0, 1), 0.176471),
    ((0, 1, 0, 2, 2, 2, 1, 0, 1), -1.000000),
    ((0, 1, 0, 2, 2, 0, 1, 2, 1), 0.000000),
    ((0, 1, 0, 2, 0, 2, 1, 0, 0), 0.300000),
    ((0, 1, 0, 2, 0, 2, 1, 1, 0), 0.000000),
    ((0, 1, 0, 2, 0, 2, 1, 1, 2), -0.200000),
    ((0, 1, 0, 2, 0, 2, 1, 0, 1), 0.411765),
    ((0, 1, 0, 2, 0, 2, 1, 2, 1), 0.333333),
    ((0, 1, 0, 2, 0, 0, 1, 2, 0), 0.518519),
    ((0, 1, 0, 2, 0, 0, 1, 2, 1), 0.333333),
    ((0, 1, 0, 2, 0, 0, 1, 0, 2), 0.215686),
    ((0, 1, 0, 2, 0, 0, 1, 1, 2), -0.142857),
    ((0, 1, 0, 2, 0, 0, 0, 1, 0), -0.034014),
    ((0, 1, 0, 2, 2, 0, 0, 1, 0), -0.347826),
    ((0, 1, 0, 2, 2, 0, 0, 1, 1), -0.529412),
    ((0, 1, 0, 2, 2, 2, 0, 1, 1), -1.000000),
    ((0, 1, 0, 2, 2, 0, 2, 1, 1), -1.000000),
    ((0, 1, 0, 2, 0, 2, 0, 1, 0), 0.017544),
    ((0, 1, 0, 2, 0, 2, 0, 1, 1), 0.000000),
    ((0, 1, 0, 2, 0, 2, 2, 1, 1), -0.200000),
    ((0, 1, 0, 2, 0, 0, 2, 1, 0), -0.044776),
    ((0, 1, 0, 2, 0, 0, 2, 1, 1), -0.529412),
    ((0, 1, 0, 2, 0, 0, 0, 1, 2), 0.139241),
    ((0, 1, 0, 2, 0, 0, 0, 0, 1), 0.194030),
    ((0, 1, 0, 2, 2, 0, 0, 0, 1), -0.069767),
    ((0, 1, 0, 2, 0, 2, 0, 0, 1), 0.300000),
    ((0, 1, 0, 2, 0, 0, 2, 0, 1), 0.000000),
    ((0, 1, 0, 2, 0, 0, 0, 2, 1), 0.450980),
    ((0, 1, 0, 0, 2, 0, 0, 0, 0), -0.056168),
    ((0, 1, 1, 0, 2, 0, 0, 0, 0), -0.006565),
    ((0, 1, 1, 0, 2, 2, 0, 0, 0), -0.068493),
    ((0, 1, 1, 1, 2, 2, 0, 0, 0), 0.238095),
    ((0, 1, 1, 1, 2, 2, 2, 0, 0), 0.600000),
    ((0, 1, 1, 1, 2, 2, 2, 1, 0), 0.500000),
    ((0, 1, 1, 1, 2, 2, 2, 1, 2), 1.000000),
    ((0, 1, 1, 1, 2, 2, 2, 0, 1), 0.500000),
    ((0, 1, 1, 1, 2, 2, 2, 2, 1), 1.000000),
    ((0, 1, 1, 1, 2, 2, 0, 2, 0), 0.600000),
    ((0, 1, 1, 1, 2, 2, 1, 2, 0), 0.500000),
    ((0, 1, 1, 1, 2, 2, 1, 2, 2), 1.000000),
    ((1, 1, 1, 1, 2, 2, 1, 2, 2), 1.000000),
    ((0, 1, 1, 1, 2, 2, 0, 2, 1), 0.500000),
    ((0, 1, 1, 1, 2, 2, 0, 0, 2), 0.200000),
    ((0, 1, 1, 1, 2, 2, 1, 0, 2), 0.000000),
    ((0, 1, 1, 1, 2, 2, 0, 1, 2), 0.000000),
    ((0, 1, 1, 0, 2, 2, 1, 0, 0), -0.294118),
    ((0, 1, 1, 0, 2, 2, 1, 2, 0), 0.200000),
    ((0, 1, 1, 0, 2, 2, 1, 2, 1), -0.500000),
    ((0, 1, 1, 0, 2, 2, 1, 0, 2), -0.200000),
    ((0, 1, 1, 0, 2, 2, 1, 1, 2), -1.000000),
    ((0, 1, 1, 0, 2, 2, 0, 1, 0), -0.294118),
    ((0, 1, 1, 0, 2, 2, 2, 1, 0), 0.200000),
    ((0, 1, 1, 0, 2, 2, 2, 1, 1), -0.500000),
    ((0, 1, 1, 0, 2, 2, 0, 1, 2), -0.200000),
    ((0, 1, 1, 0, 2, 2, 0, 0, 1), -0.058824),
    ((0, 1, 1, 0, 2, 2, 2, 0, 1), 0.200000),
    ((0, 1, 1, 0, 2, 2, 0, 2, 1), 0.200000),
    ((0, 1, 1, 0, 2, 0, 2, 0, 0), 0.139241),
    ((0, 1, 1, 1, 2, 0, 2, 0, 0), 0.238095),
    ((0, 1, 1, 1, 2, 0, 2, 2, 0), 0.600000),
    ((0, 1, 1, 1, 2, 1, 2, 2, 0), 0.000000),
    ((0, 1, 1, 1, 2, 1, 2, 2, 2), -1.000000),
    ((0, 1, 1, 1, 2, 0, 2, 2, 1), 1.000000),
    ((0, 1, 1, 1, 2, 0, 2, 0, 2), -0.200000),
    ((0, 1, 1, 1, 2, 1, 2, 0, 2), -1.000000),
    ((0, 1, 1, 1, 2, 0, 2, 1, 2), 0.000000),
    ((0, 1, 1, 0, 2, 1, 2, 0, 0), 0.111111),
    ((0, 1, 1, 0, 2, 1, 2, 2, 0), 0.500000),
    ((0, 1, 1, 0, 2, 1, 2, 2, 1), 1.000000),
    ((0, 1, 1, 0, 2, 1, 2, 0, 2), -0.200000),
    ((0, 1, 1, 0, 2, 1, 2, 1, 2), 0.000000),
    ((0, 1, 1, 0, 2, 0, 2, 1, 0), -0.142857),
    ((0, 1, 1, 0, 2, 0, 2, 1, 2), 0.200000),
    ((0, 1, 1, 0, 2, 0, 2, 0, 1), 0.333333),
    ((0, 1, 1, 0, 2, 0, 2, 2, 1), 1.000000),
    ((0, 1, 1, 0, 2, 0, 0, 2, 0), 0.341772),
    ((0, 1, 1, 1, 2, 0, 0, 2, 0), 0.238095),
    ((0, 1, 1, 1, 2, 0, 0, 2, 2), -0.200000),
    ((0, 1, 1, 1, 2, 1, 0, 2, 2), -1.000000),
    ((0, 1, 1, 1, 2, 0, 1, 2, 2), 0.000000),
    ((0, 1, 1, 0, 2, 1, 0, 2, 0), 0.333333),
    ((0, 1, 1, 0, 2, 1, 0, 2, 2), -0.200000),
    ((0, 1, 1, 0, 2, 1, 1, 2, 2), 0.000000),
    ((0, 1, 1, 0, 2, 0, 1, 2, 0), 0.238095),
    ((0, 1, 1, 0, 2, 0, 1, 2, 2), 0.200000),
    ((0, 1, 1, 0, 2, 0, 0, 2, 1), 0.555556),
    ((0, 1, 1, 0, 2, 0, 0, 0, 2), -0.107692),
    ((0, 1, 1, 1, 2, 0, 0, 0, 2), -0.125000),
    ((0, 1, 1, 0, 2, 1, 0, 0, 2), -0.125000),
    ((0, 1, 1, 0, 2, 0, 1, 0, 2), -0.125000),
    ((0, 1, 1, 0, 2, 0, 0, 1, 2), -0.125000),
    ((0, 1, 0, 1, 2, 0, 0, 0, 0), -0.047297),
    ((0, 1, 0, 1, 2, 2, 0, 0, 0), 0.185185),
    ((0, 1, 0, 1, 2, 2, 1, 0, 0), 0.238095),
    ((0, 1, 0, 1, 2, 2, 1, 2, 0), 0.600000),
    ((0, 1, 0, 1, 2, 2, 1, 2, 1), 0.500000),
    ((0, 1, 0, 1, 2, 2, 1, 0, 2), -0.200000),
    ((0, 1, 0, 1, 2, 2, 1, 1, 2), -1.000000),
    ((0, 1, 0, 1, 2, 2, 0, 1, 0), -0.166667),
    ((0, 1, 0, 1, 2, 2, 2, 1, 0), 0.000000),
    ((0, 1, 0, 1, 2, 2, 2, 1, 1), -0.500000),
    ((0, 1, 0, 1, 2, 2, 0, 1, 2), -0.333333),
    ((0, 1, 0, 1, 2, 2, 0, 0, 1), 0.333333),
    ((0, 1, 0, 1, 2, 2, 2, 0, 1), 0.000000),
    ((0, 1, 0, 1, 2, 2, 0, 2, 1), 0.666667),
    ((0, 1, 0, 1, 2, 0, 2, 0, 0), -0.148936),
    ((0, 1, 0, 1, 2, 1, 2, 0, 0), -0.578947),
    ((0, 1, 0, 1, 2, 1, 2, 2, 0), -0.333333),
    ((0, 1, 0, 1, 2, 1, 2, 2, 1), 0.000000),
    ((0, 1, 0, 1, 2, 1, 2, 0, 2), -1.000000),
    ((0, 1, 0, 1, 2, 1, 2, 1, 2), -1.000000),
    ((0, 1, 0, 1, 2, 0, 2, 1, 0), -0.368421),
    ((0, 1, 0, 1, 2, 0, 2, 1, 2), -0.333333),
    ((0, 1, 0, 1, 2, 0, 2, 0, 1), 0.052632),
    ((0, 1, 0, 1, 2, 0, 2, 2, 1), 0.333333),
    ((0, 1, 0, 1, 2, 0, 0, 2, 0), 0.185185),
    ((0, 1, 0, 1, 2, 1, 0, 2, 0), -0.166667),
    ((0, 1, 0, 1, 2, 1, 0, 2, 2), -0.333333),
    ((0, 1, 0, 1, 2, 1, 1, 2, 2), 0.000000),
    ((0, 1, 0, 1, 2, 0, 1, 2, 0), 0.238095),
    ((0, 1, 0, 1, 2, 0, 1, 2, 2), 0.200000),
    ((0, 1, 0, 1, 2, 0, 0, 2, 1), 0.333333),
    ((0, 1, 0, 1, 2, 0, 0, 0, 2), -0.272727),
    ((0, 1, 0, 1, 2, 1, 0, 0, 2), -0.578947),
    ((0, 1, 0, 1, 2, 0, 1, 0, 2), -0.125000),
    ((0, 1, 0, 1, 2, 0, 0, 1, 2), -0.578947),
    ((0, 1, 0, 0, 2, 1, 0, 0, 0), -0.047297),
    ((0, 1, 0, 0, 2, 1, 2, 0, 0), -0.272727),
    ((0, 1, 0, 0, 2, 1, 2, 1, 0), -0.578947),
    ((0, 1, 0, 0, 2, 1, 2, 1, 2), -0.333333),
    ((0, 1, 0, 0, 2, 1, 2, 0, 1), -0.125000),
    ((0, 1, 0, 0, 2, 1, 2, 2, 1), 0.200000),
    ((0, 1, 0, 0, 2, 1, 0, 2, 0), 0.185185),
    ((0, 1, 0, 0, 2, 1, 1, 2, 0), 0.333333),
    ((0, 1, 0, 0, 2, 1, 1, 2, 2), 0.333333),
    ((0, 1, 0, 0, 2, 1, 0, 2, 1), 0.238095),
    ((0, 1, 0, 0, 2, 1, 0, 0, 2), -0.148936),
    ((0, 1, 0, 0, 2, 1, 1, 0, 2), 0.052632),
    ((0, 1, 0, 0, 2, 1, 0, 1, 2), -0.368421),
    ((0, 1, 0, 0, 2, 0, 1, 0, 0), 0.055556),
    ((0, 1, 0, 0, 2, 2, 1, 0, 0), -0.069767),
    ((0, 1, 0, 0, 2, 2, 1, 1, 0), -0.529412),
    ((0, 1, 0, 0, 2, 2, 1, 1, 2), -1.000000),
    ((0, 1, 0, 0, 2, 2, 1, 0, 1), 0.176471),
    ((0, 1, 0, 0, 2, 2, 1, 2, 1), 0.000000),
    ((0, 1, 0, 0, 2, 0, 1, 2, 0), 0.296296),
    ((0, 1, 0, 0, 2, 0, 1, 2, 1), 0.166667),
    ((0, 1, 0, 0, 2, 0, 1, 0, 2), -0.136364),
    ((0, 1, 0, 0, 2, 0, 1, 1, 2), -0.789474),
    ((0, 1, 0, 0, 2, 0, 0, 1, 0), -0.385714),
    ((0, 1, 0, 0, 2, 2, 0, 1, 0), -0.347826),
    ((0, 1, 0, 0, 2, 2, 0, 1, 1), -0.294118),
    ((0, 1, 0, 0, 2, 2, 2, 1, 1), -0.666667),
    ((0, 1, 0, 0, 2, 0, 2, 1, 0), -0.404255),
    ((0, 1, 0, 0, 2, 0, 2, 1, 1), -0.789474),
    ((0, 1, 0, 0, 2, 0, 0, 1, 2), -0.404255),
    ((0, 1, 0, 0, 2, 0, 0, 0, 1), 0.055556),
    ((0, 1, 0, 0, 2, 2, 0, 0, 1), 0.043478),
    ((0, 1, 0, 0, 2, 0, 2, 0, 1), -0.136364),
    ((0, 1, 0, 0, 2, 0, 0, 2, 1), 0.296296),
    ((0, 1, 0, 0, 0, 2, 0, 0, 0), 0.223610),
    ((0, 1, 1, 0, 0, 2, 0, 0, 0), 0.238901),
    ((0, 1, 1, 0, 0, 2, 2, 0, 0), 0.291139),
    ((0, 1, 1, 1, 0, 2, 2, 0, 0), 0.428571),
    ((0, 1, 1, 1, 0, 2, 2, 2, 0), 0.200000),
    ((0, 1, 1, 1, 1, 2, 2, 2, 0), -0.500000),
    ((0, 1, 1, 1, 1, 2, 2, 2, 2), -1.000000),
    ((0, 1, 1, 1, 0, 2, 2, 2, 1), 0.500000),
    ((0, 1, 1, 1, 0, 2, 2, 0, 2), 0.600000),
    ((0, 1, 1, 1, 1, 2, 2, 0, 2), 0.000000),
    ((0, 1, 1, 1, 0, 2, 2, 1, 2), 1.000000),
    ((0, 1, 1, 0, 1, 2, 2, 0, 0), 0.333333),
    ((0, 1, 1, 0, 1, 2, 2, 2, 0), 0.200000),
    ((0, 1, 1, 0, 1, 2, 2, 2, 1), 0.500000),
    ((0, 1, 1, 0, 1, 2, 2, 0, 2), 0.500000),
    ((0, 1, 1, 0, 1, 2, 2, 1, 2), 1.000000),
    ((0, 1, 1, 0, 0, 2, 2, 1, 0), 0.333333),
    ((0, 1, 1, 0, 0, 2, 2, 1, 2), 1.000000),
    ((0, 1, 1, 0, 0, 2, 2, 0, 1), 0.047619),
    ((0, 1, 1, 0, 0, 2, 2, 2, 1), 0.600000),
    ((0, 1, 1, 0, 0, 2, 0, 2, 0), 0.493671),
    ((0, 1, 1, 1, 0, 2, 0, 2, 0), 0.428571),
    ((0, 1, 1, 1, 0, 2, 0, 2, 2), 0.600000),
    ((0, 1, 1, 1, 1, 2, 0, 2, 2), 0.000000),
    ((0, 1, 1, 1, 0, 2, 1, 2, 2), 1.000000),
    ((0, 1, 1, 0, 1, 2, 0, 2, 0), 0.555556),
    ((0, 1, 1, 0, 1, 2, 0, 2, 2), 0.500000),
    ((0, 1, 1, 0, 1, 2, 1, 2, 2), 1.000000),
    ((0, 1, 1, 0, 0, 2, 1, 2, 0), 0.555556),
    ((0, 1, 1, 0, 0, 2, 1, 2, 2), 1.000000),
    ((0, 1, 1, 0, 0, 2, 0, 2, 1), 0.428571),
    ((0, 1, 1, 0, 0, 2, 0, 0, 2), 0.452055),
    ((0, 1, 1, 1, 0, 2, 0, 0, 2), 0.428571),
    ((0, 1, 1, 0, 1, 2, 0, 0, 2), 0.733333),
    ((0, 1, 1, 0, 0, 2, 1, 0, 2), 0.333333),
    ((0, 1, 1, 0, 0, 2, 0, 1, 2), 0.333333),
    ((0, 1, 0, 1, 0, 2, 0, 0, 0), 0.220779),
    ((0, 1, 0, 1, 0, 2, 2, 0, 0), 0.259259),
    ((0, 1, 0, 1, 1, 2, 2, 0, 0), 0.238095),
    ((0, 1, 0, 1, 1, 2, 2, 2, 0), 0.000000),
    ((0, 1, 0, 1, 1, 2, 2, 2, 1), 0.500000),
    ((0, 1, 0, 1, 1, 2, 2, 0, 2), -0.200000),
    ((0, 1, 0, 1, 1, 2, 2, 1, 2), 1.000000),
    ((0, 1, 0, 1, 0, 2, 2, 1, 0), 0.238095),
    ((0, 1, 0, 1, 0, 2, 2, 1, 2), 0.600000),
    ((0, 1, 0, 1, 0, 2, 2, 0, 1), 0.333333),
    ((0, 1, 0, 1, 0, 2, 2, 2, 1), 0.666667),
    ((0, 1, 0, 1, 0, 2, 0, 2, 0), 0.370370),
    ((0, 1, 0, 1, 1, 2, 0, 2, 0), 0.000000),
    ((0, 1, 0, 1, 1, 2, 0, 2, 2), -0.333333),
    ((0, 1, 0, 1, 1, 2, 1, 2, 2), 0.000000),
    ((0, 1, 0, 1, 0, 2, 1, 2, 0), 0.428571),
    ((0, 1, 0, 1, 0, 2, 1, 2, 2), 0.600000),
    ((0, 1, 0, 1, 0, 2, 0, 2, 1), 0.500000),
    ((0, 1, 0, 1, 0, 2, 0, 0, 2), 0.069767),
    ((0, 1, 0, 1, 1, 2, 0, 0, 2), -0.058824),
    ((0, 1, 0, 1, 0, 2, 1, 0, 2), -0.058824),
    ((0, 1, 0, 1, 0, 2, 0, 1, 2), -0.058824),
    ((0, 1, 0, 0, 1, 2, 0, 0, 0), 0.438753),
    ((0, 1, 0, 0, 1, 2, 2, 0, 0), 0.397260),
    ((0, 1, 0, 0, 1, 2, 2, 1, 0), 1.000000),
    ((0, 1, 0, 0, 1, 2, 2, 0, 1), 0.555556),
    ((0, 1, 0, 0, 1, 2, 2, 2, 1), 0.600000),
    ((0, 1, 0, 0, 1, 2, 0, 2, 0), 0.372549),
    ((0, 1, 0, 0, 1, 2, 1, 2, 0), 0.428571),
    ((0, 1, 0, 0, 1, 2, 1, 2, 2), 0.200000),
    ((0, 1, 0, 0, 1, 2, 0, 2, 1), 0.619048),
    ((0, 1, 0, 0, 1, 2, 0, 0, 2), 0.245902),
    ((0, 1, 0, 0, 1, 2, 1, 0, 2), 0.285714),
    ((0, 1, 0, 0, 1, 2, 0, 1, 2), 1.000000),
    ((0, 1, 0, 0, 0, 2, 1, 0, 0), 0.194030),
    ((0, 1, 0, 0, 0, 2, 1, 2, 0), 0.450980),
    ((0, 1, 0, 0, 0, 2, 1, 2, 1), 0.333333),
    ((0, 1, 0, 0, 0, 2, 1, 0, 2), 0.000000),
    ((0, 1, 0, 0, 0, 2, 1, 1, 2), -0.529412),
    ((0, 1, 0, 0, 0, 2, 0, 1, 0), -0.034014),
    ((0, 1, 0, 0, 0, 2, 2, 1, 0), 0.139241),
    ((0, 1, 0, 0, 0, 2, 2, 1, 1), -0.142857),
    ((0, 1, 0, 0, 0, 2, 0, 1, 2), -0.044776),
    ((0, 1, 0, 0, 0, 2, 0, 0, 1), 0.310811),
    ((0, 1, 0, 0, 0, 2, 2, 0, 1), 0.215686),
    ((0, 1, 0, 0, 0, 2, 0, 2, 1), 0.518519),
    ((0, 1, 0, 0, 0, 0, 2, 0, 0), 0.057715),
    ((0, 1, 1, 0, 0, 0, 2, 0, 0), 0.047619),
    ((0, 1, 1, 0, 0, 0, 2, 2, 0), 0.134328),
    ((0, 1, 1, 1, 0, 0, 2, 2, 0), 0.176471),
    ((0, 1, 1, 1, 0, 0, 2, 2, 2), -1.000000),
    ((0, 1, 1, 0, 1, 0, 2, 2, 0), -0.294118),
    ((0, 1, 1, 0, 1, 0, 2, 2, 2), -1.000000),
    ((0, 1, 1, 0, 0, 1, 2, 2, 0), 0.000000),
    ((0, 1, 1, 0, 0, 1, 2, 2, 2), -1.000000),
    ((0, 1, 1, 0, 0, 0, 2, 2, 1), 0.555556),
    ((0, 1, 1, 0, 0, 0, 2, 0, 2), -0.044776),
    ((0, 1, 1, 1, 0, 0, 2, 0, 2), -0.058824),
    ((0, 1, 1, 0, 1, 0, 2, 0, 2), 0.000000),
    ((0, 1, 1, 0, 0, 1, 2, 0, 2), -0.529412),
    ((0, 1, 1, 0, 0, 0, 2, 1, 2), 0.333333),
    ((0, 1, 0, 1, 0, 0, 2, 0, 0), 0.055556),
    ((0, 1, 0, 1, 0, 0, 2, 2, 0), 0.173913),
    ((0, 1, 0, 1, 1, 0, 2, 2, 0), 0.176471),
    ((0, 1, 0, 1, 1, 0, 2, 2, 2), -1.000000),
    ((0, 1, 0, 1, 0, 1, 2, 2, 0), -0.058824),
    ((0, 1, 0, 1, 0, 1, 2, 2, 2), -1.000000),
    ((0, 1, 0, 1, 0, 0, 2, 2, 1), 0.500000),
    ((0, 1, 0, 1, 0, 0, 2, 0, 2), -0.209302),
    ((0, 1, 0, 1, 1, 0, 2, 0, 2), 0.000000),
    ((0, 1, 0, 1, 0, 1, 2, 0, 2), -0.529412),
    ((0, 1, 0, 1, 0, 0, 2, 1, 2), 0.047619),
    ((0, 1, 0, 0, 1, 0, 2, 0, 0), 0.251799),
    ((0, 1, 0, 0, 1, 0, 2, 2, 0), 0.116279),
    ((0, 1, 0, 0, 1, 1, 2, 2, 0), -0.058824),
    ((0, 1, 0, 0, 1, 1, 2, 2, 2), -1.000000),
    ((0, 1, 0, 0, 1, 0, 2, 2, 1), 0.428571),
    ((0, 1, 0, 0, 1, 0, 2, 0, 2), 0.017544),
    ((0, 1, 0, 0, 1, 1, 2, 0, 2), 0.000000),
    ((0, 1, 0, 0, 1, 0, 2, 1, 2), 1.000000),
    ((0, 1, 0, 0, 0, 1, 2, 0, 0), -0.092308),
    ((0, 1, 0, 0, 0, 1, 2, 2, 0), 0.069767),
    ((0, 1, 0, 0, 0, 1, 2, 2, 1), 0.428571),
    ((0, 1, 0, 0, 0, 1, 2, 0, 2), -0.209302),
    ((0, 1, 0, 0, 0, 1, 2, 1, 2), 0.047619),
    ((0, 1, 0, 0, 0, 0, 2, 1, 0), -0.085339),
    ((0, 1, 0, 0, 0, 0, 2, 1, 2), 0.189873),
    ((0, 1, 0, 0, 0, 0, 2, 0, 1), 0.144928),
    ((0, 1, 0, 0, 0, 0, 2, 2, 1), 0.490196),
    ((0, 1, 0, 0, 0, 0, 0, 2, 0), 0.309395),
    ((0, 1, 1, 0, 0, 0, 0, 2, 0), 0.315011),
    ((0, 1, 1, 0, 0, 0, 0, 2, 2), 0.194030),
    ((0, 1, 1, 1, 0, 0, 0, 2, 2), -0.058824),
    ((0, 1, 1, 0, 1, 0, 0, 2, 2), 0.285714),
    ((0, 1, 1, 0, 0, 1, 0, 2, 2), -0.058824),
    ((0, 1, 1, 0, 0, 0, 1, 2, 2), 0.555556),
    ((0, 1, 0, 1, 0, 0, 0, 2, 0), 0.220779),
    ((0, 1, 0, 1, 0, 0, 0, 2, 2), 0.069767),
    ((0, 1, 0, 1, 1, 0, 0, 2, 2), -0.058824),
    ((0, 1, 0, 1, 0, 1, 0, 2, 2), -0.058824),
    ((0, 1, 0, 1, 0, 0, 1, 2, 2), 0.428571),
    ((0, 1, 0, 0, 1, 0, 0, 2, 0), 0.270270),
    ((0, 1, 0, 0, 1, 0, 0, 2, 2), 0.116279),
    ((0, 1, 0, 0, 1, 1, 0, 2, 2), 0.176471),
    ((0, 1, 0, 0, 1, 0, 1, 2, 2), 0.428571),
    ((0, 1, 0, 0, 0, 1, 0, 2, 0), 0.220779),
    ((0, 1, 0, 0, 0, 1, 0, 2, 2), 0.173913),
    ((0, 1, 0, 0, 0, 1, 1, 2, 2), 0.500000),
    ((0, 1, 0, 0, 0, 0, 1, 2, 0), 0.407407),
    ((0, 1, 0, 0, 0, 0, 1, 2, 2), 0.490196),
    ((0, 1, 0, 0, 0, 0, 0, 2, 1), 0.407407),
    ((0, 1, 0, 0, 0, 0, 0, 0, 2), 0.057715),
    ((0, 1, 1, 0, 0, 0, 0, 0, 2), 0.103926),
    ((0, 1, 0, 1, 0, 0, 0, 0, 2), -0.092308),
    ((0, 1, 0, 0, 1, 0, 0, 0, 2), 0.251799),
    ((0, 1, 0, 0, 0, 1, 0, 0, 2), 0.055556),
    ((0, 1, 0, 0, 0, 0, 1, 0, 2), 0.144928),
    ((0, 1, 0, 0, 0, 0, 0, 1, 2), -0.085339),
    ((0, 0, 1, 0, 0, 0, 0, 0, 0), 0.243617),
    ((2, 0, 1, 0, 0, 0, 0, 0, 0), 0.197740),
    ((2, 0, 1, 1, 0, 0, 0, 0, 0), 0.263158),
    ((2, 2, 1, 1, 0, 0, 0, 0, 0), 0.490196),
    ((2, 2, 1, 1, 1, 0, 0, 0, 0), 0.777778),
    ((2, 2, 1, 1, 1, 2, 0, 0, 0), 0.600000),
    ((2, 2, 1, 1, 1, 2, 1, 0, 0), 1.000000),
    ((2, 2, 1, 1, 1, 2, 0, 1, 0), 0.500000),
    ((2, 2, 1, 1, 1, 2, 2, 1, 0), 0.000000),
    ((2, 2, 1, 1, 1, 2, 2, 1, 1), 0.000000),
    ((2, 2, 1, 1, 1, 2, 0, 1, 2), 1.000000),
    ((2, 2, 1, 1, 1, 2, 1, 1, 2), 1.000000),
    ((2, 2, 1, 1, 1, 2, 0, 0, 1), 0.500000),
    ((2, 2, 1, 1, 1, 2, 2, 0, 1), 0.000000),
    ((2, 2, 1, 1, 1, 2, 0, 2, 1), 1.000000),
    ((2, 2, 1, 1, 1, 2, 1, 2, 1), 1.000000),
    ((2, 2, 1, 1, 1, 0, 2, 0, 0), 0.600000),
    ((2, 2, 1, 1, 1, 1, 2, 0, 0), 1.000000),
    ((2, 2, 1, 1, 1, 0, 2, 1, 0), 0.500000),
    ((2, 2, 1, 1, 1, 0, 2, 1, 2), 1.000000),
    ((2, 2, 1, 1, 1, 1, 2, 1, 2), 1.000000),
    ((2, 2, 1, 1, 1, 0, 2, 0, 1), 0.500000),
    ((2, 2, 1, 1, 1, 0, 2, 2, 1), 1.000000),
    ((2, 2, 1, 1, 1, 1, 2, 2, 1), 1.000000),
    ((2, 2, 1, 1, 1, 0, 0, 2, 0), 1.000000),
    ((2, 2, 1, 1, 1, 1, 0, 2, 0), 1.000000),
    ((2, 2, 1, 1, 1, 0, 1, 2, 0), 1.000000),
    ((2, 2, 1, 1, 1, 0, 0, 2, 1), 1.000000),
    ((2, 2, 1, 1, 1, 0, 0, 0, 2), 1.000000),
    ((2, 2, 1, 1, 1, 1, 0, 0, 2), 1.000000),
    ((2, 2, 1, 1, 1, 0, 1, 0, 2), 1.000000),
    ((2, 2, 1, 1, 1, 0, 0, 1, 2), 1.000000),
    ((2, 2, 1, 1, 0, 1, 0, 0, 0), 0.333333),
    ((2, 2, 1, 1, 2, 1, 0, 0, 0), -0.200000),
    ((2, 2, 1, 1, 2, 1, 1, 0, 0), -1.000000),
    ((2, 2, 1, 1, 2, 1, 1, 2, 0), -1.000000),
    ((2, 2, 1, 1, 2, 1, 1, 0, 2), -1.000000),
    ((2, 2, 1, 1, 2, 1, 0, 1, 0), 0.000000),
    ((2, 2, 1, 1, 2, 1, 2, 1, 0), 1.000000),
    ((2, 2, 1, 1, 2, 1, 2, 1, 1), 1.000000),
    ((2, 2, 1, 1, 2, 1, 0, 1, 2), -1.000000),
    ((2, 2, 1, 1, 2, 1, 0, 0, 1), 1.000000),
    ((2, 2, 1, 1, 0, 1, 2, 0, 0), 1.000000),
    ((2, 2, 1, 1, 0, 1, 2, 1, 0), 1.000000),
    ((2, 2, 1, 1, 0, 1, 2, 1, 2), 1.000000),
    ((2, 2, 1, 1, 0, 1, 2, 0, 1), 1.000000),
    ((2, 2, 1, 1, 0, 1, 0, 2, 0), 0.500000),
    ((2, 2, 1, 1, 0, 1, 1, 2, 0), 0.000000),
    ((2, 2, 1, 1, 0, 1, 1, 2, 2), 1.000000),
    ((2, 2, 1, 1, 1, 1, 1, 2, 2), 1.000000),
    ((2, 2, 1, 1, 0, 1, 0, 2, 1), 1.000000),
    ((2, 2, 1, 1, 0, 1, 0, 0, 2), 0.200000),
    ((2, 2, 1, 1, 0, 1, 1, 0, 2), 0.000000),
    ((2, 2, 1, 1, 0, 1, 0, 1, 2), 0.000000),
    ((2, 2, 1, 1, 0, 0, 1, 0, 0), 0.238095),
    ((2, 2, 1, 1, 2, 0, 1, 0, 0), -0.333333),
    ((2, 2, 1, 1, 2, 0, 1, 1, 0), 0.000000),
    ((2, 2, 1, 1, 2, 2, 1, 1, 0), 1.000000),
    ((2, 2, 1, 1, 2, 2, 1, 1, 1), 1.000000),
    ((2, 2, 1, 1, 2, 0, 1, 1, 2), -1.000000),
    ((2, 2, 1, 1, 2, 0, 1, 0, 1), 0.000000),
    ((2, 2, 1, 1, 2, 2, 1, 0, 1), 1.000000),
    ((2, 2, 1, 1, 2, 0, 1, 2, 1), -1.000000),
    ((2, 2, 1, 1, 0, 2, 1, 0, 0), 1.000000),
    ((2, 2, 1, 1, 0, 2, 1, 1, 0), 1.000000),
    ((2, 2, 1, 1, 0, 2, 1, 1, 2), 1.000000),
    ((2, 2, 1, 1, 0, 2, 1, 0, 1), 1.000000),
    ((2, 2, 1, 1, 0, 2, 1, 2, 1), 1.000000),
    ((2, 2, 1, 1, 0, 0, 1, 2, 0), 0.200000),
    ((2, 2, 1, 1, 0, 0, 1, 2, 1), 0.000000),
    ((2, 2, 1, 1, 0, 0, 1, 0, 2), 0.200000),
    ((2, 2, 1, 1, 0, 0, 1, 1, 2), 0.000000),
    ((2, 2, 1, 1, 0, 0, 0, 1, 0), 0.500000),
    ((2, 2, 1, 1, 2, 0, 0, 1, 0), 0.333333),
    ((2, 2, 1, 1, 2, 0, 0, 1, 1), 1.000000),
    ((2, 2, 1, 1, 2, 2, 0, 1, 1), 1.000000),
    ((2, 2, 1, 1, 2, 0, 2, 1, 1), 1.000000),
    ((2, 2, 1, 1, 0, 2, 0, 1, 0), 0.666667),
    ((2, 2, 1, 1, 0, 2, 0, 1, 1), 0.500000),
    ((2, 2, 1, 1, 0, 2, 2, 1, 1), 0.000000),
    ((2, 2, 1, 1, 0, 0, 2, 1, 0), 0.666667),
    ((2, 2, 1, 1, 0, 0, 2, 1, 1), 0.500000),
    ((2, 2, 1, 1, 0, 0, 0, 1, 2), 0.333333),
    ((2, 2, 1, 1, 0, 0, 0, 0, 1), 0.619048),
    ((2, 2, 1, 1, 2, 0, 0, 0, 1), 0.600000),
    ((2, 2, 1, 1, 0, 2, 0, 0, 1), 0.666667),
    ((2, 2, 1, 1, 0, 0, 2, 0, 1), 0.600000),
    ((2, 2, 1, 1, 0, 0, 0, 2, 1), 0.600000),
    ((2, 0, 1, 1, 2, 0, 0, 0, 0), -0.021277),
    ((2, 0, 1, 1, 2, 1, 0, 0, 0), -0.125000),
    ((2, 0, 1, 1, 2, 1, 2, 0, 0), 0.200000),
    ((2, 0, 1, 1, 2, 1, 2, 1, 0), 0.000000),
    ((2, 0, 1, 1, 2, 1, 2, 1, 2), -1.000000),
    ((2, 0, 1, 1, 2, 1, 2, 0, 1), 1.000000),
    ((2, 0, 1, 1, 2, 1, 0, 2, 0), -0.200000),
    ((2, 0, 1, 1, 2, 1, 1, 2, 0), -1.000000),
    ((2, 0, 1, 1, 2, 1, 1, 2, 2), -1.000000),
    ((2, 0, 1, 1, 2, 1, 0, 2, 1), 1.000000),
    ((2, 0, 1, 1, 2, 1, 0, 0, 2), -1.000000),
    ((2, 0, 1, 1, 2, 0, 1, 0, 0), -0.368421),
    ((2, 0, 1, 1, 2, 2, 1, 0, 0), 0.000000),
    ((2, 0, 1, 1, 2, 2, 1, 1, 0), 0.000000),
    ((2, 0, 1, 1, 2, 2, 1, 1, 2), -1.000000),
    ((2, 0, 1, 1, 2, 2, 1, 0, 1), 0.500000),
    ((2, 0, 1, 1, 2, 2, 1, 2, 1), 0.000000),
    ((2, 0, 1, 1, 2, 0, 1, 2, 0), -0.666667),
    ((2, 0, 1, 1, 2, 0, 1, 2, 1), -0.500000),
    ((2, 0, 1, 1, 2, 0, 1, 0, 2), -1.000000),
    ((2, 0, 1, 1, 2, 0, 0, 1, 0), 0.052632),
    ((2, 0, 1, 1, 2, 2, 0, 1, 0), 0.000000),
    ((2, 0, 1, 1, 2, 2, 0, 1, 1), 0.500000),
    ((2, 0, 1, 1, 2, 2, 2, 1, 1), 0.000000),
    ((2, 0, 1, 1, 2, 0, 2, 1, 0), 0.000000),
    ((2, 0, 1, 1, 2, 0, 2, 1, 1), 0.500000),
    ((2, 0, 1, 1, 2, 0, 0, 1, 2), -1.000000),
    ((2, 0, 1, 1, 2, 0, 0, 0, 1), 0.428571),
    ((2, 0, 1, 1, 2, 2, 0, 0, 1), 0.333333),
    ((2, 0, 1, 1, 2, 0, 2, 0, 1), 0.600000),
    ((2, 0, 1, 1, 2, 0, 0, 2, 1), 0.200000),
    ((2, 0, 1, 1, 0, 2, 0, 0, 0), 0.368421),
    ((2, 0, 1, 1, 1, 2, 0, 0, 0), 0.619048),
    ((2, 0, 1, 1, 1, 2, 2, 0, 0), 0.333333),
    ((2, 0, 1, 1, 1, 2, 2, 1, 0), 0.500000),
    ((2, 0, 1, 1, 1, 2, 2, 1, 2), 1.000000),
    ((2, 0, 1, 1, 1, 2, 2, 0, 1), 0.000000),
    ((2, 0, 1, 1, 1, 2, 2, 2, 1), 0.000000),
    ((2, 0, 1, 1, 1, 2, 0, 2, 0), 0.600000),
    ((2, 0, 1, 1, 1, 2, 1, 2, 0), 1.000000),
    ((2, 0, 1, 1, 1, 2, 0, 2, 1), 0.500000),
    ((2, 0, 1, 1, 1, 2, 0, 0, 2), 1.000000),
    ((2, 0, 1, 1, 1, 2, 1, 0, 2), 1.000000),
    ((2, 0, 1, 1, 1, 2, 0, 1, 2), 1.000000),
    ((2, 0, 1, 1, 0, 2, 1, 0, 0), 0.428571),
    ((2, 0, 1, 1, 0, 2, 1, 2, 0), 0.600000),
    ((2, 0, 1, 1, 0, 2, 1, 2, 1), 0.500000),
    ((2, 0, 1, 1, 0, 2, 1, 0, 2), 0.200000),
    ((2, 0, 1, 1, 0, 2, 1, 1, 2), 0.000000),
    ((2, 0, 1, 1, 0, 2, 0, 1, 0), 0.333333),
    ((2, 0, 1, 1, 0, 2, 2, 1, 0), 0.333333),
    ((2, 0, 1, 1, 0, 2, 2, 1, 1), 0.000000),
    ((2, 0, 1, 1, 0, 2, 0, 1, 2), 0.333333),
    ((2, 0, 1, 1, 0, 2, 0, 0, 1), 0.333333),
    ((2, 0, 1, 1, 0, 2, 2, 0, 1), 0.000000),
    ((2, 0, 1, 1, 0, 2, 0, 2, 1), 0.333333),
    ((2, 0, 1, 1, 0, 0, 2, 0, 0), 0.296296),
    ((2, 0, 1, 1, 1, 0, 2, 0, 0), 0.428571),
    ((2, 0, 1, 1, 1, 0, 2, 2, 0), 0.200000),
    ((2, 0, 1, 1, 1, 1, 2, 2, 0), 1.000000),
    ((2, 0, 1, 1, 1, 0, 2, 2, 1), 0.500000),
    ((2, 0, 1, 1, 1, 0, 2, 0, 2), 0.600000),
    ((2, 0, 1, 1, 1, 1, 2, 0, 2), 1.000000),
    ((2, 0, 1, 1, 1, 0, 2, 1, 2), 1.000000),
    ((2, 0, 1, 1, 0, 1, 2, 0, 0), 0.333333),
    ((2, 0, 1, 1, 0, 1, 2, 2, 0), 0.500000),
    ((2, 0, 1, 1, 0, 1, 2, 2, 1), 1.000000),
    ((2, 0, 1, 1, 0, 1, 2, 0, 2), -0.200000),
    ((2, 0, 1, 1, 0, 1, 2, 1, 2), 0.000000),
    ((2, 0, 1, 1, 0, 0, 2, 1, 0), 0.333333),
    ((2, 0, 1, 1, 0, 0, 2, 1, 2), 0.333333),
    ((2, 0, 1, 1, 0, 0, 2, 0, 1), 0.428571),
    ((2, 0, 1, 1, 0, 0, 2, 2, 1), 0.600000),
    ((2, 0, 1, 1, 0, 0, 0, 2, 0), 0.215686),
    ((2, 0, 1, 1, 1, 0, 0, 2, 0), 0.555556),
    ((2, 0, 1, 1, 1, 0, 0, 2, 2), 0.500000),
    ((2, 0, 1, 1, 1, 1, 0, 2, 2), 1.000000),
    ((2, 0, 1, 1, 1, 0, 1, 2, 2), 1.000000),
    ((2, 0, 1, 1, 0, 1, 0, 2, 0), 0.111111),
    ((2, 0, 1, 1, 0, 1, 0, 2, 2), -0.200000),
    ((2, 0, 1, 1, 0, 1, 1, 2, 2), 0.000000),
    ((2, 0, 1, 1, 0, 0, 1, 2, 0), 0.047619),
    ((2, 0, 1, 1, 0, 0, 1, 2, 2), 0.200000),
    ((2, 0, 1, 1, 0, 0, 0, 2, 1), 0.428571),
    ((2, 0, 1, 1, 0, 0, 0, 0, 2), 0.181818),
    ((2, 0, 1, 1, 1, 0, 0, 0, 2), 0.777778),
    ((2, 0, 1, 1, 0, 1, 0, 0, 2), -0.125000),
    ((2, 0, 1, 1, 0, 0, 1, 0, 2), 0.125000),
    ((2, 0, 1, 1, 0, 0, 0, 1, 2), 0.263158),
    ((2, 0, 1, 0, 1, 0, 0, 0, 0), 0.401361),
    ((2, 2, 1, 0, 1, 0, 0, 0, 0), 0.616438),
    ((2, 2, 1, 0, 1, 1, 0, 0, 0), 0.733333),
    ((2, 2, 1, 2, 1, 1, 0, 0, 0), 0.500000),
    ((2, 2, 1, 2, 1, 1, 1, 0, 0), 1.000000),
    ((2, 2, 1, 2, 1, 1, 0, 1, 0), 0.000000),
    ((2, 2, 1, 2, 1, 1, 2, 1, 0), -1.000000),
    ((2, 2, 1, 2, 1, 1, 0, 1, 2), 1.000000),
    ((2, 2, 1, 2, 1, 1, 1, 1, 2), 1.000000),
    ((2, 2, 1, 2, 1, 1, 0, 0, 1), 1.000000),
    ((2, 2, 1, 0, 1, 1, 2, 0, 0), 0.500000),
    ((2, 2, 1, 0, 1, 1, 2, 1, 0), 0.000000),
    ((2, 2, 1, 0, 1, 1, 2, 1, 2), 1.000000),
    ((2, 2, 1, 0, 1, 1, 2, 0, 1), 1.000000),
    ((2, 2, 1, 0, 1, 1, 0, 2, 0), 1.000000),
    ((2, 2, 1, 0, 1, 1, 1, 2, 0), 1.000000),
    ((2, 2, 1, 0, 1, 1, 0, 2, 1), 1.000000),
    ((2, 2, 1, 0, 1, 1, 0, 0, 2), 1.000000),
    ((2, 2, 1, 0, 1, 1, 1, 0, 2), 1.000000),
    ((2, 2, 1, 0, 1, 1, 0, 1, 2), 1.000000),
    ((2, 2, 1, 0, 1, 0, 1, 0, 0), 1.000000),
    ((2, 2, 1, 0, 1, 0, 0, 1, 0), 0.428571),
    ((2, 2, 1, 2, 1, 0, 0, 1, 0), 0.200000),
    ((2, 2, 1, 2, 1, 0, 1, 1, 0), 1.000000),
    ((2, 2, 1, 2, 1, 0, 0, 1, 1), 0.000000),
    ((2, 2, 1, 2, 1, 2, 0, 1, 1), 1.000000),
    ((2, 2, 1, 2, 1, 2, 1, 1, 1), 1.000000),
    ((2, 2, 1, 2, 1, 0, 2, 1, 1), -1.000000),
    ((2, 2, 1, 0, 1, 2, 0, 1, 0), 0.600000),
    ((2, 2, 1, 0, 1, 2, 1, 1, 0), 1.000000),
    ((2, 2, 1, 0, 1, 2, 0, 1, 1), 0.500000),
    ((2, 2, 1, 0, 1, 2, 2, 1, 1), 0.000000),
    ((2, 2, 1, 0, 1, 0, 2, 1, 0), 0.000000),
    ((2, 2, 1, 0, 1, 0, 2, 1, 1), -0.500000),
    ((2, 2, 1, 0, 1, 0, 0, 1, 2), 1.000000),
    ((2, 2, 1, 0, 1, 0, 1, 1, 2), 1.000000),
    ((2, 2, 1, 0, 1, 0, 0, 0, 1), 0.555556),
    ((2, 2, 1, 2, 1, 0, 0, 0, 1), 0.500000),
    ((2, 2, 1, 2, 1, 0, 1, 0, 1), 1.000000),
    ((2, 2, 1, 0, 1, 2, 0, 0, 1), 0.600000),
    ((2, 2, 1, 0, 1, 2, 1, 0, 1), 1.000000),
    ((2, 2, 1, 0, 1, 0, 2, 0, 1), 0.200000),
    ((2, 2, 1, 0, 1, 0, 0, 2, 1), 1.000000),
    ((2, 2, 1, 0, 1, 0, 1, 2, 1), 1.000000),
    ((2, 0, 1, 2, 1, 0, 0, 0, 0), 0.298246),
    ((2, 0, 1, 2, 1, 1, 0, 0, 0), 0.285714),
    ((2, 0, 1, 2, 1, 1, 2, 0, 0), -1.000000),
    ((2, 0, 1, 2, 1, 1, 0, 2, 0), 0.500000),
    ((2, 0, 1, 2, 1, 1, 1, 2, 0), 1.000000),
    ((2, 0, 1, 2, 1, 1, 0, 2, 1), 1.000000),
    ((2, 0, 1, 2, 1, 1, 0, 0, 2), 0.200000),
    ((2, 0, 1, 2, 1, 1, 1, 0, 2), 1.000000),
    ((2, 0, 1, 2, 1, 1, 0, 1, 2), 0.000000),
    ((2, 0, 1, 2, 1, 1, 2, 1, 2), -1.000000),
    ((2, 0, 1, 2, 1, 0, 1, 0, 0), 1.000000),
    ((2, 0, 1, 2, 1, 0, 0, 1, 0), 0.285714),
    ((2, 0, 1, 2, 1, 2, 0, 1, 0), 0.500000),
    ((2, 0, 1, 2, 1, 2, 1, 1, 0), 1.000000),
    ((2, 0, 1, 2, 1, 2, 0, 1, 1), 0.000000),
    ((2, 0, 1, 2, 1, 2, 2, 1, 1), -1.000000),
    ((2, 0, 1, 2, 1, 0, 2, 1, 0), -1.000000),
    ((2, 0, 1, 2, 1, 0, 0, 1, 2), 0.500000),
    ((2, 0, 1, 2, 1, 0, 1, 1, 2), 1.000000),
    ((2, 0, 1, 2, 1, 0, 0, 0, 1), 0.285714),
    ((2, 0, 1, 2, 1, 2, 0, 0, 1), 0.200000),
    ((2, 0, 1, 2, 1, 2, 1, 0, 1), 1.000000),
    ((2, 0, 1, 2, 1, 0, 2, 0, 1), -1.000000),
    ((2, 0, 1, 2, 1, 0, 0, 2, 1), 0.500000),
    ((2, 0, 1, 2, 1, 0, 1, 2, 1), 1.000000),
    ((2, 0, 1, 0, 1, 2, 0, 0, 0), 0.493671),
    ((2, 0, 1, 0, 1, 2, 1, 0, 0), 1.000000),
    ((2, 0, 1, 0, 1, 2, 0, 1, 0), 0.555556),
    ((2, 0, 1, 0, 1, 2, 2, 1, 0), 0.200000),
    ((2, 0, 1, 0, 1, 2, 2, 1, 1), -0.500000),
    ((2, 0, 1, 0, 1, 2, 0, 1, 2), 1.000000),
    ((2, 0, 1, 0, 1, 2, 1, 1, 2), 1.000000),
    ((2, 0, 1, 0, 1, 2, 0, 0, 1), 0.238095),
    ((2, 0, 1, 0, 1, 2, 2, 0, 1), -0.333333),
    ((2, 0, 1, 0, 1, 2, 0, 2, 1), 0.600000),
    ((2, 0, 1, 0, 1, 2, 1, 2, 1), 1.000000),
    ((2, 0, 1, 0, 1, 0, 2, 0, 0), 0.069767),
    ((2, 0, 1, 0, 1, 1, 2, 0, 0), 0.000000),
    ((2, 0, 1, 0, 1, 1, 2, 2, 0), 0.000000),
    ((2, 0, 1, 0, 1, 1, 2, 2, 1), 1.000000),
    ((2, 0, 1, 0, 1, 1, 2, 0, 2), -0.200000),
    ((2, 0, 1, 0, 1, 1, 2, 1, 2), 0.000000),
    ((2, 0, 1, 0, 1, 0, 2, 1, 0), 0.176471),
    ((2, 0, 1, 0, 1, 0, 2, 1, 2), 0.600000),
    ((2, 0, 1, 0, 1, 0, 2, 0, 1), -0.058824),
    ((2, 0, 1, 0, 1, 0, 2, 2, 1), 0.200000),
    ((2, 0, 1, 0, 1, 0, 0, 2, 0), 0.397260),
    ((2, 0, 1, 0, 1, 1, 0, 2, 0), 0.466667),
    ((2, 0, 1, 0, 1, 1, 0, 2, 2), 0.500000),
    ((2, 0, 1, 0, 1, 1, 1, 2, 2), 1.000000),
    ((2, 0, 1, 0, 1, 0, 1, 2, 0), 1.000000),
    ((2, 0, 1, 0, 1, 0, 0, 2, 1), 0.555556),
    ((2, 0, 1, 0, 1, 0, 0, 0, 2), 0.561644),
    ((2, 0, 1, 0, 1, 1, 0, 0, 2), 0.333333),
    ((2, 0, 1, 0, 1, 0, 1, 0, 2), 1.000000),
    ((2, 0, 1, 0, 1, 0, 0, 1, 2), 0.777778),
    ((2, 0, 1, 0, 0, 1, 0, 0, 0), 0.103926),
    ((2, 2, 1, 0, 0, 1, 0, 0, 0), 0.452055),
    ((2, 2, 1, 0, 0, 1, 1, 0, 0), 0.333333),
    ((2, 2, 1, 2, 0, 1, 1, 0, 0), 1.000000),
    ((2, 2, 1, 2, 0, 1, 1, 1, 0), 1.000000),
    ((2, 2, 1, 2, 2, 1, 1, 1, 0), 1.000000),
    ((2, 2, 1, 2, 2, 1, 1, 1, 1), 1.000000),
    ((2, 2, 1, 2, 0, 1, 1, 1, 2), 1.000000),
    ((2, 2, 1, 2, 0, 1, 1, 0, 1), 1.000000),
    ((2, 2, 1, 0, 2, 1, 1, 0, 0), -0.200000),
    ((2, 2, 1, 0, 2, 1, 1, 1, 0), 0.000000),
    ((2, 2, 1, 0, 2, 1, 1, 1, 2), -1.000000),
    ((2, 2, 1, 0, 2, 1, 1, 0, 1), 1.000000),
    ((2, 2, 1, 0, 0, 1, 1, 2, 0), 0.500000),
    ((2, 2, 1, 0, 0, 1, 1, 2, 1), 1.000000),
    ((2, 2, 1, 0, 0, 1, 1, 0, 2), 0.200000),
    ((2, 2, 1, 0, 0, 1, 1, 1, 2), 0.000000),
    ((2, 2, 1, 0, 0, 1, 0, 1, 0), 0.428571),
    ((2, 2, 1, 2, 0, 1, 0, 1, 0), 0.600000),
    ((2, 2, 1, 2, 0, 1, 0, 1, 1), 1.000000),
    ((2, 2, 1, 0, 2, 1, 0, 1, 0), 0.200000),
    ((2, 2, 1, 0, 2, 1, 0, 1, 1), 1.000000),
    ((2, 2, 1, 0, 0, 1, 2, 1, 0), 0.600000),
    ((2, 2, 1, 0, 0, 1, 2, 1, 1), 1.000000),
    ((2, 2, 1, 0, 0, 1, 0, 1, 2), 0.333333),
    ((2, 2, 1, 0, 0, 1, 0, 0, 1), 1.000000),
    ((2, 0, 1, 2, 0, 1, 0, 0, 0), 0.194030),
    ((2, 0, 1, 2, 0, 1, 1, 0, 0), 0.555556),
    ((2, 0, 1, 2, 2, 1, 1, 0, 0), 0.200000),
    ((2, 0, 1, 2, 2, 1, 1, 1, 0), 0.000000),
    ((2, 0, 1, 2, 2, 1, 1, 1, 2), -1.000000),
    ((2, 0, 1, 2, 2, 1, 1, 0, 1), 1.000000),
    ((2, 0, 1, 2, 0, 1, 1, 2, 0), 1.000000),
    ((2, 0, 1, 2, 0, 1, 1, 2, 1), 1.000000),
    ((2, 0, 1, 2, 0, 1, 1, 0, 2), 0.200000),
    ((2, 0, 1, 2, 0, 1, 1, 1, 2), 0.000000),
    ((2, 0, 1, 2, 0, 1, 0, 1, 0), -0.058824),
    ((2, 0, 1, 2, 2, 1, 0, 1, 0), -0.200000),
    ((2, 0, 1, 2, 2, 1, 0, 1, 1), 1.000000),
    ((2, 0, 1, 2, 0, 1, 2, 1, 0), -1.000000),
    ((2, 0, 1, 2, 0, 1, 0, 1, 2), -0.333333),
    ((2, 0, 1, 2, 0, 1, 0, 0, 1), 1.000000),
    ((2, 0, 1, 0, 2, 1, 0, 0, 0), -0.107692),
    ((2, 0, 1, 0, 2, 1, 1, 0, 0), -0.125000),
    ((2, 0, 1, 0, 2, 1, 1, 2, 0), -0.200000),
    ((2, 0, 1, 0, 2, 1, 1, 2, 1), 1.000000),
    ((2, 0, 1, 0, 2, 1, 1, 0, 2), -1.000000),
    ((2, 0, 1, 0, 2, 1, 0, 1, 0), -0.125000),
    ((2, 0, 1, 0, 2, 1, 2, 1, 0), -0.200000),
    ((2, 0, 1, 0, 2, 1, 2, 1, 1), 1.000000),
    ((2, 0, 1, 0, 2, 1, 0, 1, 2), -1.000000),
    ((2, 0, 1, 0, 2, 1, 0, 0, 1), 1.000000),
    ((2, 0, 1, 0, 0, 1, 2, 0, 0), -0.044776),
    ((2, 0, 1, 0, 0, 1, 2, 1, 0), -0.058824),
    ((2, 0, 1, 0, 0, 1, 2, 1, 2), -0.333333),
    ((2, 0, 1, 0, 0, 1, 2, 0, 1), 1.000000),
    ((2, 0, 1, 0, 0, 1, 0, 2, 0), 0.232877),
    ((2, 0, 1, 0, 0, 1, 1, 2, 0), 0.333333),
    ((2, 0, 1, 0, 0, 1, 1, 2, 2), 0.200000),
    ((2, 0, 1, 0, 0, 1, 0, 2, 1), 1.000000),
    ((2, 0, 1, 0, 0, 1, 0, 0, 2), -0.090909),
    ((2, 0, 1, 0, 0, 1, 1, 0, 2), 0.125000),
    ((2, 0, 1, 0, 0, 1, 0, 1, 2), -0.157895),
    ((2, 0, 1, 0, 0, 0, 1, 0, 0), 0.256018),
    ((2, 2, 1, 0, 0, 0, 1, 0, 0), 0.452055),
    ((2, 2, 1, 0, 0, 0, 1, 1, 0), 0.555556),
    ((2, 2, 1, 2, 0, 0, 1, 1, 0), 1.000000),
    ((2, 2, 1, 2, 0, 0, 1, 1, 1), 1.000000),
    ((2, 2, 1, 0, 2, 0, 1, 1, 0), 0.200000),
    ((2, 2, 1, 0, 2, 0, 1, 1, 1), 1.000000),
    ((2, 2, 1, 0, 0, 2, 1, 1, 0), 1.000000),
    ((2, 2, 1, 0, 0, 2, 1, 1, 1), 1.000000),
    ((2, 2, 1, 0, 0, 0, 1, 1, 2), 0.200000),
    ((2, 2, 1, 0, 0, 0, 1, 0, 1), 0.733333),
    ((2, 2, 1, 2, 0, 0, 1, 0, 1), 1.000000),
    ((2, 2, 1, 0, 2, 0, 1, 0, 1), 0.500000),
    ((2, 2, 1, 0, 0, 2, 1, 0, 1), 1.000000),
    ((2, 2, 1, 0, 0, 0, 1, 2, 1), 0.500000),
    ((2, 0, 1, 2, 0, 0, 1, 0, 0), 0.452055),
    ((2, 0, 1, 2, 0, 0, 1, 1, 0), 0.333333),
    ((2, 0, 1, 2, 2, 0, 1, 1, 0), -0.200000),
    ((2, 0, 1, 2, 2, 0, 1, 1, 1), 1.000000),
    ((2, 0, 1, 2, 0, 2, 1, 1, 0), 0.500000),
    ((2, 0, 1, 2, 0, 2, 1, 1, 1), 1.000000),
    ((2, 0, 1, 2, 0, 0, 1, 1, 2), 0.200000),
    ((2, 0, 1, 2, 0, 0, 1, 0, 1), 0.733333),
    ((2, 0, 1, 2, 2, 0, 1, 0, 1), 0.500000),
    ((2, 0, 1, 2, 0, 2, 1, 0, 1), 0.500000),
    ((2, 0, 1, 2, 0, 0, 1, 2, 1), 1.000000),
    ((2, 0, 1, 0, 2, 0, 1, 0, 0), -0.136364),
    ((2, 0, 1, 0, 2, 0, 1, 1, 0), -0.125000),
    ((2, 0, 1, 0, 2, 2, 1, 1, 0), -0.200000),
    ((2, 0, 1, 0, 2, 2, 1, 1, 1), 1.000000),
    ((2, 0, 1, 0, 2, 0, 1, 1, 2), -1.000000),
    ((2, 0, 1, 0, 2, 0, 1, 0, 1), 0.333333),
    ((2, 0, 1, 0, 2, 2, 1, 0, 1), 0.200000),
    ((2, 0, 1, 0, 2, 0, 1, 2, 1), 0.200000),
    ((2, 0, 1, 0, 0, 2, 1, 0, 0), 0.341772),
    ((2, 0, 1, 0, 0, 2, 1, 1, 0), 0.333333),
    ((2, 0, 1, 0, 0, 2, 1, 1, 2), 0.200000),
    ((2, 0, 1, 0, 0, 2, 1, 0, 1), 0.555556),
    ((2, 0, 1, 0, 0, 2, 1, 2, 1), 0.600000),
    ((2, 0, 1, 0, 0, 0, 1, 2, 0), 0.341772),
    ((2, 0, 1, 0, 0, 0, 1, 2, 1), 0.555556),
    ((2, 0, 1, 0, 0, 0, 1, 0, 2), 0.138462),
    ((2, 0, 1, 0, 0, 0, 1, 1, 2), 0.125000),
    ((2, 0, 1, 0, 0, 0, 0, 1, 0), 0.144928),
    ((2, 2, 1, 0, 0, 0, 0, 1, 0), 0.490196),
    ((2, 2, 1, 0, 0, 0, 0, 1, 1), 0.555556),
    ((2, 2, 1, 2, 0, 0, 0, 1, 1), 0.500000),
    ((2, 2, 1, 0, 2, 0, 0, 1, 1), 1.000000),
    ((2, 2, 1, 0, 0, 2, 0, 1, 1), 0.600000),
    ((2, 2, 1, 0, 0, 0, 2, 1, 1), 0.200000),
    ((2, 0, 1, 2, 0, 0, 0, 1, 0), 0.000000),
    ((2, 0, 1, 2, 0, 0, 0, 1, 1), 0.000000),
    ((2, 0, 1, 2, 2, 0, 0, 1, 1), 0.000000),
    ((2, 0, 1, 2, 0, 2, 0, 1, 1), -0.200000),
    ((2, 0, 1, 2, 0, 0, 2, 1, 1), -1.000000),
    ((2, 0, 1, 0, 2, 0, 0, 1, 0), -0.136364),
    ((2, 0, 1, 0, 2, 0, 0, 1, 1), 0.333333),
    ((2, 0, 1, 0, 2, 2, 0, 1, 1), 0.200000),
    ((2, 0, 1, 0, 2, 0, 2, 1, 1), 0.200000),
    ((2, 0, 1, 0, 0, 2, 0, 1, 0), 0.215686),
    ((2, 0, 1, 0, 0, 2, 0, 1, 1), 0.047619),
    ((2, 0, 1, 0, 0, 2, 2, 1, 1), -0.333333),
    ((2, 0, 1, 0, 0, 0, 2, 1, 0), 0.043478),
    ((2, 0, 1, 0, 0, 0, 2, 1, 1), -0.058824),
    ((2, 0, 1, 0, 0, 0, 0, 1, 2), 0.181818),
    ((2, 0, 1, 0, 0, 0, 0, 0, 1), 0.315011),
    ((2, 2, 1, 0, 0, 0, 0, 0, 1), 0.616438),
    ((2, 0, 1, 2, 0, 0, 0, 0, 1), 0.245902),
    ((2, 0, 1, 0, 2, 0, 0, 0, 1), 0.291139),
    ((2, 0, 1, 0, 0, 2, 0, 0, 1), 0.185185),
    ((2, 0, 1, 0, 0, 0, 2, 0, 1), 0.095890),
    ((2, 0, 1, 0, 0, 0, 0, 2, 1), 0.493671),
    ((0, 2, 1, 0, 0, 0, 0, 0, 0), 0.352236),
    ((0, 2, 1, 1, 0, 0, 0, 0, 0), 0.310811),
    ((0, 2, 1, 1, 2, 0, 0, 0, 0), 0.043478),
    ((0, 2, 1, 1, 2, 1, 0, 0, 0), -0.294118),
    ((0, 2, 1, 1, 2, 1, 2, 0, 0), 0.200000),
    ((0, 2, 1, 1, 2, 1, 2, 1, 0), 0.500000),
    ((0, 2, 1, 1, 2, 1, 2, 1, 2), 0.000000),
    ((0, 2, 1, 1, 2, 1, 2, 0, 1), 1.000000),
    ((0, 2, 1, 1, 2, 1, 0, 2, 0), -1.000000),
    ((0, 2, 1, 1, 2, 1, 0, 0, 2), -0.666667),
    ((0, 2, 1, 1, 2, 1, 1, 0, 2), -1.000000),
    ((0, 2, 1, 1, 2, 1, 1, 2, 2), -1.000000),
    ((0, 2, 1, 1, 2, 1, 0, 1, 2), -0.500000),
    ((0, 2, 1, 1, 2, 0, 1, 0, 0), -0.058824),
    ((0, 2, 1, 1, 2, 2, 1, 0, 0), 0.600000),
    ((0, 2, 1, 1, 2, 2, 1, 1, 0), 1.000000),
    ((0, 2, 1, 1, 2, 2, 1, 1, 2), 1.000000),
    ((0, 2, 1, 1, 2, 2, 1, 0, 1), 0.000000),
    ((0, 2, 1, 1, 2, 2, 1, 2, 1), -1.000000),
    ((0, 2, 1, 1, 2, 0, 1, 2, 0), -1.000000),
    ((0, 2, 1, 1, 2, 0, 1, 0, 2), -0.200000),
    ((0, 2, 1, 1, 2, 0, 1, 1, 2), 0.000000),
    ((0, 2, 1, 1, 2, 0, 0, 1, 0), 0.333333),
    ((0, 2, 1, 1, 2, 2, 0, 1, 0), 0.666667),
    ((0, 2, 1, 1, 2, 2, 0, 1, 1), 0.500000),
    ((0, 2, 1, 1, 2, 2, 2, 1, 1), 0.000000),
    ((0, 2, 1, 1, 2, 0, 2, 1, 0), 0.333333),
    ((0, 2, 1, 1, 2, 0, 2, 1, 1), 0.500000),
    ((0, 2, 1, 1, 2, 0, 0, 1, 2), 0.000000),
    ((0, 2, 1, 1, 2, 0, 0, 0, 1), 0.176471),
    ((0, 2, 1, 1, 2, 2, 0, 0, 1), 0.000000),
    ((0, 2, 1, 1, 2, 0, 2, 0, 1), 0.200000),
    ((0, 2, 1, 1, 2, 0, 0, 2, 1), -1.000000),
    ((0, 2, 1, 1, 0, 2, 0, 0, 0), 0.518519),
    ((0, 2, 1, 1, 1, 2, 0, 0, 0), 0.619048),
    ((0, 2, 1, 1, 1, 2, 2, 0, 0), 0.333333),
    ((0, 2, 1, 1, 1, 2, 2, 1, 0), 0.000000),
    ((0, 2, 1, 1, 1, 2, 2, 1, 2), 0.000000),
    ((0, 2, 1, 1, 1, 2, 2, 0, 1), 0.500000),
    ((0, 2, 1, 1, 1, 2, 2, 2, 1), 1.000000),
    ((0, 2, 1, 1, 1, 2, 0, 2, 0), 1.000000),
    ((0, 2, 1, 1, 1, 2, 1, 2, 0), 1.000000),
    ((0, 2, 1, 1, 1, 2, 0, 2, 1), 1.000000),
    ((0, 2, 1, 1, 1, 2, 0, 0, 2), 0.600000),
    ((0, 2, 1, 1, 1, 2, 1, 0, 2), 1.000000),
    ((0, 2, 1, 1, 1, 2, 0, 1, 2), 0.500000),
    ((0, 2, 1, 1, 0, 2, 1, 0, 0), 0.777778),
    ((0, 2, 1, 1, 0, 2, 1, 2, 0), 0.500000),
    ((0, 2, 1, 1, 0, 2, 1, 2, 1), 0.000000),
    ((0, 2, 1, 1, 0, 2, 1, 0, 2), 1.000000),
    ((0, 2, 1, 1, 0, 2, 1, 1, 2), 1.000000),
    ((0, 2, 1, 1, 0, 2, 0, 1, 0), 0.500000),
    ((0, 2, 1, 1, 0, 2, 2, 1, 0), 0.000000),
    ((0, 2, 1, 1, 0, 2, 2, 1, 1), 0.000000),
    ((0, 2, 1, 1, 0, 2, 0, 1, 2), 0.666667),
    ((0, 2, 1, 1, 0, 2, 0, 0, 1), 0.333333),
    ((0, 2, 1, 1, 0, 2, 2, 0, 1), 0.333333),
    ((0, 2, 1, 1, 0, 2, 0, 2, 1), 0.333333),
    ((0, 2, 1, 1, 0, 0, 2, 0, 0), 0.259259),
    ((0, 2, 1, 1, 1, 0, 2, 0, 0), 0.428571),
    ((0, 2, 1, 1, 1, 0, 2, 2, 0), 0.600000),
    ((0, 2, 1, 1, 1, 1, 2, 2, 0), 1.000000),
    ((0, 2, 1, 1, 1, 0, 2, 2, 1), 1.000000),
    ((0, 2, 1, 1, 1, 0, 2, 0, 2), 0.200000),
    ((0, 2, 1, 1, 1, 1, 2, 0, 2), 1.000000),
    ((0, 2, 1, 1, 1, 0, 2, 1, 2), 0.500000),
    ((0, 2, 1, 1, 0, 1, 2, 0, 0), 0.333333),
    ((0, 2, 1, 1, 0, 1, 2, 2, 0), 0.000000),
    ((0, 2, 1, 1, 0, 1, 2, 2, 1), 1.000000),
    ((0, 2, 1, 1, 0, 1, 2, 0, 2), 0.200000),
    ((0, 2, 1, 1, 0, 1, 2, 1, 2), 0.500000),
    ((0, 2, 1, 1, 0, 0, 2, 1, 0), 0.333333),
    ((0, 2, 1, 1, 0, 0, 2, 1, 2), 0.333333),
    ((0, 2, 1, 1, 0, 0, 2, 0, 1), 0.428571),
    ((0, 2, 1, 1, 0, 0, 2, 2, 1), 0.600000),
    ((0, 2, 1, 1, 0, 0, 0, 2, 0), 0.300000),
    ((0, 2, 1, 1, 1, 0, 0, 2, 0), 0.777778),
    ((0, 2, 1, 1, 1, 0, 0, 2, 2), 0.500000),
    ((0, 2, 1, 1, 1, 1, 0, 2, 2), 1.000000),
    ((0, 2, 1, 1, 1, 0, 1, 2, 2), 1.000000),
    ((0, 2, 1, 1, 0, 1, 0, 2, 0), 0.000000),
    ((0, 2, 1, 1, 0, 1, 0, 2, 2), -0.200000),
    ((0, 2, 1, 1, 0, 1, 1, 2, 2), 0.000000),
    ((0, 2, 1, 1, 0, 0, 1, 2, 0), 0.285714),
    ((0, 2, 1, 1, 0, 0, 1, 2, 2), 0.500000),
    ((0, 2, 1, 1, 0, 0, 0, 2, 1), 0.411765),
    ((0, 2, 1, 1, 0, 0, 0, 0, 2), 0.215686),
    ((0, 2, 1, 1, 1, 0, 0, 0, 2), 0.555556),
    ((0, 2, 1, 1, 0, 1, 0, 0, 2), -0.142857),
    ((0, 2, 1, 1, 0, 0, 1, 0, 2), 0.333333),
    ((0, 2, 1, 1, 0, 0, 0, 1, 2), 0.333333),
    ((0, 2, 1, 0, 1, 0, 0, 0, 0), 0.568710),
    ((0, 2, 1, 2, 1, 0, 0, 0, 0), 0.616438),
    ((0, 2, 1, 2, 1, 1, 0, 0, 0), 0.555556),
    ((0, 2, 1, 2, 1, 1, 2, 0, 0), 0.200000),
    ((0, 2, 1, 2, 1, 1, 2, 1, 0), -0.500000),
    ((0, 2, 1, 2, 1, 1, 2, 1, 2), 0.000000),
    ((0, 2, 1, 2, 1, 1, 2, 0, 1), 1.000000),
    ((0, 2, 1, 2, 1, 1, 0, 2, 0), 1.000000),
    ((0, 2, 1, 2, 1, 1, 1, 2, 0), 1.000000),
    ((0, 2, 1, 2, 1, 1, 0, 2, 1), 1.000000),
    ((0, 2, 1, 2, 1, 1, 0, 0, 2), 0.600000),
    ((0, 2, 1, 2, 1, 1, 1, 0, 2), 1.000000),
    ((0, 2, 1, 2, 1, 1, 0, 1, 2), 0.500000),
    ((0, 2, 1, 2, 1, 0, 1, 0, 0), 1.000000),
    ((0, 2, 1, 2, 1, 0, 0, 1, 0), 0.428571),
    ((0, 2, 1, 2, 1, 2, 0, 1, 0), 1.000000),
    ((0, 2, 1, 2, 1, 2, 1, 1, 0), 1.000000),
    ((0, 2, 1, 2, 1, 2, 0, 1, 1), 1.000000),
    ((0, 2, 1, 2, 1, 2, 2, 1, 1), 1.000000),
    ((0, 2, 1, 2, 1, 0, 2, 1, 0), 0.000000),
    ((0, 2, 1, 2, 1, 0, 2, 1, 1), 0.000000),
    ((0, 2, 1, 2, 1, 0, 0, 1, 2), 0.600000),
    ((0, 2, 1, 2, 1, 0, 1, 1, 2), 1.000000),
    ((0, 2, 1, 2, 1, 0, 0, 0, 1), 0.733333),
    ((0, 2, 1, 2, 1, 2, 0, 0, 1), 1.000000),
    ((0, 2, 1, 2, 1, 2, 1, 0, 1), 1.000000),
    ((0, 2, 1, 2, 1, 0, 2, 0, 1), 0.500000),
    ((0, 2, 1, 2, 1, 0, 0, 2, 1), 1.000000),
    ((0, 2, 1, 2, 1, 0, 1, 2, 1), 1.000000),
    ((0, 2, 1, 0, 1, 2, 0, 0, 0), 0.696203),
    ((0, 2, 1, 0, 1, 2, 1, 0, 0), 1.000000),
    ((0, 2, 1, 0, 1, 2, 0, 1, 0), 0.619048),
    ((0, 2, 1, 0, 1, 2, 2, 1, 0), 0.333333),
    ((0, 2, 1, 0, 1, 2, 2, 1, 1), 0.500000),
    ((0, 2, 1, 0, 1, 2, 0, 1, 2), 0.600000),
    ((0, 2, 1, 0, 1, 2, 1, 1, 2), 1.000000),
    ((0, 2, 1, 0, 1, 2, 0, 0, 1), 0.777778),
    ((0, 2, 1, 0, 1, 2, 2, 0, 1), 0.600000),
    ((0, 2, 1, 0, 1, 2, 0, 2, 1), 1.000000),
    ((0, 2, 1, 0, 1, 2, 1, 2, 1), 1.000000),
    ((0, 2, 1, 0, 1, 0, 2, 0, 0), 0.333333),
    ((0, 2, 1, 0, 1, 1, 2, 0, 0), 0.333333),
    ((0, 2, 1, 0, 1, 1, 2, 2, 0), 0.500000),
    ((0, 2, 1, 0, 1, 1, 2, 2, 1), 1.000000),
    ((0, 2, 1, 0, 1, 1, 2, 0, 2), 0.200000),
    ((0, 2, 1, 0, 1, 1, 2, 1, 2), 0.500000),
    ((0, 2, 1, 0, 1, 0, 2, 1, 0), 0.166667),
    ((0, 2, 1, 0, 1, 0, 2, 1, 2), 0.333333),
    ((0, 2, 1, 0, 1, 0, 2, 0, 1), 0.555556),
    ((0, 2, 1, 0, 1, 0, 2, 2, 1), 1.000000),
    ((0, 2, 1, 0, 1, 0, 0, 2, 0), 0.761194),
    ((0, 2, 1, 0, 1, 1, 0, 2, 0), 0.733333),
    ((0, 2, 1, 0, 1, 1, 0, 2, 2), 0.500000),
    ((0, 2, 1, 0, 1, 1, 1, 2, 2), 1.000000),
    ((0, 2, 1, 0, 1, 0, 1, 2, 0), 1.000000),
    ((0, 2, 1, 0, 1, 0, 0, 2, 1), 1.000000),
    ((0, 2, 1, 0, 1, 0, 0, 0, 2), 0.493671),
    ((0, 2, 1, 0, 1, 1, 0, 0, 2), 0.555556),
    ((0, 2, 1, 0, 1, 0, 1, 0, 2), 1.000000),
    ((0, 2, 1, 0, 1, 0, 0, 1, 2), 0.619048),
    ((0, 2, 1, 0, 0, 1, 0, 0, 0), 0.238901),
    ((0, 2, 1, 2, 0, 1, 0, 0, 0), 0.493671),
    ((0, 2, 1, 2, 0, 1, 1, 0, 0), 0.555556),
    ((0, 2, 1, 2, 2, 1, 1, 0, 0), 0.200000),
    ((0, 2, 1, 2, 2, 1, 1, 1, 0), 0.500000),
    ((0, 2, 1, 2, 2, 1, 1, 1, 2), 0.000000),
    ((0, 2, 1, 2, 2, 1, 1, 0, 1), 1.000000),
    ((0, 2, 1, 2, 0, 1, 1, 2, 0), 0.500000),
    ((0, 2, 1, 2, 0, 1, 1, 2, 1), 1.000000),
    ((0, 2, 1, 2, 0, 1, 1, 0, 2), 0.600000),
    ((0, 2, 1, 2, 0, 1, 1, 1, 2), 0.500000),
    ((0, 2, 1, 2, 0, 1, 0, 1, 0), 0.428571),
    ((0, 2, 1, 2, 2, 1, 0, 1, 0), 0.600000),
    ((0, 2, 1, 2, 2, 1, 0, 1, 1), 1.000000),
    ((0, 2, 1, 2, 0, 1, 2, 1, 0), 0.200000),
    ((0, 2, 1, 2, 0, 1, 2, 1, 1), 1.000000),
    ((0, 2, 1, 2, 0, 1, 0, 1, 2), 0.333333),
    ((0, 2, 1, 2, 0, 1, 0, 0, 1), 1.000000),
    ((0, 2, 1, 0, 2, 1, 0, 0, 0), -0.068493),
    ((0, 2, 1, 0, 2, 1, 1, 0, 0), -0.294118),
    ((0, 2, 1, 0, 2, 1, 1, 2, 0), -1.000000),
    ((0, 2, 1, 0, 2, 1, 1, 0, 2), -0.666667),
    ((0, 2, 1, 0, 2, 1, 1, 1, 2), -0.500000),
    ((0, 2, 1, 0, 2, 1, 0, 1, 0), 0.238095),
    ((0, 2, 1, 0, 2, 1, 2, 1, 0), 0.600000),
    ((0, 2, 1, 0, 2, 1, 2, 1, 1), 1.000000),
    ((0, 2, 1, 0, 2, 1, 0, 1, 2), -0.333333),
    ((0, 2, 1, 0, 2, 1, 0, 0, 1), 1.000000),
    ((0, 2, 1, 0, 0, 1, 2, 0, 0), 0.291139),
    ((0, 2, 1, 0, 0, 1, 2, 1, 0), 0.428571),
    ((0, 2, 1, 0, 0, 1, 2, 1, 2), 0.333333),
    ((0, 2, 1, 0, 0, 1, 2, 0, 1), 1.000000),
    ((0, 2, 1, 0, 0, 1, 0, 2, 0), 0.245902),
    ((0, 2, 1, 0, 0, 1, 1, 2, 0), 0.285714),
    ((0, 2, 1, 0, 0, 1, 1, 2, 2), 0.200000),
    ((0, 2, 1, 0, 0, 1, 0, 2, 1), 1.000000),
    ((0, 2, 1, 0, 0, 1, 0, 0, 2), 0.074074),
    ((0, 2, 1, 0, 0, 1, 1, 0, 2), 0.047619),
    ((0, 2, 1, 0, 0, 1, 0, 1, 2), 0.166667),
    ((0, 2, 1, 0, 0, 0, 1, 0, 0), 0.319728),
    ((0, 2, 1, 2, 0, 0, 1, 0, 0), 0.397260),
    ((0, 2, 1, 2, 0, 0, 1, 1, 0), 0.555556),
    ((0, 2, 1, 2, 2, 0, 1, 1, 0), 0.200000),
    ((0, 2, 1, 2, 2, 0, 1, 1, 1), 1.000000),
    ((0, 2, 1, 2, 0, 2, 1, 1, 0), 0.500000),
    ((0, 2, 1, 2, 0, 2, 1, 1, 1), 1.000000),
    ((0, 2, 1, 2, 0, 0, 1, 1, 2), 0.600000),
    ((0, 2, 1, 2, 0, 0, 1, 0, 1), 0.466667),
    ((0, 2, 1, 2, 2, 0, 1, 0, 1), 0.000000),
    ((0, 2, 1, 2, 0, 2, 1, 0, 1), 0.500000),
    ((0, 2, 1, 2, 0, 0, 1, 2, 1), 0.500000),
    ((0, 2, 1, 0, 2, 0, 1, 0, 0), -0.069767),
    ((0, 2, 1, 0, 2, 0, 1, 1, 0), 0.238095),
    ((0, 2, 1, 0, 2, 2, 1, 1, 0), 0.600000),
    ((0, 2, 1, 0, 2, 2, 1, 1, 1), 1.000000),
    ((0, 2, 1, 0, 2, 0, 1, 1, 2), 0.000000),
    ((0, 2, 1, 0, 2, 0, 1, 0, 1), 0.000000),
    ((0, 2, 1, 0, 2, 2, 1, 0, 1), -0.200000),
    ((0, 2, 1, 0, 2, 0, 1, 2, 1), -1.000000),
    ((0, 2, 1, 0, 0, 2, 1, 0, 0), 0.561644),
    ((0, 2, 1, 0, 0, 2, 1, 1, 0), 0.777778),
    ((0, 2, 1, 0, 0, 2, 1, 1, 2), 1.000000),
    ((0, 2, 1, 0, 0, 2, 1, 0, 1), 0.333333),
    ((0, 2, 1, 0, 0, 2, 1, 2, 1), 0.200000),
    ((0, 2, 1, 0, 0, 0, 1, 2, 0), 0.298246),
    ((0, 2, 1, 0, 0, 0, 1, 2, 1), 0.285714),
    ((0, 2, 1, 0, 0, 0, 1, 0, 2), 0.341772),
    ((0, 2, 1, 0, 0, 0, 1, 1, 2), 0.428571),
    ((0, 2, 1, 0, 0, 0, 0, 1, 0), 0.407407),
    ((0, 2, 1, 2, 0, 0, 0, 1, 0), 0.450980),
    ((0, 2, 1, 2, 0, 0, 0, 1, 1), 0.555556),
    ((0, 2, 1, 2, 2, 0, 0, 1, 1), 0.500000),
    ((0, 2, 1, 2, 0, 2, 0, 1, 1), 0.600000),
    ((0, 2, 1, 2, 0, 0, 2, 1, 1), 0.600000),
    ((0, 2, 1, 0, 2, 0, 0, 1, 0), 0.296296),
    ((0, 2, 1, 0, 2, 0, 0, 1, 1), 0.555556),
    ((0, 2, 1, 0, 2, 2, 0, 1, 1), 0.200000),
    ((0, 2, 1, 0, 2, 0, 2, 1, 1), 0.600000),
    ((0, 2, 1, 0, 0, 2, 0, 1, 0), 0.518519),
    ((0, 2, 1, 0, 0, 2, 0, 1, 1), 0.428571),
    ((0, 2, 1, 0, 0, 2, 2, 1, 1), 0.333333),
    ((0, 2, 1, 0, 0, 0, 2, 1, 0), 0.333333),
    ((0, 2, 1, 0, 0, 0, 2, 1, 1), 0.428571),
    ((0, 2, 1, 0, 0, 0, 0, 1, 2), 0.368421),
    ((0, 2, 1, 0, 0, 0, 0, 0, 1), 0.438753),
    ((0, 2, 1, 2, 0, 0, 0, 0, 1), 0.522388),
    ((0, 2, 1, 0, 2, 0, 0, 0, 1), 0.134328),
    ((0, 2, 1, 0, 0, 2, 0, 0, 1), 0.372549),
    ((0, 2, 1, 0, 0, 0, 2, 0, 1), 0.493671),
    ((0, 2, 1, 0, 0, 0, 0, 2, 1), 0.508197),
    ((0, 0, 1, 2, 0, 0, 0, 0, 0), 0.309021),
    ((0, 0, 1, 2, 1, 0, 0, 0, 0), 0.453237),
    ((0, 0, 1, 2, 1, 2, 0, 0, 0), 0.761194),
    ((0, 0, 1, 2, 1, 2, 1, 0, 0), 1.000000),
    ((0, 0, 1, 2, 1, 2, 0, 1, 0), 0.777778),
    ((0, 0, 1, 2, 1, 2, 2, 1, 0), 0.600000),
    ((0, 0, 1, 2, 1, 2, 2, 1, 1), 0.000000),
    ((0, 0, 1, 2, 1, 2, 0, 1, 2), 1.000000),
    ((0, 0, 1, 2, 1, 2, 1, 1, 2), 1.000000),
    ((0, 0, 1, 2, 1, 2, 0, 0, 1), 0.555556),
    ((0, 0, 1, 2, 1, 2, 2, 0, 1), 0.200000),
    ((0, 0, 1, 2, 1, 2, 0, 2, 1), 1.000000),
    ((0, 0, 1, 2, 1, 2, 1, 2, 1), 1.000000),
    ((0, 0, 1, 2, 1, 0, 2, 0, 0), 0.150000),
    ((0, 0, 1, 2, 1, 1, 2, 0, 0), -0.294118),
    ((0, 0, 1, 2, 1, 1, 2, 2, 0), -0.200000),
    ((0, 0, 1, 2, 1, 1, 2, 2, 1), 1.000000),
    ((0, 0, 1, 2, 1, 1, 2, 0, 2), -0.666667),
    ((0, 0, 1, 2, 1, 1, 2, 1, 2), -0.500000),
    ((0, 0, 1, 2, 1, 0, 2, 1, 0), 0.176471),
    ((0, 0, 1, 2, 1, 0, 2, 1, 2), 0.200000),
    ((0, 0, 1, 2, 1, 0, 2, 0, 1), 0.285714),
    ((0, 0, 1, 2, 1, 0, 2, 2, 1), 0.500000),
    ((0, 0, 1, 2, 1, 0, 0, 2, 0), 0.522388),
    ((0, 0, 1, 2, 1, 1, 0, 2, 0), 0.333333),
    ((0, 0, 1, 2, 1, 1, 0, 2, 2), 0.200000),
    ((0, 0, 1, 2, 1, 1, 1, 2, 2), 1.000000),
    ((0, 0, 1, 2, 1, 0, 1, 2, 0), 1.000000),
    ((0, 0, 1, 2, 1, 0, 0, 2, 1), 0.733333),
    ((0, 0, 1, 2, 1, 0, 0, 0, 2), 0.397260),
    ((0, 0, 1, 2, 1, 1, 0, 0, 2), 0.047619),
    ((0, 0, 1, 2, 1, 0, 1, 0, 2), 1.000000),
    ((0, 0, 1, 2, 1, 0, 0, 1, 2), 0.555556),
    ((0, 0, 1, 2, 0, 1, 0, 0, 0), 0.315011),
    ((0, 0, 1, 2, 2, 1, 0, 0, 0), 0.341772),
    ((0, 0, 1, 2, 2, 1, 1, 0, 0), 0.238095),
    ((0, 0, 1, 2, 2, 1, 1, 2, 0), 0.600000),
    ((0, 0, 1, 2, 2, 1, 1, 2, 1), 1.000000),
    ((0, 0, 1, 2, 2, 1, 1, 0, 2), 0.000000),
    ((0, 0, 1, 2, 2, 1, 1, 1, 2), -0.500000),
    ((0, 0, 1, 2, 2, 1, 0, 1, 0), 0.238095),
    ((0, 0, 1, 2, 2, 1, 2, 1, 0), 0.600000),
    ((0, 0, 1, 2, 2, 1, 2, 1, 1), 1.000000),
    ((0, 0, 1, 2, 2, 1, 0, 1, 2), 0.000000),
    ((0, 0, 1, 2, 2, 1, 0, 0, 1), 1.000000),
    ((0, 0, 1, 2, 0, 1, 2, 0, 0), 0.134328),
    ((0, 0, 1, 2, 0, 1, 2, 1, 0), 0.176471),
    ((0, 0, 1, 2, 0, 1, 2, 1, 2), 0.000000),
    ((0, 0, 1, 2, 0, 1, 2, 0, 1), 1.000000),
    ((0, 0, 1, 2, 0, 1, 0, 2, 0), 0.561644),
    ((0, 0, 1, 2, 0, 1, 1, 2, 0), 0.777778),
    ((0, 0, 1, 2, 0, 1, 1, 2, 2), 1.000000),
    ((0, 0, 1, 2, 0, 1, 0, 2, 1), 1.000000),
    ((0, 0, 1, 2, 0, 1, 0, 0, 2), 0.185185),
    ((0, 0, 1, 2, 0, 1, 1, 0, 2), 0.428571),
    ((0, 0, 1, 2, 0, 1, 0, 1, 2), 0.000000),
    ((0, 0, 1, 2, 0, 0, 1, 0, 0), 0.319728),
    ((0, 0, 1, 2, 2, 0, 1, 0, 0), -0.069767),
    ((0, 0, 1, 2, 2, 0, 1, 1, 0), -0.294118),
    ((0, 0, 1, 2, 2, 2, 1, 1, 0), -1.000000),
    ((0, 0, 1, 2, 2, 0, 1, 1, 2), -0.666667),
    ((0, 0, 1, 2, 2, 0, 1, 0, 1), 0.000000),
    ((0, 0, 1, 2, 2, 2, 1, 0, 1), -1.000000),
    ((0, 0, 1, 2, 2, 0, 1, 2, 1), -0.200000),
    ((0, 0, 1, 2, 0, 2, 1, 0, 0), 0.298246),
    ((0, 0, 1, 2, 0, 2, 1, 1, 0), 0.285714),
    ((0, 0, 1, 2, 0, 2, 1, 1, 2), 0.200000),
    ((0, 0, 1, 2, 0, 2, 1, 0, 1), 0.285714),
    ((0, 0, 1, 2, 0, 2, 1, 2, 1), 0.200000),
    ((0, 0, 1, 2, 0, 0, 1, 2, 0), 0.561644),
    ((0, 0, 1, 2, 0, 0, 1, 2, 1), 0.333333),
    ((0, 0, 1, 2, 0, 0, 1, 0, 2), 0.341772),
    ((0, 0, 1, 2, 0, 0, 1, 1, 2), 0.047619),
    ((0, 0, 1, 2, 0, 0, 0, 1, 0), 0.194030),
    ((0, 0, 1, 2, 2, 0, 0, 1, 0), -0.069767),
    ((0, 0, 1, 2, 2, 0, 0, 1, 1), 0.000000),
    ((0, 0, 1, 2, 2, 2, 0, 1, 1), -1.000000),
    ((0, 0, 1, 2, 2, 0, 2, 1, 1), -0.200000),
    ((0, 0, 1, 2, 0, 2, 0, 1, 0), 0.300000),
    ((0, 0, 1, 2, 0, 2, 0, 1, 1), -0.058824),
    ((0, 0, 1, 2, 0, 2, 2, 1, 1), -0.333333),
    ((0, 0, 1, 2, 0, 0, 2, 1, 0), 0.209302),
    ((0, 0, 1, 2, 0, 0, 2, 1, 1), -0.058824),
    ((0, 0, 1, 2, 0, 0, 0, 1, 2), 0.215686),
    ((0, 0, 1, 2, 0, 0, 0, 0, 1), 0.297710),
    ((0, 0, 1, 2, 2, 0, 0, 0, 1), 0.017544),
    ((0, 0, 1, 2, 0, 2, 0, 0, 1), 0.200000),
    ((0, 0, 1, 2, 0, 0, 2, 0, 1), 0.245902),
    ((0, 0, 1, 2, 0, 0, 0, 2, 1), 0.522388),
    ((0, 0, 1, 0, 2, 0, 0, 0, 0), 0.035755),
    ((0, 0, 1, 1, 2, 0, 0, 0, 0), 0.055556),
    ((0, 0, 1, 1, 2, 2, 0, 0, 0), 0.296296),
    ((0, 0, 1, 1, 2, 2, 1, 0, 0), 0.238095),
    ((0, 0, 1, 1, 2, 2, 1, 2, 0), 0.200000),
    ((0, 0, 1, 1, 2, 2, 1, 2, 1), -0.500000),
    ((0, 0, 1, 1, 2, 2, 1, 0, 2), 0.200000),
    ((0, 0, 1, 1, 2, 2, 1, 1, 2), 0.000000),
    ((0, 0, 1, 1, 2, 2, 0, 1, 0), 0.333333),
    ((0, 0, 1, 1, 2, 2, 2, 1, 0), 0.333333),
    ((0, 0, 1, 1, 2, 2, 2, 1, 1), 0.000000),
    ((0, 0, 1, 1, 2, 2, 0, 1, 2), 0.333333),
    ((0, 0, 1, 1, 2, 2, 0, 0, 1), 0.166667),
    ((0, 0, 1, 1, 2, 2, 2, 0, 1), 0.333333),
    ((0, 0, 1, 1, 2, 2, 0, 2, 1), 0.000000),
    ((0, 0, 1, 1, 2, 0, 2, 0, 0), 0.148148),
    ((0, 0, 1, 1, 2, 1, 2, 0, 0), -0.142857),
    ((0, 0, 1, 1, 2, 1, 2, 2, 0), -0.200000),
    ((0, 0, 1, 1, 2, 1, 2, 2, 1), 1.000000),
    ((0, 0, 1, 1, 2, 1, 2, 0, 2), -0.666667),
    ((0, 0, 1, 1, 2, 1, 2, 1, 2), -0.500000),
    ((0, 0, 1, 1, 2, 0, 2, 1, 0), 0.166667),
    ((0, 0, 1, 1, 2, 0, 2, 1, 2), 0.000000),
    ((0, 0, 1, 1, 2, 0, 2, 0, 1), 0.428571),
    ((0, 0, 1, 1, 2, 0, 2, 2, 1), 0.600000),
    ((0, 0, 1, 1, 2, 0, 0, 2, 0), -0.069767),
    ((0, 0, 1, 1, 2, 1, 0, 2, 0), -0.529412),
    ((0, 0, 1, 1, 2, 1, 0, 2, 2), -1.000000),
    ((0, 0, 1, 1, 2, 1, 1, 2, 2), -1.000000),
    ((0, 0, 1, 1, 2, 0, 1, 2, 0), -0.294118),
    ((0, 0, 1, 1, 2, 0, 1, 2, 2), -0.200000),
    ((0, 0, 1, 1, 2, 0, 0, 2, 1), 0.176471),
    ((0, 0, 1, 1, 2, 0, 0, 0, 2), -0.136364),
    ((0, 0, 1, 1, 2, 1, 0, 0, 2), -0.789474),
    ((0, 0, 1, 1, 2, 0, 1, 0, 2), -0.125000),
    ((0, 0, 1, 1, 2, 0, 0, 1, 2), 0.052632),
    ((0, 0, 1, 0, 2, 1, 0, 0, 0), -0.006565),
    ((0, 0, 1, 0, 2, 1, 2, 0, 0), 0.139241),
    ((0, 0, 1, 0, 2, 1, 2, 1, 0), 0.238095),
    ((0, 0, 1, 0, 2, 1, 2, 1, 2), 0.000000),
    ((0, 0, 1, 0, 2, 1, 2, 0, 1), 1.000000),
    ((0, 0, 1, 0, 2, 1, 0, 2, 0), -0.044776),
    ((0, 0, 1, 0, 2, 1, 1, 2, 0), -0.058824),
    ((0, 0, 1, 0, 2, 1, 1, 2, 2), -0.333333),
    ((0, 0, 1, 0, 2, 1, 0, 2, 1), 1.000000),
    ((0, 0, 1, 0, 2, 1, 0, 0, 2), -0.276596),
    ((0, 0, 1, 0, 2, 1, 1, 0, 2), -0.368421),
    ((0, 0, 1, 0, 2, 1, 0, 1, 2), -0.157895),
    ((0, 0, 1, 0, 2, 0, 1, 0, 0), -0.092308),
    ((0, 0, 1, 0, 2, 2, 1, 0, 0), -0.069767),
    ((0, 0, 1, 0, 2, 2, 1, 1, 0), -0.058824),
    ((0, 0, 1, 0, 2, 2, 1, 1, 2), -0.333333),
    ((0, 0, 1, 0, 2, 2, 1, 0, 1), -0.294118),
    ((0, 0, 1, 0, 2, 2, 1, 2, 1), -0.666667),
    ((0, 0, 1, 0, 2, 0, 1, 2, 0), -0.069767),
    ((0, 0, 1, 0, 2, 0, 1, 2, 1), -0.294118),
    ((0, 0, 1, 0, 2, 0, 1, 0, 2), -0.136364),
    ((0, 0, 1, 0, 2, 0, 1, 1, 2), -0.368421),
    ((0, 0, 1, 0, 2, 0, 0, 1, 0), 0.055556),
    ((0, 0, 1, 0, 2, 2, 0, 1, 0), 0.043478),
    ((0, 0, 1, 0, 2, 2, 0, 1, 1), -0.058824),
    ((0, 0, 1, 0, 2, 2, 2, 1, 1), -0.333333),
    ((0, 0, 1, 0, 2, 0, 2, 1, 0), 0.148148),
    ((0, 0, 1, 0, 2, 0, 2, 1, 1), 0.047619),
    ((0, 0, 1, 0, 2, 0, 0, 1, 2), -0.021277),
    ((0, 0, 1, 0, 2, 0, 0, 0, 1), 0.129252),
    ((0, 0, 1, 0, 2, 2, 0, 0, 1), -0.086957),
    ((0, 0, 1, 0, 2, 0, 2, 0, 1), 0.291139),
    ((0, 0, 1, 0, 2, 0, 0, 2, 1), 0.134328),
    ((0, 0, 1, 0, 0, 2, 0, 0, 0), 0.352236),
    ((0, 0, 1, 1, 0, 2, 0, 0, 0), 0.407407),
    ((0, 0, 1, 1, 0, 2, 2, 0, 0), 0.333333),
    ((0, 0, 1, 1, 1, 2, 2, 0, 0), 0.166667),
    ((0, 0, 1, 1, 1, 2, 2, 2, 0), 0.000000),
    ((0, 0, 1, 1, 1, 2, 2, 2, 1), 0.500000),
    ((0, 0, 1, 1, 1, 2, 2, 0, 2), 0.000000),
    ((0, 0, 1, 1, 1, 2, 2, 1, 2), 0.500000),
    ((0, 0, 1, 1, 0, 2, 2, 1, 0), 0.333333),
    ((0, 0, 1, 1, 0, 2, 2, 1, 2), 0.666667),
    ((0, 0, 1, 1, 0, 2, 2, 0, 1), 0.333333),
    ((0, 0, 1, 1, 0, 2, 2, 2, 1), 0.666667),
    ((0, 0, 1, 1, 0, 2, 0, 2, 0), 0.450980),
    ((0, 0, 1, 1, 1, 2, 0, 2, 0), 0.428571),
    ((0, 0, 1, 1, 1, 2, 0, 2, 2), 0.200000),
    ((0, 0, 1, 1, 1, 2, 1, 2, 2), 1.000000),
    ((0, 0, 1, 1, 0, 2, 1, 2, 0), 0.555556),
    ((0, 0, 1, 1, 0, 2, 1, 2, 2), 1.000000),
    ((0, 0, 1, 1, 0, 2, 0, 2, 1), 0.333333),
    ((0, 0, 1, 1, 0, 2, 0, 0, 2), 0.490196),
    ((0, 0, 1, 1, 1, 2, 0, 0, 2), 0.428571),
    ((0, 0, 1, 1, 0, 2, 1, 0, 2), 0.555556),
    ((0, 0, 1, 1, 0, 2, 0, 1, 2), 0.500000),
    ((0, 0, 1, 0, 1, 2, 0, 0, 0), 0.568710),
    ((0, 0, 1, 0, 1, 2, 2, 0, 0), 0.333333),
    ((0, 0, 1, 0, 1, 2, 2, 1, 0), 0.428571),
    ((0, 0, 1, 0, 1, 2, 2, 1, 2), 0.600000),
    ((0, 0, 1, 0, 1, 2, 2, 0, 1), 0.238095),
    ((0, 0, 1, 0, 1, 2, 2, 2, 1), 0.600000),
    ((0, 0, 1, 0, 1, 2, 0, 2, 0), 0.616438),
    ((0, 0, 1, 0, 1, 2, 1, 2, 0), 1.000000),
    ((0, 0, 1, 0, 1, 2, 0, 2, 1), 0.777778),
    ((0, 0, 1, 0, 1, 2, 0, 0, 2), 0.616438),
    ((0, 0, 1, 0, 1, 2, 1, 0, 2), 1.000000),
    ((0, 0, 1, 0, 1, 2, 0, 1, 2), 0.777778),
    ((0, 0, 1, 0, 0, 2, 1, 0, 0), 0.319728),
    ((0, 0, 1, 0, 0, 2, 1, 2, 0), 0.397260),
    ((0, 0, 1, 0, 0, 2, 1, 2, 1), 0.047619),
    ((0, 0, 1, 0, 0, 2, 1, 0, 2), 0.452055),
    ((0, 0, 1, 0, 0, 2, 1, 1, 2), 0.238095),
    ((0, 0, 1, 0, 0, 2, 0, 1, 0), 0.310811),
    ((0, 0, 1, 0, 0, 2, 2, 1, 0), 0.259259),
    ((0, 0, 1, 0, 0, 2, 2, 1, 1), -0.166667),
    ((0, 0, 1, 0, 0, 2, 0, 1, 2), 0.490196),
    ((0, 0, 1, 0, 0, 2, 0, 0, 1), 0.209459),
    ((0, 0, 1, 0, 0, 2, 2, 0, 1), 0.185185),
    ((0, 0, 1, 0, 0, 2, 0, 2, 1), 0.372549),
    ((0, 0, 1, 0, 0, 0, 2, 0, 0), 0.199110),
    ((0, 0, 1, 1, 0, 0, 2, 0, 0), 0.220779),
    ((0, 0, 1, 1, 0, 0, 2, 2, 0), 0.209302),
    ((0, 0, 1, 1, 1, 0, 2, 2, 0), 0.176471),
    ((0, 0, 1, 1, 1, 0, 2, 2, 2), -1.000000),
    ((0, 0, 1, 1, 0, 1, 2, 2, 0), 0.000000),
    ((0, 0, 1, 1, 0, 1, 2, 2, 2), -1.000000),
    ((0, 0, 1, 1, 0, 0, 2, 2, 1), 0.619048),
    ((0, 0, 1, 1, 0, 0, 2, 0, 2), 0.043478),
    ((0, 0, 1, 1, 1, 0, 2, 0, 2), 0.176471),
    ((0, 0, 1, 1, 0, 1, 2, 0, 2), -0.294118),
    ((0, 0, 1, 1, 0, 0, 2, 1, 2), 0.333333),
    ((0, 0, 1, 0, 1, 0, 2, 0, 0), 0.194030),
    ((0, 0, 1, 0, 1, 0, 2, 2, 0), 0.150000),
    ((0, 0, 1, 0, 1, 1, 2, 2, 0), 0.000000),
    ((0, 0, 1, 0, 1, 1, 2, 2, 2), -1.000000),
    ((0, 0, 1, 0, 1, 0, 2, 2, 1), 0.555556),
    ((0, 0, 1, 0, 1, 0, 2, 0, 2), 0.069767),
    ((0, 0, 1, 0, 1, 1, 2, 0, 2), -0.294118),
    ((0, 0, 1, 0, 1, 0, 2, 1, 2), 0.428571),
    ((0, 0, 1, 0, 0, 1, 2, 0, 0), 0.047619),
    ((0, 0, 1, 0, 0, 1, 2, 2, 0), 0.017544),
    ((0, 0, 1, 0, 0, 1, 2, 2, 1), 1.000000),
    ((0, 0, 1, 0, 0, 1, 2, 0, 2), -0.217391),
    ((0, 0, 1, 0, 0, 1, 2, 1, 2), 0.000000),
    ((0, 0, 1, 0, 0, 0, 2, 1, 0), 0.220779),
    ((0, 0, 1, 0, 0, 0, 2, 1, 2), 0.296296),
    ((0, 0, 1, 0, 0, 0, 2, 0, 1), 0.315011),
    ((0, 0, 1, 0, 0, 0, 2, 2, 1), 0.616438),
    ((0, 0, 1, 0, 0, 0, 0, 2, 0), 0.309021),
    ((0, 0, 1, 1, 0, 0, 0, 2, 0), 0.194030),
    ((0, 0, 1, 1, 0, 0, 0, 2, 2), 0.000000),
    ((0, 0, 1, 1, 1, 0, 0, 2, 2), 0.285714),
    ((0, 0, 1, 1, 0, 1, 0, 2, 2), -0.529412),
    ((0, 0, 1, 1, 0, 0, 1, 2, 2), 0.333333),
    ((0, 0, 1, 0, 1, 0, 0, 2, 0), 0.453237),
    ((0, 0, 1, 0, 1, 0, 0, 2, 2), 0.298246),
    ((0, 0, 1, 0, 1, 1, 0, 2, 2), 0.285714),
    ((0, 0, 1, 0, 1, 0, 1, 2, 2), 1.000000),
    ((0, 0, 1, 0, 0, 1, 0, 2, 0), 0.165468),
    ((0, 0, 1, 0, 0, 1, 0, 2, 2), -0.023256),
    ((0, 0, 1, 0, 0, 1, 1, 2, 2), 0.238095),
    ((0, 0, 1, 0, 0, 0, 1, 2, 0), 0.319728),
    ((0, 0, 1, 0, 0, 0, 1, 2, 2), 0.452055),
    ((0, 0, 1, 0, 0, 0, 0, 2, 1), 0.438753),
    ((0, 0, 1, 0, 0, 0, 0, 0, 2), 0.197740),
    ((0, 0, 1, 1, 0, 0, 0, 0, 2), 0.144928),
    ((0, 0, 1, 0, 1, 0, 0, 0, 2), 0.401361),
    ((0, 0, 1, 0, 0, 1, 0, 0, 2), -0.048611),
    ((0, 0, 1, 0, 0, 0, 1, 0, 2), 0.256018),
    ((0, 0, 1, 0, 0, 0, 0, 1, 2), 0.263158),
    ((0, 0, 0, 1, 0, 0, 0, 0, 0), 0.137064),
    ((2, 0, 0, 1, 0, 0, 0, 0, 0), 0.132555),
    ((2, 0, 0, 1, 1, 0, 0, 0, 0), 0.391121),
    ((2, 2, 0, 1, 1, 0, 0, 0, 0), 0.373134),
    ((2, 2, 0, 1, 1, 1, 0, 0, 0), 1.000000),
    ((2, 2, 0, 1, 1, 0, 1, 0, 0), 0.285714),
    ((2, 2, 2, 1, 1, 0, 1, 0, 0), -1.000000),
    ((2, 2, 0, 1, 1, 2, 1, 0, 0), 0.200000),
    ((2, 2, 0, 1, 1, 2, 1, 1, 0), 0.000000),
    ((2, 2, 2, 1, 1, 2, 1, 1, 0), -1.000000),
    ((2, 2, 0, 1, 1, 2, 1, 1, 2), 1.000000),
    ((2, 2, 0, 1, 1, 2, 1, 0, 1), 0.000000),
    ((2, 2, 2, 1, 1, 2, 1, 0, 1), -1.000000),
    ((2, 2, 0, 1, 1, 2, 1, 2, 1), 1.000000),
    ((2, 2, 0, 1, 1, 0, 1, 2, 0), 0.500000),
    ((2, 2, 0, 1, 1, 1, 1, 2, 0), 1.000000),
    ((2, 2, 0, 1, 1, 0, 1, 2, 1), 0.000000),
    ((2, 2, 2, 1, 1, 0, 1, 2, 1), -1.000000),
    ((2, 2, 0, 1, 1, 0, 1, 0, 2), 0.500000),
    ((2, 2, 0, 1, 1, 1, 1, 0, 2), 1.000000),
    ((2, 2, 0, 1, 1, 0, 1, 1, 2), 0.000000),
    ((2, 2, 2, 1, 1, 0, 1, 1, 2), -1.000000),
    ((2, 2, 0, 1, 1, 0, 0, 1, 0), 0.176471),
    ((2, 2, 2, 1, 1, 0, 0, 1, 0), -1.000000),
    ((2, 2, 0, 1, 1, 2, 0, 1, 0), 0.000000),
    ((2, 2, 0, 1, 1, 2, 0, 1, 1), -0.500000),
    ((2, 2, 2, 1, 1, 2, 0, 1, 1), -1.000000),
    ((2, 2, 0, 1, 1, 2, 2, 1, 1), 0.000000),
    ((2, 2, 0, 1, 1, 0, 2, 1, 0), 0.200000),
    ((2, 2, 0, 1, 1, 1, 2, 1, 0), 1.000000),
    ((2, 2, 0, 1, 1, 0, 2, 1, 1), -0.500000),
    ((2, 2, 2, 1, 1, 0, 2, 1, 1), -1.000000),
    ((2, 2, 0, 1, 1, 0, 0, 1, 2), 0.600000),
    ((2, 2, 0, 1, 1, 1, 0, 1, 2), 1.000000),
    ((2, 2, 0, 1, 1, 0, 0, 0, 1), 0.176471),
    ((2, 2, 2, 1, 1, 0, 0, 0, 1), -1.000000),
    ((2, 2, 0, 1, 1, 2, 0, 0, 1), 0.000000),
    ((2, 2, 0, 1, 1, 0, 2, 0, 1), 0.200000),
    ((2, 2, 0, 1, 1, 1, 2, 0, 1), 1.000000),
    ((2, 2, 0, 1, 1, 0, 0, 2, 1), 0.600000),
    ((2, 2, 0, 1, 1, 1, 0, 2, 1), 1.000000),
    ((2, 0, 2, 1, 1, 0, 0, 0, 0), 0.134328),
    ((2, 0, 2, 1, 1, 1, 0, 0, 0), 1.000000),
    ((2, 0, 2, 1, 1, 0, 1, 0, 0), -0.294118),
    ((2, 0, 2, 1, 1, 2, 1, 0, 0), -0.666667),
    ((2, 0, 2, 1, 1, 2, 1, 1, 0), -1.000000),
    ((2, 0, 2, 1, 1, 2, 1, 1, 2), -1.000000),
    ((2, 0, 2, 1, 1, 2, 1, 0, 1), -0.500000),
    ((2, 0, 2, 1, 1, 2, 1, 2, 1), 0.000000),
    ((2, 0, 2, 1, 1, 0, 1, 2, 0), 0.200000),
    ((2, 0, 2, 1, 1, 1, 1, 2, 0), 1.000000),
    ((2, 0, 2, 1, 1, 0, 1, 2, 1), -0.500000),
    ((2, 0, 2, 1, 1, 0, 1, 0, 2), -0.200000),
    ((2, 0, 2, 1, 1, 1, 1, 0, 2), 1.000000),
    ((2, 0, 2, 1, 1, 0, 1, 1, 2), -1.000000),
    ((2, 0, 2, 1, 1, 0, 0, 1, 0), 0.000000),
    ((2, 0, 2, 1, 1, 2, 0, 1, 0), -0.200000),
    ((2, 0, 2, 1, 1, 2, 0, 1, 1), 0.000000),
    ((2, 0, 2, 1, 1, 2, 2, 1, 1), 1.000000),
    ((2, 0, 2, 1, 1, 0, 2, 1, 0), 0.500000),
    ((2, 0, 2, 1, 1, 1, 2, 1, 0), 1.000000),
    ((2, 0, 2, 1, 1, 0, 2, 1, 1), 0.000000),
    ((2, 0, 2, 1, 1, 0, 0, 1, 2), 0.000000),
    ((2, 0, 2, 1, 1, 1, 0, 1, 2), 1.000000),
    ((2, 0, 2, 1, 1, 0, 0, 0, 1), 0.176471),
    ((2, 0, 2, 1, 1, 2, 0, 0, 1), 0.000000),
    ((2, 0, 2, 1, 1, 0, 2, 0, 1), 0.600000),
    ((2, 0, 2, 1, 1, 1, 2, 0, 1), 1.000000),
    ((2, 0, 2, 1, 1, 0, 0, 2, 1), 0.200000),
    ((2, 0, 2, 1, 1, 1, 0, 2, 1), 1.000000),
    ((2, 0, 0, 1, 1, 2, 0, 0, 0), 0.296296),
    ((2, 0, 0, 1, 1, 2, 1, 0, 0), 0.047619),
    ((2, 0, 0, 1, 1, 2, 1, 2, 0), 0.600000),
    ((2, 0, 0, 1, 1, 2, 1, 2, 1), 0.500000),
    ((2, 0, 0, 1, 1, 2, 1, 0, 2), 0.200000),
    ((2, 0, 0, 1, 1, 2, 1, 1, 2), 0.000000),
    ((2, 0, 0, 1, 1, 2, 0, 1, 0), 0.238095),
    ((2, 0, 0, 1, 1, 2, 2, 1, 0), 0.600000),
    ((2, 0, 0, 1, 1, 2, 2, 1, 1), 0.500000),
    ((2, 0, 0, 1, 1, 2, 0, 1, 2), 0.600000),
    ((2, 0, 0, 1, 1, 2, 0, 0, 1), 0.166667),
    ((2, 0, 0, 1, 1, 2, 2, 0, 1), 0.333333),
    ((2, 0, 0, 1, 1, 2, 0, 2, 1), 0.333333),
    ((2, 0, 0, 1, 1, 0, 2, 0, 0), 0.493671),
    ((2, 0, 0, 1, 1, 1, 2, 0, 0), 1.000000),
    ((2, 0, 0, 1, 1, 0, 2, 1, 0), 0.555556),
    ((2, 0, 0, 1, 1, 0, 2, 1, 2), 1.000000),
    ((2, 0, 0, 1, 1, 1, 2, 1, 2), 1.000000),
    ((2, 0, 0, 1, 1, 0, 2, 0, 1), 0.428571),
    ((2, 0, 0, 1, 1, 0, 2, 2, 1), 0.600000),
    ((2, 0, 0, 1, 1, 1, 2, 2, 1), 1.000000),
    ((2, 0, 0, 1, 1, 0, 0, 2, 0), 0.493671),
    ((2, 0, 0, 1, 1, 1, 0, 2, 0), 1.000000),
    ((2, 0, 0, 1, 1, 0, 1, 2, 0), 0.555556),
    ((2, 0, 0, 1, 1, 0, 1, 2, 2), 1.000000),
    ((2, 0, 0, 1, 1, 1, 1, 2, 2), 1.000000),
    ((2, 0, 0, 1, 1, 0, 0, 2, 1), 0.428571),
    ((2, 0, 0, 1, 1, 0, 0, 0, 2), 0.561644),
    ((2, 0, 0, 1, 1, 1, 0, 0, 2), 1.000000),
    ((2, 0, 0, 1, 1, 0, 1, 0, 2), 0.333333),
    ((2, 0, 0, 1, 1, 0, 0, 1, 2), 0.555556),
    ((2, 0, 0, 1, 0, 1, 0, 0, 0), -0.085339),
    ((2, 2, 0, 1, 0, 1, 0, 0, 0), -0.044776),
    ((2, 2, 0, 1, 0, 1, 1, 0, 0), -0.529412),
    ((2, 2, 2, 1, 0, 1, 1, 0, 0), -1.000000),
    ((2, 2, 0, 1, 2, 1, 1, 0, 0), -1.000000),
    ((2, 2, 0, 1, 2, 1, 1, 1, 0), -1.000000),
    ((2, 2, 2, 1, 2, 1, 1, 1, 0), -1.000000),
    ((2, 2, 0, 1, 2, 1, 1, 1, 2), -1.000000),
    ((2, 2, 0, 1, 2, 1, 1, 0, 1), -1.000000),
    ((2, 2, 2, 1, 2, 1, 1, 0, 1), -1.000000),
    ((2, 2, 0, 1, 2, 1, 1, 2, 1), -1.000000),
    ((2, 2, 0, 1, 0, 1, 1, 2, 0), -0.200000),
    ((2, 2, 0, 1, 0, 1, 1, 2, 1), -1.000000),
    ((2, 2, 2, 1, 0, 1, 1, 2, 1), -1.000000),
    ((2, 2, 0, 1, 0, 1, 1, 0, 2), -0.200000),
    ((2, 2, 0, 1, 0, 1, 1, 1, 2), -1.000000),
    ((2, 2, 2, 1, 0, 1, 1, 1, 2), -1.000000),
    ((2, 2, 0, 1, 0, 1, 0, 1, 0), -0.058824),
    ((2, 2, 2, 1, 0, 1, 0, 1, 0), -1.000000),
    ((2, 2, 0, 1, 2, 1, 0, 1, 0), -0.333333),
    ((2, 2, 0, 1, 2, 1, 0, 1, 1), 0.000000),
    ((2, 2, 2, 1, 2, 1, 0, 1, 1), -1.000000),
    ((2, 2, 0, 1, 2, 1, 2, 1, 1), 1.000000),
    ((2, 2, 0, 1, 0, 1, 2, 1, 0), 0.600000),
    ((2, 2, 0, 1, 0, 1, 2, 1, 1), 0.000000),
    ((2, 2, 2, 1, 0, 1, 2, 1, 1), -1.000000),
    ((2, 2, 0, 1, 0, 1, 0, 1, 2), -0.200000),
    ((2, 2, 0, 1, 0, 1, 0, 0, 1), 0.000000),
    ((2, 2, 2, 1, 0, 1, 0, 0, 1), -1.000000),
    ((2, 2, 0, 1, 2, 1, 0, 0, 1), -0.200000),
    ((2, 2, 0, 1, 0, 1, 2, 0, 1), 0.500000),
    ((2, 2, 0, 1, 0, 1, 0, 2, 1), 0.000000),
    ((2, 0, 2, 1, 0, 1, 0, 0, 0), -0.232877),
    ((2, 0, 2, 1, 0, 1, 1, 0, 0), -0.294118),
    ((2, 0, 2, 1, 2, 1, 1, 0, 0), -0.666667),
    ((2, 0, 2, 1, 2, 1, 1, 1, 0), -1.000000),
    ((2, 0, 2, 1, 2, 1, 1, 1, 2), -1.000000),
    ((2, 0, 2, 1, 2, 1, 1, 0, 1), -0.500000),
    ((2, 0, 2, 1, 2, 1, 1, 2, 1), 0.000000),
    ((2, 0, 2, 1, 0, 1, 1, 2, 0), 0.200000),
    ((2, 0, 2, 1, 0, 1, 1, 2, 1), -0.500000),
    ((2, 0, 2, 1, 0, 1, 1, 0, 2), -0.200000),
    ((2, 0, 2, 1, 0, 1, 1, 1, 2), -1.000000),
    ((2, 0, 2, 1, 0, 1, 0, 1, 0), -0.529412),
    ((2, 0, 2, 1, 2, 1, 0, 1, 0), -1.000000),
    ((2, 0, 2, 1, 2, 1, 0, 1, 1), -1.000000),
    ((2, 0, 2, 1, 2, 1, 2, 1, 1), -1.000000),
    ((2, 0, 2, 1, 0, 1, 2, 1, 0), -0.200000),
    ((2, 0, 2, 1, 0, 1, 2, 1, 1), -1.000000),
    ((2, 0, 2, 1, 0, 1, 0, 1, 2), -0.200000),
    ((2, 0, 2, 1, 0, 1, 0, 0, 1), -0.294118),
    ((2, 0, 2, 1, 2, 1, 0, 0, 1), -0.666667),
    ((2, 0, 2, 1, 0, 1, 2, 0, 1), -0.200000),
    ((2, 0, 2, 1, 0, 1, 0, 2, 1), 0.200000),
    ((2, 0, 0, 1, 2, 1, 0, 0, 0), -0.404255),
    ((2, 0, 0, 1, 2, 1, 1, 0, 0), -0.789474),
    ((2, 0, 0, 1, 2, 1, 1, 2, 0), -0.666667),
    ((2, 0, 0, 1, 2, 1, 1, 2, 1), -0.500000),
    ((2, 0, 0, 1, 2, 1, 1, 0, 2), -1.000000),
    ((2, 0, 0, 1, 2, 1, 0, 1, 0), -0.578947),
    ((2, 0, 0, 1, 2, 1, 2, 1, 0), -0.333333),
    ((2, 0, 0, 1, 2, 1, 2, 1, 1), 0.000000),
    ((2, 0, 0, 1, 2, 1, 0, 1, 2), -1.000000),
    ((2, 0, 0, 1, 2, 1, 0, 0, 1), -0.142857),
    ((2, 0, 0, 1, 2, 1, 2, 0, 1), 0.200000),
    ((2, 0, 0, 1, 2, 1, 0, 2, 1), 0.200000),
    ((2, 0, 0, 1, 0, 1, 2, 0, 0), 0.189873),
    ((2, 0, 0, 1, 0, 1, 2, 1, 0), 0.047619),
    ((2, 0, 0, 1, 0, 1, 2, 1, 2), 0.200000),
    ((2, 0, 0, 1, 0, 1, 2, 0, 1), 0.333333),
    ((2, 0, 0, 1, 0, 1, 2, 2, 1), 1.000000),
    ((2, 0, 0, 1, 0, 1, 0, 2, 0), 0.139241),
    ((2, 0, 0, 1, 0, 1, 1, 2, 0), -0.142857),
    ((2, 0, 0, 1, 0, 1, 1, 2, 2), 0.200000),
    ((2, 0, 0, 1, 0, 1, 0, 2, 1), 0.333333),
    ((2, 0, 0, 1, 0, 1, 0, 0, 2), -0.107692),
    ((2, 0, 0, 1, 0, 1, 1, 0, 2), -0.125000),
    ((2, 0, 0, 1, 0, 1, 0, 1, 2), -0.125000),
    ((2, 0, 0, 1, 0, 0, 1, 0, 0), -0.048611),
    ((2, 2, 0, 1, 0, 0, 1, 0, 0), -0.023256),
    ((2, 2, 0, 1, 0, 0, 1, 1, 0), -0.058824),
    ((2, 2, 2, 1, 0, 0, 1, 1, 0), -1.000000),
    ((2, 2, 0, 1, 2, 0, 1, 1, 0), -0.200000),
    ((2, 2, 0, 1, 2, 0, 1, 1, 1), 1.000000),
    ((2, 2, 0, 1, 0, 2, 1, 1, 0), 0.600000),
    ((2, 2, 0, 1, 0, 2, 1, 1, 1), 1.000000),
    ((2, 2, 0, 1, 0, 0, 1, 1, 2), -0.333333),
    ((2, 2, 0, 1, 0, 0, 1, 0, 1), -0.058824),
    ((2, 2, 2, 1, 0, 0, 1, 0, 1), -1.000000),
    ((2, 2, 0, 1, 2, 0, 1, 0, 1), -0.200000),
    ((2, 2, 0, 1, 0, 2, 1, 0, 1), 0.600000),
    ((2, 2, 0, 1, 0, 0, 1, 2, 1), -0.333333),
    ((2, 0, 2, 1, 0, 0, 1, 0, 0), -0.217391),
    ((2, 0, 2, 1, 0, 0, 1, 1, 0), -0.529412),
    ((2, 0, 2, 1, 2, 0, 1, 1, 0), -0.200000),
    ((2, 0, 2, 1, 2, 0, 1, 1, 1), 1.000000),
    ((2, 0, 2, 1, 0, 2, 1, 1, 0), -0.200000),
    ((2, 0, 2, 1, 0, 2, 1, 1, 1), 1.000000),
    ((2, 0, 2, 1, 0, 0, 1, 1, 2), -1.000000),
    ((2, 0, 2, 1, 0, 0, 1, 0, 1), -0.058824),
    ((2, 0, 2, 1, 2, 0, 1, 0, 1), 0.200000),
    ((2, 0, 2, 1, 0, 2, 1, 0, 1), 0.200000),
    ((2, 0, 2, 1, 0, 0, 1, 2, 1), -0.333333),
    ((2, 0, 0, 1, 2, 0, 1, 0, 0), -0.276596),
    ((2, 0, 0, 1, 2, 0, 1, 1, 0), -0.125000),
    ((2, 0, 0, 1, 2, 2, 1, 1, 0), 0.200000),
    ((2, 0, 0, 1, 2, 2, 1, 1, 1), 1.000000),
    ((2, 0, 0, 1, 2, 0, 1, 1, 2), -1.000000),
    ((2, 0, 0, 1, 2, 0, 1, 0, 1), 0.047619),
    ((2, 0, 0, 1, 2, 2, 1, 0, 1), 0.600000),
    ((2, 0, 0, 1, 2, 0, 1, 2, 1), -0.333333),
    ((2, 0, 0, 1, 0, 2, 1, 0, 0), 0.185185),
    ((2, 0, 0, 1, 0, 2, 1, 1, 0), 0.047619),
    ((2, 0, 0, 1, 0, 2, 1, 1, 2), -0.333333),
    ((2, 0, 0, 1, 0, 2, 1, 0, 1), 0.428571),
    ((2, 0, 0, 1, 0, 2, 1, 2, 1), 0.333333),
    ((2, 0, 0, 1, 0, 0, 1, 2, 0), 0.074074),
    ((2, 0, 0, 1, 0, 0, 1, 2, 1), -0.166667),
    ((2, 0, 0, 1, 0, 0, 1, 0, 2), -0.090909),
    ((2, 0, 0, 1, 0, 0, 1, 1, 2), -0.578947),
    ((2, 0, 0, 1, 0, 0, 0, 1, 0), 0.055556),
    ((2, 2, 0, 1, 0, 0, 0, 1, 0), 0.173913),
    ((2, 2, 0, 1, 0, 0, 0, 1, 1), 0.176471),
    ((2, 2, 2, 1, 0, 0, 0, 1, 1), -1.000000),
    ((2, 2, 0, 1, 2, 0, 0, 1, 1), 0.600000),
    ((2, 2, 0, 1, 0, 2, 0, 1, 1), 0.200000),
    ((2, 2, 0, 1, 0, 0, 2, 1, 1), 0.000000),
    ((2, 0, 2, 1, 0, 0, 0, 1, 0), -0.209302),
    ((2, 0, 2, 1, 0, 0, 0, 1, 1), -0.058824),
    ((2, 0, 2, 1, 2, 0, 0, 1, 1), -0.200000),
    ((2, 0, 2, 1, 0, 2, 0, 1, 1), 0.600000),
    ((2, 0, 2, 1, 0, 0, 2, 1, 1), -0.333333),
    ((2, 0, 0, 1, 2, 0, 0, 1, 0), -0.148936),
    ((2, 0, 0, 1, 2, 0, 0, 1, 1), 0.238095),
    ((2, 0, 0, 1, 2, 2, 0, 1, 1), 0.600000),
    ((2, 0, 0, 1, 2, 0, 2, 1, 1), 0.000000),
    ((2, 0, 0, 1, 0, 2, 0, 1, 0), 0.259259),
    ((2, 0, 0, 1, 0, 2, 0, 1, 1), 0.428571),
    ((2, 0, 0, 1, 0, 2, 2, 1, 1), 0.333333),
    ((2, 0, 0, 1, 0, 0, 2, 1, 0), 0.185185),
    ((2, 0, 0, 1, 0, 0, 2, 1, 1), 0.000000),
    ((2, 0, 0, 1, 0, 0, 0, 1, 2), 0.000000),
    ((2, 0, 0, 1, 0, 0, 0, 0, 1), 0.220779),
    ((2, 2, 0, 1, 0, 0, 0, 0, 1), 0.209302),
    ((2, 0, 2, 1, 0, 0, 0, 0, 1), 0.043478),
    ((2, 0, 0, 1, 2, 0, 0, 0, 1), 0.148148),
    ((2, 0, 0, 1, 0, 2, 0, 0, 1), 0.333333),
    ((2, 0, 0, 1, 0, 0, 2, 0, 1), 0.296296),
    ((2, 0, 0, 1, 0, 0, 0, 2, 1), 0.259259),
    ((0, 2, 0, 1, 0, 0, 0, 0, 0), 0.223610),
    ((0, 2, 0, 1, 1, 0, 0, 0, 0), 0.438753),
    ((0, 2, 2, 1, 1, 0, 0, 0, 0), 0.245902),
    ((0, 2, 2, 1, 1, 1, 0, 0, 0), 1.000000),
    ((0, 2, 2, 1, 1, 0, 1, 0, 0), 0.000000),
    ((0, 2, 2, 1, 1, 2, 1, 0, 0), -0.200000),
    ((0, 2, 2, 1, 1, 2, 1, 1, 0), -1.000000),
    ((0, 2, 2, 1, 1, 2, 1, 1, 2), -1.000000),
    ((0, 2, 2, 1, 1, 2, 1, 0, 1), 0.000000),
    ((0, 2, 2, 1, 1, 2, 1, 2, 1), 1.000000),
    ((1, 2, 2, 1, 1, 2, 1, 2, 1), 1.000000),
    ((0, 2, 2, 1, 1, 0, 1, 2, 0), 0.500000),
    ((0, 2, 2, 1, 1, 1, 1, 2, 0), 1.000000),
    ((0, 2, 2, 1, 1, 0, 1, 2, 1), 0.000000),
    ((0, 2, 2, 1, 1, 0, 1, 0, 2), 0.000000),
    ((0, 2, 2, 1, 1, 1, 1, 0, 2), 1.000000),
    ((0, 2, 2, 1, 1, 0, 1, 1, 2), -1.000000),
    ((0, 2, 2, 1, 1, 0, 0, 1, 0), -0.058824),
    ((0, 2, 2, 1, 1, 2, 0, 1, 0), -0.333333),
    ((0, 2, 2, 1, 1, 2, 0, 1, 1), 0.000000),
    ((0, 2, 2, 1, 1, 2, 2, 1, 1), 1.000000),
    ((0, 2, 2, 1, 1, 0, 2, 1, 0), 0.600000),
    ((0, 2, 2, 1, 1, 1, 2, 1, 0), 1.000000),
    ((0, 2, 2, 1, 1, 0, 2, 1, 1), 0.000000),
    ((0, 2, 2, 1, 1, 0, 0, 1, 2), -0.200000),
    ((0, 2, 2, 1, 1, 1, 0, 1, 2), 1.000000),
    ((0, 2, 2, 1, 1, 0, 0, 0, 1), 0.285714),
    ((0, 2, 2, 1, 1, 2, 0, 0, 1), 0.200000),
    ((0, 2, 2, 1, 1, 0, 2, 0, 1), 0.500000),
    ((0, 2, 2, 1, 1, 1, 2, 0, 1), 1.000000),
    ((0, 2, 2, 1, 1, 0, 0, 2, 1), 0.500000),
    ((0, 2, 2, 1, 1, 1, 0, 2, 1), 1.000000),
    ((0, 2, 0, 1, 1, 2, 0, 0, 0), 0.372549),
    ((0, 2, 0, 1, 1, 2, 1, 0, 0), 0.333333),
    ((0, 2, 0, 1, 1, 2, 1, 2, 0), 1.000000),
    ((0, 2, 0, 1, 1, 2, 1, 2, 1), 1.000000),
    ((0, 2, 0, 1, 1, 2, 1, 0, 2), 0.500000),
    ((0, 2, 0, 1, 1, 2, 1, 1, 2), 0.000000),
    ((0, 2, 0, 1, 1, 2, 0, 1, 0), 0.000000),
    ((0, 2, 0, 1, 1, 2, 2, 1, 0), 0.333333),
    ((0, 2, 0, 1, 1, 2, 2, 1, 1), 0.500000),
    ((0, 2, 0, 1, 1, 2, 0, 1, 2), 0.000000),
    ((0, 2, 0, 1, 1, 2, 0, 0, 1), 0.428571),
    ((0, 2, 0, 1, 1, 2, 2, 0, 1), 0.600000),
    ((0, 2, 0, 1, 1, 2, 0, 2, 1), 1.000000),
    ((0, 2, 0, 1, 1, 0, 2, 0, 0), 0.493671),
    ((0, 2, 0, 1, 1, 1, 2, 0, 0), 1.000000),
    ((0, 2, 0, 1, 1, 0, 2, 1, 0), 0.428571),
    ((0, 2, 0, 1, 1, 0, 2, 1, 2), 0.600000),
    ((0, 2, 0, 1, 1, 1, 2, 1, 2), 1.000000),
    ((0, 2, 0, 1, 1, 0, 2, 0, 1), 0.555556),
    ((0, 2, 0, 1, 1, 0, 2, 2, 1), 1.000000),
    ((0, 2, 0, 1, 1, 1, 2, 2, 1), 1.000000),
    ((0, 2, 0, 1, 1, 0, 0, 2, 0), 0.761194),
    ((0, 2, 0, 1, 1, 1, 0, 2, 0), 1.000000),
    ((0, 2, 0, 1, 1, 0, 1, 2, 0), 0.733333),
    ((0, 2, 0, 1, 1, 0, 1, 2, 2), 1.000000),
    ((0, 2, 0, 1, 1, 1, 1, 2, 2), 1.000000),
    ((0, 2, 0, 1, 1, 0, 0, 2, 1), 0.777778),
    ((0, 2, 0, 1, 1, 0, 0, 0, 2), 0.397260),
    ((0, 2, 0, 1, 1, 1, 0, 0, 2), 1.000000),
    ((0, 2, 0, 1, 1, 0, 1, 0, 2), 0.466667),
    ((0, 2, 0, 1, 1, 0, 0, 1, 2), 0.238095),
    ((0, 2, 0, 1, 0, 1, 0, 0, 0), -0.034014),
    ((0, 2, 2, 1, 0, 1, 0, 0, 0), -0.044776),
    ((0, 2, 2, 1, 0, 1, 1, 0, 0), 0.000000),
    ((0, 2, 2, 1, 2, 1, 1, 0, 0), -0.200000),
    ((0, 2, 2, 1, 2, 1, 1, 1, 0), 0.000000),
    ((0, 2, 2, 1, 2, 1, 1, 1, 2), 1.000000),
    ((0, 2, 2, 1, 2, 1, 1, 0, 1), -1.000000),
    ((0, 2, 2, 1, 2, 1, 1, 2, 1), -1.000000),
    ((0, 2, 2, 1, 0, 1, 1, 2, 0), 0.000000),
    ((0, 2, 2, 1, 0, 1, 1, 2, 1), -1.000000),
    ((0, 2, 2, 1, 0, 1, 1, 0, 2), 0.500000),
    ((0, 2, 2, 1, 0, 1, 1, 1, 2), 0.000000),
    ((0, 2, 2, 1, 0, 1, 0, 1, 0), -0.058824),
    ((0, 2, 2, 1, 2, 1, 0, 1, 0), -0.333333),
    ((0, 2, 2, 1, 2, 1, 0, 1, 1), -1.000000),
    ((0, 2, 2, 1, 2, 1, 2, 1, 1), -1.000000),
    ((0, 2, 2, 1, 0, 1, 2, 1, 0), -0.200000),
    ((0, 2, 2, 1, 0, 1, 2, 1, 1), -1.000000),
    ((0, 2, 2, 1, 0, 1, 0, 1, 2), 0.600000),
    ((0, 2, 2, 1, 0, 1, 0, 0, 1), -0.529412),
    ((0, 2, 2, 1, 2, 1, 0, 0, 1), -1.000000),
    ((0, 2, 2, 1, 0, 1, 2, 0, 1), -0.200000),
    ((0, 2, 2, 1, 0, 1, 0, 2, 1), -0.200000),
    ((0, 2, 0, 1, 2, 1, 0, 0, 0), -0.347826),
    ((0, 2, 0, 1, 2, 1, 1, 0, 0), -0.529412),
    ((0, 2, 0, 1, 2, 1, 1, 2, 0), -1.000000),
    ((0, 2, 0, 1, 2, 1, 1, 0, 2), -0.200000),
    ((0, 2, 0, 1, 2, 1, 1, 1, 2), 0.000000),
    ((0, 2, 0, 1, 2, 1, 0, 1, 0), -0.166667),
    ((0, 2, 0, 1, 2, 1, 2, 1, 0), 0.000000),
    ((0, 2, 0, 1, 2, 1, 2, 1, 1), 0.000000),
    ((0, 2, 0, 1, 2, 1, 0, 1, 2), 0.000000),
    ((0, 2, 0, 1, 2, 1, 0, 0, 1), -0.529412),
    ((0, 2, 0, 1, 2, 1, 2, 0, 1), -0.200000),
    ((0, 2, 0, 1, 2, 1, 0, 2, 1), -1.000000),
    ((0, 2, 0, 1, 0, 1, 2, 0, 0), 0.139241),
    ((0, 2, 0, 1, 0, 1, 2, 1, 0), 0.238095),
    ((0, 2, 0, 1, 0, 1, 2, 1, 2), 0.600000),
    ((0, 2, 0, 1, 0, 1, 2, 0, 1), 0.111111),
    ((0, 2, 0, 1, 0, 1, 2, 2, 1), 0.500000),
    ((0, 2, 0, 1, 0, 1, 0, 2, 0), 0.017544),
    ((0, 2, 0, 1, 0, 1, 1, 2, 0), 0.000000),
    ((0, 2, 0, 1, 0, 1, 1, 2, 2), 0.500000),
    ((0, 2, 0, 1, 0, 1, 0, 2, 1), 0.000000),
    ((0, 2, 0, 1, 0, 1, 0, 0, 2), 0.139241),
    ((0, 2, 0, 1, 0, 1, 1, 0, 2), 0.111111),
    ((0, 2, 0, 1, 0, 1, 0, 1, 2), 0.238095),
    ((0, 2, 0, 1, 0, 0, 1, 0, 0), 0.165468),
    ((0, 2, 2, 1, 0, 0, 1, 0, 0), 0.017544),
    ((0, 2, 2, 1, 0, 0, 1, 1, 0), 0.000000),
    ((0, 2, 2, 1, 2, 0, 1, 1, 0), 0.500000),
    ((0, 2, 2, 1, 2, 0, 1, 1, 1), 1.000000),
    ((0, 2, 2, 1, 0, 2, 1, 1, 0), 0.000000),
    ((0, 2, 2, 1, 0, 2, 1, 1, 1), 1.000000),
    ((0, 2, 2, 1, 0, 0, 1, 1, 2), -0.200000),
    ((0, 2, 2, 1, 0, 0, 1, 0, 1), 0.000000),
    ((0, 2, 2, 1, 2, 0, 1, 0, 1), 0.000000),
    ((0, 2, 2, 1, 0, 2, 1, 0, 1), 0.500000),
    ((0, 2, 2, 1, 0, 0, 1, 2, 1), -0.200000),
    ((0, 2, 0, 1, 2, 0, 1, 0, 0), -0.044776),
    ((0, 2, 0, 1, 2, 0, 1, 1, 0), 0.333333),
    ((0, 2, 0, 1, 2, 2, 1, 1, 0), 1.000000),
    ((0, 2, 0, 1, 2, 2, 1, 1, 1), 1.000000),
    ((0, 2, 0, 1, 2, 0, 1, 1, 2), 0.200000),
    ((0, 2, 0, 1, 2, 0, 1, 0, 1), 0.000000),
    ((0, 2, 0, 1, 2, 2, 1, 0, 1), 0.500000),
    ((0, 2, 0, 1, 2, 0, 1, 2, 1), -1.000000),
    ((0, 2, 0, 1, 0, 2, 1, 0, 0), 0.561644),
    ((0, 2, 0, 1, 0, 2, 1, 1, 0), 0.555556),
    ((0, 2, 0, 1, 0, 2, 1, 1, 2), 0.600000),
    ((0, 2, 0, 1, 0, 2, 1, 0, 1), 0.555556),
    ((0, 2, 0, 1, 0, 2, 1, 2, 1), 0.600000),
    ((0, 2, 0, 1, 0, 0, 1, 2, 0), 0.245902),
    ((0, 2, 0, 1, 0, 0, 1, 2, 1), -0.058824),
    ((0, 2, 0, 1, 0, 0, 1, 0, 2), 0.232877),
    ((0, 2, 0, 1, 0, 0, 1, 1, 2), 0.047619),
    ((0, 2, 0, 1, 0, 0, 0, 1, 0), 0.220779),
    ((0, 2, 2, 1, 0, 0, 0, 1, 0), 0.069767),
    ((0, 2, 2, 1, 0, 0, 0, 1, 1), -0.058824),
    ((0, 2, 2, 1, 2, 0, 0, 1, 1), -0.200000),
    ((0, 2, 2, 1, 0, 2, 0, 1, 1), 0.600000),
    ((0, 2, 2, 1, 0, 0, 2, 1, 1), -0.333333),
    ((0, 2, 0, 1, 2, 0, 0, 1, 0), 0.185185),
    ((0, 2, 0, 1, 2, 0, 0, 1, 1), 0.238095),
    ((0, 2, 0, 1, 2, 2, 0, 1, 1), 0.600000),
    ((0, 2, 0, 1, 2, 0, 2, 1, 1), 0.000000),
    ((0, 2, 0, 1, 0, 2, 0, 1, 0), 0.370370),
    ((0, 2, 0, 1, 0, 2, 0, 1, 1), 0.428571),
    ((0, 2, 0, 1, 0, 2, 2, 1, 1), 0.333333),
    ((0, 2, 0, 1, 0, 0, 2, 1, 0), 0.228070),
    ((0, 2, 0, 1, 0, 0, 2, 1, 1), 0.000000),
    ((0, 2, 0, 1, 0, 0, 0, 1, 2), 0.259259),
    ((0, 2, 0, 1, 0, 0, 0, 0, 1), 0.194030),
    ((0, 2, 2, 1, 0, 0, 0, 0, 1), 0.000000),
    ((0, 2, 0, 1, 2, 0, 0, 0, 1), -0.069767),
    ((0, 2, 0, 1, 0, 2, 0, 0, 1), 0.450980),
    ((0, 2, 0, 1, 0, 0, 2, 0, 1), 0.215686),
    ((0, 2, 0, 1, 0, 0, 0, 2, 1), 0.300000),
    ((0, 0, 2, 1, 0, 0, 0, 0, 0), 0.057715),
    ((0, 0, 2, 1, 1, 0, 0, 0, 0), 0.251799),
    ((0, 0, 2, 1, 1, 2, 0, 0, 0), 0.116279),
    ((0, 0, 2, 1, 1, 2, 1, 0, 0), -0.294118),
    ((0, 0, 2, 1, 1, 2, 1, 2, 0), 0.200000),
    ((0, 0, 2, 1, 1, 2, 1, 2, 1), 0.500000),
    ((0, 0, 2, 1, 1, 2, 1, 0, 2), -1.000000),
    ((0, 0, 2, 1, 1, 2, 0, 1, 0), -0.058824),
    ((0, 0, 2, 1, 1, 2, 2, 1, 0), 0.600000),
    ((0, 0, 2, 1, 1, 2, 2, 1, 1), 1.000000),
    ((0, 0, 2, 1, 1, 2, 0, 1, 2), -1.000000),
    ((0, 0, 2, 1, 1, 2, 0, 0, 1), 0.428571),
    ((0, 0, 2, 1, 1, 2, 2, 0, 1), 1.000000),
    ((0, 0, 2, 1, 1, 2, 0, 2, 1), 0.600000),
    ((0, 0, 2, 1, 1, 0, 2, 0, 0), 0.561644),
    ((0, 0, 2, 1, 1, 1, 2, 0, 0), 1.000000),
    ((0, 0, 2, 1, 1, 0, 2, 1, 0), 0.555556),
    ((0, 0, 2, 1, 1, 0, 2, 1, 2), 0.500000),
    ((0, 0, 2, 1, 1, 1, 2, 1, 2), 1.000000),
    ((0, 0, 2, 1, 1, 0, 2, 0, 1), 0.777778),
    ((0, 0, 2, 1, 1, 0, 2, 2, 1), 1.000000),
    ((0, 0, 2, 1, 1, 1, 2, 2, 1), 1.000000),
    ((0, 0, 2, 1, 1, 0, 0, 2, 0), 0.397260),
    ((0, 0, 2, 1, 1, 1, 0, 2, 0), 1.000000),
    ((0, 0, 2, 1, 1, 0, 1, 2, 0), 0.333333),
    ((0, 0, 2, 1, 1, 0, 1, 2, 2), 0.500000),
    ((0, 0, 2, 1, 1, 1, 1, 2, 2), 1.000000),
    ((0, 0, 2, 1, 1, 0, 0, 2, 1), 0.555556),
    ((0, 0, 2, 1, 1, 0, 0, 0, 2), 0.017544),
    ((0, 0, 2, 1, 1, 1, 0, 0, 2), 1.000000),
    ((0, 0, 2, 1, 1, 0, 1, 0, 2), 0.000000),
    ((0, 0, 2, 1, 1, 0, 0, 1, 2), 0.000000),
    ((0, 0, 2, 1, 0, 1, 0, 0, 0), -0.085339),
    ((0, 0, 2, 1, 2, 1, 0, 0, 0), -0.404255),
    ((0, 0, 2, 1, 2, 1, 1, 0, 0), -0.142857),
    ((0, 0, 2, 1, 2, 1, 1, 2, 0), 0.200000),
    ((0, 0, 2, 1, 2, 1, 1, 2, 1), -0.500000),
    ((0, 0, 2, 1, 2, 1, 1, 0, 2), 0.200000),
    ((0, 0, 2, 1, 2, 1, 1, 1, 2), 0.000000),
    ((0, 0, 2, 1, 2, 1, 0, 1, 0), -0.578947),
    ((0, 0, 2, 1, 2, 1, 2, 1, 0), -1.000000),
    ((0, 0, 2, 1, 2, 1, 0, 1, 2), -0.333333),
    ((0, 0, 2, 1, 2, 1, 0, 0, 1), -0.789474),
    ((0, 0, 2, 1, 2, 1, 2, 0, 1), -1.000000),
    ((0, 0, 2, 1, 2, 1, 0, 2, 1), -0.666667),
    ((0, 0, 2, 1, 0, 1, 2, 0, 0), -0.107692),
    ((0, 0, 2, 1, 0, 1, 2, 1, 0), -0.125000),
    ((0, 0, 2, 1, 0, 1, 2, 1, 2), 0.200000),
    ((0, 0, 2, 1, 0, 1, 2, 0, 1), -0.125000),
    ((0, 0, 2, 1, 0, 1, 2, 2, 1), 0.200000),
    ((0, 0, 2, 1, 0, 1, 0, 2, 0), 0.139241),
    ((0, 0, 2, 1, 0, 1, 1, 2, 0), 0.333333),
    ((0, 0, 2, 1, 0, 1, 1, 2, 2), 1.000000),
    ((0, 0, 2, 1, 0, 1, 0, 2, 1), -0.142857),
    ((0, 0, 2, 1, 0, 1, 0, 0, 2), 0.189873),
    ((0, 0, 2, 1, 0, 1, 1, 0, 2), 0.333333),
    ((0, 0, 2, 1, 0, 1, 0, 1, 2), 0.047619),
    ((0, 0, 2, 1, 0, 0, 1, 0, 0), 0.047619),
    ((0, 0, 2, 1, 2, 0, 1, 0, 0), 0.139241),
    ((0, 0, 2, 1, 2, 0, 1, 1, 0), 0.111111),
    ((0, 0, 2, 1, 2, 2, 1, 1, 0), 0.500000),
    ((0, 0, 2, 1, 2, 2, 1, 1, 1), 1.000000),
    ((0, 0, 2, 1, 2, 0, 1, 1, 2), -0.200000),
    ((0, 0, 2, 1, 2, 0, 1, 0, 1), 0.333333),
    ((0, 0, 2, 1, 2, 2, 1, 0, 1), 1.000000),
    ((0, 0, 2, 1, 2, 0, 1, 2, 1), 0.200000),
    ((0, 0, 2, 1, 0, 2, 1, 0, 0), 0.134328),
    ((0, 0, 2, 1, 0, 2, 1, 1, 0), 0.000000),
    ((0, 0, 2, 1, 0, 2, 1, 1, 2), -1.000000),
    ((0, 0, 2, 1, 0, 2, 1, 0, 1), 0.555556),
    ((0, 0, 2, 1, 0, 2, 1, 2, 1), 0.600000),
    ((0, 0, 2, 1, 0, 0, 1, 2, 0), 0.291139),
    ((0, 0, 2, 1, 0, 0, 1, 2, 1), 0.047619),
    ((0, 0, 2, 1, 0, 0, 1, 0, 2), -0.044776),
    ((0, 0, 2, 1, 0, 0, 1, 1, 2), -0.529412),
    ((0, 0, 2, 1, 0, 0, 0, 1, 0), -0.092308),
    ((0, 0, 2, 1, 2, 0, 0, 1, 0), -0.272727),
    ((0, 0, 2, 1, 2, 0, 0, 1, 1), -0.125000),
    ((0, 0, 2, 1, 2, 2, 0, 1, 1), 0.200000),
    ((0, 0, 2, 1, 2, 0, 2, 1, 1), -1.000000),
    ((0, 0, 2, 1, 0, 2, 0, 1, 0), 0.069767),
    ((0, 0, 2, 1, 0, 2, 0, 1, 1), 0.428571),
    ((0, 0, 2, 1, 0, 2, 2, 1, 1), 0.333333),
    ((0, 0, 2, 1, 0, 0, 2, 1, 0), 0.000000),
    ((0, 0, 2, 1, 0, 0, 2, 1, 1), -0.157895),
    ((0, 0, 2, 1, 0, 0, 0, 1, 2), -0.209302),
    ((0, 0, 2, 1, 0, 0, 0, 0, 1), 0.144928),
    ((0, 0, 2, 1, 2, 0, 0, 0, 1), -0.136364),
    ((0, 0, 2, 1, 0, 2, 0, 0, 1), 0.490196),
    ((0, 0, 2, 1, 0, 0, 2, 0, 1), 0.181818),
    ((0, 0, 2, 1, 0, 0, 0, 2, 1), 0.215686),
    ((0, 0, 0, 1, 2, 0, 0, 0, 0), -0.056168),
    ((0, 0, 0, 1, 2, 1, 0, 0, 0), -0.385714),
    ((0, 0, 0, 1, 2, 1, 2, 0, 0), -0.404255),
    ((0, 0, 0, 1, 2, 1, 2, 1, 0), -0.368421),
    ((0, 0, 0, 1, 2, 1, 2, 1, 2), -0.666667),
    ((0, 0, 0, 1, 2, 1, 2, 0, 1), -0.125000),
    ((0, 0, 0, 1, 2, 1, 2, 2, 1), -0.200000),
    ((0, 0, 0, 1, 2, 1, 0, 2, 0), -0.347826),
    ((0, 0, 0, 1, 2, 1, 1, 2, 0), -0.294118),
    ((0, 0, 0, 1, 2, 1, 1, 2, 2), -0.200000),
    ((0, 0, 0, 1, 2, 1, 0, 2, 1), -0.294118),
    ((0, 0, 0, 1, 2, 1, 0, 0, 2), -0.404255),
    ((0, 0, 0, 1, 2, 1, 1, 0, 2), -0.125000),
    ((0, 0, 0, 1, 2, 1, 0, 1, 2), -0.368421),
    ((0, 0, 0, 1, 2, 0, 1, 0, 0), -0.006565),
    ((0, 0, 0, 1, 2, 2, 1, 0, 0), 0.341772),
    ((0, 0, 0, 1, 2, 2, 1, 1, 0), 0.333333),
    ((0, 0, 0, 1, 2, 2, 1, 1, 2), -0.200000),
    ((0, 0, 0, 1, 2, 2, 1, 0, 1), 0.555556),
    ((0, 0, 0, 1, 2, 2, 1, 2, 1), 0.200000),
    ((0, 0, 0, 1, 2, 0, 1, 2, 0), -0.068493),
    ((0, 0, 0, 1, 2, 0, 1, 2, 1), -0.058824),
    ((0, 0, 0, 1, 2, 0, 1, 0, 2), -0.107692),
    ((0, 0, 0, 1, 2, 0, 1, 1, 2), -0.125000),
    ((0, 0, 0, 1, 2, 0, 0, 1, 0), -0.047297),
    ((0, 0, 0, 1, 2, 2, 0, 1, 0), 0.185185),
    ((0, 0, 0, 1, 2, 2, 0, 1, 1), 0.238095),
    ((0, 0, 0, 1, 2, 2, 2, 1, 1), -0.333333),
    ((0, 0, 0, 1, 2, 0, 2, 1, 0), -0.160000),
    ((0, 0, 0, 1, 2, 0, 2, 1, 1), -0.157895),
    ((0, 0, 0, 1, 2, 0, 0, 1, 2), -0.148936),
    ((0, 0, 0, 1, 2, 0, 0, 0, 1), 0.055556),
    ((0, 0, 0, 1, 2, 2, 0, 0, 1), 0.296296),
    ((0, 0, 0, 1, 2, 0, 2, 0, 1), -0.021277),
    ((0, 0, 0, 1, 2, 0, 0, 2, 1), 0.043478),
    ((0, 0, 0, 1, 0, 2, 0, 0, 0), 0.309395),
    ((0, 0, 0, 1, 1, 2, 0, 0, 0), 0.270270),
    ((0, 0, 0, 1, 1, 2, 2, 0, 0), 0.296296),
    ((0, 0, 0, 1, 1, 2, 2, 1, 0), 0.428571),
    ((0, 0, 0, 1, 1, 2, 2, 1, 2), 0.200000),
    ((0, 0, 0, 1, 1, 2, 2, 0, 1), 0.619048),
    ((0, 0, 0, 1, 1, 2, 2, 2, 1), 0.600000),
    ((0, 0, 0, 1, 1, 2, 0, 2, 0), 0.372549),
    ((0, 0, 0, 1, 1, 2, 1, 2, 0), 0.555556),
    ((0, 0, 0, 1, 1, 2, 1, 2, 2), 0.500000),
    ((0, 0, 0, 1, 1, 2, 0, 2, 1), 0.619048),
    ((0, 0, 0, 1, 1, 2, 0, 0, 2), 0.116279),
    ((0, 0, 0, 1, 1, 2, 1, 0, 2), 0.285714),
    ((0, 0, 0, 1, 1, 2, 0, 1, 2), 0.176471),
    ((0, 0, 0, 1, 0, 2, 1, 0, 0), 0.315011),
    ((0, 0, 0, 1, 0, 2, 1, 2, 0), 0.493671),
    ((0, 0, 0, 1, 0, 2, 1, 2, 1), 0.428571),
    ((0, 0, 0, 1, 0, 2, 1, 0, 2), 0.194030),
    ((0, 0, 0, 1, 0, 2, 1, 1, 2), -0.058824),
    ((0, 0, 0, 1, 0, 2, 0, 1, 0), 0.220779),
    ((0, 0, 0, 1, 0, 2, 2, 1, 0), 0.228070),
    ((0, 0, 0, 1, 0, 2, 2, 1, 1), 0.166667),
    ((0, 0, 0, 1, 0, 2, 0, 1, 2), 0.173913),
    ((0, 0, 0, 1, 0, 2, 0, 0, 1), 0.407407),
    ((0, 0, 0, 1, 0, 2, 2, 0, 1), 0.368421),
    ((0, 0, 0, 1, 0, 2, 0, 2, 1), 0.518519),
    ((0, 0, 0, 1, 0, 0, 2, 0, 0), 0.132555),
    ((0, 0, 0, 1, 1, 0, 2, 0, 0), 0.391121),
    ((0, 0, 0, 1, 1, 0, 2, 2, 0), 0.373134),
    ((0, 0, 0, 1, 1, 1, 2, 2, 0), 1.000000),
    ((0, 0, 0, 1, 1, 0, 2, 2, 1), 0.777778),
    ((0, 0, 0, 1, 1, 0, 2, 0, 2), 0.134328),
    ((0, 0, 0, 1, 1, 1, 2, 0, 2), 1.000000),
    ((0, 0, 0, 1, 1, 0, 2, 1, 2), 0.555556),
    ((0, 0, 0, 1, 0, 1, 2, 0, 0), -0.085339),
    ((0, 0, 0, 1, 0, 1, 2, 2, 0), -0.044776),
    ((0, 0, 0, 1, 0, 1, 2, 2, 1), 0.333333),
    ((0, 0, 0, 1, 0, 1, 2, 0, 2), -0.232877),
    ((0, 0, 0, 1, 0, 1, 2, 1, 2), 0.047619),
    ((0, 0, 0, 1, 0, 0, 2, 1, 0), 0.120253),
    ((0, 0, 0, 1, 0, 0, 2, 1, 2), 0.185185),
    ((0, 0, 0, 1, 0, 0, 2, 0, 1), 0.263158),
    ((0, 0, 0, 1, 0, 0, 2, 2, 1), 0.490196),
    ((0, 0, 0, 1, 0, 0, 0, 2, 0), 0.223610),
    ((0, 0, 0, 1, 1, 0, 0, 2, 0), 0.438753),
    ((0, 0, 0, 1, 1, 0, 0, 2, 2), 0.245902),
    ((0, 0, 0, 1, 1, 1, 0, 2, 2), 1.000000),
    ((0, 0, 0, 1, 1, 0, 1, 2, 2), 0.733333),
    ((0, 0, 0, 1, 0, 1, 0, 2, 0), -0.034014),
    ((0, 0, 0, 1, 0, 1, 0, 2, 2), -0.044776),
    ((0, 0, 0, 1, 0, 1, 1, 2, 2), 0.333333),
    ((0, 0, 0, 1, 0, 0, 1, 2, 0), 0.238901),
    ((0, 0, 0, 1, 0, 0, 1, 2, 2), 0.452055),
    ((0, 0, 0, 1, 0, 0, 0, 2, 1), 0.310811),
    ((0, 0, 0, 1, 0, 0, 0, 0, 2), 0.057715),
    ((0, 0, 0, 1, 1, 0, 0, 0, 2), 0.251799),
    ((0, 0, 0, 1, 0, 1, 0, 0, 2), -0.085339),
    ((0, 0, 0, 1, 0, 0, 1, 0, 2), 0.103926),
    ((0, 0, 0, 1, 0, 0, 0, 1, 2), 0.055556),
    ((0, 0, 0, 0, 1, 0, 0, 0, 0), 0.387755),
    ((2, 0, 0, 0, 1, 0, 0, 0, 0), 0.324578),
    ((2, 0, 0, 0, 1, 1, 0, 0, 0), 0.251799),
    ((2, 2, 0, 0, 1, 1, 0, 0, 0), 0.245902),
    ((2, 2, 0, 0, 1, 1, 1, 0, 0), 0.285714),
    ((2, 2, 2, 0, 1, 1, 1, 0, 0), -1.000000),
    ((2, 2, 0, 2, 1, 1, 1, 0, 0), 0.200000),
    ((2, 2, 0, 2, 1, 1, 1, 1, 0), 0.000000),
    ((2, 2, 2, 2, 1, 1, 1, 1, 0), -1.000000),
    ((2, 2, 0, 2, 1, 1, 1, 1, 2), 1.000000),
    ((2, 2, 0, 2, 1, 1, 1, 0, 1), 0.000000),
    ((2, 2, 2, 2, 1, 1, 1, 0, 1), -1.000000),
    ((2, 2, 0, 2, 1, 1, 1, 2, 1), 1.000000),
    ((2, 2, 1, 2, 1, 1, 1, 2, 1), 1.000000),
    ((2, 2, 0, 0, 1, 1, 1, 2, 0), 0.500000),
    ((2, 2, 0, 0, 1, 1, 1, 2, 1), 0.000000),
    ((2, 2, 2, 0, 1, 1, 1, 2, 1), -1.000000),
    ((2, 2, 0, 0, 1, 1, 1, 0, 2), 0.500000),
    ((2, 2, 0, 0, 1, 1, 1, 1, 2), 0.000000),
    ((2, 2, 2, 0, 1, 1, 1, 1, 2), -1.000000),
    ((2, 2, 0, 0, 1, 1, 0, 1, 0), -0.058824),
    ((2, 2, 2, 0, 1, 1, 0, 1, 0), -1.000000),
    ((2, 2, 0, 2, 1, 1, 0, 1, 0), -0.333333),
    ((2, 2, 0, 2, 1, 1, 0, 1, 1), -1.000000),
    ((2, 2, 2, 2, 1, 1, 0, 1, 1), -1.000000),
    ((2, 2, 0, 2, 1, 1, 2, 1, 1), -1.000000),
    ((2, 2, 0, 0, 1, 1, 2, 1, 0), -0.200000),
    ((2, 2, 0, 0, 1, 1, 2, 1, 1), -1.000000),
    ((2, 2, 2, 0, 1, 1, 2, 1, 1), -1.000000),
    ((2, 2, 0, 0, 1, 1, 0, 1, 2), 0.600000),
    ((2, 2, 0, 0, 1, 1, 0, 0, 1), 0.000000),
    ((2, 2, 2, 0, 1, 1, 0, 0, 1), -1.000000),
    ((2, 2, 0, 2, 1, 1, 0, 0, 1), -0.200000),
    ((2, 2, 0, 0, 1, 1, 2, 0, 1), 0.000000),
    ((2, 2, 0, 0, 1, 1, 0, 2, 1), 0.500000),
    ((2, 0, 2, 0, 1, 1, 0, 0, 0), 0.134328),
    ((2, 0, 2, 0, 1, 1, 1, 0, 0), 0.176471),
    ((2, 0, 2, 2, 1, 1, 1, 0, 0), 0.000000),
    ((2, 0, 2, 2, 1, 1, 1, 1, 0), 0.000000),
    ((2, 0, 2, 2, 1, 1, 1, 1, 2), 1.000000),
    ((2, 0, 2, 2, 1, 1, 1, 0, 1), -0.500000),
    ((2, 0, 2, 2, 1, 1, 1, 2, 1), 0.000000),
    ((2, 0, 2, 0, 1, 1, 1, 2, 0), 0.200000),
    ((2, 0, 2, 0, 1, 1, 1, 2, 1), -0.500000),
    ((2, 0, 2, 0, 1, 1, 1, 0, 2), 0.600000),
    ((2, 0, 2, 0, 1, 1, 1, 1, 2), 0.000000),
    ((2, 0, 2, 0, 1, 1, 0, 1, 0), 0.000000),
    ((2, 0, 2, 2, 1, 1, 0, 1, 0), -0.200000),
    ((2, 0, 2, 2, 1, 1, 0, 1, 1), -1.000000),
    ((2, 0, 2, 2, 1, 1, 2, 1, 1), -1.000000),
    ((2, 0, 2, 0, 1, 1, 2, 1, 0), 0.000000),
    ((2, 0, 2, 0, 1, 1, 2, 1, 1), -1.000000),
    ((2, 0, 2, 0, 1, 1, 0, 1, 2), 0.500000),
    ((2, 0, 2, 0, 1, 1, 0, 0, 1), -0.294118),
    ((2, 0, 2, 2, 1, 1, 0, 0, 1), -0.666667),
    ((2, 0, 2, 0, 1, 1, 2, 0, 1), -0.200000),
    ((2, 0, 2, 0, 1, 1, 0, 2, 1), 0.200000),
    ((2, 0, 0, 2, 1, 1, 0, 0, 0), 0.116279),
    ((2, 0, 0, 2, 1, 1, 1, 0, 0), 0.428571),
    ((2, 0, 0, 2, 1, 1, 1, 2, 0), 0.600000),
    ((2, 0, 0, 2, 1, 1, 1, 2, 1), 0.500000),
    ((2, 0, 0, 2, 1, 1, 1, 0, 2), 1.000000),
    ((2, 0, 0, 2, 1, 1, 1, 1, 2), 1.000000),
    ((2, 0, 0, 2, 1, 1, 0, 1, 0), -0.058824),
    ((2, 0, 0, 2, 1, 1, 2, 1, 0), -1.000000),
    ((2, 0, 0, 2, 1, 1, 0, 1, 2), 0.600000),
    ((2, 0, 0, 2, 1, 1, 0, 0, 1), -0.294118),
    ((2, 0, 0, 2, 1, 1, 2, 0, 1), -1.000000),
    ((2, 0, 0, 2, 1, 1, 0, 2, 1), 0.200000),
    ((2, 0, 0, 0, 1, 1, 2, 0, 0), 0.017544),
    ((2, 0, 0, 0, 1, 1, 2, 1, 0), 0.000000),
    ((2, 0, 0, 0, 1, 1, 2, 1, 2), 0.500000),
    ((2, 0, 0, 0, 1, 1, 2, 0, 1), 0.000000),
    ((2, 0, 0, 0, 1, 1, 2, 2, 1), 0.500000),
    ((2, 0, 0, 0, 1, 1, 0, 2, 0), 0.397260),
    ((2, 0, 0, 0, 1, 1, 1, 2, 0), 0.555556),
    ((2, 0, 0, 0, 1, 1, 1, 2, 2), 1.000000),
    ((2, 0, 0, 0, 1, 1, 0, 2, 1), 0.333333),
    ((2, 0, 0, 0, 1, 1, 0, 0, 2), 0.561644),
    ((2, 0, 0, 0, 1, 1, 1, 0, 2), 0.777778),
    ((2, 0, 0, 0, 1, 1, 0, 1, 2), 0.555556),
    ((2, 0, 0, 0, 1, 0, 1, 0, 0), 0.401361),
    ((2, 2, 0, 0, 1, 0, 1, 0, 0), 0.298246),
    ((2, 2, 0, 0, 1, 0, 1, 1, 0), 0.285714),
    ((2, 2, 2, 0, 1, 0, 1, 1, 0), -1.000000),
    ((2, 2, 0, 2, 1, 0, 1, 1, 0), 0.500000),
    ((2, 2, 0, 2, 1, 0, 1, 1, 1), 1.000000),
    ((2, 2, 0, 0, 1, 2, 1, 1, 0), 0.500000),
    ((2, 2, 0, 0, 1, 2, 1, 1, 1), 1.000000),
    ((2, 2, 0, 0, 1, 0, 1, 1, 2), 0.200000),
    ((2, 2, 0, 0, 1, 0, 1, 0, 1), 0.285714),
    ((2, 2, 2, 0, 1, 0, 1, 0, 1), -1.000000),
    ((2, 2, 0, 2, 1, 0, 1, 0, 1), 0.500000),
    ((2, 2, 0, 0, 1, 2, 1, 0, 1), 0.500000),
    ((2, 2, 0, 0, 1, 0, 1, 2, 1), 0.200000),
    ((2, 0, 2, 0, 1, 0, 1, 0, 0), 0.069767),
    ((2, 0, 2, 0, 1, 0, 1, 1, 0), 0.000000),
    ((2, 0, 2, 2, 1, 0, 1, 1, 0), 0.500000),
    ((2, 0, 2, 2, 1, 0, 1, 1, 1), 1.000000),
    ((2, 0, 2, 0, 1, 2, 1, 1, 0), 0.000000),
    ((2, 0, 2, 0, 1, 2, 1, 1, 1), 1.000000),
    ((2, 0, 2, 0, 1, 0, 1, 1, 2), -0.200000),
    ((2, 0, 2, 0, 1, 0, 1, 0, 1), -0.058824),
    ((2, 0, 2, 2, 1, 0, 1, 0, 1), 0.200000),
    ((2, 0, 2, 0, 1, 2, 1, 0, 1), 0.200000),
    ((2, 0, 2, 0, 1, 0, 1, 2, 1), -0.333333),
    ((2, 0, 0, 2, 1, 0, 1, 0, 0), 0.616438),
    ((2, 0, 0, 2, 1, 0, 1, 1, 0), 0.733333),
    ((2, 0, 0, 2, 1, 2, 1, 1, 0), 1.000000),
    ((2, 0, 0, 2, 1, 2, 1, 1, 1), 1.000000),
    ((2, 0, 0, 2, 1, 0, 1, 1, 2), 1.000000),
    ((2, 0, 0, 2, 1, 0, 1, 0, 1), 0.555556),
    ((2, 0, 0, 2, 1, 2, 1, 0, 1), 1.000000),
    ((2, 0, 0, 2, 1, 0, 1, 2, 1), 0.600000),
    ((2, 0, 0, 0, 1, 2, 1, 0, 0), 0.397260),
    ((2, 0, 0, 0, 1, 2, 1, 1, 0), 0.466667),
    ((2, 0, 0, 0, 1, 2, 1, 1, 2), 0.500000),
    ((2, 0, 0, 0, 1, 2, 1, 0, 1), 0.555556),
    ((2, 0, 0, 0, 1, 2, 1, 2, 1), 0.600000),
    ((2, 0, 0, 0, 1, 0, 1, 2, 0), 0.493671),
    ((2, 0, 0, 0, 1, 0, 1, 2, 1), 0.238095),
    ((2, 0, 0, 0, 1, 0, 1, 0, 2), 0.561644),
    ((2, 0, 0, 0, 1, 0, 1, 1, 2), 0.333333),
    ((2, 0, 0, 0, 1, 0, 0, 1, 0), 0.251799),
    ((2, 2, 0, 0, 1, 0, 0, 1, 0), 0.116279),
    ((2, 2, 0, 0, 1, 0, 0, 1, 1), -0.294118),
    ((2, 2, 2, 0, 1, 0, 0, 1, 1), -1.000000),
    ((2, 2, 0, 2, 1, 0, 0, 1, 1), -0.200000),
    ((2, 2, 0, 0, 1, 2, 0, 1, 1), 0.200000),
    ((2, 2, 0, 0, 1, 0, 2, 1, 1), -0.666667),
    ((2, 0, 2, 0, 1, 0, 0, 1, 0), 0.017544),
    ((2, 0, 2, 0, 1, 0, 0, 1, 1), 0.000000),
    ((2, 0, 2, 2, 1, 0, 0, 1, 1), 0.000000),
    ((2, 0, 2, 0, 1, 2, 0, 1, 1), 0.500000),
    ((2, 0, 2, 0, 1, 0, 2, 1, 1), -0.200000),
    ((2, 0, 0, 2, 1, 0, 0, 1, 0), 0.245902),
    ((2, 0, 0, 2, 1, 0, 0, 1, 1), 0.000000),
    ((2, 0, 0, 2, 1, 2, 0, 1, 1), 0.500000),
    ((2, 0, 0, 2, 1, 0, 2, 1, 1), -1.000000),
    ((2, 0, 0, 0, 1, 2, 0, 1, 0), 0.397260),
    ((2, 0, 0, 0, 1, 2, 0, 1, 1), 0.333333),
    ((2, 0, 0, 0, 1, 2, 2, 1, 1), 0.200000),
    ((2, 0, 0, 0, 1, 0, 2, 1, 0), 0.134328),
    ((2, 0, 0, 0, 1, 0, 2, 1, 1), -0.294118),
    ((2, 0, 0, 0, 1, 0, 0, 1, 2), 0.561644),
    ((2, 0, 0, 0, 1, 0, 0, 0, 1), 0.194030),
    ((2, 2, 0, 0, 1, 0, 0, 0, 1), 0.150000),
    ((2, 0, 2, 0, 1, 0, 0, 0, 1), 0.069767),
    ((2, 0, 0, 2, 1, 0, 0, 0, 1), 0.150000),
    ((2, 0, 0, 0, 1, 2, 0, 0, 1), 0.333333),
    ((2, 0, 0, 0, 1, 0, 2, 0, 1), 0.069767),
    ((2, 0, 0, 0, 1, 0, 0, 2, 1), 0.333333),
    ((0, 2, 0, 0, 1, 0, 0, 0, 0), 0.449541),
    ((0, 2, 0, 0, 1, 1, 0, 0, 0), 0.438753),
    ((0, 2, 2, 0, 1, 1, 0, 0, 0), 0.373134),
    ((0, 2, 2, 0, 1, 1, 1, 0, 0), 0.176471),
    ((0, 2, 2, 2, 1, 1, 1, 0, 0), 0.000000),
    ((0, 2, 2, 2, 1, 1, 1, 1, 0), -0.500000),
    ((0, 2, 2, 2, 1, 1, 1, 1, 2), 0.000000),
    ((0, 2, 2, 2, 1, 1, 1, 0, 1), 0.000000),
    ((0, 2, 2, 2, 1, 1, 1, 2, 1), 1.000000),
    ((0, 2, 2, 0, 1, 1, 1, 2, 0), 0.600000),
    ((0, 2, 2, 0, 1, 1, 1, 2, 1), 0.000000),
    ((0, 2, 2, 0, 1, 1, 1, 0, 2), 0.200000),
    ((0, 2, 2, 0, 1, 1, 1, 1, 2), -0.500000),
    ((0, 2, 2, 0, 1, 1, 0, 1, 0), 0.176471),
    ((0, 2, 2, 2, 1, 1, 0, 1, 0), 0.000000),
    ((0, 2, 2, 2, 1, 1, 0, 1, 1), 0.000000),
    ((0, 2, 2, 2, 1, 1, 2, 1, 1), 1.000000),
    ((0, 2, 2, 0, 1, 1, 2, 1, 0), 0.600000),
    ((0, 2, 2, 0, 1, 1, 2, 1, 1), 0.000000),
    ((0, 2, 2, 0, 1, 1, 0, 1, 2), 0.200000),
    ((0, 2, 2, 0, 1, 1, 0, 0, 1), 0.285714),
    ((0, 2, 2, 2, 1, 1, 0, 0, 1), 0.200000),
    ((0, 2, 2, 0, 1, 1, 2, 0, 1), 0.500000),
    ((0, 2, 2, 0, 1, 1, 0, 2, 1), 0.500000),
    ((0, 2, 0, 2, 1, 1, 0, 0, 0), 0.372549),
    ((0, 2, 0, 2, 1, 1, 1, 0, 0), 0.428571),
    ((0, 2, 0, 2, 1, 1, 1, 2, 0), 1.000000),
    ((0, 2, 0, 2, 1, 1, 1, 2, 1), 1.000000),
    ((0, 2, 0, 2, 1, 1, 1, 0, 2), 0.600000),
    ((0, 2, 0, 2, 1, 1, 1, 1, 2), 0.500000),
    ((0, 2, 0, 2, 1, 1, 0, 1, 0), 0.000000),
    ((0, 2, 0, 2, 1, 1, 2, 1, 0), 0.000000),
    ((0, 2, 0, 2, 1, 1, 2, 1, 1), 0.000000),
    ((0, 2, 0, 2, 1, 1, 0, 1, 2), 0.333333),
    ((0, 2, 0, 2, 1, 1, 0, 0, 1), 0.333333),
    ((0, 2, 0, 2, 1, 1, 2, 0, 1), 0.500000),
    ((0, 2, 0, 2, 1, 1, 0, 2, 1), 1.000000),
    ((0, 2, 0, 0, 1, 1, 2, 0, 0), 0.397260),
    ((0, 2, 0, 0, 1, 1, 2, 1, 0), 0.238095),
    ((0, 2, 0, 0, 1, 1, 2, 1, 2), 0.600000),
    ((0, 2, 0, 0, 1, 1, 2, 0, 1), 0.466667),
    ((0, 2, 0, 0, 1, 1, 2, 2, 1), 1.000000),
    ((0, 2, 0, 0, 1, 1, 0, 2, 0), 0.761194),
    ((0, 2, 0, 0, 1, 1, 1, 2, 0), 0.777778),
    ((0, 2, 0, 0, 1, 1, 1, 2, 2), 1.000000),
    ((0, 2, 0, 0, 1, 1, 0, 2, 1), 0.733333),
    ((0, 2, 0, 0, 1, 1, 0, 0, 2), 0.493671),
    ((0, 2, 0, 0, 1, 1, 1, 0, 2), 0.555556),
    ((0, 2, 0, 0, 1, 1, 0, 1, 2), 0.428571),
    ((0, 2, 0, 0, 1, 0, 1, 0, 0), 0.453237),
    ((0, 2, 2, 0, 1, 0, 1, 0, 0), 0.150000),
    ((0, 2, 2, 0, 1, 0, 1, 1, 0), -0.294118),
    ((0, 2, 2, 2, 1, 0, 1, 1, 0), 0.200000),
    ((0, 2, 2, 2, 1, 0, 1, 1, 1), 1.000000),
    ((0, 2, 2, 0, 1, 2, 1, 1, 0), -0.200000),
    ((0, 2, 2, 0, 1, 2, 1, 1, 1), 1.000000),
    ((0, 2, 2, 0, 1, 0, 1, 1, 2), -0.666667),
    ((0, 2, 2, 0, 1, 0, 1, 0, 1), 0.285714),
    ((0, 2, 2, 2, 1, 0, 1, 0, 1), 0.500000),
    ((0, 2, 2, 0, 1, 2, 1, 0, 1), 0.500000),
    ((0, 2, 2, 0, 1, 0, 1, 2, 1), 0.200000),
    ((0, 2, 0, 2, 1, 0, 1, 0, 0), 0.616438),
    ((0, 2, 0, 2, 1, 0, 1, 1, 0), 0.555556),
    ((0, 2, 0, 2, 1, 2, 1, 1, 0), 1.000000),
    ((0, 2, 0, 2, 1, 2, 1, 1, 1), 1.000000),
    ((0, 2, 0, 2, 1, 0, 1, 1, 2), 0.600000),
    ((0, 2, 0, 2, 1, 0, 1, 0, 1), 0.733333),
    ((0, 2, 0, 2, 1, 2, 1, 0, 1), 1.000000),
    ((0, 2, 0, 2, 1, 0, 1, 2, 1), 1.000000),
    ((0, 2, 0, 0, 1, 2, 1, 0, 0), 0.522388),
    ((0, 2, 0, 0, 1, 2, 1, 1, 0), 0.333333),
    ((0, 2, 0, 0, 1, 2, 1, 1, 2), 0.200000),
    ((0, 2, 0, 0, 1, 2, 1, 0, 1), 0.733333),
    ((0, 2, 0, 0, 1, 2, 1, 2, 1), 1.000000),
    ((0, 2, 0, 0, 1, 0, 1, 2, 0), 0.761194),
    ((0, 2, 0, 0, 1, 0, 1, 2, 1), 0.555556),
    ((0, 2, 0, 0, 1, 0, 1, 0, 2), 0.397260),
    ((0, 2, 0, 0, 1, 0, 1, 1, 2), 0.047619),
    ((0, 2, 0, 0, 1, 0, 0, 1, 0), 0.270270),
    ((0, 2, 2, 0, 1, 0, 0, 1, 0), 0.116279),
    ((0, 2, 2, 0, 1, 0, 0, 1, 1), 0.285714),
    ((0, 2, 2, 2, 1, 0, 0, 1, 1), 0.500000),
    ((0, 2, 2, 0, 1, 2, 0, 1, 1), 0.500000),
    ((0, 2, 2, 0, 1, 0, 2, 1, 1), 0.200000),
    ((0, 2, 0, 2, 1, 0, 0, 1, 0), 0.372549),
    ((0, 2, 0, 2, 1, 0, 0, 1, 1), 0.333333),
    ((0, 2, 0, 2, 1, 2, 0, 1, 1), 1.000000),
    ((0, 2, 0, 2, 1, 0, 2, 1, 1), 0.200000),
    ((0, 2, 0, 0, 1, 2, 0, 1, 0), 0.372549),
    ((0, 2, 0, 0, 1, 2, 0, 1, 1), 0.555556),
    ((0, 2, 0, 0, 1, 2, 2, 1, 1), 0.600000),
    ((0, 2, 0, 0, 1, 0, 2, 1, 0), 0.296296),
    ((0, 2, 0, 0, 1, 0, 2, 1, 1), 0.047619),
    ((0, 2, 0, 0, 1, 0, 0, 1, 2), 0.296296),
    ((0, 2, 0, 0, 1, 0, 0, 0, 1), 0.453237),
    ((0, 2, 2, 0, 1, 0, 0, 0, 1), 0.298246),
    ((0, 2, 0, 2, 1, 0, 0, 0, 1), 0.522388),
    ((0, 2, 0, 0, 1, 2, 0, 0, 1), 0.616438),
    ((0, 2, 0, 0, 1, 0, 2, 0, 1), 0.397260),
    ((0, 2, 0, 0, 1, 0, 0, 2, 1), 0.761194),
    ((0, 0, 2, 0, 1, 0, 0, 0, 0), 0.324578),
    ((0, 0, 2, 0, 1, 1, 0, 0, 0), 0.391121),
    ((0, 0, 2, 2, 1, 1, 0, 0, 0), 0.296296),
    ((0, 0, 2, 2, 1, 1, 1, 0, 0), 0.166667),
    ((0, 0, 2, 2, 1, 1, 1, 2, 0), 0.333333),
    ((0, 0, 2, 2, 1, 1, 1, 2, 1), 0.500000),
    ((0, 0, 2, 2, 1, 1, 1, 0, 2), 0.333333),
    ((0, 0, 2, 2, 1, 1, 1, 1, 2), 0.500000),
    ((0, 0, 2, 2, 1, 1, 0, 1, 0), 0.238095),
    ((0, 0, 2, 2, 1, 1, 2, 1, 0), 0.600000),
    ((0, 0, 2, 2, 1, 1, 2, 1, 1), 0.000000),
    ((0, 0, 2, 2, 1, 1, 0, 1, 2), 0.600000),
    ((0, 0, 2, 2, 1, 1, 0, 0, 1), 0.047619),
    ((0, 0, 2, 2, 1, 1, 2, 0, 1), 0.200000),
    ((0, 0, 2, 2, 1, 1, 0, 2, 1), 0.600000),
    ((0, 0, 2, 0, 1, 1, 2, 0, 0), 0.561644),
    ((0, 0, 2, 0, 1, 1, 2, 1, 0), 0.555556),
    ((0, 0, 2, 0, 1, 1, 2, 1, 2), 1.000000),
    ((0, 0, 2, 0, 1, 1, 2, 0, 1), 0.333333),
    ((0, 0, 2, 0, 1, 1, 2, 2, 1), 1.000000),
    ((0, 0, 2, 0, 1, 1, 0, 2, 0), 0.493671),
    ((0, 0, 2, 0, 1, 1, 1, 2, 0), 0.428571),
    ((0, 0, 2, 0, 1, 1, 1, 2, 2), 0.600000),
    ((0, 0, 2, 0, 1, 1, 0, 2, 1), 0.555556),
    ((0, 0, 2, 0, 1, 1, 0, 0, 2), 0.493671),
    ((0, 0, 2, 0, 1, 1, 1, 0, 2), 0.428571),
    ((0, 0, 2, 0, 1, 1, 0, 1, 2), 0.555556),
    ((0, 0, 2, 0, 1, 0, 1, 0, 0), 0.194030),
    ((0, 0, 2, 2, 1, 0, 1, 0, 0), 0.333333),
    ((0, 0, 2, 2, 1, 0, 1, 1, 0), 0.333333),
    ((0, 0, 2, 2, 1, 2, 1, 1, 0), 0.500000),
    ((0, 0, 2, 2, 1, 2, 1, 1, 1), 1.000000),
    ((0, 0, 2, 2, 1, 0, 1, 1, 2), 0.200000),
    ((0, 0, 2, 2, 1, 0, 1, 0, 1), 0.555556),
    ((0, 0, 2, 2, 1, 2, 1, 0, 1), 1.000000),
    ((0, 0, 2, 2, 1, 0, 1, 2, 1), 0.600000),
    ((0, 0, 2, 0, 1, 2, 1, 0, 0), 0.150000),
    ((0, 0, 2, 0, 1, 2, 1, 1, 0), 0.000000),
    ((0, 0, 2, 0, 1, 2, 1, 1, 2), -1.000000),
    ((0, 0, 2, 0, 1, 2, 1, 0, 1), 0.555556),
    ((0, 0, 2, 0, 1, 2, 1, 2, 1), 0.600000),
    ((0, 0, 2, 0, 1, 0, 1, 2, 0), 0.333333),
    ((0, 0, 2, 0, 1, 0, 1, 2, 1), 0.238095),
    ((0, 0, 2, 0, 1, 0, 1, 0, 2), 0.069767),
    ((0, 0, 2, 0, 1, 0, 1, 1, 2), -0.294118),
    ((0, 0, 2, 0, 1, 0, 0, 1, 0), 0.251799),
    ((0, 0, 2, 2, 1, 0, 0, 1, 0), 0.397260),
    ((0, 0, 2, 2, 1, 0, 0, 1, 1), 0.466667),
    ((0, 0, 2, 2, 1, 2, 0, 1, 1), 1.000000),
    ((0, 0, 2, 2, 1, 0, 2, 1, 1), 0.500000),
    ((0, 0, 2, 0, 1, 2, 0, 1, 0), 0.245902),
    ((0, 0, 2, 0, 1, 2, 0, 1, 1), 0.733333),
    ((0, 0, 2, 0, 1, 2, 2, 1, 1), 1.000000),
    ((0, 0, 2, 0, 1, 0, 2, 1, 0), 0.561644),
    ((0, 0, 2, 0, 1, 0, 2, 1, 1), 0.333333),
    ((0, 0, 2, 0, 1, 0, 0, 1, 2), 0.134328),
    ((0, 0, 2, 0, 1, 0, 0, 0, 1), 0.401361),
    ((0, 0, 2, 2, 1, 0, 0, 0, 1), 0.397260),
    ((0, 0, 2, 0, 1, 2, 0, 0, 1), 0.616438),
    ((0, 0, 2, 0, 1, 0, 2, 0, 1), 0.561644),
    ((0, 0, 2, 0, 1, 0, 0, 2, 1), 0.493671),
    ((0, 0, 0, 2, 1, 0, 0, 0, 0), 0.449541),
    ((0, 0, 0, 2, 1, 1, 0, 0, 0), 0.270270),
    ((0, 0, 0, 2, 1, 1, 2, 0, 0), 0.116279),
    ((0, 0, 0, 2, 1, 1, 2, 1, 0), 0.176471),
    ((0, 0, 0, 2, 1, 1, 2, 1, 2), 0.200000),
    ((0, 0, 0, 2, 1, 1, 2, 0, 1), 0.285714),
    ((0, 0, 0, 2, 1, 1, 2, 2, 1), 0.500000),
    ((0, 0, 0, 2, 1, 1, 0, 2, 0), 0.372549),
    ((0, 0, 0, 2, 1, 1, 1, 2, 0), 0.619048),
    ((0, 0, 0, 2, 1, 1, 1, 2, 2), 0.600000),
    ((0, 0, 0, 2, 1, 1, 0, 2, 1), 0.555556),
    ((0, 0, 0, 2, 1, 1, 0, 0, 2), 0.296296),
    ((0, 0, 0, 2, 1, 1, 1, 0, 2), 0.619048),
    ((0, 0, 0, 2, 1, 1, 0, 1, 2), 0.428571),
    ((0, 0, 0, 2, 1, 0, 1, 0, 0), 0.568710),
    ((0, 0, 0, 2, 1, 2, 1, 0, 0), 0.761194),
    ((0, 0, 0, 2, 1, 2, 1, 1, 0), 0.733333),
    ((0, 0, 0, 2, 1, 2, 1, 1, 2), 0.500000),
    ((0, 0, 0, 2, 1, 2, 1, 0, 1), 1.000000),
    ((0, 0, 0, 2, 1, 2, 1, 2, 1), 1.000000),
    ((0, 0, 0, 2, 1, 0, 1, 2, 0), 0.696203),
    ((0, 0, 0, 2, 1, 0, 1, 2, 1), 0.777778),
    ((0, 0, 0, 2, 1, 0, 1, 0, 2), 0.493671),
    ((0, 0, 0, 2, 1, 0, 1, 1, 2), 0.555556),
    ((0, 0, 0, 2, 1, 0, 0, 1, 0), 0.438753),
    ((0, 0, 0, 2, 1, 2, 0, 1, 0), 0.761194),
    ((0, 0, 0, 2, 1, 2, 0, 1, 1), 0.733333),
    ((0, 0, 0, 2, 1, 2, 2, 1, 1), 0.500000),
    ((0, 0, 0, 2, 1, 0, 2, 1, 0), 0.373134),
    ((0, 0, 0, 2, 1, 0, 2, 1, 1), 0.285714),
    ((0, 0, 0, 2, 1, 0, 0, 1, 2), 0.493671),
    ((0, 0, 0, 2, 1, 0, 0, 0, 1), 0.453237),
    ((0, 0, 0, 2, 1, 2, 0, 0, 1), 0.761194),
    ((0, 0, 0, 2, 1, 0, 2, 0, 1), 0.298246),
    ((0, 0, 0, 2, 1, 0, 0, 2, 1), 0.616438),
    ((0, 0, 0, 0, 1, 2, 0, 0, 0), 0.449541),
    ((0, 0, 0, 0, 1, 2, 1, 0, 0), 0.453237),
    ((0, 0, 0, 0, 1, 2, 1, 2, 0), 0.616438),
    ((0, 0, 0, 0, 1, 2, 1, 2, 1), 0.777778),
    ((0, 0, 0, 0, 1, 2, 1, 0, 2), 0.298246),
    ((0, 0, 0, 0, 1, 2, 1, 1, 2), 0.285714),
    ((0, 0, 0, 0, 1, 2, 0, 1, 0), 0.438753),
    ((0, 0, 0, 0, 1, 2, 2, 1, 0), 0.493671),
    ((0, 0, 0, 0, 1, 2, 2, 1, 1), 0.555556),
    ((0, 0, 0, 0, 1, 2, 0, 1, 2), 0.373134),
    ((0, 0, 0, 0, 1, 2, 0, 0, 1), 0.568710),
    ((0, 0, 0, 0, 1, 2, 2, 0, 1), 0.493671),
    ((0, 0, 0, 0, 1, 2, 0, 2, 1), 0.696203),
    ((0, 0, 0, 0, 1, 0, 2, 0, 0), 0.324578),
    ((0, 0, 0, 0, 1, 1, 2, 0, 0), 0.251799),
    ((0, 0, 0, 0, 1, 1, 2, 2, 0), 0.245902),
    ((0, 0, 0, 0, 1, 1, 2, 2, 1), 0.733333),
    ((0, 0, 0, 0, 1, 1, 2, 0, 2), 0.134328),
    ((0, 0, 0, 0, 1, 1, 2, 1, 2), 0.555556),
    ((0, 0, 0, 0, 1, 0, 2, 1, 0), 0.391121),
    ((0, 0, 0, 0, 1, 0, 2, 1, 2), 0.493671),
    ((0, 0, 0, 0, 1, 0, 2, 0, 1), 0.401361),
    ((0, 0, 0, 0, 1, 0, 2, 2, 1), 0.616438),
    ((0, 0, 0, 0, 1, 0, 0, 2, 0), 0.449541),
    ((0, 0, 0, 0, 1, 1, 0, 2, 0), 0.438753),
    ((0, 0, 0, 0, 1, 1, 0, 2, 2), 0.373134),
    ((0, 0, 0, 0, 1, 1, 1, 2, 2), 0.777778),
    ((0, 0, 0, 0, 1, 0, 1, 2, 0), 0.568710),
    ((0, 0, 0, 0, 1, 0, 1, 2, 2), 0.616438),
    ((0, 0, 0, 0, 1, 0, 0, 2, 1), 0.568710),
    ((0, 0, 0, 0, 1, 0, 0, 0, 2), 0.324578),
    ((0, 0, 0, 0, 1, 1, 0, 0, 2), 0.391121),
    ((0, 0, 0, 0, 1, 0, 1, 0, 2), 0.401361),
    ((0, 0, 0, 0, 1, 0, 0, 1, 2), 0.391121),
    ((0, 0, 0, 0, 0, 1, 0, 0, 0), 0.137064),
    ((2, 0, 0, 0, 0, 1, 0, 0, 0), 0.057715),
    ((2, 0, 0, 0, 0, 1, 1, 0, 0), 0.144928),
    ((2, 2, 0, 0, 0, 1, 1, 0, 0), 0.000000),
    ((2, 2, 0, 0, 0, 1, 1, 1, 0), -0.058824),
    ((2, 2, 2, 0, 0, 1, 1, 1, 0), -1.000000),
    ((2, 2, 0, 2, 0, 1, 1, 1, 0), 0.600000),
    ((2, 2, 0, 2, 0, 1, 1, 1, 1), 1.000000),
    ((2, 2, 0, 0, 2, 1, 1, 1, 0), -0.200000),
    ((2, 2, 0, 0, 2, 1, 1, 1, 1), 1.000000),
    ((2, 2, 0, 0, 0, 1, 1, 1, 2), -0.333333),
    ((2, 2, 0, 0, 0, 1, 1, 0, 1), 0.000000),
    ((2, 2, 2, 0, 0, 1, 1, 0, 1), -1.000000),
    ((2, 2, 0, 2, 0, 1, 1, 0, 1), 0.500000),
    ((2, 2, 0, 0, 2, 1, 1, 0, 1), 0.000000),
    ((2, 2, 0, 0, 0, 1, 1, 2, 1), -0.200000),
    ((2, 0, 2, 0, 0, 1, 1, 0, 0), 0.043478),
    ((2, 0, 2, 0, 0, 1, 1, 1, 0), -0.058824),
    ((2, 0, 2, 2, 0, 1, 1, 1, 0), 0.600000),
    ((2, 0, 2, 2, 0, 1, 1, 1, 1), 1.000000),
    ((2, 0, 2, 0, 2, 1, 1, 1, 0), -0.200000),
    ((2, 0, 2, 0, 2, 1, 1, 1, 1), 1.000000),
    ((2, 0, 2, 0, 0, 1, 1, 1, 2), -0.333333),
    ((2, 0, 2, 0, 0, 1, 1, 0, 1), -0.058824),
    ((2, 0, 2, 2, 0, 1, 1, 0, 1), 0.200000),
    ((2, 0, 2, 0, 2, 1, 1, 0, 1), 0.200000),
    ((2, 0, 2, 0, 0, 1, 1, 2, 1), -0.333333),
    ((2, 0, 0, 2, 0, 1, 1, 0, 0), 0.490196),
    ((2, 0, 0, 2, 0, 1, 1, 1, 0), 0.428571),
    ((2, 0, 0, 2, 2, 1, 1, 1, 0), 0.200000),
    ((2, 0, 0, 2, 2, 1, 1, 1, 1), 1.000000),
    ((2, 0, 0, 2, 0, 1, 1, 1, 2), 0.333333),
    ((2, 0, 0, 2, 0, 1, 1, 0, 1), 0.555556),
    ((2, 0, 0, 2, 2, 1, 1, 0, 1), 1.000000),
    ((2, 0, 0, 2, 0, 1, 1, 2, 1), 0.600000),
    ((2, 0, 0, 0, 2, 1, 1, 0, 0), -0.136364),
    ((2, 0, 0, 0, 2, 1, 1, 1, 0), -0.125000),
    ((2, 0, 0, 0, 2, 1, 1, 1, 2), -1.000000),
    ((2, 0, 0, 0, 2, 1, 1, 0, 1), 0.333333),
    ((2, 0, 0, 0, 2, 1, 1, 2, 1), 0.200000),
    ((2, 0, 0, 0, 0, 1, 1, 2, 0), 0.215686),
    ((2, 0, 0, 0, 0, 1, 1, 2, 1), 0.047619),
    ((2, 0, 0, 0, 0, 1, 1, 0, 2), 0.181818),
    ((2, 0, 0, 0, 0, 1, 1, 1, 2), -0.157895),
    ((2, 0, 0, 0, 0, 1, 0, 1, 0), -0.092308),
    ((2, 2, 0, 0, 0, 1, 0, 1, 0), 0.069767),
    ((2, 2, 0, 0, 0, 1, 0, 1, 1), 0.000000),
    ((2, 2, 2, 0, 0, 1, 0, 1, 1), -1.000000),
    ((2, 2, 0, 2, 0, 1, 0, 1, 1), 0.000000),
    ((2, 2, 0, 0, 2, 1, 0, 1, 1), 0.500000),
    ((2, 2, 0, 0, 0, 1, 2, 1, 1), -0.200000),
    ((2, 0, 2, 0, 0, 1, 0, 1, 0), -0.209302),
    ((2, 0, 2, 0, 0, 1, 0, 1, 1), -0.529412),
    ((2, 0, 2, 2, 0, 1, 0, 1, 1), -0.200000),
    ((2, 0, 2, 0, 2, 1, 0, 1, 1), -0.200000),
    ((2, 0, 2, 0, 0, 1, 2, 1, 1), -1.000000),
    ((2, 0, 0, 2, 0, 1, 0, 1, 0), 0.069767),
    ((2, 0, 0, 2, 0, 1, 0, 1, 1), 0.000000),
    ((2, 0, 0, 2, 2, 1, 0, 1, 1), 0.500000),
    ((2, 0, 0, 2, 0, 1, 2, 1, 1), -1.000000),
    ((2, 0, 0, 0, 2, 1, 0, 1, 0), -0.272727),
    ((2, 0, 0, 0, 2, 1, 0, 1, 1), 0.111111),
    ((2, 0, 0, 0, 2, 1, 2, 1, 1), -0.200000),
    ((2, 0, 0, 0, 0, 1, 2, 1, 0), -0.209302),
    ((2, 0, 0, 0, 0, 1, 2, 1, 1), -0.529412),
    ((2, 0, 0, 0, 0, 1, 0, 1, 2), 0.000000),
    ((2, 0, 0, 0, 0, 1, 0, 0, 1), 0.047619),
    ((2, 2, 0, 0, 0, 1, 0, 0, 1), 0.017544),
    ((2, 0, 2, 0, 0, 1, 0, 0, 1), -0.217391),
    ((2, 0, 0, 2, 0, 1, 0, 0, 1), 0.134328),
    ((2, 0, 0, 0, 2, 1, 0, 0, 1), 0.139241),
    ((2, 0, 0, 0, 0, 1, 2, 0, 1), -0.044776),
    ((2, 0, 0, 0, 0, 1, 0, 2, 1), 0.291139),
    ((0, 2, 0, 0, 0, 1, 0, 0, 0), 0.223610),
    ((0, 2, 0, 0, 0, 1, 1, 0, 0), 0.194030),
    ((0, 2, 2, 0, 0, 1, 1, 0, 0), 0.209302),
    ((0, 2, 2, 0, 0, 1, 1, 1, 0), 0.176471),
    ((0, 2, 2, 2, 0, 1, 1, 1, 0), 0.200000),
    ((0, 2, 2, 2, 0, 1, 1, 1, 1), 1.000000),
    ((0, 2, 2, 0, 2, 1, 1, 1, 0), 0.600000),
    ((0, 2, 2, 0, 2, 1, 1, 1, 1), 1.000000),
    ((0, 2, 2, 0, 0, 1, 1, 1, 2), 0.000000),
    ((0, 2, 2, 0, 0, 1, 1, 0, 1), -0.058824),
    ((0, 2, 2, 2, 0, 1, 1, 0, 1), 0.600000),
    ((0, 2, 2, 0, 2, 1, 1, 0, 1), -0.200000),
    ((0, 2, 2, 0, 0, 1, 1, 2, 1), -0.333333),
    ((0, 2, 0, 2, 0, 1, 1, 0, 0), 0.450980),
    ((0, 2, 0, 2, 0, 1, 1, 1, 0), 0.428571),
    ((0, 2, 0, 2, 2, 1, 1, 1, 0), 0.600000),
    ((0, 2, 0, 2, 2, 1, 1, 1, 1), 1.000000),
    ((0, 2, 0, 2, 0, 1, 1, 1, 2), 0.333333),
    ((0, 2, 0, 2, 0, 1, 1, 0, 1), 0.555556),
    ((0, 2, 0, 2, 2, 1, 1, 0, 1), 0.500000),
    ((0, 2, 0, 2, 0, 1, 1, 2, 1), 0.600000),
    ((0, 2, 0, 0, 2, 1, 1, 0, 0), -0.069767),
    ((0, 2, 0, 0, 2, 1, 1, 1, 0), 0.238095),
    ((0, 2, 0, 0, 2, 1, 1, 1, 2), 0.000000),
    ((0, 2, 0, 0, 2, 1, 1, 0, 1), 0.000000),
    ((0, 2, 0, 0, 2, 1, 1, 2, 1), -1.000000),
    ((0, 2, 0, 0, 0, 1, 1, 2, 0), 0.300000),
    ((0, 2, 0, 0, 0, 1, 1, 2, 1), -0.058824),
    ((0, 2, 0, 0, 0, 1, 1, 0, 2), 0.215686),
    ((0, 2, 0, 0, 0, 1, 1, 1, 2), 0.000000),
    ((0, 2, 0, 0, 0, 1, 0, 1, 0), 0.220779),
    ((0, 2, 2, 0, 0, 1, 0, 1, 0), 0.173913),
    ((0, 2, 2, 0, 0, 1, 0, 1, 1), -0.058824),
    ((0, 2, 2, 2, 0, 1, 0, 1, 1), 0.600000),
    ((0, 2, 2, 0, 2, 1, 0, 1, 1), -0.200000),
    ((0, 2, 2, 0, 0, 1, 2, 1, 1), -0.333333),
    ((0, 2, 0, 2, 0, 1, 0, 1, 0), 0.370370),
    ((0, 2, 0, 2, 0, 1, 0, 1, 1), 0.555556),
    ((0, 2, 0, 2, 2, 1, 0, 1, 1), 1.000000),
    ((0, 2, 0, 2, 0, 1, 2, 1, 1), 0.600000),
    ((0, 2, 0, 0, 2, 1, 0, 1, 0), 0.185185),
    ((0, 2, 0, 0, 2, 1, 0, 1, 1), 0.333333),
    ((0, 2, 0, 0, 2, 1, 2, 1, 1), 0.200000),
    ((0, 2, 0, 0, 0, 1, 2, 1, 0), 0.259259),
    ((0, 2, 0, 0, 0, 1, 2, 1, 1), 0.047619),
    ((0, 2, 0, 0, 0, 1, 0, 1, 2), 0.228070),
    ((0, 2, 0, 0, 0, 1, 0, 0, 1), 0.165468),
    ((0, 2, 2, 0, 0, 1, 0, 0, 1), -0.023256),
    ((0, 2, 0, 2, 0, 1, 0, 0, 1), 0.561644),
    ((0, 2, 0, 0, 2, 1, 0, 0, 1), -0.044776),
    ((0, 2, 0, 0, 0, 1, 2, 0, 1), 0.232877),
    ((0, 2, 0, 0, 0, 1, 0, 2, 1), 0.245902),
    ((0, 0, 2, 0, 0, 1, 0, 0, 0), 0.132555),
    ((0, 0, 2, 0, 0, 1, 1, 0, 0), 0.220779),
    ((0, 0, 2, 2, 0, 1, 1, 0, 0), 0.333333),
    ((0, 0, 2, 2, 0, 1, 1, 1, 0), 0.428571),
    ((0, 0, 2, 2, 2, 1, 1, 1, 0), 0.600000),
    ((0, 0, 2, 2, 2, 1, 1, 1, 1), 1.000000),
    ((0, 0, 2, 2, 0, 1, 1, 1, 2), 0.333333),
    ((0, 0, 2, 2, 0, 1, 1, 0, 1), 0.428571),
    ((0, 0, 2, 2, 2, 1, 1, 0, 1), 0.600000),
    ((0, 0, 2, 2, 0, 1, 1, 2, 1), 0.333333),
    ((0, 0, 2, 0, 2, 1, 1, 0, 0), 0.148148),
    ((0, 0, 2, 0, 2, 1, 1, 1, 0), 0.238095),
    ((0, 0, 2, 0, 2, 1, 1, 1, 2), 0.000000),
    ((0, 0, 2, 0, 2, 1, 1, 0, 1), 0.047619),
    ((0, 0, 2, 0, 2, 1, 1, 2, 1), -0.333333),
    ((0, 0, 2, 0, 0, 1, 1, 2, 0), 0.259259),
    ((0, 0, 2, 0, 0, 1, 1, 2, 1), -0.166667),
    ((0, 0, 2, 0, 0, 1, 1, 0, 2), 0.296296),
    ((0, 0, 2, 0, 0, 1, 1, 1, 2), 0.000000),
    ((0, 0, 2, 0, 0, 1, 0, 1, 0), 0.055556),
    ((0, 0, 2, 2, 0, 1, 0, 1, 0), 0.259259),
    ((0, 0, 2, 2, 0, 1, 0, 1, 1), 0.047619),
    ((0, 0, 2, 2, 2, 1, 0, 1, 1), 0.200000),
    ((0, 0, 2, 2, 0, 1, 2, 1, 1), -0.333333),
    ((0, 0, 2, 0, 2, 1, 0, 1, 0), -0.148936),
    ((0, 0, 2, 0, 2, 1, 0, 1, 1), -0.125000),
    ((0, 0, 2, 0, 2, 1, 2, 1, 1), -1.000000),
    ((0, 0, 2, 0, 0, 1, 2, 1, 0), 0.000000),
    ((0, 0, 2, 0, 0, 1, 2, 1, 1), -0.578947),
    ((0, 0, 2, 0, 0, 1, 0, 1, 2), 0.185185),
    ((0, 0, 2, 0, 0, 1, 0, 0, 1), -0.048611),
    ((0, 0, 2, 2, 0, 1, 0, 0, 1), 0.185185),
    ((0, 0, 2, 0, 2, 1, 0, 0, 1), -0.276596),
    ((0, 0, 2, 0, 0, 1, 2, 0, 1), -0.090909),
    ((0, 0, 2, 0, 0, 1, 0, 2, 1), 0.074074),
    ((0, 0, 0, 2, 0, 1, 0, 0, 0), 0.309395),
    ((0, 0, 0, 2, 0, 1, 1, 0, 0), 0.407407),
    ((0, 0, 0, 2, 2, 1, 1, 0, 0), 0.296296),
    ((0, 0, 0, 2, 2, 1, 1, 1, 0), 0.238095),
    ((0, 0, 0, 2, 2, 1, 1, 1, 2), -0.333333),
    ((0, 0, 0, 2, 2, 1, 1, 0, 1), 0.555556),
    ((0, 0, 0, 2, 2, 1, 1, 2, 1), 0.200000),
    ((0, 0, 0, 2, 0, 1, 1, 2, 0), 0.518519),
    ((0, 0, 0, 2, 0, 1, 1, 2, 1), 0.428571),
    ((0, 0, 0, 2, 0, 1, 1, 0, 2), 0.368421),
    ((0, 0, 0, 2, 0, 1, 1, 1, 2), 0.166667),
    ((0, 0, 0, 2, 0, 1, 0, 1, 0), 0.220779),
    ((0, 0, 0, 2, 2, 1, 0, 1, 0), 0.185185),
    ((0, 0, 0, 2, 2, 1, 0, 1, 1), 0.333333),
    ((0, 0, 0, 2, 2, 1, 2, 1, 1), -0.200000),
    ((0, 0, 0, 2, 0, 1, 2, 1, 0), 0.173913),
    ((0, 0, 0, 2, 0, 1, 2, 1, 1), -0.058824),
    ((0, 0, 0, 2, 0, 1, 0, 1, 2), 0.228070),
    ((0, 0, 0, 2, 0, 1, 0, 0, 1), 0.315011),
    ((0, 0, 0, 2, 2, 1, 0, 0, 1), 0.341772),
    ((0, 0, 0, 2, 0, 1, 2, 0, 1), 0.194030),
    ((0, 0, 0, 2, 0, 1, 0, 2, 1), 0.493671),
    ((0, 0, 0, 0, 2, 1, 0, 0, 0), -0.056168),
    ((0, 0, 0, 0, 2, 1, 1, 0, 0), 0.055556),
    ((0, 0, 0, 0, 2, 1, 1, 2, 0), 0.043478),
    ((0, 0, 0, 0, 2, 1, 1, 2, 1), -0.058824),
    ((0, 0, 0, 0, 2, 1, 1, 0, 2), -0.021277),
    ((0, 0, 0, 0, 2, 1, 1, 1, 2), -0.157895),
    ((0, 0, 0, 0, 2, 1, 0, 1, 0), -0.047297),
    ((0, 0, 0, 0, 2, 1, 2, 1, 0), -0.148936),
    ((0, 0, 0, 0, 2, 1, 2, 1, 1), -0.125000),
    ((0, 0, 0, 0, 2, 1, 0, 1, 2), -0.160000),
    ((0, 0, 0, 0, 2, 1, 0, 0, 1), -0.006565),
    ((0, 0, 0, 0, 2, 1, 2, 0, 1), -0.107692),
    ((0, 0, 0, 0, 2, 1, 0, 2, 1), -0.068493),
    ((0, 0, 0, 0, 0, 1, 2, 0, 0), 0.057715),
    ((0, 0, 0, 0, 0, 1, 2, 1, 0), 0.055556),
    ((0, 0, 0, 0, 0, 1, 2, 1, 2), 0.185185),
    ((0, 0, 0, 0, 0, 1, 2, 0, 1), 0.103926),
    ((0, 0, 0, 0, 0, 1, 2, 2, 1), 0.452055),
    ((0, 0, 0, 0, 0, 1, 0, 2, 0), 0.223610),
    ((0, 0, 0, 0, 0, 1, 1, 2, 0), 0.310811),
    ((0, 0, 0, 0, 0, 1, 1, 2, 2), 0.490196),
    ((0, 0, 0, 0, 0, 1, 0, 2, 1), 0.238901),
    ((0, 0, 0, 0, 0, 1, 0, 0, 2), 0.132555),
    ((0, 0, 0, 0, 0, 1, 1, 0, 2), 0.263158),
    ((0, 0, 0, 0, 0, 1, 0, 1, 2), 0.120253),
    ((0, 0, 0, 0, 0, 0, 1, 0, 0), 0.243617),
    ((2, 0, 0, 0, 0, 0, 1, 0, 0), 0.197740),
    ((2, 0, 0, 0, 0, 0, 1, 1, 0), 0.103926),
    ((2, 2, 0, 0, 0, 0, 1, 1, 0), 0.194030),
    ((2, 2, 0, 0, 0, 0, 1, 1, 1), 1.000000),
    ((2, 0, 2, 0, 0, 0, 1, 1, 0), -0.044776),
    ((2, 0, 2, 0, 0, 0, 1, 1, 1), 1.000000),
    ((2, 0, 0, 2, 0, 0, 1, 1, 0), 0.452055),
    ((2, 0, 0, 2, 0, 0, 1, 1, 1), 1.000000),
    ((2, 0, 0, 0, 2, 0, 1, 1, 0), -0.107692),
    ((2, 0, 0, 0, 2, 0, 1, 1, 1), 1.000000),
    ((2, 0, 0, 0, 0, 2, 1, 1, 0), 0.232877),
    ((2, 0, 0, 0, 0, 2, 1, 1, 1), 1.000000),
    ((2, 0, 0, 0, 0, 0, 1, 1, 2), -0.090909),
    ((2, 0, 0, 0, 0, 0, 1, 0, 1), 0.315011),
    ((2, 2, 0, 0, 0, 0, 1, 0, 1), 0.245902),
    ((2, 0, 2, 0, 0, 0, 1, 0, 1), 0.095890),
    ((2, 0, 0, 2, 0, 0, 1, 0, 1), 0.616438),
    ((2, 0, 0, 0, 2, 0, 1, 0, 1), 0.291139),
    ((2, 0, 0, 0, 0, 2, 1, 0, 1), 0.493671),
    ((2, 0, 0, 0, 0, 0, 1, 2, 1), 0.185185),
    ((0, 2, 0, 0, 0, 0, 1, 0, 0), 0.309021),
    ((0, 2, 0, 0, 0, 0, 1, 1, 0), 0.315011),
    ((0, 2, 2, 0, 0, 0, 1, 1, 0), 0.134328),
    ((0, 2, 2, 0, 0, 0, 1, 1, 1), 1.000000),
    ((0, 2, 0, 2, 0, 0, 1, 1, 0), 0.493671),
    ((0, 2, 0, 2, 0, 0, 1, 1, 1), 1.000000),
    ((0, 2, 0, 0, 2, 0, 1, 1, 0), 0.341772),
    ((0, 2, 0, 0, 2, 0, 1, 1, 1), 1.000000),
    ((0, 2, 0, 0, 0, 2, 1, 1, 0), 0.561644),
    ((0, 2, 0, 0, 0, 2, 1, 1, 1), 1.000000),
    ((0, 2, 0, 0, 0, 0, 1, 1, 2), 0.185185),
    ((0, 2, 0, 0, 0, 0, 1, 0, 1), 0.297710),
    ((0, 2, 2, 0, 0, 0, 1, 0, 1), 0.245902),
    ((0, 2, 0, 2, 0, 0, 1, 0, 1), 0.522388),
    ((0, 2, 0, 0, 2, 0, 1, 0, 1), 0.017544),
    ((0, 2, 0, 0, 0, 2, 1, 0, 1), 0.522388),
    ((0, 2, 0, 0, 0, 0, 1, 2, 1), 0.200000),
    ((0, 0, 2, 0, 0, 0, 1, 0, 0), 0.199110),
    ((0, 0, 2, 0, 0, 0, 1, 1, 0), 0.047619),
    ((0, 0, 2, 2, 0, 0, 1, 1, 0), 0.291139),
    ((0, 0, 2, 2, 0, 0, 1, 1, 1), 1.000000),
    ((0, 0, 2, 0, 2, 0, 1, 1, 0), 0.139241),
    ((0, 0, 2, 0, 2, 0, 1, 1, 1), 1.000000),
    ((0, 0, 2, 0, 0, 2, 1, 1, 0), 0.017544),
    ((0, 0, 2, 0, 0, 2, 1, 1, 1), 1.000000),
    ((0, 0, 2, 0, 0, 0, 1, 1, 2), -0.217391),
    ((0, 0, 2, 0, 0, 0, 1, 0, 1), 0.315011),
    ((0, 0, 2, 2, 0, 0, 1, 0, 1), 0.493671),
    ((0, 0, 2, 0, 2, 0, 1, 0, 1), 0.291139),
    ((0, 0, 2, 0, 0, 2, 1, 0, 1), 0.616438),
    ((0, 0, 2, 0, 0, 0, 1, 2, 1), 0.185185),
    ((0, 0, 0, 2, 0, 0, 1, 0, 0), 0.352236),
    ((0, 0, 0, 2, 0, 0, 1, 1, 0), 0.238901),
    ((0, 0, 0, 2, 2, 0, 1, 1, 0), -0.068493),
    ((0, 0, 0, 2, 2, 0, 1, 1, 1), 1.000000),
    ((0, 0, 0, 2, 0, 2, 1, 1, 0), 0.245902),
    ((0, 0, 0, 2, 0, 2, 1, 1, 1), 1.000000),
    ((0, 0, 0, 2, 0, 0, 1, 1, 2), 0.074074),
    ((0, 0, 0, 2, 0, 0, 1, 0, 1), 0.438753),
    ((0, 0, 0, 2, 2, 0, 1, 0, 1), 0.134328),
    ((0, 0, 0, 2, 0, 2, 1, 0, 1), 0.508197),
    ((0, 0, 0, 2, 0, 0, 1, 2, 1), 0.372549),
    ((0, 0, 0, 0, 2, 0, 1, 0, 0), 0.035755),
    ((0, 0, 0, 0, 2, 0, 1, 1, 0), -0.006565),
    ((0, 0, 0, 0, 2, 2, 1, 1, 0), -0.044776),
    ((0, 0, 0, 0, 2, 2, 1, 1, 1), 1.000000),
    ((0, 0, 0, 0, 2, 0, 1, 1, 2), -0.276596),
    ((0, 0, 0, 0, 2, 0, 1, 0, 1), 0.129252),
    ((0, 0, 0, 0, 2, 2, 1, 0, 1), 0.134328),
    ((0, 0, 0, 0, 2, 0, 1, 2, 1), -0.086957),
    ((0, 0, 0, 0, 0, 2, 1, 0, 0), 0.309021),
    ((0, 0, 0, 0, 0, 2, 1, 1, 0), 0.165468),
    ((0, 0, 0, 0, 0, 2, 1, 1, 2), -0.023256),
    ((0, 0, 0, 0, 0, 2, 1, 0, 1), 0.438753),
    ((0, 0, 0, 0, 0, 2, 1, 2, 1), 0.372549),
    ((0, 0, 0, 0, 0, 0, 1, 2, 0), 0.352236),
    ((0, 0, 0, 0, 0, 0, 1, 2, 1), 0.209459),
    ((0, 0, 0, 0, 0, 0, 1, 0, 2), 0.197740),
    ((0, 0, 0, 0, 0, 0, 1, 1, 2), -0.048611),
    ((0, 0, 0, 0, 0, 0, 0, 1, 0), 0.137064),
    ((2, 0, 0, 0, 0, 0, 0, 1, 0), 0.057715),
    ((2, 0, 0, 0, 0, 0, 0, 1, 1), 0.047619),
    ((2, 2, 0, 0, 0, 0, 0, 1, 1), 0.134328),
    ((2, 0, 2, 0, 0, 0, 0, 1, 1), -0.044776),
    ((2, 0, 0, 2, 0, 0, 0, 1, 1), 0.017544),
    ((2, 0, 0, 0, 2, 0, 0, 1, 1), 0.139241),
    ((2, 0, 0, 0, 0, 2, 0, 1, 1), 0.291139),
    ((2, 0, 0, 0, 0, 0, 2, 1, 1), -0.217391),
    ((0, 2, 0, 0, 0, 0, 0, 1, 0), 0.309395),
    ((0, 2, 0, 0, 0, 0, 0, 1, 1), 0.315011),
    ((0, 2, 2, 0, 0, 0, 0, 1, 1), 0.194030),
    ((0, 2, 0, 2, 0, 0, 0, 1, 1), 0.561644),
    ((0, 2, 0, 0, 2, 0, 0, 1, 1), 0.341772),
    ((0, 2, 0, 0, 0, 2, 0, 1, 1), 0.493671),
    ((0, 2, 0, 0, 0, 0, 2, 1, 1), 0.185185),
    ((0, 0, 2, 0, 0, 0, 0, 1, 0), 0.057715),
    ((0, 0, 2, 0, 0, 0, 0, 1, 1), 0.103926),
    ((0, 0, 2, 2, 0, 0, 0, 1, 1), 0.232877),
    ((0, 0, 2, 0, 2, 0, 0, 1, 1), -0.107692),
    ((0, 0, 2, 0, 0, 2, 0, 1, 1), 0.452055),
    ((0, 0, 2, 0, 0, 0, 2, 1, 1), -0.090909),
    ((0, 0, 0, 2, 0, 0, 0, 1, 0), 0.223610),
    ((0, 0, 0, 2, 0, 0, 0, 1, 1), 0.165468),
    ((0, 0, 0, 2, 2, 0, 0, 1, 1), -0.044776),
    ((0, 0, 0, 2, 0, 2, 0, 1, 1), 0.245902),
    ((0, 0, 0, 2, 0, 0, 2, 1, 1), -0.023256),
    ((0, 0, 0, 0, 2, 0, 0, 1, 0), -0.056168),
    ((0, 0, 0, 0, 2, 0, 0, 1, 1), -0.006565),
    ((0, 0, 0, 0, 2, 2, 0, 1, 1), -0.068493),
    ((0, 0, 0, 0, 2, 0, 2, 1, 1), -0.276596),
    ((0, 0, 0, 0, 0, 2, 0, 1, 0), 0.223610),
    ((0, 0, 0, 0, 0, 2, 0, 1, 1), 0.238901),
    ((0, 0, 0, 0, 0, 2, 2, 1, 1), 0.074074),
    ((0, 0, 0, 0, 0, 0, 2, 1, 0), 0.132555),
    ((0, 0, 0, 0, 0, 0, 2, 1, 1), -0.048611),
    ((0, 0, 0, 0, 0, 0, 0, 1, 2), 0.132555),
    ((0, 0, 0, 0, 0, 0, 0, 0, 1), 0.243617),
    ((2, 0, 0, 0, 0, 0, 0, 0, 1), 0.199110),
    ((0, 2, 0, 0, 0, 0, 0, 0, 1), 0.309021),
    ((0, 0, 2, 0, 0, 0, 0, 0, 1), 0.197740),
    ((0, 0, 0, 2, 0, 0, 0, 0, 1), 0.309021),
    ((0, 0, 0, 0, 2, 0, 0, 0, 1), 0.035755),
    ((0, 0, 0, 0, 0, 2, 0, 0, 1), 0.352236),
    ((0, 0, 0, 0, 0, 0, 2, 0, 1), 0.197740),
    ((0, 0, 0, 0, 0, 0, 0, 2, 1), 0.352236),
])

def eval_local_boards(state_board: np.ndarray) -> list[float]:
    """
    Accepts a (3, 3, 3, 3) Ultimate Tic Tac Toe board:
    3x3 macro-board, each with a 3x3 local board.
    """
    evals = []
    for macro_row in range(3):
        for macro_col in range(3):
            local_board = state_board[macro_row, macro_col]  # shape (3, 3)
            score = states.get(tuple(local_board.flatten()), 0.0)
            evals.append(score)
    return evals

def eval_aug_data(data) -> tuple[np.ndarray, np.ndarray]:
    X = []
    y = []
    for state, utility in data:
        evals = eval_local_boards(state.board)  # list of 9 floats
        fill_num = state.fill_num
        # Check if prev_local_action is None; if so, set a default (e.g., [0, 0])
        if state.prev_local_action is None:
            prev_action = [0, 0]
        else:
            prev_action = list(state.prev_local_action)  # 2 ints: (row, col)
        feature_vector = evals + [fill_num] + prev_action  # shape: (12,)
        X.append(feature_vector)
        y.append(utility)
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

X_train, y_train = eval_aug_data(train_data)
X_eval, y_eval = eval_aug_data(test_data)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pprint

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(12, 32)   # Input → Layer 1
        self.fc2 = nn.Linear(32, 32)   # Layer 1 → Layer 2
        self.fc3 = nn.Linear(32, 32)    # Layer 2 → Layer 3
        self.out = nn.Linear(32, 1)     # Layer 3 → Output

    def forward(self, x):
        x = F.gelu(self.fc1(x))   # Layer 1
        x = F.gelu(self.fc2(x))   # Layer 2
        x = F.gelu(self.fc3(x))   # Layer 3
        x = self.out(x)           # Output layer (no activation for regression)
        return x

# Set up GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_eval_tensor = torch.tensor(X_eval, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)  # Add extra dimension
y_eval_tensor = torch.tensor(y_eval, dtype=torch.float32).unsqueeze(1).to(device)

# --- Training ---
net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
loss_fn = nn.MSELoss()
losses = []

num_epochs = 2000
for epoch in range(num_epochs):
    net.train()
    y_pred_train = net(X_train_tensor)  # Use the tensor version.
    loss = loss_fn(y_pred_train, y_train_tensor)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item():.4f}")
    if epoch + 1 == num_epochs:
        print(f"Epoch {num_epochs}, Training Loss: {loss.item():.4f}")

# --- Evaluation ---
net.eval()
with torch.no_grad():
    y_pred_eval = net(X_eval_tensor)
eval_mse = loss_fn(y_pred_eval, y_eval_tensor).item()
print(f"Evaluation MSE: {eval_mse:.4f}")

trained_weights = net.state_dict()
weights_dict = OrderedDict()
for name, param in trained_weights.items():
    values = param.detach().cpu().numpy().tolist()
    weights_dict[name] = values

with open("weights2.py", "w") as f:
    f.write("weights = OrderedDict([\n")
    for key, value in weights_dict.items():
        f.write(f"    ('{key}', torch.tensor(\n")
        f.write(pprint.pformat(value, indent=8))
        f.write("\n    )),\n")  # close parentheses here
    f.write("])\n")

Using device: cuda
Epoch 50/2000, Training Loss: 0.3269
Epoch 100/2000, Training Loss: 0.3181
Epoch 150/2000, Training Loss: 0.3151
Epoch 200/2000, Training Loss: 0.3138
Epoch 250/2000, Training Loss: 0.3127
Epoch 300/2000, Training Loss: 0.3119
Epoch 350/2000, Training Loss: 0.3115
Epoch 400/2000, Training Loss: 0.3112
Epoch 450/2000, Training Loss: 0.3109
Epoch 500/2000, Training Loss: 0.3106
Epoch 550/2000, Training Loss: 0.3104
Epoch 600/2000, Training Loss: 0.3102
Epoch 650/2000, Training Loss: 0.3099
Epoch 700/2000, Training Loss: 0.3097
Epoch 750/2000, Training Loss: 0.3094
Epoch 800/2000, Training Loss: 0.3091
Epoch 850/2000, Training Loss: 0.3087
Epoch 900/2000, Training Loss: 0.3084
Epoch 950/2000, Training Loss: 0.3079
Epoch 1000/2000, Training Loss: 0.3074
Epoch 1050/2000, Training Loss: 0.3070
Epoch 1100/2000, Training Loss: 0.3066
Epoch 1150/2000, Training Loss: 0.3063
Epoch 1200/2000, Training Loss: 0.3060
Epoch 1250/2000, Training Loss: 0.3058
Epoch 1300/2000, Training 